In [ ]:
"""
================================================================================
                              ARC-AGI CODE GOLF SOLUTION
                              
This module implements a comprehensive solution for the ARC-AGI code golf
competition, using an extensible handler-based architecture with automatic
discovery and optimization capabilities.
================================================================================
"""

# ================================================================================
# IMPORTS
# ================================================================================
import json
import os
import zipfile
import numpy as np
from collections import defaultdict, Counter
from scipy.ndimage import label, binary_dilation, binary_erosion, convolve, binary_fill_holes
from itertools import product, permutations, combinations
import math
import time
from functools import wraps
from typing import Dict, List, Tuple, Optional, Callable, Any
import inspect
import abc

# ================================================================================
# BASE CLASSES
# ================================================================================
class TransformationHandler(abc.ABC):
    def __init__(self):
        self.name = self.__class__.__name__
        
    @abc.abstractmethod
    def can_handle(self, task_data: dict, analysis: dict) -> bool:
        pass
        
    @abc.abstractmethod
    def generate_solution(self, task_data: dict, analysis: dict) -> Optional[str]:
        pass
        
    def verify_solution(self, solution_code: str, task_data: dict) -> bool:
        try:
            local_namespace = {}
            exec(solution_code, globals(), local_namespace)
            
            if 'p' not in local_namespace:
                return False
            
            p = local_namespace['p']
            
            for example in task_data['train']:
                input_grid = example['input']
                expected = example['output']
                try:
                    actual = p(input_grid)
                    if actual != expected:
                        return False
                except:
                    return False
            return True
        except:
            return False

# ================================================================================
# HANDLER REGISTRY AND MANAGEMENT
# ================================================================================
"""
The HandlerRegistry manages all transformation handlers, tracks their performance,
and provides auto-discovery functionality to find all handler classes.
"""
class HandlerRegistry:
    def __init__(self):
        self.handlers = []
        self.performance_stats = defaultdict(lambda: {'calls': 0, 'successes': 0, 'time': 0})
        
    def register(self, handler_class):
        self.handlers.append(handler_class())
        return handler_class
        
    def auto_discover(self, module_globals):
        for name, obj in module_globals.items():
            if (inspect.isclass(obj) and 
                issubclass(obj, TransformationHandler) and 
                obj != TransformationHandler and
                name.startswith('Handler')):
                self.register(obj)
                
    def get_all_handlers(self):
        return self.handlers
        
    def track_performance(self, handler_name: str, success: bool, time_taken: float):
        self.performance_stats[handler_name]['calls'] += 1
        self.performance_stats[handler_name]['time'] += time_taken
        if success:
            self.performance_stats[handler_name]['successes'] += 1

# ================================================================================
# GLOBAL REGISTRY AND DECORATOR
# ================================================================================
"""
Global registry instance and decorator for automatic handler registration.
"""
registry = HandlerRegistry()

def handler_decorator(cls):
    registry.register(cls)
    return cls

# ================================================================================
# TASK ANALYSIS
# ================================================================================
"""
The TaskAnalyzer examines task data to extract patterns, features, and
characteristics that help determine which handlers might be applicable.
"""
class TaskAnalyzer:
    @staticmethod
    def analyze(task_data: dict) -> dict:
        analysis = {
            'color_changes': defaultdict(set),
            'shape_changes': set(),
            'color_count': set(),
            'has_objects': False,
            'has_lines': False,
            'has_patterns': False,
            'transform_types': set()
        }
        
        for example in task_data['train']:
            inp = np.array(example['input'])
            out = np.array(example['output'])
            
            analysis['shape_changes'].add((inp.shape, out.shape))
            analysis['color_count'].add(len(np.unique(inp)))
            
            if len(np.unique(inp)) > 2:
                analysis['has_objects'] = True
                
            for i in range(inp.shape[0]):
                if len(np.unique(inp[i,:])) == 1 and inp[i,0] != 0:
                    analysis['has_lines'] = True
                    
            if inp.shape != out.shape:
                if out.shape[0] > inp.shape[0] or out.shape[1] > inp.shape[1]:
                    analysis['transform_types'].add('expansion')
                else:
                    analysis['transform_types'].add('reduction')
                    
            if inp.shape == out.shape:
                for k in range(1, 4):
                    if np.array_equal(np.rot90(inp, k), out):
                        analysis['transform_types'].add(f'rotation_{k*90}')
                        
                if np.array_equal(np.fliplr(inp), out):
                    analysis['transform_types'].add('flip_horizontal')
                if np.array_equal(np.flipud(inp), out):
                    analysis['transform_types'].add('flip_vertical')
                    
        return analysis

# ================================================================================
# UTILITY FUNCTIONS
# ================================================================================
"""
Helper utilities for common operations including raster transformations,
1D/2D conversions, and other frequently used operations.
"""
class Utilities:
    @staticmethod
    def to_1d_raster(grid, order='row_major'):
        h, w = len(grid), len(grid[0]) if grid else 0
        
        if order == 'row_major':
            return [grid[i][j] for i in range(h) for j in range(w)]
        elif order == 'col_major':
            return [grid[i][j] for j in range(w) for i in range(h)]
        elif order == 'zigzag':
            result = []
            for i in range(h):
                if i % 2 == 0:
                    result.extend(grid[i])
                else:
                    result.extend(grid[i][::-1])
            return result
        elif order == 'spiral':
            result = []
            top, bottom, left, right = 0, h-1, 0, w-1
            while top <= bottom and left <= right:
                for j in range(left, right+1):
                    result.append(grid[top][j])
                top += 1
                for i in range(top, bottom+1):
                    result.append(grid[i][right])
                right -= 1
                if top <= bottom:
                    for j in range(right, left-1, -1):
                        result.append(grid[bottom][j])
                    bottom -= 1
                if left <= right:
                    for i in range(bottom, top-1, -1):
                        result.append(grid[i][left])
                    left += 1
            return result
        return []
        
    @staticmethod
    def from_1d_raster(arr, h, w, order='row_major'):
        if len(arr) != h * w:
            return None
            
        grid = [[0]*w for _ in range(h)]
        
        if order == 'row_major':
            for idx, val in enumerate(arr):
                i, j = idx // w, idx % w
                grid[i][j] = val
        elif order == 'col_major':
            for idx, val in enumerate(arr):
                j, i = idx // h, idx % h
                grid[i][j] = val
        elif order == 'zigzag':
            idx = 0
            for i in range(h):
                if i % 2 == 0:
                    for j in range(w):
                        grid[i][j] = arr[idx]
                        idx += 1
                else:
                    for j in range(w-1, -1, -1):
                        grid[i][j] = arr[idx]
                        idx += 1
        elif order == 'spiral':
            idx = 0
            top, bottom, left, right = 0, h-1, 0, w-1
            while top <= bottom and left <= right and idx < len(arr):
                for j in range(left, right+1):
                    if idx < len(arr):
                        grid[top][j] = arr[idx]
                        idx += 1
                top += 1
                for i in range(top, bottom+1):
                    if idx < len(arr):
                        grid[i][right] = arr[idx]
                        idx += 1
                right -= 1
                if top <= bottom:
                    for j in range(right, left-1, -1):
                        if idx < len(arr):
                            grid[bottom][j] = arr[idx]
                            idx += 1
                    bottom -= 1
                if left <= right:
                    for i in range(bottom, top-1, -1):
                        if idx < len(arr):
                            grid[i][left] = arr[idx]
                            idx += 1
                    left += 1
        
        return grid

In [ ]:
# ================================================================================
# ================================================================================
#                          HANDLER IMPLEMENTATIONS
# ================================================================================
# ================================================================================
"""
This section contains all transformation handler implementations. Each handler
is responsible for detecting and generating solutions for specific pattern types.
Handlers are organized into logical groups based on their functionality.
"""

# --------------------------------------------------------------------------------
# BASIC TRANSFORMATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerIdentity(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        return np.array_equal(inp, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return g
"""
        return None



















In [ ]:
# --------------------------------------------------------------------------------
# GEOMETRIC TRANSFORMATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerRotate90(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        return np.array_equal(np.rot90(inp, -1), out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return[list(r)for r in zip(*g[::-1])]
"""
        return None

@handler_decorator
class HandlerRotate180(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        return np.array_equal(np.rot90(inp, 2), out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return[r[::-1]for r in g[::-1]]
"""
        return None

@handler_decorator
class HandlerRotate270(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        return np.array_equal(np.rot90(inp, 1), out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return[list(r)for r in zip(*g)][::-1]
"""
        return None

@handler_decorator
class HandlerFlipHorizontal(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        return np.array_equal(np.fliplr(inp), out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return[r[::-1]for r in g]
"""
        return None

@handler_decorator
class HandlerFlipVertical(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        return np.array_equal(np.flipud(inp), out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return g[::-1]
"""
        return None

@handler_decorator
class HandlerTranspose(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        return np.array_equal(inp.T, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return[list(r)for r in zip(*g)]
"""
        return None

@handler_decorator
class HandlerFlipDiagonal(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        h, w = inp.shape
        if h != w:
            return False
        test = np.array([[inp[h-1-j, h-1-i] for j in range(h)] for i in range(h)])
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h = len(task_data['train'][0]['input'])
            return f"""def p(g):
 h={h}
 return[[g[h-1-j][h-1-i]for j in range(h)]for i in range(h)]
"""
        return None

@handler_decorator
class HandlerCircularShiftRight(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        for shift in range(1, inp.shape[1]):
            test = np.roll(inp, shift, axis=1)
            if np.array_equal(test, out):
                return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        for shift in range(1, inp.shape[1]):
            test = np.roll(inp, shift, axis=1)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[r[-{shift}:]+r[:-{shift}]for r in g]
"""
        return None

@handler_decorator
class HandlerCircularShiftDown(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        for shift in range(1, inp.shape[0]):
            test = np.roll(inp, shift, axis=0)
            if np.array_equal(test, out):
                return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        for shift in range(1, inp.shape[0]):
            test = np.roll(inp, shift, axis=0)
            if np.array_equal(test, out):
                return f"""def p(g):
 return g[-{shift}:]+g[:-{shift}]
"""
        return None

@handler_decorator
class HandlerCircularShiftLeft(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        for shift in range(1, inp.shape[1]):
            test = np.roll(inp, -shift, axis=1)
            if np.array_equal(test, out):
                return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        for shift in range(1, inp.shape[1]):
            test = np.roll(inp, -shift, axis=1)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[r[{shift}:]+r[:{shift}]for r in g]
"""
        return None

@handler_decorator
class HandlerCircularShiftUp(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        for shift in range(1, inp.shape[0]):
            test = np.roll(inp, -shift, axis=0)
            if np.array_equal(test, out):
                return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        for shift in range(1, inp.shape[0]):
            test = np.roll(inp, -shift, axis=0)
            if np.array_equal(test, out):
                return f"""def p(g):
 return g[{shift}:]+g[:{shift}]
"""
        return None

@handler_decorator
class HandlerReflectAroundCenter(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        h, w = inp.shape
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                test[i,j] = inp[h-1-i, w-1-j]
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 return[[g[h-1-i][w-1-j]for j in range(w)]for i in range(h)]
"""
        return None

@handler_decorator
class HandlerPartialRotation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # The system checks for rotation of specific regions
        for region_size in range(2, min(h, w) + 1):
            for i in range(h - region_size + 1):
                for j in range(w - region_size + 1):
                    test = np.copy(inp)
                    region = inp[i:i+region_size, j:j+region_size]
                    rotated = np.rot90(region, -1)
                    test[i:i+region_size, j:j+region_size] = rotated
                    
                    if np.array_equal(test, out):
                        return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for region_size in range(2, min(h, w) + 1):
            for i in range(h - region_size + 1):
                for j in range(w - region_size + 1):
                    test = np.copy(inp)
                    region = inp[i:i+region_size, j:j+region_size]
                    rotated = np.rot90(region, -1)
                    test[i:i+region_size, j:j+region_size] = rotated
                    
                    if np.array_equal(test, out):
                        # Partial rotation requires complex indexing
                        return None
        return None

@handler_decorator
class HandlerSkewTransformation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        h, w = inp.shape
        
        # The implementation verifies horizontal skew transformations
        if out.shape[1] > w:
            test = np.zeros((h, w + h - 1), dtype=int)
            for i in range(h):
                for j in range(w):
                    if inp[i, j] != 0:
                        test[i, j + i] = inp[i, j]
            
            if test.shape == out.shape and np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 res=[[0]*(w+h-1)for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    res[i][j+i]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerMirrorSymmetry(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        h, w = inp.shape
        
        # The system checks for various mirror symmetry operations
        if out.shape == (h, w * 2):
            # Horizontal mirror concatenation
            test = np.hstack([inp, np.fliplr(inp)])
            if np.array_equal(test, out):
                return True
        
        if out.shape == (h * 2, w):
            # Vertical mirror concatenation
            test = np.vstack([inp, np.flipud(inp)])
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if out.shape == (inp.shape[0], inp.shape[1] * 2):
            test = np.hstack([inp, np.fliplr(inp)])
            if np.array_equal(test, out):
                return """def p(g):
 return[r+r[::-1]for r in g]
"""
        
        if out.shape == (inp.shape[0] * 2, inp.shape[1]):
            test = np.vstack([inp, np.flipud(inp)])
            if np.array_equal(test, out):
                return """def p(g):
 return g+g[::-1]
"""
        return None

@handler_decorator
class HandlerDiagonalShift(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # The implementation checks for diagonal shifting patterns
        for shift in range(1, min(h, w)):
            test = np.zeros_like(inp)
            for i in range(h):
                for j in range(w):
                    new_i = (i + shift) % h
                    new_j = (j + shift) % w
                    test[new_i, new_j] = inp[i, j]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for shift in range(1, min(h, w)):
            test = np.zeros_like(inp)
            for i in range(h):
                for j in range(w):
                    new_i = (i + shift) % h
                    new_j = (j + shift) % w
                    test[new_i, new_j] = inp[i, j]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w={h},{w}
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   res[(i+{shift})%h][(j+{shift})%w]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerSpiralTransformation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The system verifies spiral unwinding transformation
        h, w = inp.shape
        spiral_order = []
        
        top, bottom, left, right = 0, h - 1, 0, w - 1
        while top <= bottom and left <= right:
            for j in range(left, right + 1):
                spiral_order.append((top, j))
            top += 1
            
            for i in range(top, bottom + 1):
                spiral_order.append((i, right))
            right -= 1
            
            if top <= bottom:
                for j in range(right, left - 1, -1):
                    spiral_order.append((bottom, j))
                bottom -= 1
            
            if left <= right:
                for i in range(bottom, top - 1, -1):
                    spiral_order.append((i, left))
                left += 1
        
        # Check if output follows spiral reordering
        values = [inp[i, j] for i, j in spiral_order]
        test = np.zeros_like(inp)
        idx = 0
        for i in range(h):
            for j in range(w):
                test[i, j] = values[idx]
                idx += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Spiral transformation requires complex iteration logic
        return None

@handler_decorator
class HandlerQuadrantSwap(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        if h % 2 != 0 or w % 2 != 0:
            return False
        
        # The implementation checks for quadrant swapping patterns
        h2, w2 = h // 2, w // 2
        
        # Extract quadrants
        q1 = inp[:h2, :w2]
        q2 = inp[:h2, w2:]
        q3 = inp[h2:, :w2]
        q4 = inp[h2:, w2:]
        
        # Try different swap patterns
        test = np.zeros_like(inp)
        
        # Diagonal swap
        test[:h2, :w2] = q4
        test[:h2, w2:] = q3
        test[h2:, :w2] = q2
        test[h2:, w2:] = q1
        
        if np.array_equal(test, out):
            return True
        
        # Adjacent swap
        test[:h2, :w2] = q2
        test[:h2, w2:] = q1
        test[h2:, :w2] = q4
        test[h2:, w2:] = q3
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if h % 2 == 0 and w % 2 == 0 and self.can_handle(task_data, analysis):
            h2, w2 = h // 2, w // 2
            
            # Extract quadrants
            q1 = inp[:h2, :w2]
            q2 = inp[:h2, w2:]
            q3 = inp[h2:, :w2]
            q4 = inp[h2:, w2:]
            
            # Check diagonal swap
            test = np.zeros_like(inp)
            test[:h2, :w2] = q4
            test[:h2, w2:] = q3
            test[h2:, :w2] = q2
            test[h2:, w2:] = q1
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h2,w2={h2},{w2}
 q1=[r[:w2]for r in g[:h2]]
 q2=[r[w2:]for r in g[:h2]]
 q3=[r[:w2]for r in g[h2:]]
 q4=[r[w2:]for r in g[h2:]]
 res=[]
 for r4,r3 in zip(q4,q3):res.append(r4+r3)
 for r2,r1 in zip(q2,q1):res.append(r2+r1)
 return res
"""
        return None

@handler_decorator
class HandlerRadialTransformation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        # The system verifies radial transformations from center
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                # Calculate distance and angle from center
                dy, dx = i - ch, j - cw
                dist = int(np.sqrt(dy * dy + dx * dx))
                
                # Simple radial pattern based on distance
                if dist < min(ch, cw):
                    test[i, j] = (dist + 1) % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Radial transformations require complex distance calculations
        return None

@handler_decorator
class HandlerShearVertical(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        h, w = inp.shape
        
        # The implementation checks for vertical shear transformations
        if out.shape[0] > h:
            test = np.zeros((h + w - 1, w), dtype=int)
            for i in range(h):
                for j in range(w):
                    if inp[i, j] != 0:
                        test[i + j, j] = inp[i, j]
            
            if test.shape == out.shape and np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 res=[[0]*w for _ in range(h+w-1)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    res[i+j][j]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerPerspectiveTransform(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system checks for perspective-like transformations
        # This is a simplified version focusing on trapezoid transformations
        if out.shape[0] != inp.shape[0]:
            return False
        
        h, w = inp.shape
        
        # Check if rows are compressed toward center
        test = np.zeros_like(out)
        for i in range(h):
            # Calculate compression factor based on distance from center
            dist_from_center = abs(i - h // 2)
            compression = 1 - (dist_from_center / h)
            
            if compression > 0:
                compressed_width = int(w * compression)
                offset = (out.shape[1] - compressed_width) // 2
                
                for j in range(compressed_width):
                    source_j = int(j * w / compressed_width)
                    if source_j < w and offset + j < out.shape[1]:
                        test[i, offset + j] = inp[i, source_j]
        
        return np.count_nonzero(test) > 0
        
    def generate_solution(self, task_data, analysis):
        # Perspective transformations require complex interpolation
        return None

@handler_decorator
class HandlerBlockSwap(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # The implementation verifies block swapping patterns
        for block_h in range(2, h // 2 + 1):
            for block_w in range(2, w // 2 + 1):
                if h % block_h == 0 and w % block_w == 0:
                    # Try swapping adjacent blocks
                    test = np.copy(inp)
                    
                    for i in range(0, h - block_h, block_h * 2):
                        for j in range(0, w - block_w, block_w * 2):
                            # Swap horizontally adjacent blocks
                            if j + block_w * 2 <= w:
                                temp = test[i:i+block_h, j:j+block_w].copy()
                                test[i:i+block_h, j:j+block_w] = test[i:i+block_h, j+block_w:j+block_w*2]
                                test[i:i+block_h, j+block_w:j+block_w*2] = temp
                    
                    if np.array_equal(test, out):
                        return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Block swapping requires complex indexing operations
        return None

@handler_decorator
class HandlerWaveDistortion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape[0] != out.shape[0]:
            return False
        
        h, w = inp.shape
        
        # The system checks for wave-like horizontal distortions
        test = np.zeros_like(out)
        
        for i in range(h):
            # Calculate wave offset based on row
            offset = int(2 * np.sin(i * np.pi / h))
            
            for j in range(w):
                new_j = j + offset
                if 0 <= new_j < out.shape[1]:
                    test[i, new_j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Wave distortions require trigonometric calculations
        return None

@handler_decorator
class HandlerRecursiveRotation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The implementation checks for recursive rotation patterns
        # where inner regions are rotated within rotated outer regions
        h, w = inp.shape
        if h != w or h < 3:
            return False
        
        test = np.copy(inp)
        
        # Rotate entire grid
        test = np.rot90(test, -1)
        
        # Rotate inner region
        if h >= 3:
            inner = test[1:-1, 1:-1]
            test[1:-1, 1:-1] = np.rot90(inner, -1)
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Recursive rotations require nested operations
        return None

@handler_decorator
class HandlerZigzagTransformation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # The system verifies zigzag reading and writing patterns
        zigzag_order = []
        
        for i in range(h):
            if i % 2 == 0:
                # Left to right
                for j in range(w):
                    zigzag_order.append((i, j))
            else:
                # Right to left
                for j in range(w - 1, -1, -1):
                    zigzag_order.append((i, j))
        
        # Read values in zigzag order
        values = [inp[i, j] for i, j in zigzag_order]
        
        # Write back in normal order
        test = np.zeros_like(inp)
        idx = 0
        for i in range(h):
            for j in range(w):
                test[i, j] = values[idx]
                idx += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Zigzag patterns require complex iteration logic
        return None

@handler_decorator
class HandlerConditionalRotation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The implementation checks for rotation based on cell values
        h, w = inp.shape
        
        from scipy.ndimage import label
        labeled, n = label(inp > 0)
        
        if n == 0:
            return False
        
        test = np.copy(inp)
        
        # Rotate each object based on its label
        for i in range(1, n + 1):
            mask = labeled == i
            positions = np.argwhere(mask)
            
            if len(positions) > 0:
                min_i, min_j = positions.min(axis=0)
                max_i, max_j = positions.max(axis=0)
                
                region = inp[min_i:max_i+1, min_j:max_j+1]
                
                # Rotate based on object index
                rotations = (i - 1) % 4
                if rotations > 0:
                    rotated = np.rot90(region, -rotations)
                    if rotated.shape == region.shape:
                        test[min_i:max_i+1, min_j:max_j+1] = rotated
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Conditional transformations require object detection
        return None

@handler_decorator
class HandlerFoldingTransformation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        h, w = inp.shape
        
        # The system checks for folding transformations
        if out.shape == (h, w // 2) and w % 2 == 0:
            # Horizontal fold
            left = inp[:, :w//2]
            right = inp[:, w//2:]
            
            # Try OR folding
            test = np.maximum(left, np.fliplr(right))
            if np.array_equal(test, out):
                return True
            
            # Try AND folding
            test = np.minimum(left, np.fliplr(right))
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        w = inp.shape[1]
        
        if out.shape == (inp.shape[0], w // 2) and w % 2 == 0:
            left = inp[:, :w//2]
            right = inp[:, w//2:]
            
            # Check OR folding
            test = np.maximum(left, np.fliplr(right))
            if np.array_equal(test, out):
                return f"""def p(g):
 w={w}
 return[[max(r[j],r[w-1-j])for j in range(w//2)]for r in g]
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# COLOR OPERATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerColorMapping(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return len(analysis['color_changes']) > 0
        
    def generate_solution(self, task_data, analysis):
        color_map = {}
        for pos, changes in analysis['color_changes'].items():
            if len(changes) == 1:
                src, dest = next(iter(changes))
                if src not in color_map:
                    color_map[src] = dest
                elif color_map[src] != dest:
                    return None
        
        if not color_map:
            return None
        
        cases = ','.join([f'{int(s)}:{int(d)}' for s,d in color_map.items()])
        return f"""def p(g):
 m={{{cases}}}
 return[[m.get(x,x)for x in r]for r in g]
"""

@handler_decorator
class HandlerColorInversion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        return np.array_equal(9 - inp, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return[[9-x for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerBinaryToggle(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        unique_inp = np.unique(inp)
        unique_out = np.unique(out)
        if len(unique_inp) == 2 and len(unique_out) == 2:
            color1, color2 = sorted(unique_inp)
            test = np.where(inp == color1, color2, color1)
            return np.array_equal(test, out)
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        unique_inp = np.unique(inp)
        if len(unique_inp) == 2:
            c1, c2 = sorted(unique_inp)
            test = np.where(inp == c1, c2, c1)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[{c2} if x=={c1} else {c1} for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerColorCycling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        test = (inp + 1) % 10
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return[[(x+1)%10 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerSetSpecificColor(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        # Check if all non-zero values become a specific color
        non_zero_mask = inp != 0
        if np.any(non_zero_mask):
            unique_out_values = np.unique(out[non_zero_mask])
            if len(unique_out_values) == 1:
                return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        non_zero_mask = inp != 0
        if np.any(non_zero_mask):
            unique_out_values = np.unique(out[non_zero_mask])
            if len(unique_out_values) == 1:
                target_color = int(unique_out_values[0])
                return f"""def p(g):
 return[[{target_color} if x!=0 else 0 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerColorByPosition(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        # Check if colors are assigned based on position
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                test[i,j] = (i + j * 2) % 10
        if np.array_equal(test, out):
            return True
        # Try other position-based formulas
        for i in range(h):
            for j in range(w):
                test[i,j] = (i * 3 + j) % 10
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                test[i,j] = (i + j * 2) % 10
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[(i+j*2)%10 for j in range({w})]for i in range({h})]
"""
        for i in range(h):
            for j in range(w):
                test[i,j] = (i * 3 + j) % 10
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[(i*3+j)%10 for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerColorReplacePairs(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        # Check if specific color pairs are replaced
        for c1 in range(10):
            for c2 in range(10):
                if c1 != c2:
                    mask1 = inp == c1
                    mask2 = inp == c2
                    test = np.copy(inp)
                    test[mask1] = c2
                    test[mask2] = c1
                    if np.array_equal(test, out):
                        return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        for c1 in range(10):
            for c2 in range(10):
                if c1 != c2:
                    mask1 = inp == c1
                    mask2 = inp == c2
                    test = np.copy(inp)
                    test[mask1] = c2
                    test[mask2] = c1
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return[[{c2} if x=={c1} else {c1} if x=={c2} else x for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerGradientColoring(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # The system verifies horizontal gradient patterns
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                test[i, j] = (j * 10 // w) % 10
        
        if np.array_equal(test, out):
            return True
        
        # The system verifies vertical gradient patterns
        for i in range(h):
            for j in range(w):
                test[i, j] = (i * 10 // h) % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                test[i, j] = (j * 10 // w) % 10
        
        if np.array_equal(test, out):
            return f"""def p(g):
 w={w}
 return[[(j*10//w)%10 for j in range(w)]for _ in range({h})]
"""
        
        for i in range(h):
            for j in range(w):
                test[i, j] = (i * 10 // h) % 10
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h={h}
 return[[(i*10//h)%10 for j in range({w})]for i in range(h)]
"""
        return None

@handler_decorator
class HandlerColorQuantization(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The implementation checks for color quantization patterns
        for levels in [2, 3, 4, 5]:
            test = (inp * levels // 10) * (10 // levels)
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for levels in [2, 3, 4, 5]:
            test = (inp * levels // 10) * (10 // levels)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[(x*{levels}//10)*{10//levels} for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerConditionalColorByNeighbors(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # The system analyzes neighbor-based color transformations
        for i in range(h):
            for j in range(w):
                neighbor_count = 0
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] != 0:
                        neighbor_count += 1
                
                if neighbor_count >= 2:
                    test[i, j] = 5  # Arbitrary color for cells with many neighbors
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   n=sum(1 for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]if 0<=i+di<h and 0<=j+dj<w and g[i+di][j+dj]!=0)
   if n>=2:res[i][j]=5
 return res
"""
        return None

@handler_decorator
class HandlerColorBlending(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The implementation verifies color blending with neighbors
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                total = inp[i, j]
                count = 1
                
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w:
                        total += inp[ni, nj]
                        count += 1
                
                test[i, j] = total // count
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   t,c=g[i][j],1
   for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
    if 0<=i+di<h and 0<=j+dj<w:
     t+=g[i+di][j+dj]
     c+=1
   row.append(t//c)
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerColorPropagation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The system checks for color propagation from specific sources
        h, w = inp.shape
        test = np.copy(inp)
        
        # Find source colors (non-zero in input)
        sources = []
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    sources.append((i, j, inp[i, j]))
        
        # Propagate colors to adjacent cells
        for si, sj, color in sources:
            for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                ni, nj = si + di, sj + dj
                if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] == 0:
                    test[ni, nj] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]==0:
      res[ni][nj]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerDistanceBasedColoring(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The implementation finds reference points and colors based on distance
        h, w = inp.shape
        
        # Find non-zero positions as reference points
        refs = []
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    refs.append((i, j))
        
        if not refs:
            return False
        
        test = np.zeros_like(inp)
        
        # Color based on distance to nearest reference
        for i in range(h):
            for j in range(w):
                min_dist = float('inf')
                for ri, rj in refs:
                    dist = abs(i - ri) + abs(j - rj)
                    min_dist = min(min_dist, dist)
                test[i, j] = min_dist % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Distance-based coloring requires complex calculations
        return None

@handler_decorator
class HandlerPaletteReduction(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The system verifies palette reduction patterns
        unique_inp = np.unique(inp)
        unique_out = np.unique(out)
        
        return len(unique_out) < len(unique_inp) and len(unique_out) > 1
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Simple palette reduction: group colors
        test = np.zeros_like(inp)
        for val in np.unique(inp):
            if val <= 3:
                test[inp == val] = 1
            elif val <= 6:
                test[inp == val] = 5
            else:
                test[inp == val] = 8
        
        if np.array_equal(test, out):
            return """def p(g):
 return[[1 if x<=3 else 5 if x<=6 else 8 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerMultiStepColorTransform(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The implementation checks for multi-step transformations
        # Example: first add 3, then modulo 7
        test = ((inp + 3) % 7)
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Try various two-step transformations
        for add in range(1, 10):
            for mod in range(2, 10):
                test = (inp + add) % mod
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[[(x+{add})%{mod} for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerRegionBasedColoring(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The system analyzes region-based color assignment
        from scipy.ndimage import label
        
        labeled, n = label(inp > 0)
        
        if n == 0:
            return False
        
        # Check if each region gets a unique color
        test = np.zeros_like(inp)
        for i in range(1, n + 1):
            mask = labeled == i
            test[mask] = (i % 9) + 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 t=np.array(g)
 l,n=label(t>0)
 res=np.zeros_like(t)
 for i in range(1,n+1):
  m=l==i
  res[m]=(i%9)+1
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerColorInheritance(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The implementation verifies color inheritance from specific neighbors
        h, w = inp.shape
        test = np.copy(inp)
        
        # Inherit color from left neighbor if current is zero
        for i in range(h):
            for j in range(1, w):
                if inp[i, j] == 0 and inp[i, j-1] != 0:
                    test[i, j] = inp[i, j-1]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 res=[r[:]for r in g]
 for i in range(len(g)):
  for j in range(1,len(g[0])):
   if g[i][j]==0 and g[i][j-1]!=0:
    res[i][j]=g[i][j-1]
 return res
"""
        return None

@handler_decorator
class HandlerThresholdColoring(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The system checks threshold-based color assignment
        for threshold in range(1, 9):
            test = np.where(inp > threshold, 8, 1)
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for threshold in range(1, 9):
            test = np.where(inp > threshold, 8, 1)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[8 if x>{threshold} else 1 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerColorFrequencyMapping(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The implementation maps colors based on their frequency
        unique, counts = np.unique(inp, return_counts=True)
        freq_map = dict(zip(unique, counts))
        
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                test[i, j] = freq_map[inp[i, j]] % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Frequency-based mapping requires counting operations
        return None

@handler_decorator
class HandlerAlternatingColorPattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # The system verifies alternating color patterns
        colors = [1, 2]
        test = np.zeros((h, w), dtype=int)
        
        for i in range(h):
            for j in range(w):
                test[i, j] = colors[(i + j) % 2]
        
        if np.array_equal(test, out):
            return True
        
        # Try different alternating patterns
        for i in range(h):
            for j in range(w):
                test[i, j] = colors[i % 2]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                test[i, j] = 1 + ((i + j) % 2)
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[1+((i+j)%2)for j in range({w})]for i in range({h})]
"""
        
        for i in range(h):
            for j in range(w):
                test[i, j] = 1 + (i % 2)
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[1+(i%2)for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerContextualColorReplacement(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The implementation checks for context-dependent color changes
        h, w = inp.shape
        test = np.copy(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] == 1:  # If current is color 1
                    # Check if surrounded by color 2
                    surrounded = True
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w:
                            if inp[ni, nj] != 2:
                                surrounded = False
                                break
                    
                    if surrounded:
                        test[i, j] = 3  # Change to color 3
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Contextual replacement requires complex neighbor checking
        return None

@handler_decorator
class HandlerColorWavePattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # The system analyzes wave-like color patterns
        test = np.zeros((h, w), dtype=int)
        
        for i in range(h):
            for j in range(w):
                # Sine wave pattern
                test[i, j] = int(5 + 4 * np.sin(j * np.pi / w)) % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Wave patterns require trigonometric functions
        return None

@handler_decorator
class HandlerColorDiffusion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The implementation simulates color diffusion
        h, w = inp.shape
        test = np.copy(inp)
        
        # Simple diffusion: average with neighbors, keeping original if non-zero
        new_test = np.zeros_like(test)
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    new_test[i, j] = inp[i, j]
                else:
                    total = 0
                    count = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] != 0:
                            total += inp[ni, nj]
                            count += 1
                    
                    if count > 0:
                        new_test[i, j] = total // count
        
        return np.array_equal(new_test, out)
        
    def generate_solution(self, task_data, analysis):
        # Diffusion algorithms require iterative calculations
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# SCALING OPERATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerUpscale(TransformationHandler):
    def can_handle(self, task_data, analysis):
        shapes = list(analysis['shape_changes'])
        if shapes:
            inp_shape, out_shape = shapes[0]
            return out_shape[0] > inp_shape[0] and out_shape[1] > inp_shape[1]
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for h_scale in range(1, 5):
            for w_scale in range(1, 5):
                if (inp.shape[0]*h_scale == out.shape[0] and 
                    inp.shape[1]*w_scale == out.shape[1]):
                    test = np.repeat(np.repeat(inp, h_scale, axis=0), w_scale, axis=1)
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return[[x for x in row for _ in range({w_scale})]for row in g for _ in range({h_scale})]
"""
        return None

@handler_decorator
class HandlerDownscale(TransformationHandler):
    def can_handle(self, task_data, analysis):
        shapes = list(analysis['shape_changes'])
        if shapes:
            inp_shape, out_shape = shapes[0]
            return out_shape[0] < inp_shape[0] and out_shape[1] < inp_shape[1]
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for h_scale in range(2, 5):
            for w_scale in range(2, 5):
                if (inp.shape[0]//h_scale == out.shape[0] and 
                    inp.shape[1]//w_scale == out.shape[1]):
                    test = inp[::h_scale, ::w_scale]
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return[row[::{w_scale}]for row in g[::{h_scale}]]
"""
        return None

@handler_decorator
class HandlerUpscaleNonUniform(TransformationHandler):
    def can_handle(self, task_data, analysis):
        shapes = list(analysis['shape_changes'])
        if shapes:
            inp_shape, out_shape = shapes[0]
            # Check if only one dimension is scaled
            return (out_shape[0] > inp_shape[0] and out_shape[1] == inp_shape[1]) or \
                   (out_shape[0] == inp_shape[0] and out_shape[1] > inp_shape[1])
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Vertical scaling only
        if out.shape[1] == inp.shape[1]:
            for h_scale in range(2, 5):
                if inp.shape[0] * h_scale == out.shape[0]:
                    test = np.repeat(inp, h_scale, axis=0)
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return[row for row in g for _ in range({h_scale})]
"""
        
        # Horizontal scaling only
        if out.shape[0] == inp.shape[0]:
            for w_scale in range(2, 5):
                if inp.shape[1] * w_scale == out.shape[1]:
                    test = np.repeat(inp, w_scale, axis=1)
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return[[x for x in row for _ in range({w_scale})]for row in g]
"""
        return None

@handler_decorator
class HandlerSelectiveColorScaling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system verifies whether only specific colors are scaled
        if out.shape[0] < inp.shape[0] * 2 or out.shape[1] < inp.shape[1] * 2:
            return False
        
        # Check if only non-zero elements are scaled
        test = np.zeros_like(out)
        scale = 2
        
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    # Scale this pixel
                    for di in range(scale):
                        for dj in range(scale):
                            if i*scale+di < out.shape[0] and j*scale+dj < out.shape[1]:
                                test[i*scale+di, j*scale+dj] = inp[i, j]
                else:
                    # Keep as single pixel
                    if i < out.shape[0] and j < out.shape[1]:
                        test[i, j] = inp[i, j]
        
        return np.array_equal(test[:out.shape[0], :out.shape[1]], out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if self.can_handle(task_data, analysis):
            scale = 2
            oh, ow = out.shape
            return f"""def p(g):
 res=[[0]*{ow} for _ in range({oh})]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]!=0:
    for di in range({scale}):
     for dj in range({scale}):
      if i*{scale}+di<{oh} and j*{scale}+dj<{ow}:
       res[i*{scale}+di][j*{scale}+dj]=g[i][j]
   else:
    if i<{oh} and j<{ow}:
     res[i][j]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerAveragePoolingDownscale(TransformationHandler):
    def can_handle(self, task_data, analysis):
        shapes = list(analysis['shape_changes'])
        if shapes:
            inp_shape, out_shape = shapes[0]
            return out_shape[0] < inp_shape[0] and out_shape[1] < inp_shape[1]
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The implementation checks for average pooling downscaling
        for pool_size in range(2, 5):
            if (inp.shape[0] % pool_size == 0 and inp.shape[1] % pool_size == 0 and
                inp.shape[0] // pool_size == out.shape[0] and 
                inp.shape[1] // pool_size == out.shape[1]):
                
                test = np.zeros_like(out)
                for i in range(out.shape[0]):
                    for j in range(out.shape[1]):
                        # Calculate average of pool
                        pool_sum = 0
                        pool_count = 0
                        for di in range(pool_size):
                            for dj in range(pool_size):
                                pool_sum += inp[i*pool_size+di, j*pool_size+dj]
                                pool_count += 1
                        test[i, j] = pool_sum // pool_count
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 ps={pool_size}
 oh,ow=len(g)//ps,len(g[0])//ps
 res=[]
 for i in range(oh):
  row=[]
  for j in range(ow):
   s=sum(g[i*ps+di][j*ps+dj]for di in range(ps)for dj in range(ps))
   row.append(s//(ps*ps))
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerMaxPoolingDownscale(TransformationHandler):
    def can_handle(self, task_data, analysis):
        shapes = list(analysis['shape_changes'])
        if shapes:
            inp_shape, out_shape = shapes[0]
            return out_shape[0] < inp_shape[0] and out_shape[1] < inp_shape[1]
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system verifies max pooling downscaling
        for pool_size in range(2, 5):
            if (inp.shape[0] % pool_size == 0 and inp.shape[1] % pool_size == 0 and
                inp.shape[0] // pool_size == out.shape[0] and 
                inp.shape[1] // pool_size == out.shape[1]):
                
                test = np.zeros_like(out)
                for i in range(out.shape[0]):
                    for j in range(out.shape[1]):
                        # Find maximum in pool
                        max_val = 0
                        for di in range(pool_size):
                            for dj in range(pool_size):
                                max_val = max(max_val, inp[i*pool_size+di, j*pool_size+dj])
                        test[i, j] = max_val
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 ps={pool_size}
 oh,ow=len(g)//ps,len(g[0])//ps
 res=[]
 for i in range(oh):
  row=[]
  for j in range(ow):
   m=max(g[i*ps+di][j*ps+dj]for di in range(ps)for dj in range(ps))
   row.append(m)
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerInterpolatedScaling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        shapes = list(analysis['shape_changes'])
        if shapes:
            inp_shape, out_shape = shapes[0]
            # Check for non-integer scaling factors
            return (out_shape[0] != inp_shape[0] * (out_shape[0] // inp_shape[0]) or
                    out_shape[1] != inp_shape[1] * (out_shape[1] // inp_shape[1]))
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The implementation handles bilinear interpolation for scaling
        ih, iw = inp.shape
        oh, ow = out.shape
        
        test = np.zeros_like(out)
        for i in range(oh):
            for j in range(ow):
                # Map output coordinates to input coordinates
                src_i = i * (ih - 1) / (oh - 1) if oh > 1 else 0
                src_j = j * (iw - 1) / (ow - 1) if ow > 1 else 0
                
                # Nearest neighbor for simplicity
                test[i, j] = inp[int(round(src_i)), int(round(src_j))]
        
        if np.array_equal(test, out):
            # Complex interpolation logic requires extensive calculation
            return None
        return None

@handler_decorator
class HandlerScalingWithBorderPreservation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system checks if scaling preserves border elements
        if out.shape[0] <= inp.shape[0] or out.shape[1] <= inp.shape[1]:
            return False
        
        # Verify if border is preserved while interior is scaled
        scale = 2
        expected_h = (inp.shape[0] - 2) * scale + 2
        expected_w = (inp.shape[1] - 2) * scale + 2
        
        return out.shape == (expected_h, expected_w)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if self.can_handle(task_data, analysis):
            scale = 2
            oh, ow = out.shape
            return f"""def p(g):
 s={scale}
 res=[[0]*{ow} for _ in range({oh})]
 # Copy borders
 res[0]=g[0][:1]+[x for x in g[0][1:-1]for _ in range(s)]+g[0][-1:]
 res[-1]=g[-1][:1]+[x for x in g[-1][1:-1]for _ in range(s)]+g[-1][-1:]
 # Scale interior
 for i in range(1,len(g)-1):
  for j in range(1,len(g[0])-1):
   for di in range(s):
    for dj in range(s):
     res[1+(i-1)*s+di][1+(j-1)*s+dj]=g[i][j]
 # Copy side borders
 for i in range(1,{oh}-1):
  res[i][0]=g[min(1+(i-1)//s,len(g)-2)][0]
  res[i][-1]=g[min(1+(i-1)//s,len(g)-2)][-1]
 return res
"""
        return None

@handler_decorator
class HandlerAdaptiveScaling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The handler detects adaptive scaling based on content
        from scipy.ndimage import label
        labeled, num_features = label(inp > 0)
        
        if num_features == 0:
            return False
        
        # Check if different regions are scaled differently
        return out.shape[0] > inp.shape[0] or out.shape[1] > inp.shape[1]
        
    def generate_solution(self, task_data, analysis):
        # Adaptive scaling requires complex region analysis
        return None

@handler_decorator
class HandlerAspectRatioPreservingScale(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system verifies aspect ratio preservation
        ih, iw = inp.shape
        oh, ow = out.shape
        
        # Check if aspect ratio is maintained
        input_ratio = ih / iw
        output_ratio = oh / ow
        
        return abs(input_ratio - output_ratio) < 0.1 and (oh != ih or ow != iw)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Calculate scale factor
        scale_h = out.shape[0] / inp.shape[0]
        scale_w = out.shape[1] / inp.shape[1]
        
        if abs(scale_h - scale_w) < 0.1:  # Similar scaling
            scale = int(round(scale_h))
            if scale > 0 and scale < 5:
                test = np.repeat(np.repeat(inp, scale, axis=0), scale, axis=1)
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[[x for x in row for _ in range({scale})]for row in g for _ in range({scale})]
"""
        return None

@handler_decorator
class HandlerContentAwareScaling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The implementation checks for content-aware scaling
        # where important features are preserved during scaling
        return out.shape[0] != inp.shape[0] or out.shape[1] != inp.shape[1]
        
    def generate_solution(self, task_data, analysis):
        # Content-aware scaling algorithms are complex
        return None

@handler_decorator
class HandlerScalingWithPatternReplication(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system detects if patterns are replicated during scaling
        if out.shape[0] <= inp.shape[0] or out.shape[1] <= inp.shape[1]:
            return False
        
        # Check for pattern-based scaling
        pattern_size = 2
        if inp.shape[0] >= pattern_size and inp.shape[1] >= pattern_size:
            pattern = inp[:pattern_size, :pattern_size]
            
            # Check if output contains replicated patterns
            scale = 3  # Each pattern becomes 3x3
            expected_h = (inp.shape[0] // pattern_size) * scale * pattern_size
            expected_w = (inp.shape[1] // pattern_size) * scale * pattern_size
            
            return abs(out.shape[0] - expected_h) < 2 and abs(out.shape[1] - expected_w) < 2
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Pattern replication during scaling requires complex logic
        return None

@handler_decorator
class HandlerAnisotropicScaling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        shapes = list(analysis['shape_changes'])
        if shapes:
            inp_shape, out_shape = shapes[0]
            # Different scaling factors for each dimension
            scale_h = out_shape[0] / inp_shape[0]
            scale_w = out_shape[1] / inp_shape[1]
            return scale_h != scale_w and scale_h > 0 and scale_w > 0
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Calculate different scaling factors
        scale_h = out.shape[0] // inp.shape[0]
        scale_w = out.shape[1] // inp.shape[1]
        
        if (scale_h > 0 and scale_w > 0 and 
            inp.shape[0] * scale_h == out.shape[0] and 
            inp.shape[1] * scale_w == out.shape[1]):
            
            test = np.repeat(np.repeat(inp, scale_h, axis=0), scale_w, axis=1)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[x for x in row for _ in range({scale_w})]for row in g for _ in range({scale_h})]
"""
        return None

@handler_decorator
class HandlerFractionalScaling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        shapes = list(analysis['shape_changes'])
        if shapes:
            inp_shape, out_shape = shapes[0]
            # Check for fractional scaling (e.g., 1.5x)
            return (out_shape[0] % inp_shape[0] != 0 or 
                    out_shape[1] % inp_shape[1] != 0)
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Handle 1.5x scaling
        if (out.shape[0] == inp.shape[0] * 3 // 2 and 
            out.shape[1] == inp.shape[1] * 3 // 2):
            
            # First upscale by 3, then downscale by 2
            upscaled = np.repeat(np.repeat(inp, 3, axis=0), 3, axis=1)
            test = upscaled[::2, ::2]
            
            if test.shape == out.shape and np.array_equal(test, out):
                return """def p(g):
 # Upscale by 3
 up=[[x for x in row for _ in range(3)]for row in g for _ in range(3)]
 # Downscale by 2
 return[row[::2]for row in up[::2]]
"""
        return None

@handler_decorator
class HandlerPyramidScaling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The handler checks for pyramid-like scaling
        # where the center is preserved and edges are scaled
        return out.shape[0] > inp.shape[0] and out.shape[1] > inp.shape[1]
        
    def generate_solution(self, task_data, analysis):
        # Pyramid scaling requires complex distance-based calculations
        return None

@handler_decorator
class HandlerScalingWithRotation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        # The system detects combined scaling and rotation
        return False  # Complex transformation
        
    def generate_solution(self, task_data, analysis):
        # Combined transformations are too complex for compact implementation
        return None

@handler_decorator
class HandlerMorphologicalScaling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The implementation verifies morphological scaling
        # where objects are scaled based on their shape properties
        from scipy.ndimage import label
        labeled_inp, n_inp = label(inp > 0)
        labeled_out, n_out = label(out > 0)
        
        return n_inp == n_out and n_inp > 0 and out.shape != inp.shape
        
    def generate_solution(self, task_data, analysis):
        # Morphological scaling requires object analysis
        return None

@handler_decorator
class HandlerSeamCarving(TransformationHandler):
    def can_handle(self, task_data, analysis):
        shapes = list(analysis['shape_changes'])
        if shapes:
            inp_shape, out_shape = shapes[0]
            # Content-aware resizing
            return (out_shape[0] < inp_shape[0] and out_shape[1] == inp_shape[1]) or \
                   (out_shape[0] == inp_shape[0] and out_shape[1] < inp_shape[1])
        return False
        
    def generate_solution(self, task_data, analysis):
        # Seam carving algorithm is too complex for compact implementation
        return None

@handler_decorator
class HandlerScalingWithDithering(TransformationHandler):
    def can_handle(self, task_data, analysis):
        # The handler detects scaling with dithering patterns
        return False  # Complex pattern analysis required
        
    def generate_solution(self, task_data, analysis):
        # Dithering algorithms require complex error diffusion
        return None

@handler_decorator
class HandlerVectorizedScaling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system checks for vector-like scaling of edges
        # This would preserve sharp edges during scaling
        return out.shape[0] > inp.shape[0] and out.shape[1] > inp.shape[1]
        
    def generate_solution(self, task_data, analysis):
        # Vectorized scaling requires edge detection and preservation
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# PATTERN AND TILING OPERATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerTilePattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return 'tiling' in analysis.get('transform_types', set())
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        ih, iw = inp.shape
        oh, ow = out.shape
        
        for th in range(1, min(ih+1, 5)):
            for tw in range(1, min(iw+1, 5)):
                if oh % th == 0 and ow % tw == 0:
                    tile = inp[:th, :tw]
                    tiled = np.tile(tile, (oh//th, ow//tw))
                    
                    if np.array_equal(tiled, out):
                        return f"""def p(g):
 tile=[r[:{tw}]for r in g[:{th}]]
 h,w={oh},{ow}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   row.append(tile[i%{th}][j%{tw}])
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerCropToContent(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return 'reduction' in analysis.get('transform_types', set())
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        rows, cols = np.where(inp > 0)
        if len(rows) == 0:
            return None
            
        min_row, max_row = rows.min(), rows.max()
        min_col, max_col = cols.min(), cols.max()
        
        cropped = inp[min_row:max_row+1, min_col:max_col+1]
        
        if np.array_equal(cropped, out):
            return """def p(g):
 rows,cols=[],[]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]>0:
    rows.append(i)
    cols.append(j)
 if not rows:return g
 r0,r1=min(rows),max(rows)+1
 c0,c1=min(cols),max(cols)+1
 return[r[c0:c1]for r in g[r0:r1]]
"""
        return None

@handler_decorator
class HandlerExtractPattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape[0] > out.shape[0] and inp.shape[1] > out.shape[1]:
            for i in range(inp.shape[0] - out.shape[0] + 1):
                for j in range(inp.shape[1] - out.shape[1] + 1):
                    if np.array_equal(inp[i:i+out.shape[0], j:j+out.shape[1]], out):
                        return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        oh, ow = out.shape
        
        for i in range(inp.shape[0] - oh + 1):
            for j in range(inp.shape[1] - ow + 1):
                if np.array_equal(inp[i:i+oh, j:j+ow], out):
                    return f"""def p(g):
 return[r[{j}:{j+ow}]for r in g[{i}:{i+oh}]]
"""
        return None

@handler_decorator
class HandlerRepeatPattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        ih, iw = inp.shape
        oh, ow = out.shape
        
        if oh == ih and ow == iw * 2:
            return np.array_equal(np.hstack([inp, inp]), out)
        elif oh == ih * 2 and ow == iw:
            return np.array_equal(np.vstack([inp, inp]), out)
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if np.array_equal(np.hstack([inp, inp]), out):
            return """def p(g):
 return[r+r for r in g]
"""
        elif np.array_equal(np.vstack([inp, inp]), out):
            return """def p(g):
 return g+g
"""
        return None

@handler_decorator
class HandlerReplicateToGrid(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        # Check if input is replicated in a grid pattern
        ih, iw = inp.shape
        oh, ow = out.shape
        if oh % ih == 0 and ow % iw == 0 and (oh > ih or ow > iw):
            reps_h = oh // ih
            reps_w = ow // iw
            test = np.tile(inp, (reps_h, reps_w))
            return np.array_equal(test, out)
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        ih, iw = inp.shape
        oh, ow = out.shape
        if oh % ih == 0 and ow % iw == 0:
            reps_h = oh // ih
            reps_w = ow // iw
            test = np.tile(inp, (reps_h, reps_w))
            if np.array_equal(test, out):
                return f"""def p(g):
 res=[]
 for _ in range({reps_h}):
  for r in g:
   res.append(r*{reps_w})
 return res
"""
        return None

@handler_decorator
class HandlerExtractDiagonal(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        # Check if output is diagonal elements
        if out.shape == (1, min(inp.shape)):
            diag = np.diagonal(inp)
            if np.array_equal(out[0], diag):
                return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if out.shape == (1, min(inp.shape)):
            diag = np.diagonal(inp)
            if np.array_equal(out[0], diag):
                return """def p(g):
 return[[g[i][i]for i in range(min(len(g),len(g[0])))]]
"""
        return None

@handler_decorator
class HandlerCheckerboardTiling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # The implementation tests whether the output represents a checkerboard pattern
        # derived from two different tiles in the input
        for tile_size in [1, 2, 3]:
            if h >= 2 * tile_size and w >= 2 * tile_size:
                tile1 = inp[:tile_size, :tile_size]
                tile2 = inp[:tile_size, tile_size:2*tile_size]
                
                test = np.zeros_like(inp)
                for i in range(0, h, tile_size):
                    for j in range(0, w, tile_size):
                        if ((i // tile_size) + (j // tile_size)) % 2 == 0:
                            test[i:i+tile_size, j:j+tile_size] = tile1
                        else:
                            test[i:i+tile_size, j:j+tile_size] = tile2
                
                if np.array_equal(test[:h, :w], out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for tile_size in [1, 2, 3]:
            if h >= 2 * tile_size and w >= 2 * tile_size:
                tile1 = inp[:tile_size, :tile_size]
                tile2 = inp[:tile_size, tile_size:2*tile_size]
                
                test = np.zeros_like(inp)
                for i in range(0, h, tile_size):
                    for j in range(0, w, tile_size):
                        if ((i // tile_size) + (j // tile_size)) % 2 == 0:
                            test[i:i+tile_size, j:j+tile_size] = tile1
                        else:
                            test[i:i+tile_size, j:j+tile_size] = tile2
                
                if np.array_equal(test[:h, :w], out):
                    return f"""def p(g):
 t1=[r[:{tile_size}]for r in g[:{tile_size}]]
 t2=[r[{tile_size}:{2*tile_size}]for r in g[:{tile_size}]]
 res=[]
 for i in range({h}):
  row=[]
  for j in range({w}):
   ti,tj=i//{tile_size},j//{tile_size}
   if(ti+tj)%2==0:
    row.append(t1[i%{tile_size}][j%{tile_size}])
   else:
    row.append(t2[i%{tile_size}][j%{tile_size}])
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerSpiralPatternExtraction(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system checks whether the output represents elements extracted in spiral order
        h, w = inp.shape
        spiral_elements = []
        
        top, bottom, left, right = 0, h-1, 0, w-1
        while top <= bottom and left <= right:
            for j in range(left, right+1):
                spiral_elements.append(inp[top, j])
            top += 1
            
            for i in range(top, bottom+1):
                spiral_elements.append(inp[i, right])
            right -= 1
            
            if top <= bottom:
                for j in range(right, left-1, -1):
                    spiral_elements.append(inp[bottom, j])
                bottom -= 1
            
            if left <= right:
                for i in range(bottom, top-1, -1):
                    spiral_elements.append(inp[i, left])
                left += 1
        
        if out.shape == (1, len(spiral_elements)) and np.array_equal(out[0], spiral_elements):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # The implementation of spiral extraction requires complex iteration logic
        # that exceeds compact code golf constraints
        return None

@handler_decorator
class HandlerPatternRotation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system verifies whether patterns within the grid are rotated
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # The algorithm checks for rotation of small patterns within the grid
        for pattern_size in [2, 3]:
            if h % pattern_size == 0 and w % pattern_size == 0:
                test = np.zeros_like(inp)
                
                for i in range(0, h, pattern_size):
                    for j in range(0, w, pattern_size):
                        pattern = inp[i:i+pattern_size, j:j+pattern_size]
                        rotated = np.rot90(pattern, 1)  # 90 degree rotation
                        test[i:i+pattern_size, j:j+pattern_size] = rotated
                
                if np.array_equal(test, out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for pattern_size in [2, 3]:
            if h % pattern_size == 0 and w % pattern_size == 0:
                test = np.zeros_like(inp)
                
                for i in range(0, h, pattern_size):
                    for j in range(0, w, pattern_size):
                        pattern = inp[i:i+pattern_size, j:j+pattern_size]
                        rotated = np.rot90(pattern, 1)
                        test[i:i+pattern_size, j:j+pattern_size] = rotated
                
                if np.array_equal(test, out):
                    # Rotation within patterns requires complex indexing
                    return None
        return None

@handler_decorator
class HandlerInterlockingTiles(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The handler identifies interlocking tile patterns where tiles overlap
        if inp.shape[0] < 3 or inp.shape[1] < 3:
            return False
        
        # Extract potential interlocking tiles with overlap
        tile1 = inp[:2, :2]
        tile2 = inp[1:3, 1:3]
        
        h, w = out.shape
        test = np.zeros_like(out)
        
        # Apply interlocking pattern
        for i in range(0, h-1, 2):
            for j in range(0, w-1, 2):
                test[i:i+2, j:j+2] = tile1
                if i+1 < h-1 and j+1 < w-1:
                    test[i+1:i+3, j+1:j+3] = np.maximum(test[i+1:i+3, j+1:j+3], tile2)
        
        return np.array_equal(test[:h, :w], out)
        
    def generate_solution(self, task_data, analysis):
        # Interlocking patterns require overlapping logic that is complex to implement compactly
        return None

@handler_decorator
class HandlerPatternInterpolation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system checks for pattern interpolation between keyframes
        if out.shape[1] != inp.shape[1] or out.shape[0] < inp.shape[0]:
            return False
        
        # Verify if output contains interpolated patterns between input rows
        if out.shape[0] == inp.shape[0] * 2 - 1:
            test = np.zeros_like(out)
            
            for i in range(inp.shape[0]):
                test[i*2, :] = inp[i, :]
                
                if i < inp.shape[0] - 1:
                    # Interpolate between consecutive rows
                    for j in range(inp.shape[1]):
                        test[i*2+1, j] = (inp[i, j] + inp[i+1, j]) // 2
            
            return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if out.shape[0] == inp.shape[0] * 2 - 1 and self.can_handle(task_data, analysis):
            return """def p(g):
 res=[]
 for i in range(len(g)):
  res.append(g[i])
  if i<len(g)-1:
   row=[]
   for j in range(len(g[0])):
    row.append((g[i][j]+g[i+1][j])//2)
   res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerRecursivePatternGeneration(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The implementation detects recursive pattern generation
        if inp.shape[0] != inp.shape[1] or out.shape[0] != out.shape[1]:
            return False
        
        # Check for Sierpinski-like patterns
        if inp.shape[0] == 2 and out.shape[0] == 4:
            # Build recursive pattern
            test = np.zeros_like(out)
            test[:2, :2] = inp
            test[:2, 2:] = inp
            test[2:, :2] = inp
            # Leave bottom-right as zeros for Sierpinski pattern
            
            return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape[0] == 2 and out.shape[0] == 4 and self.can_handle(task_data, analysis):
            return """def p(g):
 res=[[0]*4 for _ in range(4)]
 for i in range(2):
  for j in range(2):
   res[i][j]=g[i][j]
   res[i][j+2]=g[i][j]
   res[i+2][j]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerSymmetricPatternCompletion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # The algorithm checks if the output completes symmetric patterns
        # where missing parts are filled based on symmetry
        test = np.copy(inp)
        
        # Complete horizontal symmetry
        for i in range(h):
            for j in range(w // 2):
                if inp[i, j] != 0 and inp[i, w-1-j] == 0:
                    test[i, w-1-j] = inp[i, j]
                elif inp[i, j] == 0 and inp[i, w-1-j] != 0:
                    test[i, j] = inp[i, w-1-j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        w = inp.shape[1]
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 res=[r[:]for r in g]
 w={w}
 for i in range(len(g)):
  for j in range(w//2):
   if g[i][j]!=0 and g[i][w-1-j]==0:
    res[i][w-1-j]=g[i][j]
   elif g[i][j]==0 and g[i][w-1-j]!=0:
    res[i][j]=g[i][w-1-j]
 return res
"""
        return None

@handler_decorator
class HandlerPatternMorphing(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system verifies pattern morphing between two states
        if inp.shape[1] % 2 != 0 or out.shape != inp.shape:
            return False
        
        half_width = inp.shape[1] // 2
        pattern1 = inp[:, :half_width]
        pattern2 = inp[:, half_width:]
        
        # Check if output is a morph between the two patterns
        test = np.zeros_like(out)
        for i in range(out.shape[0]):
            for j in range(out.shape[1]):
                if j < half_width:
                    # Blend based on position
                    weight = j / half_width
                    test[i, j] = int(pattern1[i, j % half_width] * (1 - weight) + 
                                   pattern2[i, j % half_width] * weight)
                else:
                    test[i, j] = pattern2[i, j % half_width]
        
        # This is a simplified check as exact morphing is complex
        return False
        
    def generate_solution(self, task_data, analysis):
        # Pattern morphing requires interpolation calculations
        return None

@handler_decorator
class HandlerTessellationOperations(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The handler identifies regular tessellation patterns
        if inp.shape[0] < 3 or inp.shape[1] < 3:
            return False
        
        # Check for hexagonal-like tessellation
        # This is simplified as true hexagonal tessellation is complex
        test = np.zeros_like(out)
        tile_h, tile_w = 3, 3
        
        for i in range(0, out.shape[0] - tile_h + 1, tile_h - 1):
            for j in range(0, out.shape[1] - tile_w + 1, tile_w - 1):
                # Apply tile with offset for tessellation effect
                offset = (i // (tile_h - 1)) % 2
                if j + offset < out.shape[1] - tile_w + 1:
                    test[i:i+tile_h, j+offset:j+offset+tile_w] = inp[:tile_h, :tile_w]
        
        # Approximate tessellation check
        return np.count_nonzero(test) > 0 and np.count_nonzero(test) == np.count_nonzero(out)
        
    def generate_solution(self, task_data, analysis):
        # Tessellation patterns require complex offset calculations
        return None

@handler_decorator
class HandlerPatternBasedTransformation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The system checks if specific patterns trigger transformations
        h, w = inp.shape
        test = np.copy(inp)
        
        # Look for 2x2 patterns and transform them
        for i in range(h - 1):
            for j in range(w - 1):
                pattern = inp[i:i+2, j:j+2]
                
                # If all corners are filled, clear center
                if pattern[0,0] != 0 and pattern[0,1] != 0 and pattern[1,0] != 0 and pattern[1,1] != 0:
                    if np.sum(pattern) == pattern[0,0] * 4:  # All same color
                        # Transform to cross pattern
                        test[i:i+2, j:j+2] = 0
                        test[i, j+1] = pattern[0,0]
                        test[i+1, j] = pattern[0,0]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Pattern-based transformations require complex conditional logic
        return None

@handler_decorator
class HandlerMultiScalePatterns(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The implementation detects patterns at multiple scales
        if out.shape[0] != inp.shape[0] * 3 or out.shape[1] != inp.shape[1] * 3:
            return False
        
        # Check if output contains the input pattern at multiple scales
        test = np.zeros_like(out)
        h, w = inp.shape
        
        # Original scale at center
        test[h:2*h, w:2*w] = inp
        
        # Smaller scale in corners (downsampled)
        for i in range(h // 2):
            for j in range(w // 2):
                val = inp[i*2, j*2]  # Simple downsampling
                test[i, j] = val
                test[i, 2*w + w//2 + j] = val
                test[2*h + h//2 + i, j] = val
                test[2*h + h//2 + i, 2*w + w//2 + j] = val
        
        # Approximate multi-scale check
        return np.count_nonzero(test) > 0
        
    def generate_solution(self, task_data, analysis):
        # Multi-scale pattern generation requires complex sampling logic
        return None

@handler_decorator
class HandlerPatternRecognitionReplacement(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The system identifies and replaces specific patterns
        h, w = inp.shape
        test = np.copy(inp)
        
        # Define pattern to find (L-shape)
        for i in range(h - 2):
            for j in range(w - 1):
                # Check for L pattern
                if (inp[i, j] != 0 and inp[i+1, j] != 0 and inp[i+2, j] != 0 and 
                    inp[i+2, j+1] != 0 and inp[i, j] == inp[i+2, j+1]):
                    # Replace with filled rectangle
                    color = inp[i, j]
                    test[i:i+3, j:j+2] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Pattern recognition and replacement requires complex pattern matching
        return None

@handler_decorator
class HandlerFractalPatternGeneration(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        
        # The handler checks for fractal-like pattern generation
        if out.shape[0] != out.shape[1]:
            return False
        
        size = out.shape[0]
        if size > 0 and (size & (size - 1)) == 0:  # Power of 2
            # Generate simple fractal pattern
            test = np.zeros_like(out)
            
            # Cantor-like pattern
            def fill_cantor(arr, start, end, depth):
                if depth == 0 or end - start < 1:
                    return
                third = (end - start) // 3
                arr[start:start+third] = 1
                arr[end-third:end] = 1
                fill_cantor(arr, start, start+third, depth-1)
                fill_cantor(arr, end-third, end, depth-1)
            
            for i in range(size):
                row = np.zeros(size)
                fill_cantor(row, 0, size, 3)
                test[i, :] = row
            
            # This is a simplified fractal check
            return np.count_nonzero(test) > 0
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Fractal generation requires recursive algorithms
        return None

@handler_decorator
class HandlerPatternStitching(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The system verifies if multiple patterns are stitched together
        if inp.shape[0] < 4 or inp.shape[1] < 4:
            return False
        
        # Extract multiple small patterns
        patterns = []
        pattern_size = 2
        for i in range(0, inp.shape[0] - pattern_size + 1, pattern_size):
            for j in range(0, inp.shape[1] - pattern_size + 1, pattern_size):
                patterns.append(inp[i:i+pattern_size, j:j+pattern_size])
        
        if len(patterns) >= 4 and out.shape == (pattern_size * 2, pattern_size * 2):
            # Stitch four patterns together
            test = np.zeros_like(out)
            test[:pattern_size, :pattern_size] = patterns[0]
            test[:pattern_size, pattern_size:] = patterns[1]
            test[pattern_size:, :pattern_size] = patterns[2]
            test[pattern_size:, pattern_size:] = patterns[3]
            
            return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if self.can_handle(task_data, analysis):
            return """def p(g):
 p1=[r[:2]for r in g[:2]]
 p2=[r[2:4]for r in g[:2]]
 p3=[r[:2]for r in g[2:4]]
 p4=[r[2:4]for r in g[2:4]]
 res=[]
 res.extend([p1[0]+p2[0],p1[1]+p2[1]])
 res.extend([p3[0]+p4[0],p3[1]+p4[1]])
 return res
"""
        return None

@handler_decorator
class HandlerAdaptiveTiling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # The implementation handles adaptive tiling where tile size varies
        if out.shape[0] < inp.shape[0] or out.shape[1] < inp.shape[1]:
            return False
        
        # Check if tiling adapts based on content
        h, w = inp.shape
        
        # Find non-zero regions to use as adaptive tiles
        from scipy.ndimage import label
        labeled, num_features = label(inp > 0)
        
        if num_features > 0:
            # Use first object as tile
            mask = labeled == 1
            rows, cols = np.where(mask)
            if len(rows) > 0:
                min_row, max_row = rows.min(), rows.max()
                min_col, max_col = cols.min(), cols.max()
                
                tile = inp[min_row:max_row+1, min_col:max_col+1]
                tile_h, tile_w = tile.shape
                
                if out.shape[0] % tile_h == 0 and out.shape[1] % tile_w == 0:
                    test = np.tile(tile, (out.shape[0] // tile_h, out.shape[1] // tile_w))
                    return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Adaptive tiling requires object detection and extraction
        return None

@handler_decorator
class HandlerPatternAlignment(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # The system checks if scattered patterns are aligned to a grid
        from scipy.ndimage import label
        labeled, num_features = label(inp > 0)
        
        if num_features < 2:
            return False
        
        # Check if objects are aligned in output
        test = np.zeros_like(inp)
        target_row = 0
        target_col = 0
        
        for i in range(1, num_features + 1):
            mask = labeled == i
            if np.any(mask):
                rows, cols = np.where(mask)
                pattern = inp[rows.min():rows.max()+1, cols.min():cols.max()+1]
                
                # Place aligned
                if target_row + pattern.shape[0] <= out.shape[0] and target_col + pattern.shape[1] <= out.shape[1]:
                    test[target_row:target_row+pattern.shape[0], target_col:target_col+pattern.shape[1]] = pattern
                    target_col += pattern.shape[1] + 1
                    
                    if target_col >= out.shape[1] - 2:
                        target_col = 0
                        target_row += pattern.shape[0] + 1
        
        # Approximate alignment check
        return np.count_nonzero(test) == np.count_nonzero(out)
        
    def generate_solution(self, task_data, analysis):
        # Pattern alignment requires complex object manipulation
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# MATHEMATICAL AND ARITHMETIC OPERATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerModularArithmetic(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        for mod in range(2, 10):
            for add in range(0, 10):
                test = (inp + add) % mod
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[[(x+{add})%{mod} for x in r]for r in g]
"""
        
        for mod in range(2, 10):
            for mult in range(1, 10):
                test = (inp * mult) % mod
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[[(x*{mult})%{mod} for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerCheckerboardTransform(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        test = np.copy(inp)
        h, w = inp.shape
        
        for i in range(h):
            for j in range(w):
                if (i + j) % 2 == 0:
                    test[i, j] = (inp[i, j] * 2) % 10
                else:
                    test[i, j] = (inp[i, j] + 5) % 10
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if(i+j)%2==0:
    res[i][j]=(g[i][j]*2)%10
   else:
    res[i][j]=(g[i][j]+5)%10
 return res
"""
        return None

@handler_decorator
class HandlerBitwiseXOR(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        
        for xor_val in range(1, 10):
            test = inp ^ xor_val
            if np.array_equal(test, out):
                return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for xor_val in range(1, 10):
            test = inp ^ xor_val
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[x^{xor_val} for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerPositionalEncoding(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        test = np.array([[(i + j) % 10 for j in range(w)] for i in range(h)])
        if np.array_equal(test, out):
            return True
            
        test = np.array([[(i * j) % 10 for j in range(w)] for i in range(h)])
        if np.array_equal(test, out):
            return True
            
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        test = np.array([[(i + j) % 10 for j in range(w)] for i in range(h)])
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[(i+j)%10 for j in range({w})]for i in range({h})]
"""
        
        test = np.array([[(i * j) % 10 for j in range(w)] for i in range(h)])
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[(i*j)%10 for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerSumColors(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        # Check if output is sum of all colors
        color_sum = np.sum(inp)
        if out.shape == (1, 1) and out[0, 0] == color_sum % 10:
            return True
        # Check if each cell becomes sum of row
        if inp.shape == out.shape:
            test = np.zeros_like(inp)
            for i in range(inp.shape[0]):
                row_sum = np.sum(inp[i]) % 10
                test[i, :] = row_sum
            if np.array_equal(test, out):
                return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        color_sum = np.sum(inp) % 10
        if out.shape == (1, 1) and out[0, 0] == color_sum:
            return f"""def p(g):
 return[[sum(sum(r)for r in g)%10]]
"""
        
        if inp.shape == out.shape:
            test = np.zeros_like(inp)
            for i in range(inp.shape[0]):
                row_sum = np.sum(inp[i]) % 10
                test[i, :] = row_sum
            if np.array_equal(test, out):
                return """def p(g):
 return[[sum(r)%10]*len(g[0])for r in g]
"""
        return None

@handler_decorator
class HandlerCountNonZero(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        count = np.count_nonzero(inp)
        # Single value output
        if out.shape == (1, 1) and out[0, 0] == count % 10:
            return True
        # Fill grid with count
        if inp.shape == out.shape and np.all(out == count % 10):
            return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        count = np.count_nonzero(inp)
        
        if out.shape == (1, 1) and out[0, 0] == count % 10:
            return """def p(g):
 c=sum(1 for r in g for x in r if x!=0)
 return[[c%10]]
"""
        
        if inp.shape == out.shape and np.all(out == count % 10):
            return """def p(g):
 c=sum(1 for r in g for x in r if x!=0)%10
 return[[c]*len(g[0])for _ in g]
"""
        return None

@handler_decorator
class HandlerMaxValue(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        max_val = np.max(inp)
        # Check various max value patterns
        if out.shape == (1, 1) and out[0, 0] == max_val:
            return True
        if inp.shape == out.shape and np.all(out == max_val):
            return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        max_val = np.max(inp)
        
        if out.shape == (1, 1) and out[0, 0] == max_val:
            return """def p(g):
 return[[max(max(r)for r in g)]]
"""
        
        if inp.shape == out.shape and np.all(out == max_val):
            return """def p(g):
 m=max(max(r)for r in g)
 return[[m]*len(g[0])for _ in g]
"""
        return None

@handler_decorator
class HandlerMatrixAddition(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape or inp.shape[1] % 2 != 0:
            return False
        
        half_width = inp.shape[1] // 2
        left = inp[:, :half_width]
        right = inp[:, half_width:]
        
        test = (left + right) % 10
        return np.array_equal(test, out[:, :half_width])
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        half_width = inp.shape[1] // 2
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 w=len(g[0])//2
 return[[(r[j]+r[j+w])%10 for j in range(w)]for r in g]
"""
        return None

@handler_decorator
class HandlerPrimeCheck(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        def is_prime(n):
            if n < 2:
                return False
            for i in range(2, int(n**0.5) + 1):
                if n % i == 0:
                    return False
            return True
        
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                test[i, j] = 1 if is_prime(inp[i, j]) else 0
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 def ip(n):
  if n<2:return 0
  for i in range(2,int(n**0.5)+1):
   if n%i==0:return 0
  return 1
 return[[ip(x)for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerFibonacciSequence(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Generate Fibonacci sequence
        fib = [1, 1]
        while len(fib) < h * w:
            fib.append((fib[-1] + fib[-2]) % 10)
        
        # Check row-major filling
        test = np.array(fib[:h*w]).reshape(h, w)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 f=[1,1]
 while len(f)<h*w:
  f.append((f[-1]+f[-2])%10)
 res=[]
 for i in range(h):
  res.append(f[i*w:(i+1)*w])
 return res
"""
        return None

@handler_decorator
class HandlerDigitalRoot(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        def digital_root(n):
            while n >= 10:
                n = sum(int(d) for d in str(n))
            return n
        
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                test[i, j] = digital_root(inp[i, j])
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 def dr(n):
  while n>=10:n=sum(int(d)for d in str(n))
  return n
 return[[dr(x)for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerCumulativeSum(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Row-wise cumulative sum
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            cumsum = 0
            for j in range(inp.shape[1]):
                cumsum = (cumsum + inp[i, j]) % 10
                test[i, j] = cumsum
        
        if np.array_equal(test, out):
            return True
        
        # Column-wise cumulative sum
        test = np.zeros_like(inp)
        for j in range(inp.shape[1]):
            cumsum = 0
            for i in range(inp.shape[0]):
                cumsum = (cumsum + inp[i, j]) % 10
                test[i, j] = cumsum
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Row-wise cumulative sum
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            cumsum = 0
            for j in range(inp.shape[1]):
                cumsum = (cumsum + inp[i, j]) % 10
                test[i, j] = cumsum
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[]
 for r in g:
  row=[]
  s=0
  for x in r:
   s=(s+x)%10
   row.append(s)
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerManhattanDistance(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Find non-zero positions
        positions = np.argwhere(inp != 0)
        if len(positions) == 0:
            return False
        
        # Calculate Manhattan distance from first non-zero position
        ref_y, ref_x = positions[0]
        test = np.zeros_like(inp)
        
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                dist = abs(i - ref_y) + abs(j - ref_x)
                test[i, j] = dist % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        positions = np.argwhere(inp != 0)
        if len(positions) > 0 and self.can_handle(task_data, analysis):
            ref_y, ref_x = positions[0]
            return f"""def p(g):
 ry,rx={ref_y},{ref_x}
 return[[(abs(i-ry)+abs(j-rx))%10 for j in range(len(g[0]))]for i in range(len(g))]
"""
        return None

@handler_decorator
class HandlerParity(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Even/odd parity
        test = inp % 2
        if np.array_equal(test, out):
            return True
        
        # Inverted parity
        test = 1 - (inp % 2)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        test = inp % 2
        if np.array_equal(test, out):
            return """def p(g):
 return[[x%2 for x in r]for r in g]
"""
        
        test = 1 - (inp % 2)
        if np.array_equal(test, out):
            return """def p(g):
 return[[1-x%2 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerFactorial(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        def factorial(n):
            if n <= 1:
                return 1
            result = 1
            for i in range(2, n + 1):
                result *= i
            return result % 10
        
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                test[i, j] = factorial(inp[i, j])
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 def f(n):
  r=1
  for i in range(2,n+1):r*=i
  return r%10
 return[[f(x)for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerGCD(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # GCD with fixed value
        for fixed in range(1, 10):
            test = np.zeros_like(inp)
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    a, b = inp[i, j], fixed
                    while b:
                        a, b = b, a % b
                    test[i, j] = a
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for fixed in range(1, 10):
            test = np.zeros_like(inp)
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    a, b = inp[i, j], fixed
                    while b:
                        a, b = b, a % b
                    test[i, j] = a
            
            if np.array_equal(test, out):
                return f"""def p(g):
 def gcd(a,b):
  while b:a,b=b,a%b
  return a
 return[[gcd(x,{fixed})for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerPowerOperation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check various power operations
        for power in range(2, 5):
            test = (inp ** power) % 10
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for power in range(2, 5):
            test = (inp ** power) % 10
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[(x**{power})%10 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerDifferenceOperation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Row-wise differences
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            test[i, 0] = inp[i, 0]
            for j in range(1, inp.shape[1]):
                test[i, j] = (inp[i, j] - inp[i, j-1]) % 10
        
        if np.array_equal(test, out):
            return True
        
        # Column-wise differences
        test = np.zeros_like(inp)
        test[0, :] = inp[0, :]
        for i in range(1, inp.shape[0]):
            for j in range(inp.shape[1]):
                test[i, j] = (inp[i, j] - inp[i-1, j]) % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Row-wise differences
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            test[i, 0] = inp[i, 0]
            for j in range(1, inp.shape[1]):
                test[i, j] = (inp[i, j] - inp[i, j-1]) % 10
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[]
 for r in g:
  row=[r[0]]
  for j in range(1,len(r)):
   row.append((r[j]-r[j-1])%10)
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerGeometricSeries(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Check geometric series with different ratios
        for ratio in range(2, 5):
            test = np.zeros((h, w), dtype=int)
            val = 1
            for i in range(h):
                for j in range(w):
                    test[i, j] = val % 10
                    val = (val * ratio) % 10
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        for ratio in range(2, 5):
            test = np.zeros((h, w), dtype=int)
            val = 1
            for i in range(h):
                for j in range(w):
                    test[i, j] = val % 10
                    val = (val * ratio) % 10
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w={h},{w}
 res=[]
 v=1
 for i in range(h):
  row=[]
  for j in range(w):
   row.append(v%10)
   v=(v*{ratio})%10
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerBinaryConversion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Convert to binary representation (1s digit)
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                test[i, j] = 1 if inp[i, j] & 1 else 0
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return[[x&1 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerTriangularNumbers(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Generate triangular numbers
        tri = []
        n = 1
        while len(tri) < h * w:
            tri.append((n * (n + 1) // 2) % 10)
            n += 1
        
        test = np.array(tri[:h*w]).reshape(h, w)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 t=[]
 n=1
 while len(t)<h*w:
  t.append((n*(n+1)//2)%10)
  n+=1
 res=[]
 for i in range(h):
  res.append(t[i*w:(i+1)*w])
 return res
"""
        return None

@handler_decorator
class HandlerSlidingWindowSum(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Sliding window sum with window size 3
        test = np.zeros_like(inp)
        h, w = inp.shape
        
        for i in range(h):
            for j in range(w):
                window_sum = 0
                count = 0
                for dj in range(-1, 2):
                    nj = j + dj
                    if 0 <= nj < w:
                        window_sum += inp[i, nj]
                        count += 1
                test[i, j] = window_sum % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   s=0
   for dj in[-1,0,1]:
    if 0<=j+dj<w:
     s+=g[i][j+dj]
   row.append(s%10)
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerMatrixDeterminant2x2(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check if processing 2x2 submatrices
        if inp.shape[0] < 2 or inp.shape[1] < 2:
            return False
        
        if out.shape == (1, 1):
            # Compute determinant of top-left 2x2
            det = (inp[0, 0] * inp[1, 1] - inp[0, 1] * inp[1, 0]) % 10
            return out[0, 0] == det
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if out.shape == (1, 1) and self.can_handle(task_data, analysis):
            return """def p(g):
 return[[(g[0][0]*g[1][1]-g[0][1]*g[1][0])%10]]
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# PHYSICS AND MOVEMENT OPERATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerGravity(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        
        # Gravity down
        test = np.zeros_like(inp)
        for j in range(w):
            col = inp[:,j]
            non_zero = col[col != 0]
            test[h-len(non_zero):,j] = non_zero
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for j in range(w):
  nz=[g[i][j]for i in range(h)if g[i][j]!=0]
  for k,v in enumerate(nz):
   res[h-len(nz)+k][j]=v
 return res
"""
        
        # Gravity up
        test = np.zeros_like(inp)
        for j in range(w):
            col = inp[:,j]
            non_zero = col[col != 0]
            test[:len(non_zero),j] = non_zero
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for j in range(w):
  nz=[g[i][j]for i in range(h)if g[i][j]!=0]
  for k,v in enumerate(nz):
   res[k][j]=v
 return res
"""
        return None

@handler_decorator
class HandlerGravityLeft(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        for i in range(h):
            row = inp[i,:]
            non_zero = row[row != 0]
            test[i,:len(non_zero)] = non_zero
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 res=[]
 for r in g:
  nz=[x for x in r if x!=0]
  res.append(nz+[0]*(len(r)-len(nz)))
 return res
"""
        return None

@handler_decorator
class HandlerGravityRight(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        for i in range(h):
            row = inp[i,:]
            non_zero = row[row != 0]
            test[i,w-len(non_zero):] = non_zero
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 res=[]
 for r in g:
  nz=[x for x in r if x!=0]
  res.append([0]*(len(r)-len(nz))+nz)
 return res
"""
        return None

@handler_decorator
class HandlerMoveObjects(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        for dy in range(-3, 4):
            for dx in range(-3, 4):
                if dy == 0 and dx == 0:
                    continue
                test = np.zeros_like(inp)
                h, w = inp.shape
                for i in range(h):
                    for j in range(w):
                        if inp[i,j] != 0:
                            ni, nj = i + dy, j + dx
                            if 0 <= ni < h and 0 <= nj < w:
                                test[ni,nj] = inp[i,j]
                
                if np.array_equal(test, out):
                    return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for dy in range(-3, 4):
            for dx in range(-3, 4):
                if dy == 0 and dx == 0:
                    continue
                test = np.zeros_like(inp)
                for i in range(h):
                    for j in range(w):
                        if inp[i,j] != 0:
                            ni, nj = i + dy, j + dx
                            if 0 <= ni < h and 0 <= nj < w:
                                test[ni,nj] = inp[i,j]
                
                if np.array_equal(test, out):
                    if dy >= 0 and dx >= 0:
                        return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h-{dy}):
  for j in range(w-{dx}):
   if g[i][j]!=0:
    res[i+{dy}][j+{dx}]=g[i][j]
 return res
"""
                    elif dy >= 0 and dx < 0:
                        return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h-{dy}):
  for j in range({-dx},w):
   if g[i][j]!=0:
    res[i+{dy}][j{dx}]=g[i][j]
 return res
"""
                    elif dy < 0 and dx >= 0:
                        return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range({-dy},h):
  for j in range(w-{dx}):
   if g[i][j]!=0:
    res[i{dy}][j+{dx}]=g[i][j]
 return res
"""
                    else:
                        return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range({-dy},h):
  for j in range({-dx},w):
   if g[i][j]!=0:
    res[i{dy}][j{dx}]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerBounce(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        if inp.shape != out.shape:
            return False
        # Check if objects move and bounce off edges
        # This is a simplified check - would need more complex logic
        return False
        
    def generate_solution(self, task_data, analysis):
        # Complex bouncing logic would go here
        return None

@handler_decorator
class HandlerCollapse(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        # Check if all non-zero elements collapse to center
        if inp.shape != out.shape:
            return False
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        # Check if all non-zero values moved to center region
        center_region = out[max(0, ch-1):ch+2, max(0, cw-1):cw+2]
        if np.count_nonzero(out) == np.count_nonzero(center_region):
            return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        # Simple collapse to center
        if np.count_nonzero(out) == 1 and out[ch, cw] != 0:
            colors = [c for row in inp for c in row if c != 0]
            if colors and out[ch, cw] == colors[0]:
                return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 c=[x for r in g for x in r if x!=0]
 if c:res[h//2][w//2]=c[0]
 return res
"""
        return None

@handler_decorator
class HandlerStackingGravity(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Apply gravity with stacking behavior
        for j in range(w):
            column_elements = []
            for i in range(h):
                if inp[i, j] != 0:
                    column_elements.append(inp[i, j])
            
            # Place elements from bottom up
            test[:, j] = 0
            for idx, elem in enumerate(column_elements):
                test[h - 1 - idx, j] = elem
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for j in range(w):
  ce=[g[i][j]for i in range(h)if g[i][j]!=0]
  for idx,e in enumerate(ce):
   res[h-1-idx][j]=e
 return res
"""
        return None

@handler_decorator
class HandlerRotationalMovement(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        # Check for clockwise rotation of objects around center
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    # Calculate rotated position (90 degrees clockwise)
                    rel_i, rel_j = i - ch, j - cw
                    new_rel_i, new_rel_j = rel_j, -rel_i
                    new_i, new_j = new_rel_i + ch, new_rel_j + cw
                    
                    if 0 <= new_i < h and 0 <= new_j < w:
                        test[new_i, new_j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 ch,cw={ch},{cw}
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    ri,rj=i-ch,j-cw
    ni,nj=rj+ch,-ri+cw
    if 0<=ni<h and 0<=nj<w:
     res[ni][nj]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerOrbitalMovement(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if objects move in circular orbits
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        # Detect objects and their potential orbital movement
        test = np.zeros_like(inp)
        angle_step = np.pi / 4  # 45 degrees
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    # Calculate distance from center
                    dx, dy = j - cw, i - ch
                    radius = np.sqrt(dx**2 + dy**2)
                    
                    if radius > 0:
                        # Current angle
                        angle = np.arctan2(dy, dx)
                        # New angle after rotation
                        new_angle = angle + angle_step
                        
                        # New position
                        new_x = int(round(cw + radius * np.cos(new_angle)))
                        new_y = int(round(ch + radius * np.sin(new_angle)))
                        
                        if 0 <= new_y < h and 0 <= new_x < w:
                            test[new_y, new_x] = inp[i, j]
        
        return np.count_nonzero(test) > 0 and np.count_nonzero(test) == np.count_nonzero(out)
        
    def generate_solution(self, task_data, analysis):
        # Orbital movement requires trigonometric calculations
        return None

@handler_decorator
class HandlerWavePropagation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if pattern propagates like a wave
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Find source points
        sources = []
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    sources.append((i, j, inp[i, j]))
        
        # Propagate from each source
        for si, sj, color in sources:
            # Create wave front at distance 1
            for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                ni, nj = si + di, sj + dj
                if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] == 0:
                    test[ni, nj] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]==0:
      res[ni][nj]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerCollisionDetection(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if colliding objects merge or change
        from scipy.ndimage import label
        
        labeled_inp, n_inp = label(inp > 0)
        labeled_out, n_out = label(out > 0)
        
        # Output should have fewer objects if collision occurred
        if n_out < n_inp and n_inp >= 2:
            # Check if objects that were close merged
            test = np.copy(inp)
            
            for i in range(1, n_inp + 1):
                for j in range(i + 1, n_inp + 1):
                    mask_i = labeled_inp == i
                    mask_j = labeled_inp == j
                    
                    # Check if objects are adjacent
                    from scipy.ndimage import binary_dilation
                    dilated_i = binary_dilation(mask_i)
                    
                    if np.any(dilated_i & mask_j):
                        # Merge objects
                        color = max(inp[mask_i].max(), inp[mask_j].max())
                        test[mask_i | mask_j] = color
            
            return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Collision detection requires complex spatial analysis
        return None

@handler_decorator
class HandlerAttraction(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if objects move toward each other
        from scipy.ndimage import label, center_of_mass
        
        labeled, n = label(inp > 0)
        
        if n < 2:
            return False
        
        # Calculate centers
        centers = []
        for i in range(1, n + 1):
            mask = labeled == i
            cy, cx = center_of_mass(mask)
            centers.append((cy, cx, i))
        
        # Check if objects moved closer
        test = np.zeros_like(inp)
        
        # Move each object toward the center of all objects
        avg_y = sum(c[0] for c in centers) / len(centers)
        avg_x = sum(c[1] for c in centers) / len(centers)
        
        for cy, cx, obj_id in centers:
            mask = labeled == obj_id
            dy = 1 if avg_y > cy else -1 if avg_y < cy else 0
            dx = 1 if avg_x > cx else -1 if avg_x < cx else 0
            
            # Move object
            positions = np.argwhere(mask)
            for y, x in positions:
                new_y, new_x = y + dy, x + dx
                if 0 <= new_y < inp.shape[0] and 0 <= new_x < inp.shape[1]:
                    test[new_y, new_x] = inp[y, x]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Attraction physics requires complex calculations
        return None

@handler_decorator
class HandlerRepulsion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if objects move away from each other
        from scipy.ndimage import label, center_of_mass
        
        labeled, n = label(inp > 0)
        
        if n < 2:
            return False
        
        # Calculate centers and check if they moved apart
        centers_inp = []
        centers_out = []
        
        for i in range(1, n + 1):
            mask = labeled == i
            if np.any(mask):
                cy, cx = center_of_mass(mask)
                centers_inp.append((cy, cx))
        
        # Simple repulsion: objects move away from center
        avg_y = sum(c[0] for c in centers_inp) / len(centers_inp)
        avg_x = sum(c[1] for c in centers_inp) / len(centers_inp)
        
        test = np.zeros_like(inp)
        
        for i in range(1, n + 1):
            mask = labeled == i
            if np.any(mask):
                cy, cx = center_of_mass(mask)
                
                # Direction away from center
                dy = 1 if cy > avg_y else -1 if cy < avg_y else 0
                dx = 1 if cx > avg_x else -1 if cx < avg_x else 0
                
                # Move object
                positions = np.argwhere(mask)
                for y, x in positions:
                    new_y, new_x = y + dy, x + dx
                    if 0 <= new_y < inp.shape[0] and 0 <= new_x < inp.shape[1]:
                        test[new_y, new_x] = inp[y, x]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Repulsion physics requires complex calculations
        return None

@handler_decorator
class HandlerSlidingWithObstacles(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if objects slide until they hit obstacles
        h, w = inp.shape
        
        # Try sliding in each direction
        for direction in ['down', 'up', 'left', 'right']:
            test = np.copy(inp)
            
            if direction == 'down':
                for j in range(w):
                    for i in range(h - 1):
                        if test[i, j] != 0 and test[i, j] != 1:  # Assuming 1 is obstacle
                            # Slide down until obstacle or edge
                            current_i = i
                            while current_i < h - 1 and test[current_i + 1, j] == 0:
                                test[current_i + 1, j] = test[current_i, j]
                                test[current_i, j] = 0
                                current_i += 1
            
            elif direction == 'right':
                for i in range(h):
                    for j in range(w - 1):
                        if test[i, j] != 0 and test[i, j] != 1:
                            current_j = j
                            while current_j < w - 1 and test[i, current_j + 1] == 0:
                                test[i, current_j + 1] = test[i, current_j]
                                test[i, current_j] = 0
                                current_j += 1
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Sliding with obstacles requires complex iteration logic
        return None

@handler_decorator
class HandlerExplosion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if objects explode outward from their positions
        h, w = inp.shape
        test = np.copy(inp)
        
        # Find explosion centers
        explosion_points = []
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    explosion_points.append((i, j, inp[i, j]))
        
        # Create explosion pattern
        test = np.zeros_like(inp)
        for ei, ej, color in explosion_points:
            # Add fragments in all 8 directions
            for di, dj in [(0,1), (1,1), (1,0), (1,-1), (0,-1), (-1,-1), (-1,0), (-1,1)]:
                ni, nj = ei + di * 2, ej + dj * 2  # Distance 2 explosion
                if 0 <= ni < h and 0 <= nj < w:
                    test[ni, nj] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    for di,dj in[(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]:
     ni,nj=i+di*2,j+dj*2
     if 0<=ni<h and 0<=nj<w:
      res[ni][nj]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerMagneticField(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if objects align along field lines
        h, w = inp.shape
        
        # Vertical magnetic field alignment
        test = np.zeros_like(inp)
        for j in range(w):
            column_objects = []
            for i in range(h):
                if inp[i, j] != 0:
                    column_objects.append(inp[i, j])
            
            # Distribute evenly along column
            if column_objects:
                spacing = h // (len(column_objects) + 1)
                for idx, obj in enumerate(column_objects):
                    pos = (idx + 1) * spacing
                    if pos < h:
                        test[pos, j] = obj
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Magnetic field alignment requires specific spacing calculations
        return None

@handler_decorator
class HandlerPendulumMotion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check for pendulum-like swing motion
        h, w = inp.shape
        pivot_row = 0  # Assume top row is pivot
        
        test = np.zeros_like(inp)
        
        # Find hanging objects
        for j in range(w):
            for i in range(1, h):
                if inp[i, j] != 0:
                    # Calculate swing position
                    distance_from_pivot = i
                    # Simple pendulum: move horizontally based on vertical distance
                    swing_offset = min(distance_from_pivot // 2, 2)
                    
                    # Try swinging right
                    new_j = j + swing_offset
                    if new_j < w:
                        test[i, new_j] = inp[i, j]
                        break
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Pendulum motion requires physics simulation
        return None

@handler_decorator
class HandlerProjectileMotion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check for parabolic trajectory
        h, w = inp.shape
        
        # Find launch points (bottom row)
        launch_points = []
        for j in range(w):
            if inp[h-1, j] != 0:
                launch_points.append((h-1, j, inp[h-1, j]))
        
        if not launch_points:
            return False
        
        test = np.zeros_like(inp)
        
        # Simple projectile: move up and right
        for li, lj, color in launch_points:
            # Trajectory parameters
            vx, vy = 1, -3  # Initial velocity
            
            t = 0
            while t < 10:
                # Position at time t
                x = lj + vx * t
                y = li + vy * t + t * t // 2  # Gravity effect
                
                if 0 <= y < h and 0 <= x < w:
                    test[int(y), int(x)] = color
                
                t += 1
                
                if y >= h:
                    break
        
        return np.count_nonzero(test) > 0
        
    def generate_solution(self, task_data, analysis):
        # Projectile motion requires physics calculations
        return None

@handler_decorator
class HandlerFluidSpreading(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if pattern spreads like fluid
        h, w = inp.shape
        test = np.copy(inp)
        
        # Fluid spreads downward and sideways when blocked
        changed = True
        iterations = 0
        
        while changed and iterations < 10:
            changed = False
            new_test = np.copy(test)
            
            for i in range(h-1, -1, -1):
                for j in range(w):
                    if test[i, j] != 0:
                        # Try to flow down
                        if i < h-1 and test[i+1, j] == 0:
                            new_test[i+1, j] = test[i, j]
                            changed = True
                        else:
                            # Flow sideways if blocked
                            if j > 0 and test[i, j-1] == 0 and (i == h-1 or test[i+1, j-1] != 0):
                                new_test[i, j-1] = test[i, j]
                                changed = True
                            if j < w-1 and test[i, j+1] == 0 and (i == h-1 or test[i+1, j+1] != 0):
                                new_test[i, j+1] = test[i, j]
                                changed = True
            
            test = new_test
            iterations += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Fluid dynamics simulation is too complex for compact implementation
        return None

@handler_decorator
class HandlerElasticCollision(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if objects bounce off each other
        from scipy.ndimage import label
        
        labeled_inp, n_inp = label(inp > 0)
        labeled_out, n_out = label(out > 0)
        
        # Same number of objects but in different positions
        if n_inp == n_out and n_inp >= 2:
            # Check if objects moved in opposite directions
            # This would require velocity tracking
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Elastic collision requires velocity and momentum calculations
        return None

@handler_decorator
class HandlerCentrifugalMotion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if objects move outward from center
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    # Calculate direction from center
                    di = i - ch
                    dj = j - cw
                    
                    # Move outward
                    if di != 0 or dj != 0:
                        # Normalize direction
                        magnitude = max(abs(di), abs(dj))
                        step_i = di // magnitude if magnitude > 0 else 0
                        step_j = dj // magnitude if magnitude > 0 else 0
                        
                        new_i = i + step_i
                        new_j = j + step_j
                        
                        if 0 <= new_i < h and 0 <= new_j < w:
                            test[new_i, new_j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 ch,cw={ch},{cw}
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    di=i-ch
    dj=j-cw
    if di!=0 or dj!=0:
     m=max(abs(di),abs(dj))
     si=di//m if m>0 else 0
     sj=dj//m if m>0 else 0
     ni=i+si
     nj=j+sj
     if 0<=ni<h and 0<=nj<w:
      res[ni][nj]=g[i][j]
 return res
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# OBJECT DETECTION AND MANIPULATION
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerFillObjects(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        test = np.copy(inp)
        for color in range(1, 10):
            mask = inp == color
            if np.any(mask):
                filled = binary_fill_holes(mask)
                test[filled] = color
        
        if np.array_equal(test, out):
            return """def p(g):
 from scipy.ndimage import binary_fill_holes
 import numpy as np
 t=np.array(g)
 res=t.copy()
 for c in range(1,10):
  m=t==c
  if np.any(m):
   f=binary_fill_holes(m)
   res[f]=c
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerColorCount(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        unique_colors = len(np.unique(inp))
        
        if np.all(out == unique_colors):
            return f"""def p(g):
 colors=set()
 for r in g:
  for c in r:
   colors.add(c)
 n=len(colors)
 return[[n]*len(g[0])for _ in g]
"""
        return None

@handler_decorator
class HandlerExtractLargestObject(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        labeled, n = label(inp > 0)
        if n == 0:
            return None
        
        sizes = [np.sum(labeled == i) for i in range(1, n+1)]
        largest_idx = np.argmax(sizes) + 1
        
        test = np.where(labeled == largest_idx, inp, 0)
        
        if np.array_equal(test, out):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 t=np.array(g)
 l,n=label(t>0)
 if n==0:return g
 sizes=[np.sum(l==i)for i in range(1,n+1)]
 idx=np.argmax(sizes)+1
 res=np.where(l==idx,t,0)
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerExtractByColor(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        for color in range(1, 10):
            test = np.where(inp == color, inp, 0)
            if np.array_equal(test, out):
                return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for color in range(1, 10):
            test = np.where(inp == color, inp, 0)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[x if x=={color} else 0 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerRemoveNoise(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i,j] != 0:
                    neighbors = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni,nj] != 0:
                            neighbors += 1
                    if neighbors == 0:
                        test[i,j] = 0
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    n=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
      n+=1
    if n==0:
     res[i][j]=0
 return res
"""
        return None

@handler_decorator
class HandlerExtractSmallestObject(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        labeled, n = label(inp > 0)
        if n == 0:
            return None
        
        sizes = [np.sum(labeled == i) for i in range(1, n+1)]
        smallest_idx = np.argmin(sizes) + 1
        
        test = np.where(labeled == smallest_idx, inp, 0)
        
        if np.array_equal(test, out):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 t=np.array(g)
 l,n=label(t>0)
 if n==0:return g
 sizes=[np.sum(l==i)for i in range(1,n+1)]
 idx=np.argmin(sizes)+1
 res=np.where(l==idx,t,0)
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerDilateObjects(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        # Try dilation
        struct = np.array([[0,1,0],[1,1,1],[0,1,0]])
        test = np.copy(inp)
        for color in range(1, 10):
            mask = inp == color
            if np.any(mask):
                dilated = binary_dilation(mask, struct)
                test[dilated] = color
        
        if np.array_equal(test, out):
            return """def p(g):
 from scipy.ndimage import binary_dilation
 import numpy as np
 t=np.array(g)
 res=t.copy()
 s=np.array([[0,1,0],[1,1,1],[0,1,0]])
 for c in range(1,10):
  m=t==c
  if np.any(m):
   d=binary_dilation(m,s)
   res[d]=c
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerErodeObjects(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        # Try erosion
        struct = np.array([[0,1,0],[1,1,1],[0,1,0]])
        test = np.copy(inp)
        for color in range(1, 10):
            mask = inp == color
            if np.any(mask):
                eroded = binary_erosion(mask, struct)
                test[mask] = 0
                test[eroded] = color
        
        if np.array_equal(test, out):
            return """def p(g):
 from scipy.ndimage import binary_erosion
 import numpy as np
 t=np.array(g)
 res=np.zeros_like(t)
 s=np.array([[0,1,0],[1,1,1],[0,1,0]])
 for c in range(1,10):
  m=t==c
  if np.any(m):
   e=binary_erosion(m,s)
   res[e]=c
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerObjectCentroids(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        from scipy.ndimage import label, center_of_mass
        
        labeled, n = label(inp > 0)
        test = np.zeros_like(inp)
        
        for i in range(1, n+1):
            mask = labeled == i
            cy, cx = center_of_mass(mask)
            cy, cx = int(round(cy)), int(round(cx))
            
            if 0 <= cy < inp.shape[0] and 0 <= cx < inp.shape[1]:
                color = inp[mask][0]
                test[cy, cx] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 from scipy.ndimage import label,center_of_mass
 import numpy as np
 t=np.array(g)
 l,n=label(t>0)
 res=np.zeros_like(t)
 for i in range(1,n+1):
  m=l==i
  cy,cx=center_of_mass(m)
  cy,cx=int(round(cy)),int(round(cx))
  if 0<=cy<t.shape[0]and 0<=cx<t.shape[1]:
   c=t[m][0]
   res[cy,cx]=c
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerMoveObjects(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        from scipy.ndimage import label
        
        labeled_inp, n_inp = label(inp > 0)
        labeled_out, n_out = label(out > 0)
        
        if n_inp != n_out:
            return False
        
        # Check if objects have moved
        for offset_y in range(-5, 6):
            for offset_x in range(-5, 6):
                if offset_y == 0 and offset_x == 0:
                    continue
                
                test = np.zeros_like(inp)
                for i in range(inp.shape[0]):
                    for j in range(inp.shape[1]):
                        if inp[i, j] != 0:
                            new_i = i + offset_y
                            new_j = j + offset_x
                            if 0 <= new_i < inp.shape[0] and 0 <= new_j < inp.shape[1]:
                                test[new_i, new_j] = inp[i, j]
                
                if np.array_equal(test, out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for offset_y in range(-5, 6):
            for offset_x in range(-5, 6):
                if offset_y == 0 and offset_x == 0:
                    continue
                
                test = np.zeros_like(inp)
                for i in range(inp.shape[0]):
                    for j in range(inp.shape[1]):
                        if inp[i, j] != 0:
                            new_i = i + offset_y
                            new_j = j + offset_x
                            if 0 <= new_i < inp.shape[0] and 0 <= new_j < inp.shape[1]:
                                test[new_i, new_j] = inp[i, j]
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    ni=i+{offset_y}
    nj=j+{offset_x}
    if 0<=ni<h and 0<=nj<w:
     res[ni][nj]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerDuplicateObjects(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        from scipy.ndimage import label
        
        labeled_inp, n_inp = label(inp > 0)
        labeled_out, n_out = label(out > 0)
        
        # Check if output has twice as many objects
        return n_out == n_inp * 2
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Try duplicating objects in various directions
        from scipy.ndimage import label
        
        labeled, n = label(inp > 0)
        
        for offset_y in range(1, 5):
            for offset_x in range(1, 5):
                test = np.copy(inp)
                
                for i in range(1, n+1):
                    mask = labeled == i
                    positions = np.argwhere(mask)
                    
                    for pos in positions:
                        y, x = pos
                        new_y = y + offset_y
                        new_x = x + offset_x
                        
                        if 0 <= new_y < inp.shape[0] and 0 <= new_x < inp.shape[1]:
                            if test[new_y, new_x] == 0:
                                test[new_y, new_x] = inp[y, x]
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 from scipy.ndimage import label
 import numpy as np
 t=np.array(g)
 l,n=label(t>0)
 res=t.copy()
 for i in range(1,n+1):
  m=l==i
  pos=np.argwhere(m)
  for y,x in pos:
   ny=y+{offset_y}
   nx=x+{offset_x}
   if 0<=ny<t.shape[0]and 0<=nx<t.shape[1]:
    if res[ny,nx]==0:
     res[ny,nx]=t[y,x]
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerFilterBySize(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        from scipy.ndimage import label
        
        labeled, n = label(inp > 0)
        
        # Try different size thresholds
        for threshold in range(2, 20):
            test = np.zeros_like(inp)
            
            for i in range(1, n+1):
                mask = labeled == i
                size = np.sum(mask)
                
                if size >= threshold:
                    test[mask] = inp[mask]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 from scipy.ndimage import label
 import numpy as np
 t=np.array(g)
 l,n=label(t>0)
 res=np.zeros_like(t)
 for i in range(1,n+1):
  m=l==i
  if np.sum(m)>={threshold}:
   res[m]=t[m]
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerObjectBoundingBoxes(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        from scipy.ndimage import label
        
        labeled, n = label(inp > 0)
        test = np.zeros_like(inp)
        
        for i in range(1, n+1):
            mask = labeled == i
            positions = np.argwhere(mask)
            
            if len(positions) > 0:
                min_y, min_x = positions.min(axis=0)
                max_y, max_x = positions.max(axis=0)
                
                color = inp[mask][0]
                
                # Draw bounding box
                test[min_y:max_y+1, min_x] = color
                test[min_y:max_y+1, max_x] = color
                test[min_y, min_x:max_x+1] = color
                test[max_y, min_x:max_x+1] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 t=np.array(g)
 l,n=label(t>0)
 res=np.zeros_like(t)
 for i in range(1,n+1):
  m=l==i
  pos=np.argwhere(m)
  if len(pos)>0:
   miy,mix=pos.min(axis=0)
   may,max=pos.max(axis=0)
   c=t[m][0]
   res[miy:may+1,mix]=c
   res[miy:may+1,max]=c
   res[miy,mix:max+1]=c
   res[may,mix:max+1]=c
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerMergeOverlappingObjects(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if overlapping objects are merged
        from scipy.ndimage import label
        
        test = np.copy(inp)
        
        # Merge objects of the same color that are adjacent
        for color in range(1, 10):
            mask = test == color
            if np.any(mask):
                labeled, n = label(mask)
                if n > 1:
                    # Multiple components of same color - merge them
                    test[mask] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Merging logic would be complex to implement compactly
        return None

@handler_decorator
class HandlerObjectAlignment(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        from scipy.ndimage import label, center_of_mass
        
        labeled, n = label(inp > 0)
        
        if n < 2:
            return False
        
        # Check if objects are aligned horizontally or vertically
        centers = []
        for i in range(1, n+1):
            mask = labeled == i
            cy, cx = center_of_mass(mask)
            centers.append((int(round(cy)), int(round(cx)), i))
        
        # Try horizontal alignment
        target_y = centers[0][0]
        test = np.zeros_like(inp)
        
        for cy, cx, obj_id in centers:
            mask = labeled == obj_id
            positions = np.argwhere(mask)
            
            for pos in positions:
                y, x = pos
                new_y = y - cy + target_y
                
                if 0 <= new_y < inp.shape[0]:
                    test[new_y, x] = inp[y, x]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Object alignment requires complex coordinate transformations
        return None

@handler_decorator
class HandlerReplaceObjectColors(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        from scipy.ndimage import label
        
        labeled, n = label(inp > 0)
        
        # Check if each object gets a unique color based on its index
        test = np.zeros_like(inp)
        for i in range(1, n+1):
            mask = labeled == i
            test[mask] = (i % 9) + 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 t=np.array(g)
 l,n=label(t>0)
 res=np.zeros_like(t)
 for i in range(1,n+1):
  m=l==i
  res[m]=(i%9)+1
 return res.tolist()
"""
        return None

@handler_decorator
class HandlerObjectIntersection(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if output shows intersection of different colored objects
        test = np.zeros_like(inp)
        
        for color1 in range(1, 10):
            for color2 in range(color1+1, 10):
                mask1 = inp == color1
                mask2 = inp == color2
                
                if np.any(mask1) and np.any(mask2):
                    # Dilate masks to find intersection
                    from scipy.ndimage import binary_dilation
                    
                    dilated1 = binary_dilation(mask1)
                    dilated2 = binary_dilation(mask2)
                    
                    intersection = dilated1 & dilated2
                    if np.any(intersection):
                        test[intersection] = 5  # Mark intersection
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Intersection detection requires complex morphological operations
        return None

@handler_decorator
class HandlerObjectDistance(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        from scipy.ndimage import label, center_of_mass
        
        labeled, n = label(inp > 0)
        
        if n != 2:
            return False
        
        # Calculate distance between two objects
        centers = []
        for i in range(1, n+1):
            mask = labeled == i
            cy, cx = center_of_mass(mask)
            centers.append((cy, cx))
        
        if len(centers) == 2:
            dist = int(((centers[0][0] - centers[1][0])**2 + 
                       (centers[0][1] - centers[1][1])**2)**0.5)
            
            # Check if output encodes distance
            if np.all(out == (dist % 10)):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Distance-based transformations require specific implementations
        return None

@handler_decorator
class HandlerObjectConnectivity(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if objects are connected based on proximity
        h, w = inp.shape
        test = np.copy(inp)
        
        # Connect objects within certain distance
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    # Look for nearby objects
                    for radius in range(1, 4):
                        for di in range(-radius, radius+1):
                            for dj in range(-radius, radius+1):
                                ni, nj = i + di, j + dj
                                if (0 <= ni < h and 0 <= nj < w and 
                                    inp[ni, nj] != 0 and inp[ni, nj] != inp[i, j]):
                                    # Connect with line
                                    if i == ni:
                                        for jj in range(min(j, nj), max(j, nj)+1):
                                            if test[i, jj] == 0:
                                                test[i, jj] = inp[i, j]
                                    elif j == nj:
                                        for ii in range(min(i, ni), max(i, ni)+1):
                                            if test[ii, j] == 0:
                                                test[ii, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Connectivity algorithms are complex to implement compactly
        return None

@handler_decorator
class HandlerObjectSegmentation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if large objects are segmented into smaller parts
        from scipy.ndimage import label
        
        labeled_inp, n_inp = label(inp > 0)
        labeled_out, n_out = label(out > 0)
        
        # Output should have more objects than input
        return n_out > n_inp and n_inp > 0
        
    def generate_solution(self, task_data, analysis):
        # Object segmentation requires complex algorithms
        return None

@handler_decorator
class HandlerObjectGrowthPattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if objects grow by specific pattern
        test = np.copy(inp)
        
        # Grow each object by adding layers
        for iteration in range(1, 4):
            new_test = np.copy(test)
            
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    if test[i, j] != 0:
                        # Add neighbors
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if (0 <= ni < inp.shape[0] and 0 <= nj < inp.shape[1] and
                                new_test[ni, nj] == 0):
                                new_test[ni, nj] = test[i, j]
            
            test = new_test
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for iterations in range(1, 4):
            test = np.copy(inp)
            
            for _ in range(iterations):
                new_test = np.copy(test)
                
                for i in range(inp.shape[0]):
                    for j in range(inp.shape[1]):
                        if test[i, j] != 0:
                            for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                                ni, nj = i + di, j + dj
                                if (0 <= ni < inp.shape[0] and 0 <= nj < inp.shape[1] and
                                    new_test[ni, nj] == 0):
                                    new_test[ni, nj] = test[i, j]
                
                test = new_test
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for _ in range({iterations}):
  tmp=[r[:]for r in res]
  for i in range(h):
   for j in range(w):
    if res[i][j]!=0:
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if 0<=ni<h and 0<=nj<w and tmp[ni][nj]==0:
       tmp[ni][nj]=res[i][j]
  res=tmp
 return res
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# LINE AND SHAPE DRAWING
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerDrawLines(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        
        ch, cw = h // 2, w // 2
        test = np.copy(inp)
        
        if h > 0:
            test[ch,:] = 1
        if w > 0:
            test[:,cw] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 if h>0:
  res[h//2]=[1]*w
 if w>0:
  for i in range(h):
   res[i][w//2]=1
 return res
"""
        
        test = np.copy(inp)
        for i in range(min(h,w)):
            test[i,i] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 for i in range(min(len(g),len(g[0]))):
  res[i][i]=1
 return res
"""
        return None

@handler_decorator
class HandlerDrawDiagonals(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Main diagonal
        for i in range(min(h, w)):
            test[i, i] = 1
        
        if np.array_equal(test, out):
            return True
        
        # Anti-diagonal
        test = np.copy(inp)
        for i in range(min(h, w)):
            test[i, w-1-i] = 1
        
        if np.array_equal(test, out):
            return True
        
        # Both diagonals
        test = np.copy(inp)
        for i in range(min(h, w)):
            test[i, i] = 1
            test[i, w-1-i] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.copy(inp)
        for i in range(min(h, w)):
            test[i, i] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 for i in range(min(len(g),len(g[0]))):
  res[i][i]=1
 return res
"""
        
        test = np.copy(inp)
        for i in range(min(h, w)):
            test[i, w-1-i] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 w=len(g[0])
 for i in range(min(len(g),w)):
  res[i][w-1-i]=1
 return res
"""
        
        test = np.copy(inp)
        for i in range(min(h, w)):
            test[i, i] = 1
            test[i, w-1-i] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 w=len(g[0])
 for i in range(min(len(g),w)):
  res[i][i]=1
  res[i][w-1-i]=1
 return res
"""
        return None

@handler_decorator
class HandlerDrawRectangle(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Draw border
        test[0, :] = 1
        test[-1, :] = 1
        test[:, 0] = 1
        test[:, -1] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 res[0]=[1]*w
 res[-1]=[1]*w
 for i in range(h):
  res[i][0]=1
  res[i][-1]=1
 return res
"""
        return None

@handler_decorator
class HandlerDrawCircle(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape or inp.shape[0] != inp.shape[1]:
            return False
        
        h, w = inp.shape
        cx, cy = w // 2, h // 2
        radius = min(h, w) // 2 - 1
        
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                dist = ((i - cy) ** 2 + (j - cx) ** 2) ** 0.5
                if abs(dist - radius) < 1:
                    test[i, j] = 1
        
        # Rough check for circle pattern
        if np.count_nonzero(test) > 0 and np.count_nonzero(out) > 0:
            return True
        return False
        
    def generate_solution(self, task_data, analysis):
        # Complex circle drawing would require more precise implementation
        return None

@handler_decorator
class HandlerDrawTriangle(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check for simple triangle patterns
        h, w = inp.shape
        test = np.copy(inp)
        
        # Upward triangle
        for i in range(h):
            for j in range(w):
                if j >= i and j < w - i:
                    test[i, j] = 1
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                if j >= i and j < w - i:
                    test[i, j] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if j>=i and j<w-i:
    res[i][j]=1
 return res
"""
        return None

@handler_decorator
class HandlerDrawGrid(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for grid patterns with different spacings
        for spacing in range(2, min(h, w) // 2):
            test = np.copy(inp)
            
            # Draw vertical lines
            for j in range(0, w, spacing):
                test[:, j] = 1
            
            # Draw horizontal lines
            for i in range(0, h, spacing):
                test[i, :] = 1
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for spacing in range(2, min(h, w) // 2):
            test = np.copy(inp)
            
            for j in range(0, w, spacing):
                test[:, j] = 1
            
            for i in range(0, h, spacing):
                test[i, :] = 1
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for j in range(0,w,{spacing}):
  for i in range(h):
   res[i][j]=1
 for i in range(0,h,{spacing}):
  res[i]=[1]*w
 return res
"""
        return None

@handler_decorator
class HandlerDrawFilledRectangle(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for filled rectangles at various positions
        for top in range(h // 2):
            for left in range(w // 2):
                for bottom in range(top + 1, h):
                    for right in range(left + 1, w):
                        test = np.copy(inp)
                        test[top:bottom, left:right] = 1
                        
                        if np.array_equal(test, out):
                            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Find the filled rectangle bounds
        rows, cols = np.where(out == 1)
        if len(rows) > 0:
            top, bottom = rows.min(), rows.max() + 1
            left, right = cols.min(), cols.max() + 1
            
            test = np.copy(inp)
            test[top:bottom, left:right] = 1
            
            if np.array_equal(test, out):
                return f"""def p(g):
 res=[r[:]for r in g]
 for i in range({top},{bottom}):
  for j in range({left},{right}):
   res[i][j]=1
 return res
"""
        return None

@handler_decorator
class HandlerDrawDiamond(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                manhattan_dist = abs(i - ch) + abs(j - cw)
                if manhattan_dist <= min(ch, cw):
                    test[i, j] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        max_dist = min(ch, cw)
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 ch,cw={ch},{cw}
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if abs(i-ch)+abs(j-cw)<={max_dist}:
    res[i][j]=1
 return res
"""
        return None

@handler_decorator
class HandlerDrawStar(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        test = np.copy(inp)
        
        # Draw star pattern (cross + diagonals)
        test[ch, :] = 1  # Horizontal
        test[:, cw] = 1  # Vertical
        
        # Add diagonals through center
        for i in range(min(h, w)):
            if ch - i >= 0 and cw - i >= 0:
                test[ch - i, cw - i] = 1
            if ch - i >= 0 and cw + i < w:
                test[ch - i, cw + i] = 1
            if ch + i < h and cw - i >= 0:
                test[ch + i, cw - i] = 1
            if ch + i < h and cw + i < w:
                test[ch + i, cw + i] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Star pattern is complex to implement compactly
        return None

@handler_decorator
class HandlerDrawParallelLines(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for parallel horizontal lines
        for spacing in range(2, h):
            test = np.copy(inp)
            for i in range(0, h, spacing):
                test[i, :] = 1
            
            if np.array_equal(test, out):
                return True
        
        # Check for parallel vertical lines
        for spacing in range(2, w):
            test = np.copy(inp)
            for j in range(0, w, spacing):
                test[:, j] = 1
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for spacing in range(2, h):
            test = np.copy(inp)
            for i in range(0, h, spacing):
                test[i, :] = 1
            
            if np.array_equal(test, out):
                return f"""def p(g):
 res=[r[:]for r in g]
 for i in range(0,len(g),{spacing}):
  res[i]=[1]*len(g[0])
 return res
"""
        
        for spacing in range(2, w):
            test = np.copy(inp)
            for j in range(0, w, spacing):
                test[:, j] = 1
            
            if np.array_equal(test, out):
                return f"""def p(g):
 res=[r[:]for r in g]
 for i in range(len(g)):
  for j in range(0,len(g[0]),{spacing}):
   res[i][j]=1
 return res
"""
        return None

@handler_decorator
class HandlerDrawZigzagPattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Draw zigzag pattern
        amplitude = min(h, w) // 4
        for j in range(w):
            i = int(h // 2 + amplitude * np.sin(2 * np.pi * j / w))
            if 0 <= i < h:
                test[i, j] = 1
        
        # Check if roughly matches output pattern
        return np.count_nonzero(test) > 0 and np.count_nonzero(out) > 0
        
    def generate_solution(self, task_data, analysis):
        # Zigzag patterns require trigonometric functions
        return None

@handler_decorator
class HandlerDrawHexagon(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        # Approximate hexagon using Manhattan distance with modifications
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                dx = abs(j - cw)
                dy = abs(i - ch)
                
                # Hexagon-like condition
                if dx + dy * 2 <= min(h, w) // 2:
                    test[i, j] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Hexagon drawing is complex
        return None

@handler_decorator
class HandlerDrawConnectedPoints(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Find non-zero points in input
        points = []
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    points.append((i, j))
        
        if len(points) < 2:
            return False
        
        # Check if output connects these points
        test = np.copy(inp)
        
        # Try connecting consecutive points
        for idx in range(len(points) - 1):
            y1, x1 = points[idx]
            y2, x2 = points[idx + 1]
            
            # Draw line between points (simple horizontal/vertical only)
            if y1 == y2:
                for x in range(min(x1, x2), max(x1, x2) + 1):
                    test[y1, x] = inp[y1, x1]
            elif x1 == x2:
                for y in range(min(y1, y2), max(y1, y2) + 1):
                    test[y, x1] = inp[y1, x1]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Line connection algorithms are complex to implement compactly
        return None

@handler_decorator
class HandlerDrawConcentricShapes(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for concentric rectangles
        test = np.copy(inp)
        layers = min(h, w) // 2
        
        for layer in range(layers):
            color = (layer % 9) + 1
            # Top and bottom edges
            test[layer, layer:w-layer] = color
            test[h-1-layer, layer:w-layer] = color
            # Left and right edges
            test[layer:h-layer, layer] = color
            test[layer:h-layer, w-1-layer] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        layers = min(h, w) // 2
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 res=[r[:]for r in g]
 for l in range({layers}):
  c=(l%9)+1
  for j in range(l,w-l):
   res[l][j]=c
   res[h-1-l][j]=c
  for i in range(l,h-l):
   res[i][l]=c
   res[i][w-1-l]=c
 return res
"""
        return None

@handler_decorator
class HandlerDrawChessPattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for various chess-like patterns
        for size in [1, 2, 3]:
            test = np.copy(inp)
            for i in range(h):
                for j in range(w):
                    if ((i // size) + (j // size)) % 2 == 0:
                        test[i, j] = 1
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for size in [1, 2, 3]:
            test = np.copy(inp)
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    if ((i // size) + (j // size)) % 2 == 0:
                        test[i, j] = 1
            
            if np.array_equal(test, out):
                if size == 1:
                    return """def p(g):
 return[[1 if(i+j)%2==0 else 0 for j in range(len(g[0]))]for i in range(len(g))]
"""
                else:
                    return f"""def p(g):
 return[[1 if((i//{size})+(j//{size}))%2==0 else 0 for j in range(len(g[0]))]for i in range(len(g))]
"""
        return None

@handler_decorator
class HandlerDrawSpiralLine(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Draw spiral from outside to inside
        top, bottom, left, right = 0, h-1, 0, w-1
        
        while top <= bottom and left <= right:
            # Top row
            for j in range(left, right+1):
                test[top, j] = 1
            top += 1
            
            # Right column
            for i in range(top, bottom+1):
                test[i, right] = 1
            right -= 1
            
            # Bottom row
            if top <= bottom:
                for j in range(right, left-1, -1):
                    test[bottom, j] = 1
                bottom -= 1
            
            # Left column
            if left <= right:
                for i in range(bottom, top-1, -1):
                    test[i, left] = 1
                left += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Spiral line drawing requires complex loop logic
        return None

@handler_decorator
class HandlerDrawArrow(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        ch = h // 2
        
        # Check for arrow patterns
        test = np.copy(inp)
        
        # Right-pointing arrow
        test[ch, :] = 1  # Horizontal line
        for i in range(min(h//2, w//2)):
            if ch-i >= 0 and w-1-i >= 0:
                test[ch-i, w-1-i] = 1
            if ch+i < h and w-1-i >= 0:
                test[ch+i, w-1-i] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Arrow patterns are complex to implement compactly
        return None

@handler_decorator
class HandlerDrawRadialLines(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        test = np.copy(inp)
        
        # Draw 8 radial lines from center
        # Horizontal and vertical
        test[ch, :] = 1
        test[:, cw] = 1
        
        # Diagonals
        for i in range(min(h, w)):
            if i < h and i < w:
                test[i, i] = 1
            if i < h and w-1-i >= 0:
                test[i, w-1-i] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        ch, cw = h // 2, w // 2
        
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w={h},{w}
 res=[r[:]for r in g]
 res[{ch}]=[1]*w
 for i in range(h):
  res[i][{cw}]=1
 for i in range(min(h,w)):
  if i<h and i<w:
   res[i][i]=1
  if i<h and w-1-i>=0:
   res[i][w-1-i]=1
 return res
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# FILLING AND REGION OPERATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerFloodFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp[0,0] == 0 and out[0,0] != 0:
            fill_color = out[0,0]
            
            h, w = inp.shape
            filled = np.copy(inp)
            to_fill = [(0, 0)]
            filled[0,0] = fill_color
            
            while to_fill:
                y, x = to_fill.pop()
                for dy, dx in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ny, nx = y + dy, x + dx
                    if 0 <= ny < h and 0 <= nx < w and filled[ny,nx] == 0:
                        filled[ny,nx] = fill_color
                        to_fill.append((ny, nx))
            
            if np.array_equal(filled, out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 fc={fill_color}
 q=[(0,0)]
 res[0][0]=fc
 while q:
  y,x=q.pop()
  for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
   ny,nx=y+dy,x+dx
   if 0<=ny<h and 0<=nx<w and res[ny][nx]==0:
    res[ny][nx]=fc
    q.append((ny,nx))
 return res
"""
        return None

@handler_decorator
class HandlerMirrorPattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        mirrored = np.hstack([inp, np.fliplr(inp)])
        if np.array_equal(mirrored, out):
            return """def p(g):
 return[r+r[::-1]for r in g]
"""
        
        mirrored = np.vstack([inp, np.flipud(inp)])
        if np.array_equal(mirrored, out):
            return """def p(g):
 return g+g[::-1]
"""
        
        top = np.hstack([inp, np.fliplr(inp)])
        bottom = np.hstack([np.flipud(inp), np.flipud(np.fliplr(inp))])
        mirrored = np.vstack([top, bottom])
        if np.array_equal(mirrored, out):
            return """def p(g):
 t=[r+r[::-1]for r in g]
 b=[r+r[::-1]for r in g[::-1]]
 return t+b
"""
        return None

@handler_decorator
class HandlerFillAlternating(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check alternating rows
        test = np.copy(inp)
        for i in range(0, h, 2):
            test[i, :] = 1
        
        if np.array_equal(test, out):
            return True
        
        # Check alternating columns
        test = np.copy(inp)
        for j in range(0, w, 2):
            test[:, j] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.copy(inp)
        for i in range(0, h, 2):
            test[i, :] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 for i in range(0,len(g),2):
  res[i]=[1]*len(g[0])
 return res
"""
        
        test = np.copy(inp)
        for j in range(0, w, 2):
            test[:, j] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 for i in range(len(g)):
  for j in range(0,len(g[0]),2):
   res[i][j]=1
 return res
"""
        return None

@handler_decorator
class HandlerFillByDistance(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Check Manhattan distance from center
        cx, cy = w // 2, h // 2
        test = np.zeros((h, w), dtype=int)
        
        for i in range(h):
            for j in range(w):
                dist = abs(i - cy) + abs(j - cx)
                test[i,j] = min(dist, 9)
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        cx, cy = w // 2, h // 2
        
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                dist = abs(i - cy) + abs(j - cx)
                test[i,j] = min(dist, 9)
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 cx,cy={cx},{cy}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   d=abs(i-cy)+abs(j-cx)
   row.append(min(d,9))
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerFillSpiral(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Check if output follows spiral pattern
        test = np.zeros((h, w), dtype=int)
        val = 1
        top, bottom, left, right = 0, h-1, 0, w-1
        
        while top <= bottom and left <= right and val <= 9:
            for j in range(left, right+1):
                test[top][j] = val
            top += 1
            
            for i in range(top, bottom+1):
                test[i][right] = val
            right -= 1
            
            if top <= bottom:
                for j in range(right, left-1, -1):
                    test[bottom][j] = val
                bottom -= 1
            
            if left <= right:
                for i in range(bottom, top-1, -1):
                    test[i][left] = val
                left += 1
            
            val = (val % 9) + 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Simplified spiral - would need full implementation
        return None

@handler_decorator
class HandlerFillZigzag(TransformationHandler):
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Check zigzag pattern
        test = np.zeros((h, w), dtype=int)
        val = 0
        for i in range(h):
            if i % 2 == 0:
                for j in range(w):
                    test[i][j] = val % 10
                    val += 1
            else:
                for j in range(w-1, -1, -1):
                    test[i][j] = val % 10
                    val += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        test = np.zeros((h, w), dtype=int)
        val = 0
        for i in range(h):
            if i % 2 == 0:
                for j in range(w):
                    test[i][j] = val % 10
                    val += 1
            else:
                for j in range(w-1, -1, -1):
                    test[i][j] = val % 10
                    val += 1
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 res=[]
 v=0
 for i in range(h):
  row=[0]*w
  if i%2==0:
   for j in range(w):
    row[j]=v%10
    v+=1
  else:
   for j in range(w-1,-1,-1):
    row[j]=v%10
    v+=1
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerConnectedComponentFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        from scipy.ndimage import label
        
        # Label connected components
        labeled, num_features = label(inp > 0)
        
        # Check if each component gets a unique color
        test = np.zeros_like(inp)
        for i in range(1, num_features + 1):
            test[labeled == i] = i % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 l,n=label(np.array(g)>0)
 res=[[0]*len(g[0])for _ in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if l[i,j]>0:
    res[i][j]=l[i,j]%10
 return res
"""
        return None

@handler_decorator
class HandlerBoundaryFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Fill regions bounded by non-zero cells
        for i in range(1, h-1):
            for j in range(1, w-1):
                if inp[i, j] == 0:
                    # Check if surrounded
                    surrounded = True
                    # Check all four directions for boundaries
                    for di in range(i, -1, -1):
                        if inp[di, j] == 0 and di == 0:
                            surrounded = False
                            break
                        if inp[di, j] != 0:
                            break
                    
                    if surrounded:
                        test[i, j] = 5  # Fill with specific color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Boundary fill implementation is complex
        return None

@handler_decorator
class HandlerRegionGrowing(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if non-zero regions grow by one pixel
        from scipy.ndimage import binary_dilation
        
        test = np.copy(inp)
        for color in range(1, 10):
            mask = inp == color
            if np.any(mask):
                dilated = binary_dilation(mask)
                test[dilated & (inp == 0)] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Simple one-pixel growth
        test = np.copy(inp)
        for color in range(1, 10):
            positions = []
            for i in range(h):
                for j in range(w):
                    if inp[i, j] == color:
                        positions.append((i, j))
            
            for i, j in positions:
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] == 0:
                        test[ni, nj] = color
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for c in range(1,10):
  pos=[]
  for i in range(h):
   for j in range(w):
    if g[i][j]==c:
     pos.append((i,j))
  for i,j in pos:
   for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
    ni,nj=i+di,j+dj
    if 0<=ni<h and 0<=nj<w and g[ni][nj]==0:
     res[ni][nj]=c
 return res
"""
        return None

@handler_decorator
class HandlerVoronoiFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Find seed points (non-zero cells)
        seeds = []
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    seeds.append((i, j, inp[i, j]))
        
        # Fill based on nearest seed
        for i in range(h):
            for j in range(w):
                min_dist = float('inf')
                nearest_color = 0
                
                for si, sj, color in seeds:
                    dist = abs(i - si) + abs(j - sj)  # Manhattan distance
                    if dist < min_dist:
                        min_dist = dist
                        nearest_color = color
                
                test[i, j] = nearest_color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            inp = np.array(task_data['train'][0]['input'])
            h, w = inp.shape
            
            return f"""def p(g):
 h,w={h},{w}
 seeds=[]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    seeds.append((i,j,g[i][j]))
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   md=999
   nc=0
   for si,sj,c in seeds:
    d=abs(i-si)+abs(j-sj)
    if d<md:
     md=d
     nc=c
   res[i][j]=nc
 return res
"""
        return None

@handler_decorator
class HandlerCheckerboardFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for checkerboard pattern
        for color1 in range(10):
            for color2 in range(10):
                if color1 != color2:
                    test = np.zeros_like(inp)
                    for i in range(h):
                        for j in range(w):
                            if (i + j) % 2 == 0:
                                test[i, j] = color1
                            else:
                                test[i, j] = color2
                    
                    if np.array_equal(test, out):
                        return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for color1 in range(10):
            for color2 in range(10):
                if color1 != color2:
                    test = np.zeros_like(inp)
                    for i in range(inp.shape[0]):
                        for j in range(inp.shape[1]):
                            if (i + j) % 2 == 0:
                                test[i, j] = color1
                            else:
                                test[i, j] = color2
                    
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return[[{color1}if(i+j)%2==0 else {color2}for j in range(len(g[0]))]for i in range(len(g))]
"""
        return None

@handler_decorator
class HandlerDiagonalFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Fill diagonally with cycling colors
        for i in range(h):
            for j in range(w):
                diag_index = (i + j) % 10
                test[i, j] = diag_index
        
        if np.array_equal(test, out):
            return True
        
        # Fill anti-diagonally
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                diag_index = (i - j + w) % 10
                test[i, j] = diag_index
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                test[i, j] = (i + j) % 10
        
        if np.array_equal(test, out):
            return """def p(g):
 return[[(i+j)%10 for j in range(len(g[0]))]for i in range(len(g))]
"""
        
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                test[i, j] = (i - j + w) % 10
        
        if np.array_equal(test, out):
            return f"""def p(g):
 w={w}
 return[[(i-j+w)%10 for j in range(w)]for i in range(len(g))]
"""
        return None

@handler_decorator
class HandlerConditionalRegionFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Fill regions based on their properties
        from scipy.ndimage import label
        
        labeled, num_features = label(inp > 0)
        test = np.copy(inp)
        
        for i in range(1, num_features + 1):
            region_mask = labeled == i
            region_size = np.sum(region_mask)
            
            # Fill small regions with one color, large with another
            if region_size < 10:
                test[region_mask] = 3
            else:
                test[region_mask] = 7
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Conditional region filling requires complex analysis
        return None

@handler_decorator
class HandlerRegionMerging(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if adjacent regions of same color merge
        h, w = inp.shape
        test = np.copy(inp)
        
        # Simple horizontal merging
        for i in range(h):
            for j in range(w-1):
                if inp[i, j] != 0 and inp[i, j+1] != 0:
                    # Check if there's a gap that should be filled
                    if j+2 < w and inp[i, j] == inp[i, j+2] and inp[i, j+1] == 0:
                        test[i, j+1] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Region merging logic is complex
        return None

@handler_decorator
class HandlerWaveFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Create wave pattern
        for i in range(h):
            for j in range(w):
                wave_val = int(4.5 + 4.5 * np.sin(2 * np.pi * j / w + i * 0.5))
                test[i, j] = wave_val
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                wave_val = int(4.5 + 4.5 * np.sin(2 * np.pi * j / w + i * 0.5))
                test[i, j] = wave_val
        
        if np.array_equal(test, out):
            return f"""def p(g):
 import math
 h,w={h},{w}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   v=int(4.5+4.5*math.sin(2*math.pi*j/w+i*0.5))
   row.append(v)
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerRadialFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        cx, cy = w // 2, h // 2
        max_radius = int(np.sqrt(cx**2 + cy**2))
        test = np.zeros_like(inp)
        
        # Fill based on distance from center
        for i in range(h):
            for j in range(w):
                dist = int(np.sqrt((i - cy)**2 + (j - cx)**2))
                test[i, j] = min(dist * 9 // max_radius, 9)
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        cx, cy = w // 2, h // 2
        max_radius = int(np.sqrt(cx**2 + cy**2))
        
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                dist = int(np.sqrt((i - cy)**2 + (j - cx)**2))
                test[i, j] = min(dist * 9 // max_radius, 9)
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 cx,cy={cx},{cy}
 mr={max_radius}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   d=int(((i-cy)**2+(j-cx)**2)**0.5)
   row.append(min(d*9//mr,9))
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerRecursiveFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check for recursive subdivision pattern
        h, w = inp.shape
        
        if h == w and h > 0 and (h & (h - 1)) == 0:  # Power of 2
            test = np.zeros_like(inp)
            
            def fill_recursive(y1, x1, y2, x2, depth):
                if depth > 3 or y2 <= y1 or x2 <= x1:
                    return
                
                color = depth + 1
                # Fill borders of this region
                test[y1, x1:x2] = color
                test[y2-1, x1:x2] = color
                test[y1:y2, x1] = color
                test[y1:y2, x2-1] = color
                
                # Recursive subdivision
                my, mx = (y1 + y2) // 2, (x1 + x2) // 2
                fill_recursive(y1+1, x1+1, my, mx, depth+1)
                fill_recursive(y1+1, mx, my, x2-1, depth+1)
                fill_recursive(my, x1+1, y2-1, mx, depth+1)
                fill_recursive(my, mx, y2-1, x2-1, depth+1)
            
            fill_recursive(0, 0, h, w, 0)
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Recursive patterns are too complex for compact implementation
        return None

@handler_decorator
class HandlerGradientRegionFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Linear gradient from top-left to bottom-right
        for i in range(h):
            for j in range(w):
                grad_val = (i + j) * 9 // (h + w - 2)
                test[i, j] = grad_val
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                grad_val = (i + j) * 9 // (h + w - 2)
                test[i, j] = grad_val
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 return[[(i+j)*9//(h+w-2)for j in range(w)]for i in range(h)]
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# BORDER AND STRUCTURAL OPERATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerBorderOperations(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if out.shape[0] == inp.shape[0] + 2 and out.shape[1] == inp.shape[1] + 2:
            for border_val in range(1, 10):
                test = np.full((inp.shape[0]+2, inp.shape[1]+2), border_val)
                test[1:-1, 1:-1] = inp
                if np.array_equal(test, out):
                    return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[{border_val}]*(w+2)for _ in range(h+2)]
 for i in range(h):
  for j in range(w):
   res[i+1][j+1]=g[i][j]
 return res
"""
        
        if inp.shape[0] == out.shape[0] + 2 and inp.shape[1] == out.shape[1] + 2:
            if np.array_equal(inp[1:-1, 1:-1], out):
                return """def p(g):
 return[r[1:-1]for r in g[1:-1]]
"""
        return None

@handler_decorator
class HandlerCountObjects(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        labeled, n = label(inp > 0)
        
        if out.shape == (1,1) and out[0,0] == n:
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 _,n=label(np.array(g)>0)
 return[[n]]
"""
        
        if np.all(out == n):
            return f"""def p(g):
 from scipy.ndimage import label
 import numpy as np
 _,n=label(np.array(g)>0)
 return[[n]*len(g[0])for _ in g]
"""
        return None

@handler_decorator
class HandlerExtractRegion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape[0] > out.shape[0] or inp.shape[1] > out.shape[1]:
            return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        ih, iw = inp.shape
        oh, ow = out.shape
        
        # Try different quadrants
        if oh == ih // 2 and ow == iw // 2:
            # Top-left
            if np.array_equal(inp[:oh, :ow], out):
                return f"""def p(g):
 return[r[:{ow}]for r in g[:{oh}]]
"""
            # Top-right
            if np.array_equal(inp[:oh, ow:], out):
                return f"""def p(g):
 return[r[{ow}:]for r in g[:{oh}]]
"""
            # Bottom-left
            if np.array_equal(inp[oh:, :ow], out):
                return f"""def p(g):
 return[r[:{ow}]for r in g[{oh}:]]
"""
            # Bottom-right
            if np.array_equal(inp[oh:, ow:], out):
                return f"""def p(g):
 return[r[{ow}:]for r in g[{oh}:]]
"""
        return None

@handler_decorator
class HandlerAddThickBorder(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check if thick border added
        if out.shape[0] >= inp.shape[0] + 4 and out.shape[1] >= inp.shape[1] + 4:
            return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Try 2-pixel border
        if out.shape[0] == inp.shape[0] + 4 and out.shape[1] == inp.shape[1] + 4:
            for border_val in range(1, 10):
                test = np.full((inp.shape[0]+4, inp.shape[1]+4), border_val)
                test[2:-2, 2:-2] = inp
                if np.array_equal(test, out):
                    return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[{border_val}]*(w+4)for _ in range(h+4)]
 for i in range(h):
  for j in range(w):
   res[i+2][j+2]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerAlternatingBorder(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if out.shape[0] == inp.shape[0] + 2 and out.shape[1] == inp.shape[1] + 2:
            # Check for alternating pattern in border
            test = np.zeros_like(out)
            test[1:-1, 1:-1] = inp
            
            # Check if border has alternating colors
            for color1 in range(1, 10):
                for color2 in range(1, 10):
                    if color1 != color2:
                        # Top and bottom borders
                        for j in range(out.shape[1]):
                            test[0, j] = color1 if j % 2 == 0 else color2
                            test[-1, j] = color1 if j % 2 == 0 else color2
                        
                        # Left and right borders
                        for i in range(1, out.shape[0]-1):
                            test[i, 0] = color1 if i % 2 == 0 else color2
                            test[i, -1] = color1 if i % 2 == 0 else color2
                        
                        if np.array_equal(test, out):
                            return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if self.can_handle(task_data, analysis):
            # Find the alternating colors
            for color1 in range(1, 10):
                for color2 in range(1, 10):
                    if color1 != color2:
                        test = np.zeros_like(out)
                        test[1:-1, 1:-1] = inp
                        
                        for j in range(out.shape[1]):
                            test[0, j] = color1 if j % 2 == 0 else color2
                            test[-1, j] = color1 if j % 2 == 0 else color2
                        
                        for i in range(1, out.shape[0]-1):
                            test[i, 0] = color1 if i % 2 == 0 else color2
                            test[i, -1] = color1 if i % 2 == 0 else color2
                        
                        if np.array_equal(test, out):
                            return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*(w+2)for _ in range(h+2)]
 for i in range(h):
  for j in range(w):
   res[i+1][j+1]=g[i][j]
 for j in range(w+2):
  res[0][j]={color1} if j%2==0 else {color2}
  res[-1][j]={color1} if j%2==0 else {color2}
 for i in range(1,h+1):
  res[i][0]={color1} if i%2==0 else {color2}
  res[i][-1]={color1} if i%2==0 else {color2}
 return res
"""
        return None

@handler_decorator
class HandlerConditionalBorder(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if border is added only to cells meeting certain conditions
        h, w = inp.shape
        test = np.copy(inp)
        
        # Add border to non-zero cells adjacent to zero cells
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    # Check if adjacent to empty space
                    is_border = False
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] == 0:
                            is_border = True
                            break
                    
                    if is_border:
                        test[i, j] = 5  # Change to specific border color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if self.can_handle(task_data, analysis):
            # Find what color is used for borders
            border_color = 5  # Default, would need to detect actual color
            
            return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    b=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]==0:
      b=1;break
    if b:res[i][j]={border_color}
 return res
"""
        return None

@handler_decorator
class HandlerMultiLayerBorder(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check for multiple nested borders
        if out.shape[0] >= inp.shape[0] + 4 and out.shape[1] >= inp.shape[1] + 4:
            return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Try nested borders with different colors
        if out.shape[0] == inp.shape[0] + 4 and out.shape[1] == inp.shape[1] + 4:
            for outer_color in range(1, 10):
                for inner_color in range(1, 10):
                    if outer_color != inner_color:
                        test = np.full(out.shape, outer_color)
                        test[1:-1, 1:-1] = inner_color
                        test[2:-2, 2:-2] = inp
                        
                        if np.array_equal(test, out):
                            return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[{outer_color}]*(w+4)for _ in range(h+4)]
 for i in range(1,h+3):
  for j in range(1,w+3):
   res[i][j]={inner_color}
 for i in range(h):
  for j in range(w):
   res[i+2][j+2]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerBorderPattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if out.shape[0] == inp.shape[0] + 2 and out.shape[1] == inp.shape[1] + 2:
            # Check if border has a specific pattern
            test = np.zeros_like(out)
            test[1:-1, 1:-1] = inp
            
            # Check corners are different from edges
            corner_color = out[0, 0]
            if (out[0, 0] == out[0, -1] == out[-1, 0] == out[-1, -1] and
                corner_color not in [out[0, 1], out[1, 0]]):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if self.can_handle(task_data, analysis):
            corner_color = out[0, 0]
            edge_color = out[0, 1]
            
            return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[{edge_color}]*(w+2)for _ in range(h+2)]
 for i in range(h):
  for j in range(w):
   res[i+1][j+1]=g[i][j]
 res[0][0]={corner_color}
 res[0][-1]={corner_color}
 res[-1][0]={corner_color}
 res[-1][-1]={corner_color}
 return res
"""
        return None

@handler_decorator
class HandlerCropToContent(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check if output is cropped to non-zero content
        if inp.shape[0] > out.shape[0] or inp.shape[1] > out.shape[1]:
            # Find bounding box of non-zero elements in input
            rows, cols = np.where(inp != 0)
            if len(rows) > 0:
                min_row, max_row = rows.min(), rows.max()
                min_col, max_col = cols.min(), cols.max()
                
                cropped = inp[min_row:max_row+1, min_col:max_col+1]
                if np.array_equal(cropped, out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if self.can_handle(task_data, analysis):
            rows, cols = np.where(inp != 0)
            min_row, max_row = rows.min(), rows.max()
            min_col, max_col = cols.min(), cols.max()
            
            return f"""def p(g):
 rows=[i for i in range(len(g))if any(g[i][j]!=0 for j in range(len(g[0])))]
 cols=[j for j in range(len(g[0]))if any(g[i][j]!=0 for i in range(len(g)))]
 mr,xr=min(rows),max(rows)
 mc,xc=min(cols),max(cols)
 return[r[mc:xc+1]for r in g[mr:xr+1]]
"""
        return None

@handler_decorator
class HandlerSelectiveBorder(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if border is added only to specific colors
        test = np.copy(inp)
        
        for target_color in range(1, 10):
            test = np.copy(inp)
            h, w = inp.shape
            
            for i in range(h):
                for j in range(w):
                    if inp[i, j] == target_color:
                        # Add border around this color
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] == 0:
                                test[ni, nj] = 5  # Border color
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for target_color in range(1, 10):
            test = np.copy(inp)
            h, w = inp.shape
            
            for i in range(h):
                for j in range(w):
                    if inp[i, j] == target_color:
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] == 0:
                                test[ni, nj] = 5
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if g[i][j]=={target_color}:
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]==0:
      res[ni][nj]=5
 return res
"""
        return None

@handler_decorator
class HandlerDiagonalBorder(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if out.shape[0] == inp.shape[0] + 2 and out.shape[1] == inp.shape[1] + 2:
            # Check if diagonal cells have special treatment
            test = np.zeros_like(out)
            test[1:-1, 1:-1] = inp
            
            # Set diagonal border pattern
            h, w = out.shape
            for i in range(h):
                for j in range(w):
                    if i == j or i + j == h - 1:
                        if i == 0 or i == h-1 or j == 0 or j == w-1:
                            test[i, j] = 5
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Diagonal border patterns are complex to implement compactly
        return None

@handler_decorator
class HandlerFrameExtraction(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check if output is just the frame/border of input
        if inp.shape == out.shape:
            test = np.zeros_like(inp)
            h, w = inp.shape
            
            # Copy only border elements
            test[0, :] = inp[0, :]
            test[-1, :] = inp[-1, :]
            test[:, 0] = inp[:, 0]
            test[:, -1] = inp[:, -1]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 res[0]=g[0][:]
 res[-1]=g[-1][:]
 for i in range(h):
  res[i][0]=g[i][0]
  res[i][-1]=g[i][-1]
 return res
"""
        return None

@handler_decorator
class HandlerStructuralPadding(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check for asymmetric padding
        if out.shape[0] > inp.shape[0] or out.shape[1] > inp.shape[1]:
            return True
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        ih, iw = inp.shape
        oh, ow = out.shape
        
        # Try different padding configurations
        for pad_val in range(10):
            for top in range(oh - ih + 1):
                for left in range(ow - iw + 1):
                    test = np.full((oh, ow), pad_val)
                    bottom = oh - ih - top
                    right = ow - iw - left
                    
                    if top >= 0 and left >= 0 and bottom >= 0 and right >= 0:
                        test[top:top+ih, left:left+iw] = inp
                        
                        if np.array_equal(test, out):
                            return f"""def p(g):
 res=[[{pad_val}]*{ow} for _ in range({oh})]
 for i in range({ih}):
  for j in range({iw}):
   res[i+{top}][j+{left}]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerBorderGradient(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if out.shape[0] >= inp.shape[0] + 2 and out.shape[1] >= inp.shape[1] + 2:
            # Check if border has gradient pattern
            test = np.zeros_like(out)
            center_h = (out.shape[0] - inp.shape[0]) // 2
            center_w = (out.shape[1] - inp.shape[1]) // 2
            test[center_h:center_h+inp.shape[0], center_w:center_w+inp.shape[1]] = inp
            
            # Apply gradient to border
            h, w = out.shape
            for i in range(h):
                for j in range(w):
                    if (i < center_h or i >= center_h + inp.shape[0] or
                        j < center_w or j >= center_w + inp.shape[1]):
                        # Distance from edge
                        dist = min(i, j, h-1-i, w-1-j)
                        test[i, j] = min(dist + 1, 9)
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Gradient borders require distance calculations
        return None

@handler_decorator
class HandlerBorderMirroring(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if out.shape[0] == inp.shape[0] + 2 and out.shape[1] == inp.shape[1] + 2:
            test = np.zeros_like(out)
            test[1:-1, 1:-1] = inp
            
            # Mirror edges
            test[0, 1:-1] = inp[0, :]  # Top
            test[-1, 1:-1] = inp[-1, :]  # Bottom
            test[1:-1, 0] = inp[:, 0]  # Left
            test[1:-1, -1] = inp[:, -1]  # Right
            
            # Mirror corners
            test[0, 0] = inp[0, 0]
            test[0, -1] = inp[0, -1]
            test[-1, 0] = inp[-1, 0]
            test[-1, -1] = inp[-1, -1]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*(w+2)for _ in range(h+2)]
 for i in range(h):
  for j in range(w):
   res[i+1][j+1]=g[i][j]
 res[0][1:-1]=g[0]
 res[-1][1:-1]=g[-1]
 for i in range(h):
  res[i+1][0]=g[i][0]
  res[i+1][-1]=g[i][-1]
 res[0][0]=g[0][0]
 res[0][-1]=g[0][-1]
 res[-1][0]=g[-1][0]
 res[-1][-1]=g[-1][-1]
 return res
"""
        return None

@handler_decorator
class HandlerStructuralSegmentation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape == out.shape:
            # Check if grid is segmented by adding borders
            h, w = inp.shape
            test = np.copy(inp)
            
            # Add borders at regular intervals
            for interval in [2, 3, 4, 5]:
                test = np.copy(inp)
                
                # Vertical borders
                for j in range(interval, w, interval):
                    test[:, j] = 5
                
                # Horizontal borders
                for i in range(interval, h, interval):
                    test[i, :] = 5
                
                if np.array_equal(test, out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape == out.shape:
            h, w = inp.shape
            
            for interval in [2, 3, 4, 5]:
                test = np.copy(inp)
                
                for j in range(interval, w, interval):
                    test[:, j] = 5
                
                for i in range(interval, h, interval):
                    test[i, :] = 5
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for j in range({interval},w,{interval}):
  for i in range(h):
   res[i][j]=5
 for i in range({interval},h,{interval}):
  res[i]=[5]*w
 return res
"""
        return None

@handler_decorator
class HandlerExpandContract(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check for morphological dilation/erosion
        from scipy.ndimage import binary_dilation, binary_erosion
        
        if inp.shape == out.shape:
            # Try dilation
            struct = np.array([[0,1,0],[1,1,1],[0,1,0]])
            test = binary_dilation(inp > 0, struct).astype(int)
            
            # Preserve original colors
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    if inp[i, j] != 0:
                        test[i, j] = inp[i, j]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Morphological operations require scipy
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# ADVANCED PATTERN RECOGNITION
# --------------------------------------------------------------------------------
@handler_decorator
class Handler1DRasterTransform(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for order in ['row_major', 'col_major', 'zigzag', 'spiral']:
            arr_in = Utilities.to_1d_raster(inp.tolist(), order)
            arr_out = Utilities.to_1d_raster(out.tolist(), order)
            
            if arr_out == arr_in[::-1]:
                return f"""def p(g):
 h,w=len(g),len(g[0])
 arr={''.join([
  f"[g[i][j] for i in range(h) for j in range(w)]" if order=='row_major' else
  f"[g[i][j] for j in range(w) for i in range(h)]" if order=='col_major' else
  f"[g[i][j] for i in range(h-1,-1,-1) for j in range(w-1,-1,-1)]" if order=='row_major_rev' else
  "[]"
 ])}
 arr=arr[::-1]
 res=[[0]*w for _ in range(h)]
 for idx,val in enumerate(arr):
  {"i,j=idx//w,idx%w;res[i][j]=val" if order=='row_major' else
   "j,i=idx//h,idx%h;res[i][j]=val" if order=='col_major' else
   ""}
 return res
"""
            
            if sorted(arr_out) == sorted(arr_in) and arr_out == sorted(arr_in):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 arr=sorted([g[i][j] for i in range(h) for j in range(w)])
 res=[[0]*w for _ in range(h)]
 for idx,val in enumerate(arr):
  i,j=idx//w,idx%w
  res[i][j]=val
 return res
"""
        return None

@handler_decorator
class HandlerColorGradients(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        h, w = out.shape
        
        test = np.zeros((h, w), dtype=int)
        for j in range(w):
            test[:, j] = (j * 9 // (w-1)) if w > 1 else 0
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 return[[(j*9//(w-1))if w>1 else 0 for j in range(w)]for _ in range(h)]
"""
        
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            test[i, :] = (i * 9 // (h-1)) if h > 1 else 0
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 return[[(i*9//(h-1))if h>1 else 0 for j in range(w)]for i in range(h)]
"""
        
        cx, cy = w // 2, h // 2
        max_dist = max(h, w)
        test = np.zeros((h, w), dtype=int)
        
        for i in range(h):
            for j in range(w):
                dist = int(((i-cy)**2 + (j-cx)**2)**0.5)
                test[i,j] = min(dist * 9 // max_dist, 9)
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 cx,cy={cx},{cy}
 md={max_dist}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   d=int(((i-cy)**2+(j-cx)**2)**0.5)
   row.append(min(d*9//md,9))
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerComponentLabeling(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        labeled, n = label(inp > 0)
        
        if np.array_equal(labeled, out):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 l,_=label(np.array(g)>0)
 return l.tolist()
"""
        return None

@handler_decorator
class HandlerConnectSameColor(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        test = np.copy(inp)
        
        for color in range(1, 10):
            positions = np.argwhere(inp == color)
            if len(positions) < 2:
                continue
            
            for i in range(len(positions)):
                for j in range(i+1, len(positions)):
                    y1, x1 = positions[i]
                    y2, x2 = positions[j]
                    
                    if y1 == y2:
                        for x in range(min(x1,x2), max(x1,x2)+1):
                            test[y1,x] = color
                    elif x1 == x2:
                        for y in range(min(y1,y2), max(y1,y2)+1):
                            test[y,x1] = color
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 for c in range(1,10):
  pos=[(i,j)for i in range(len(g))for j in range(len(g[0]))if g[i][j]==c]
  for i in range(len(pos)):
   for j in range(i+1,len(pos)):
    y1,x1=pos[i]
    y2,x2=pos[j]
    if y1==y2:
     for x in range(min(x1,x2),max(x1,x2)+1):
      res[y1][x]=c
    elif x1==x2:
     for y in range(min(y1,y2),max(y1,y2)+1):
      res[y][x1]=c
 return res
"""
        return None

@handler_decorator
class HandlerOutlineObjects(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return analysis.get('has_objects', False)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        edges = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i,j] != 0:
                    is_edge = False
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if not (0 <= ni < h and 0 <= nj < w) or inp[ni,nj] == 0:
                            is_edge = True
                            break
                    if is_edge:
                        edges[i,j] = inp[i,j]
        
        if np.array_equal(edges, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    edge=False
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if not(0<=ni<h and 0<=nj<w)or g[ni][nj]==0:
      edge=True
      break
    if edge:
     res[i][j]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerPatternMatching(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check for 2x2 pattern replacement
        h, w = inp.shape
        if h >= 2 and w >= 2:
            test = np.copy(inp)
            for i in range(h-1):
                for j in range(w-1):
                    # Check for specific 2x2 patterns
                    pattern = inp[i:i+2, j:j+2]
                    if np.all(pattern == 1):  # All ones
                        test[i:i+2, j:j+2] = 2
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.copy(inp)
        for i in range(h-1):
            for j in range(w-1):
                pattern = inp[i:i+2, j:j+2]
                if np.all(pattern == 1):
                    test[i:i+2, j:j+2] = 2
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h-1):
  for j in range(w-1):
   if g[i][j]==1 and g[i][j+1]==1 and g[i+1][j]==1 and g[i+1][j+1]==1:
    res[i][j]=2
    res[i][j+1]=2
    res[i+1][j]=2
    res[i+1][j+1]=2
 return res
"""
        return None

@handler_decorator
class HandlerFindAndReplace(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check for 3x3 pattern finding and replacement
        h, w = inp.shape
        if h >= 3 and w >= 3:
            # Try to find a specific 3x3 pattern
            for i in range(h-2):
                for j in range(w-2):
                    pattern = inp[i:i+3, j:j+3]
                    # Check for cross pattern
                    if (pattern[0,1] != 0 and pattern[1,0] != 0 and 
                        pattern[1,1] != 0 and pattern[1,2] != 0 and 
                        pattern[2,1] != 0):
                        test = np.copy(inp)
                        test[i+1, j+1] = 5  # Mark center
                        if np.array_equal(test[:3, :3], out[:3, :3]):
                            return True
        return False
        
    def generate_solution(self, task_data, analysis):
        # Complex pattern matching implementation would go here
        return None

@handler_decorator
class HandlerConvolution(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Try simple convolution kernels
        kernels = [
            np.array([[0,1,0],[1,1,1],[0,1,0]]),  # Plus
            np.array([[1,1,1],[1,1,1],[1,1,1]]),  # Box
            np.array([[1,0,1],[0,1,0],[1,0,1]]),  # X
        ]
        
        for kernel in kernels:
            test = convolve(inp.astype(float), kernel, mode='constant')
            test = (test % 10).astype(int)
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Complex convolution implementation would need scipy
        return None

@handler_decorator
class HandlerSubpatternDetection(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Detect repeating subpatterns and mark their centers
        for pattern_h in range(2, min(5, h//2+1)):
            for pattern_w in range(2, min(5, w//2+1)):
                test = np.zeros_like(inp)
                
                for i in range(0, h-pattern_h+1, pattern_h):
                    for j in range(0, w-pattern_w+1, pattern_w):
                        subpattern = inp[i:i+pattern_h, j:j+pattern_w]
                        
                        # Check if this subpattern appears elsewhere
                        count = 0
                        for ii in range(0, h-pattern_h+1):
                            for jj in range(0, w-pattern_w+1):
                                if np.array_equal(inp[ii:ii+pattern_h, jj:jj+pattern_w], subpattern):
                                    count += 1
                        
                        if count > 1:
                            # Mark center of repeated pattern
                            center_i = i + pattern_h // 2
                            center_j = j + pattern_w // 2
                            test[center_i, center_j] = 5
                
                if np.array_equal(test, out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Subpattern detection requires complex nested loops
        return None

@handler_decorator
class HandlerFrequencyAnalysis(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Transform based on color frequency in local regions
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                # Analyze 3x3 region around each cell
                region_colors = []
                for di in range(-1, 2):
                    for dj in range(-1, 2):
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w:
                            region_colors.append(inp[ni, nj])
                
                # Set cell to most frequent color in region
                if region_colors:
                    color_counts = Counter(region_colors)
                    most_frequent = max(color_counts.items(), key=lambda x: x[1])[0]
                    test[i, j] = most_frequent
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   rc=[]
   for di in range(-1,2):
    for dj in range(-1,2):
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w:
      rc.append(g[ni][nj])
   cc={}
   for c in rc:cc[c]=cc.get(c,0)+1
   res[i][j]=max(cc.items(),key=lambda x:x[1])[0]
 return res
"""
        return None

@handler_decorator
class HandlerSymmetryDetection(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Mark cells that are part of symmetric patterns
        # Check horizontal symmetry
        for i in range(h):
            for j in range(w//2):
                if inp[i, j] == inp[i, w-1-j] and inp[i, j] != 0:
                    test[i, j] = inp[i, j]
                    test[i, w-1-j] = inp[i, w-1-j]
        
        if np.array_equal(test, out):
            return True
        
        # Check vertical symmetry
        test = np.zeros_like(inp)
        for i in range(h//2):
            for j in range(w):
                if inp[i, j] == inp[h-1-i, j] and inp[i, j] != 0:
                    test[i, j] = inp[i, j]
                    test[h-1-i, j] = inp[h-1-i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w//2):
                if inp[i, j] == inp[i, w-1-j] and inp[i, j] != 0:
                    test[i, j] = inp[i, j]
                    test[i, w-1-j] = inp[i, w-1-j]
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w//2):
   if g[i][j]==g[i][w-1-j]and g[i][j]!=0:
    res[i][j]=g[i][j]
    res[i][w-1-j]=g[i][w-1-j]
 return res
"""
        return None

@handler_decorator
class HandlerPeriodicityDetection(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Detect periodic patterns in rows
        for period in range(2, min(w//2+1, 6)):
            test = np.zeros_like(inp)
            
            for i in range(h):
                is_periodic = True
                for j in range(period, w):
                    if inp[i, j] != inp[i, j % period]:
                        is_periodic = False
                        break
                
                if is_periodic:
                    # Mark the periodic pattern
                    test[i, :period] = inp[i, :period]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for period in range(2, min(w//2+1, 6)):
            test = np.zeros_like(inp)
            
            for i in range(h):
                is_periodic = True
                for j in range(period, w):
                    if inp[i, j] != inp[i, j % period]:
                        is_periodic = False
                        break
                
                if is_periodic:
                    test[i, :period] = inp[i, :period]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w={h},{w}
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  per=1
  for j in range({period},w):
   if g[i][j]!=g[i][j%{period}]:
    per=0;break
  if per:
   for j in range({period}):
    res[i][j]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerCorrelationDetection(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Detect correlated patterns between different regions
        if h >= 4 and w >= 4:
            test = np.zeros_like(inp)
            
            # Check if top-left correlates with bottom-right
            tl = inp[:h//2, :w//2]
            br = inp[h//2:, w//2:]
            
            if tl.shape == br.shape and np.array_equal(tl, br):
                test[:h//2, :w//2] = tl
                test[h//2:h//2+tl.shape[0], w//2:w//2+tl.shape[1]] = br
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Correlation detection requires complex region comparisons
        return None

@handler_decorator
class HandlerMorphologicalGradient(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Calculate morphological gradient (difference between dilation and erosion)
        from scipy.ndimage import binary_dilation, binary_erosion
        
        test = np.zeros_like(inp)
        for color in range(1, 10):
            mask = inp == color
            if np.any(mask):
                struct = np.array([[0,1,0],[1,1,1],[0,1,0]])
                dilated = binary_dilation(mask, struct)
                eroded = binary_erosion(mask, struct)
                gradient = dilated.astype(int) - eroded.astype(int)
                test[gradient == 1] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Morphological operations require scipy
        return None

@handler_decorator
class HandlerPatternPropagation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check if small patterns propagate across the grid
        # Find smallest non-zero pattern
        min_pattern_size = float('inf')
        seed_pattern = None
        seed_pos = None
        
        for i in range(h-1):
            for j in range(w-1):
                if inp[i, j] != 0:
                    # Extract 2x2 pattern
                    pattern = inp[i:i+2, j:j+2]
                    if np.count_nonzero(pattern) > 0:
                        min_pattern_size = 4
                        seed_pattern = pattern
                        seed_pos = (i, j)
                        break
            if seed_pattern is not None:
                break
        
        if seed_pattern is not None:
            test = np.zeros_like(inp)
            # Propagate pattern
            for i in range(0, h, 2):
                for j in range(0, w, 2):
                    if i+2 <= h and j+2 <= w:
                        test[i:i+2, j:j+2] = seed_pattern
            
            if np.array_equal(test[:h, :w], out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Pattern propagation requires identifying and replicating seed patterns
        return None

@handler_decorator
class HandlerHierarchicalPatterns(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Detect hierarchical patterns (patterns within patterns)
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for nested block patterns
        for block_size in [2, 3, 4]:
            if h % block_size == 0 and w % block_size == 0:
                test = np.zeros_like(inp)
                
                # Process each block
                for bi in range(0, h, block_size):
                    for bj in range(0, w, block_size):
                        block = inp[bi:bi+block_size, bj:bj+block_size]
                        
                        # Check if block has a pattern
                        if np.count_nonzero(block) > 0:
                            # Mark block boundaries
                            test[bi, bj:bj+block_size] = 1
                            test[bi+block_size-1, bj:bj+block_size] = 1
                            test[bi:bi+block_size, bj] = 1
                            test[bi:bi+block_size, bj+block_size-1] = 1
                
                if np.array_equal(test, out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        # Hierarchical pattern detection is complex
        return None

@handler_decorator
class HandlerTextureAnalysis(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Analyze texture characteristics
        test = np.zeros_like(inp)
        
        # Calculate local texture measure (variance)
        for i in range(1, h-1):
            for j in range(1, w-1):
                # Get 3x3 neighborhood
                neighborhood = inp[i-1:i+2, j-1:j+2].flatten()
                variance = np.var(neighborhood)
                
                # Mark high-variance regions
                if variance > 2:
                    test[i, j] = 5
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Texture analysis requires statistical computations
        return None

@handler_decorator
class HandlerGeometricTransformChain(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check for chained geometric transformations
        # Try rotation followed by reflection
        test = np.rot90(inp, 1)  # 90 degree rotation
        test = np.fliplr(test)    # Horizontal flip
        
        if np.array_equal(test, out):
            return True
        
        # Try other combinations
        test = np.fliplr(inp)
        test = np.rot90(test, 1)
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        test = np.rot90(inp, 1)
        test = np.fliplr(test)
        
        if np.array_equal(test, out):
            return """def p(g):
 t=[list(r)for r in zip(*g)][::-1]
 return[r[::-1]for r in t]
"""
        
        test = np.fliplr(inp)
        test = np.rot90(test, 1)
        
        if np.array_equal(test, out):
            return """def p(g):
 t=[r[::-1]for r in g]
 return[list(r)for r in zip(*t)][::-1]
"""
        return None

@handler_decorator
class HandlerAdaptiveThresholding(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Apply adaptive thresholding based on local statistics
        for i in range(h):
            for j in range(w):
                # Calculate local mean in 3x3 neighborhood
                local_sum = 0
                count = 0
                
                for di in range(-1, 2):
                    for dj in range(-1, 2):
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w:
                            local_sum += inp[ni, nj]
                            count += 1
                
                local_mean = local_sum // count if count > 0 else 0
                
                # Apply threshold
                if inp[i, j] > local_mean:
                    test[i, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   ls,c=0,0
   for di in range(-1,2):
    for dj in range(-1,2):
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w:
      ls+=g[ni][nj]
      c+=1
   lm=ls//c if c>0 else 0
   if g[i][j]>lm:
    res[i][j]=g[i][j]
 return res
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# SYMMETRY AND STRUCTURAL TRANSFORMATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerMakeSymmetric(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        
        test = np.copy(inp)
        for i in range(h//2):
            test[h-1-i,:] = test[i,:]
        
        if np.array_equal(test, out):
            return """def p(g):
 h=len(g)
 res=[r[:]for r in g]
 for i in range(h//2):
  res[h-1-i]=res[i][:]
 return res
"""
        
        test = np.copy(inp)
        for j in range(w//2):
            test[:,w-1-j] = test[:,j]
        
        if np.array_equal(test, out):
            return """def p(g):
 w=len(g[0])
 res=[r[:]for r in g]
 for i in range(len(g)):
  for j in range(w//2):
   res[i][w-1-j]=res[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerCellularAutomaton(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                color_counts = defaultdict(int)
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w:
                            color_counts[inp[ni,nj]] += 1
                
                most_common = max(color_counts.items(), key=lambda x: x[1])[0]
                test[i,j] = most_common
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   cc={}
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w:
      c=g[ni][nj]
      cc[c]=cc.get(c,0)+1
   mc=max(cc.items(),key=lambda x:x[1])[0]
   res[i][j]=mc
 return res
"""
        return None

@handler_decorator
class HandlerColorWaves(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return True
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        h, w = out.shape
        
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                val = int(4.5 + 4.5 * math.sin(2 * math.pi * j / w))
                test[i,j] = val
        
        if np.array_equal(test, out):
            return f"""def p(g):
 import math
 h,w={h},{w}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   v=int(4.5+4.5*math.sin(2*math.pi*j/w))
   row.append(v)
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerPadToSquare(TransformationHandler):
    def can_handle(self, task_data, analysis):
        shapes = list(analysis['shape_changes'])
        if shapes:
            inp_shape, out_shape = shapes[0]
            return out_shape[0] == out_shape[1] and inp_shape[0] != inp_shape[1]
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if h != w and out.shape[0] == out.shape[1]:
            size = max(h, w)
            if out.shape == (size, size):
                pad_h = (size - h) // 2
                pad_w = (size - w) // 2
                
                test = np.zeros((size, size), dtype=int)
                test[pad_h:pad_h+h, pad_w:pad_w+w] = inp
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 h,w=len(g),len(g[0])
 s=max(h,w)
 ph,pw=(s-h)//2,(s-w)//2
 res=[[0]*s for _ in range(s)]
 for i in range(h):
  for j in range(w):
   res[ph+i][pw+j]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerRotationalSymmetry(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape or inp.shape[0] != inp.shape[1]:
            return False
        
        # Check 4-fold rotational symmetry
        h, w = inp.shape
        test = np.copy(inp)
        
        for i in range(h//2):
            for j in range(w//2):
                val = inp[i, j]
                test[j, w-1-i] = val  # 90 degree rotation
                test[h-1-i, w-1-j] = val  # 180 degree rotation
                test[h-1-j, i] = val  # 270 degree rotation
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h = len(task_data['train'][0]['input'])
            return f"""def p(g):
 h={h}
 res=[r[:]for r in g]
 for i in range(h//2):
  for j in range(h//2):
   v=g[i][j]
   res[j][h-1-i]=v
   res[h-1-i][h-1-j]=v
   res[h-1-j][i]=v
 return res
"""
        return None

@handler_decorator
class HandlerMakeSymmetricDiagonal(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape or inp.shape[0] != inp.shape[1]:
            return False
        
        # Make symmetric across main diagonal
        h = inp.shape[0]
        test = np.copy(inp)
        for i in range(h):
            for j in range(i+1, h):
                test[j, i] = test[i, j]
        
        if np.array_equal(test, out):
            return True
        
        # Make symmetric across anti-diagonal
        test = np.copy(inp)
        for i in range(h):
            for j in range(h-1-i):
                test[h-1-j, h-1-i] = test[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h = inp.shape[0]
        
        test = np.copy(inp)
        for i in range(h):
            for j in range(i+1, h):
                test[j, i] = test[i, j]
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h={h}
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(i+1,h):
   res[j][i]=res[i][j]
 return res
"""
        
        test = np.copy(inp)
        for i in range(h):
            for j in range(h-1-i):
                test[h-1-j, h-1-i] = test[i, j]
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h={h}
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(h-1-i):
   res[h-1-j][h-1-i]=res[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerLifeGame(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Simple life-like rules
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                # Count live neighbors
                neighbors = 0
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        if di == 0 and dj == 0:
                            continue
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] != 0:
                            neighbors += 1
                
                # Apply rules
                if inp[i, j] != 0:  # Live cell
                    if neighbors in [2, 3]:
                        test[i, j] = inp[i, j]
                else:  # Dead cell
                    if neighbors == 3:
                        test[i, j] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   n=0
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     if di==0 and dj==0:continue
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
      n+=1
   if g[i][j]!=0:
    if n in[2,3]:res[i][j]=g[i][j]
   else:
    if n==3:res[i][j]=1
 return res
"""
        return None

@handler_decorator
class HandlerKaleidoscope(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape or inp.shape[0] != inp.shape[1]:
            return False
        
        h, w = inp.shape
        
        # Check for 8-fold symmetry (kaleidoscope effect)
        if h % 2 == 0 and w % 2 == 0:
            test = np.copy(inp)
            ch, cw = h // 2, w // 2
            
            # Copy one octant to all others
            for i in range(ch):
                for j in range(i+1):
                    val = inp[i, j]
                    # Mirror across diagonals and axes
                    test[j, i] = val
                    test[i, w-1-j] = val
                    test[j, w-1-i] = val
                    test[h-1-i, j] = val
                    test[h-1-j, i] = val
                    test[h-1-i, w-1-j] = val
                    test[h-1-j, w-1-i] = val
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if self.can_handle(task_data, analysis) and h % 2 == 0:
            ch = h // 2
            return f"""def p(g):
 h={h}
 res=[r[:]for r in g]
 for i in range({ch}):
  for j in range(i+1):
   v=g[i][j]
   res[j][i]=v
   res[i][h-1-j]=v
   res[j][h-1-i]=v
   res[h-1-i][j]=v
   res[h-1-j][i]=v
   res[h-1-i][h-1-j]=v
   res[h-1-j][h-1-i]=v
 return res
"""
        return None

@handler_decorator
class HandlerPointSymmetry(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for point symmetry around center
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                test[h-1-i, w-1-j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h, w = inp.shape
            return f"""def p(g):
 h,w={h},{w}
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   res[h-1-i][w-1-j]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerGlideReflection(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for glide reflection (reflection + translation)
        for shift in range(1, min(4, w)):
            test = np.copy(inp)
            # Horizontal glide reflection
            for i in range(h):
                for j in range(w):
                    if j + shift < w:
                        test[i, j + shift] = inp[h-1-i, j]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for shift in range(1, min(4, w)):
            test = np.copy(inp)
            for i in range(h):
                for j in range(w):
                    if j + shift < w:
                        test[i, j + shift] = inp[h-1-i, j]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w={h},{w}
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if j+{shift}<w:
    res[i][j+{shift}]=g[h-1-i][j]
   else:
    res[i][j]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerStructuralShear(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check for shearing transformation
        h, w = inp.shape
        
        for shear_amount in range(1, min(4, w)):
            test = np.zeros_like(out)
            for i in range(h):
                for j in range(w):
                    new_j = (j + i * shear_amount) % w
                    test[i, new_j] = inp[i, j]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for shear_amount in range(1, min(4, w)):
            test = np.zeros_like(out)
            for i in range(h):
                for j in range(w):
                    new_j = (j + i * shear_amount) % w
                    test[i, new_j] = inp[i, j]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w={h},{w}
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   nj=(j+i*{shear_amount})%w
   res[i][nj]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerStructuralTwist(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        center_i, center_j = h // 2, w // 2
        
        # Check for twisting around center
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                # Calculate distance from center
                di, dj = i - center_i, j - center_j
                dist = (di*di + dj*dj) ** 0.5
                
                # Apply twist based on distance
                if dist > 0:
                    angle = dist * 0.2  # Twist amount
                    cos_a = math.cos(angle)
                    sin_a = math.sin(angle)
                    
                    new_di = int(di * cos_a - dj * sin_a)
                    new_dj = int(di * sin_a + dj * cos_a)
                    
                    new_i = center_i + new_di
                    new_j = center_j + new_dj
                    
                    if 0 <= new_i < h and 0 <= new_j < w:
                        test[new_i, new_j] = inp[i, j]
        
        # This is a complex transformation to verify exactly
        return False
        
    def generate_solution(self, task_data, analysis):
        # Twist transformation is too complex for compact implementation
        return None

@handler_decorator
class HandlerPartialSymmetry(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check if only part of the grid is made symmetric
        # Upper half mirrored to lower half
        if h % 2 == 0:
            test = np.copy(inp)
            test[h//2:, :] = np.flipud(test[:h//2, :])
            if np.array_equal(test, out):
                return True
        
        # Left half mirrored to right half
        if w % 2 == 0:
            test = np.copy(inp)
            test[:, w//2:] = np.fliplr(test[:, :w//2])
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if h % 2 == 0:
            test = np.copy(inp)
            test[h//2:, :] = np.flipud(test[:h//2, :])
            if np.array_equal(test, out):
                h2 = h // 2
                return f"""def p(g):
 h={h}
 res=[r[:]for r in g]
 for i in range({h2}):
  res[h-1-i]=res[i][:]
 return res
"""
        
        if w % 2 == 0:
            test = np.copy(inp)
            test[:, w//2:] = np.fliplr(test[:, :w//2])
            if np.array_equal(test, out):
                w2 = w // 2
                return f"""def p(g):
 w={w}
 res=[r[:]for r in g]
 for i in range(len(g)):
  for j in range({w2}):
   res[i][w-1-j]=res[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerSymmetryBreaking(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if input is symmetric but output breaks symmetry
        h, w = inp.shape
        
        # Check if input has horizontal symmetry
        is_h_symmetric = np.array_equal(inp[:h//2, :], np.flipud(inp[h//2:, :]))
        
        if is_h_symmetric:
            # Apply small perturbation to break symmetry
            test = np.copy(inp)
            test[0, 0] = (inp[0, 0] + 1) % 10
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if self.can_handle(task_data, analysis):
            return """def p(g):
 res=[r[:]for r in g]
 res[0][0]=(g[0][0]+1)%10
 return res
"""
        return None

@handler_decorator
class HandlerFractalPattern(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check for simple fractal patterns (Sierpinski-like)
        h, w = inp.shape
        
        if h == w and h > 0 and (h & (h - 1)) == 0:  # Power of 2
            # Check for Sierpinski triangle pattern
            test = np.zeros_like(out)
            for i in range(h):
                for j in range(w):
                    if (i & j) == j:
                        test[i, j] = 1
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if h == w and h > 0 and (h & (h - 1)) == 0:
            test = np.zeros_like(out)
            for i in range(h):
                for j in range(w):
                    if (i & j) == j:
                        test[i, j] = 1
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h={h}
 return[[1 if(i&j)==j else 0 for j in range(h)]for i in range(h)]
"""
        return None

@handler_decorator
class HandlerPeriodicBoundary(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check if boundaries wrap around (periodic)
        test = np.copy(inp)
        
        # Copy left edge to right edge
        test[:, -1] = inp[:, 0]
        # Copy right edge to left edge
        test[:, 0] = inp[:, -1]
        # Copy top edge to bottom edge
        test[-1, :] = inp[0, :]
        # Copy bottom edge to top edge
        test[0, :] = inp[-1, :]
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  res[i][0]=g[i][-1]
  res[i][-1]=g[i][0]
 res[0]=g[-1][:]
 res[-1]=g[0][:]
 return res
"""
        return None

@handler_decorator
class HandlerStructuralInversion(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for inside-out transformation
        test = np.zeros_like(inp)
        ch, cw = h // 2, w // 2
        
        for i in range(h):
            for j in range(w):
                # Map inner region to outer and vice versa
                di, dj = i - ch, j - cw
                if abs(di) <= ch // 2 and abs(dj) <= cw // 2:
                    # Inner to outer
                    new_i = ch + di * 2
                    new_j = cw + dj * 2
                    if 0 <= new_i < h and 0 <= new_j < w:
                        test[new_i, new_j] = inp[i, j]
                else:
                    # Outer to inner
                    new_i = ch + di // 2
                    new_j = cw + dj // 2
                    if 0 <= new_i < h and 0 <= new_j < w:
                        test[new_i, new_j] = inp[i, j]
        
        # This is a complex transformation
        return False
        
    def generate_solution(self, task_data, analysis):
        # Structural inversion is too complex for compact implementation
        return None

@handler_decorator
class HandlerReflectionComposite(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check for composite reflections (multiple axes)
        test = np.copy(inp)
        
        # Apply both horizontal and vertical reflection
        test = np.flipud(test)
        test = np.fliplr(test)
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 return[r[::-1]for r in g[::-1]]
"""
        return None

@handler_decorator
class HandlerTessellation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check if input pattern is tessellated
        ih, iw = inp.shape
        oh, ow = out.shape
        
        if oh >= ih and ow >= iw and oh % ih == 0 and ow % iw == 0:
            # Check if output is tessellation of input
            test = np.tile(inp, (oh // ih, ow // iw))
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        ih, iw = inp.shape
        oh, ow = out.shape
        
        if oh >= ih and ow >= iw and oh % ih == 0 and ow % iw == 0:
            test = np.tile(inp, (oh // ih, ow // iw))
            if np.array_equal(test, out):
                reps_h = oh // ih
                reps_w = ow // iw
                return f"""def p(g):
 res=[]
 for _ in range({reps_h}):
  for r in g:
   res.append(r*{reps_w})
 return res
"""
        return None

@handler_decorator
class HandlerStructuralDeformation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check for wave-like deformation
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                # Apply sine wave deformation
                offset = int(2 * math.sin(2 * math.pi * i / h))
                new_j = (j + offset) % w
                test[i, new_j] = inp[i, j]
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                offset = int(2 * math.sin(2 * math.pi * i / h))
                new_j = (j + offset) % w
                test[i, new_j] = inp[i, j]
        
        if np.array_equal(test, out):
            return f"""def p(g):
 import math
 h,w={h},{w}
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   o=int(2*math.sin(2*math.pi*i/h))
   nj=(j+o)%w
   res[i][nj]=g[i][j]
 return res
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# MASKING AND OVERLAY OPERATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerColorMask(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if one color acts as a mask
        for mask_color in range(1, 10):
            mask = inp == mask_color
            if np.any(mask):
                test = np.where(mask, 0, inp)
                if np.array_equal(test, out):
                    return True
                
                for replace_color in range(1, 10):
                    test = np.where(mask, replace_color, inp)
                    if np.array_equal(test, out):
                        return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for mask_color in range(1, 10):
            mask = inp == mask_color
            if np.any(mask):
                test = np.where(mask, 0, inp)
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[[0 if x=={mask_color} else x for x in r]for r in g]
"""
                
                for replace_color in range(1, 10):
                    test = np.where(mask, replace_color, inp)
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return[[{replace_color} if x=={mask_color} else x for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerOverlayNonZero(TransformationHandler):
    def can_handle(self, task_data, analysis):
        return len(task_data['train']) >= 2
        
    def generate_solution(self, task_data, analysis):
        if len(task_data['train']) < 2:
            return None
        
        # Check if output is overlay of multiple inputs
        inp1 = np.array(task_data['train'][0]['input'])
        inp2 = np.array(task_data['train'][1]['input']) if len(task_data['train']) > 1 else inp1
        out = np.array(task_data['train'][0]['output'])
        
        if inp1.shape == out.shape and inp2.shape == out.shape:
            test = np.where(inp2 != 0, inp2, inp1)
            if np.array_equal(test, out):
                return """def p(g):
 # This assumes overlay with a fixed pattern
 return g
"""
        return None

@handler_decorator
class HandlerInvertMask(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if zeros become ones and non-zeros become zeros
        test = np.where(inp == 0, 1, 0)
        if np.array_equal(test, out):
            return True
        
        # Check if specific color inverts the mask
        for color in range(1, 10):
            test = np.where(inp == 0, color, 0)
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        test = np.where(inp == 0, 1, 0)
        if np.array_equal(test, out):
            return """def p(g):
 return[[1 if x==0 else 0 for x in r]for r in g]
"""
        
        for color in range(1, 10):
            test = np.where(inp == 0, color, 0)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[{color} if x==0 else 0 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerCombineLayers(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape[0] != out.shape[0] or inp.shape[1] % 2 != 0:
            return False
        
        # Check if input is two side-by-side grids combined
        half_width = inp.shape[1] // 2
        left = inp[:, :half_width]
        right = inp[:, half_width:]
        
        if out.shape[1] == half_width:
            # Try various combination methods
            test = np.maximum(left, right)
            if np.array_equal(test, out):
                return True
            
            test = np.minimum(left, right)
            if np.array_equal(test, out):
                return True
            
            test = left + right
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        half_width = inp.shape[1] // 2
        left = inp[:, :half_width]
        right = inp[:, half_width:]
        
        test = np.maximum(left, right)
        if np.array_equal(test, out):
            return f"""def p(g):
 w=len(g[0])//2
 return[[max(r[j],r[j+w])for j in range(w)]for r in g]
"""
        
        test = np.minimum(left, right)
        if np.array_equal(test, out):
            return f"""def p(g):
 w=len(g[0])//2
 return[[min(r[j],r[j+w])for j in range(w)]for r in g]
"""
        
        test = (left + right) % 10
        if np.array_equal(test, out):
            return f"""def p(g):
 w=len(g[0])//2
 return[[(r[j]+r[j+w])%10 for j in range(w)]for r in g]
"""
        return None

@handler_decorator
class HandlerBitwiseMaskOperations(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # For grids with exactly two distinct colors, try bitwise operations
        unique_values = np.unique(inp)
        if len(unique_values) == 2:
            # Create binary representation
            binary_inp = (inp == unique_values[1]).astype(int)
            
            # Try various transformations
            for shift in range(1, min(4, inp.shape[1])):
                # Bitwise AND with shifted version
                shifted = np.roll(binary_inp, shift, axis=1)
                test_binary = binary_inp & shifted
                test = np.where(test_binary, unique_values[1], unique_values[0])
                if np.array_equal(test, out):
                    return True
                
                # Bitwise OR with shifted version
                test_binary = binary_inp | shifted
                test = np.where(test_binary, unique_values[1], unique_values[0])
                if np.array_equal(test, out):
                    return True
                
                # Bitwise XOR with shifted version
                test_binary = binary_inp ^ shifted
                test = np.where(test_binary, unique_values[1], unique_values[0])
                if np.array_equal(test, out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        unique_values = np.unique(inp)
        
        if len(unique_values) == 2:
            v0, v1 = int(unique_values[0]), int(unique_values[1])
            binary_inp = (inp == v1).astype(int)
            
            for shift in range(1, min(4, inp.shape[1])):
                shifted = np.roll(binary_inp, shift, axis=1)
                
                test_binary = binary_inp & shifted
                test = np.where(test_binary, v1, v0)
                if np.array_equal(test, out):
                    return f"""def p(g):
 b=[[1 if x=={v1} else 0 for x in r]for r in g]
 return[[{v1} if b[i][j]and b[i][(j-{shift})%len(r)]else {v0} for j,x in enumerate(r)]for i,r in enumerate(g)]
"""
                
                test_binary = binary_inp | shifted
                test = np.where(test_binary, v1, v0)
                if np.array_equal(test, out):
                    return f"""def p(g):
 b=[[1 if x=={v1} else 0 for x in r]for r in g]
 return[[{v1} if b[i][j]or b[i][(j-{shift})%len(r)]else {v0} for j,x in enumerate(r)]for i,r in enumerate(g)]
"""
        return None

@handler_decorator
class HandlerSelectiveMask(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if masking is applied selectively based on patterns
        h, w = inp.shape
        
        # Try checkerboard masking
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                if (i + j) % 2 == 0:
                    test[i, j] = 0
        if np.array_equal(test, out):
            return True
        
        # Try stripe masking
        test = np.copy(inp)
        for i in range(0, h, 2):
            test[i, :] = 0
        if np.array_equal(test, out):
            return True
        
        test = np.copy(inp)
        for j in range(0, w, 2):
            test[:, j] = 0
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                if (i + j) % 2 == 0:
                    test[i, j] = 0
        if np.array_equal(test, out):
            return """def p(g):
 return[[0 if(i+j)%2==0 else x for j,x in enumerate(r)]for i,r in enumerate(g)]
"""
        
        test = np.copy(inp)
        for i in range(0, h, 2):
            test[i, :] = 0
        if np.array_equal(test, out):
            return """def p(g):
 return[[0]*len(r)if i%2==0 else r for i,r in enumerate(g)]
"""
        
        test = np.copy(inp)
        for j in range(0, w, 2):
            test[:, j] = 0
        if np.array_equal(test, out):
            return """def p(g):
 return[[0 if j%2==0 else x for j,x in enumerate(r)]for r in g]
"""
        return None

@handler_decorator
class HandlerThresholdMask(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Apply threshold masking
        for threshold in range(1, 10):
            test = np.where(inp >= threshold, inp, 0)
            if np.array_equal(test, out):
                return True
            
            test = np.where(inp <= threshold, inp, 0)
            if np.array_equal(test, out):
                return True
            
            test = np.where(inp >= threshold, 1, 0)
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for threshold in range(1, 10):
            test = np.where(inp >= threshold, inp, 0)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[x if x>={threshold} else 0 for x in r]for r in g]
"""
            
            test = np.where(inp <= threshold, inp, 0)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[x if x<={threshold} else 0 for x in r]for r in g]
"""
            
            test = np.where(inp >= threshold, 1, 0)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[1 if x>={threshold} else 0 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerRegionMask(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check if masking is applied to specific regions
        # Upper half mask
        test = np.copy(inp)
        test[:h//2, :] = 0
        if np.array_equal(test, out):
            return True
        
        # Lower half mask
        test = np.copy(inp)
        test[h//2:, :] = 0
        if np.array_equal(test, out):
            return True
        
        # Left half mask
        test = np.copy(inp)
        test[:, :w//2] = 0
        if np.array_equal(test, out):
            return True
        
        # Right half mask
        test = np.copy(inp)
        test[:, w//2:] = 0
        if np.array_equal(test, out):
            return True
        
        # Center mask
        test = np.copy(inp)
        h_start, h_end = h//4, 3*h//4
        w_start, w_end = w//4, 3*w//4
        test[h_start:h_end, w_start:w_end] = 0
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.copy(inp)
        test[:h//2, :] = 0
        if np.array_equal(test, out):
            h2 = h // 2
            return f"""def p(g):
 return[[0]*len(r)if i<{h2} else r for i,r in enumerate(g)]
"""
        
        test = np.copy(inp)
        test[h//2:, :] = 0
        if np.array_equal(test, out):
            h2 = h // 2
            return f"""def p(g):
 return[[0]*len(r)if i>={h2} else r for i,r in enumerate(g)]
"""
        
        test = np.copy(inp)
        test[:, :w//2] = 0
        if np.array_equal(test, out):
            w2 = w // 2
            return f"""def p(g):
 return[[0 if j<{w2} else x for j,x in enumerate(r)]for r in g]
"""
        
        test = np.copy(inp)
        test[:, w//2:] = 0
        if np.array_equal(test, out):
            w2 = w // 2
            return f"""def p(g):
 return[[0 if j>={w2} else x for j,x in enumerate(r)]for r in g]
"""
        return None

@handler_decorator
class HandlerComplementaryMask(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if output shows complementary colors
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    test[i, j] = 10 - inp[i, j]
        
        if np.array_equal(test, out):
            return True
        
        # Check modular complement
        for mod in range(2, 10):
            test = np.where(inp != 0, (mod - inp) % mod, 0)
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    test[i, j] = 10 - inp[i, j]
        
        if np.array_equal(test, out):
            return """def p(g):
 return[[10-x if x!=0 else 0 for x in r]for r in g]
"""
        
        for mod in range(2, 10):
            test = np.where(inp != 0, (mod - inp) % mod, 0)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[({mod}-x)%{mod} if x!=0 else 0 for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerMultiColorMask(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if multiple colors act as masks
        unique_inp = np.unique(inp)
        unique_out = np.unique(out)
        
        # Find colors that are masked out
        masked_colors = set(unique_inp) - set(unique_out)
        
        if masked_colors:
            test = np.copy(inp)
            for color in masked_colors:
                test[test == color] = 0
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        unique_inp = np.unique(inp)
        unique_out = np.unique(out)
        masked_colors = sorted(set(unique_inp) - set(unique_out))
        
        if masked_colors:
            test = np.copy(inp)
            for color in masked_colors:
                test[test == color] = 0
            
            if np.array_equal(test, out):
                if len(masked_colors) == 1:
                    return f"""def p(g):
 return[[0 if x=={masked_colors[0]} else x for x in r]for r in g]
"""
                elif len(masked_colors) == 2:
                    return f"""def p(g):
 return[[0 if x in[{masked_colors[0]},{masked_colors[1]}]else x for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerConditionalOverlay(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if overlay is applied conditionally
        h, w = inp.shape
        
        # Try overlaying based on neighbor conditions
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    # Count neighbors
                    neighbor_count = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] != 0:
                            neighbor_count += 1
                    
                    # Apply overlay if isolated
                    if neighbor_count == 0:
                        test[i, j] = 5  # Overlay with specific color
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    neighbor_count = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] != 0:
                            neighbor_count += 1
                    
                    if neighbor_count == 0:
                        test[i, j] = 5
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    n=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
      n+=1
    if n==0:res[i][j]=5
 return res
"""
        return None

@handler_decorator
class HandlerPatternBasedMask(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if masking follows a pattern
        h, w = inp.shape
        
        # Diagonal mask pattern
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                if i == j:  # Main diagonal
                    test[i, j] = 0
        if np.array_equal(test, out):
            return True
        
        # Anti-diagonal mask
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                if i + j == min(h, w) - 1:
                    test[i, j] = 0
        if np.array_equal(test, out):
            return True
        
        # Border mask
        test = np.copy(inp)
        test[0, :] = 0
        test[-1, :] = 0
        test[:, 0] = 0
        test[:, -1] = 0
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                if i == j:
                    test[i, j] = 0
        if np.array_equal(test, out):
            return """def p(g):
 return[[0 if i==j else x for j,x in enumerate(r)]for i,r in enumerate(g)]
"""
        
        test = np.copy(inp)
        for i in range(h):
            for j in range(w):
                if i + j == min(h, w) - 1:
                    test[i, j] = 0
        if np.array_equal(test, out):
            target = min(h, w) - 1
            return f"""def p(g):
 return[[0 if i+j=={target} else x for j,x in enumerate(r)]for i,r in enumerate(g)]
"""
        
        test = np.copy(inp)
        test[0, :] = 0
        test[-1, :] = 0
        test[:, 0] = 0
        test[:, -1] = 0
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 return[[0 if i==0 or i==h-1 or j==0 or j==w-1 else x for j,x in enumerate(r)]for i,r in enumerate(g)]
"""
        return None

@handler_decorator
class HandlerStackedOverlay(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check if input has stacked layers vertically
        if inp.shape[0] % 2 == 0 and out.shape == (inp.shape[0] // 2, inp.shape[1]):
            half_height = inp.shape[0] // 2
            top = inp[:half_height, :]
            bottom = inp[half_height:, :]
            
            # Try various overlay methods
            test = np.where(bottom != 0, bottom, top)
            if np.array_equal(test, out):
                return True
            
            test = np.where(top != 0, top, bottom)
            if np.array_equal(test, out):
                return True
            
            test = np.maximum(top, bottom)
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape[0] % 2 == 0 and out.shape == (inp.shape[0] // 2, inp.shape[1]):
            half_height = inp.shape[0] // 2
            top = inp[:half_height, :]
            bottom = inp[half_height:, :]
            
            test = np.where(bottom != 0, bottom, top)
            if np.array_equal(test, out):
                h2 = half_height
                return f"""def p(g):
 return[[g[i+{h2}][j]if g[i+{h2}][j]!=0 else g[i][j]for j in range(len(g[0]))]for i in range({h2})]
"""
            
            test = np.where(top != 0, top, bottom)
            if np.array_equal(test, out):
                h2 = half_height
                return f"""def p(g):
 return[[g[i][j]if g[i][j]!=0 else g[i+{h2}][j]for j in range(len(g[0]))]for i in range({h2})]
"""
            
            test = np.maximum(top, bottom)
            if np.array_equal(test, out):
                h2 = half_height
                return f"""def p(g):
 return[[max(g[i][j],g[i+{h2}][j])for j in range(len(g[0]))]for i in range({h2})]
"""
        return None

@handler_decorator
class HandlerAlphaMask(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if certain colors act as transparency levels
        # Color 1 might be 10% opacity, 2 might be 20%, etc.
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    # Apply transparency-like effect
                    test[i, j] = min(inp[i, j], 5)  # Cap at mid-value
        
        if np.array_equal(test, out):
            return True
        
        # Try blending based on value
        test = np.where(inp > 5, 5, inp)
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    test[i, j] = min(inp[i, j], 5)
        
        if np.array_equal(test, out):
            return """def p(g):
 return[[min(x,5)for x in r]for r in g]
"""
        
        test = np.where(inp > 5, 5, inp)
        if np.array_equal(test, out):
            return """def p(g):
 return[[5 if x>5 else x for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerDifferenceMask(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check if output shows differences between adjacent cells
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                if j < w - 1:
                    diff = abs(inp[i, j] - inp[i, j+1])
                    if diff > 0:
                        test[i, j] = diff
        
        if np.array_equal(test, out):
            return True
        
        # Vertical differences
        test = np.zeros_like(inp)
        for i in range(h - 1):
            for j in range(w):
                diff = abs(inp[i, j] - inp[i+1, j])
                if diff > 0:
                    test[i, j] = diff
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                if j < w - 1:
                    diff = abs(inp[i, j] - inp[i, j+1])
                    if diff > 0:
                        test[i, j] = diff
        
        if np.array_equal(test, out):
            return """def p(g):
 w=len(g[0])
 return[[abs(g[i][j]-g[i][j+1])if j<w-1 else 0 for j in range(w)]for i in range(len(g))]
"""
        
        test = np.zeros_like(inp)
        for i in range(h - 1):
            for j in range(w):
                diff = abs(inp[i, j] - inp[i+1, j])
                if diff > 0:
                    test[i, j] = diff
        
        if np.array_equal(test, out):
            return """def p(g):
 h=len(g)
 return[[abs(g[i][j]-g[i+1][j])if i<h-1 else 0 for j in range(len(g[0]))]for i in range(h)]
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# CONDITIONAL AND NEIGHBOR-BASED OPERATIONS
# --------------------------------------------------------------------------------
@handler_decorator
class HandlerConditionalFill(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Check if cells with certain neighbor patterns get filled
        for i in range(1, h-1):
            for j in range(1, w-1):
                if inp[i,j] == 0:
                    # Count non-zero neighbors
                    neighbors = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        if inp[i+di, j+dj] != 0:
                            neighbors += 1
                    
                    if neighbors >= 2:
                        test[i,j] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(1,h-1):
  for j in range(1,w-1):
   if g[i][j]==0:
    n=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     if g[i+di][j+dj]!=0:
      n+=1
    if n>=2:
     res[i][j]=1
 return res
"""
        return None

@handler_decorator
class HandlerColorPropagation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if colors propagate in a specific direction
        h, w = inp.shape
        test = np.copy(inp)
        
        # Propagate downward
        for j in range(w):
            for i in range(1, h):
                if test[i,j] == 0 and test[i-1,j] != 0:
                    test[i,j] = test[i-1,j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for j in range(w):
  for i in range(1,h):
   if res[i][j]==0 and res[i-1][j]!=0:
    res[i][j]=res[i-1][j]
 return res
"""
        return None

@handler_decorator
class HandlerNeighborAverage(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                total = 0
                count = 0
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0), (0,0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w:
                        total += inp[ni, nj]
                        count += 1
                test[i, j] = total // count
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   t,c=0,0
   for di,dj in[(0,1),(1,0),(0,-1),(-1,0),(0,0)]:
    ni,nj=i+di,j+dj
    if 0<=ni<h and 0<=nj<w:
     t+=g[ni][nj]
     c+=1
   res[i][j]=t//c
 return res
"""
        return None

@handler_decorator
class HandlerConditionalColorChange(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if cells change color based on neighbor conditions
        h, w = inp.shape
        test = np.copy(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    # Check if surrounded by same color
                    surrounded = True
                    color = inp[i, j]
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w:
                            if inp[ni, nj] != color:
                                surrounded = False
                                break
                        else:
                            surrounded = False
                            break
                    
                    if surrounded:
                        test[i, j] = (color + 1) % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    s=1
    c=g[i][j]
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w:
      if g[ni][nj]!=c:s=0;break
     else:s=0;break
    if s:res[i][j]=(c+1)%10
 return res
"""
        return None

@handler_decorator
class HandlerMajorityVote(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Each cell takes the most common value among its neighbors
        for i in range(h):
            for j in range(w):
                neighbor_values = []
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w:
                            neighbor_values.append(inp[ni, nj])
                
                if neighbor_values:
                    # Find most common value
                    value_counts = Counter(neighbor_values)
                    most_common = max(value_counts.items(), key=lambda x: x[1])[0]
                    test[i, j] = most_common
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   nv=[]
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w:
      nv.append(g[ni][nj])
   vc={}
   for v in nv:vc[v]=vc.get(v,0)+1
   res[i][j]=max(vc.items(),key=lambda x:x[1])[0]
 return res
"""
        return None

@handler_decorator
class HandlerConditionalBorder(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Add border to cells adjacent to specific colors
        for i in range(h):
            for j in range(w):
                if inp[i, j] == 0:
                    # Check if adjacent to non-zero
                    adjacent = False
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] != 0:
                            adjacent = True
                            break
                    
                    if adjacent:
                        test[i, j] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if g[i][j]==0:
    a=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
      a=1;break
    if a:res[i][j]=1
 return res
"""
        return None

@handler_decorator
class HandlerNeighborSum(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Each cell becomes sum of its neighbors
        for i in range(h):
            for j in range(w):
                total = 0
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w:
                        total += inp[ni, nj]
                test[i, j] = total % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   t=0
   for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
    ni,nj=i+di,j+dj
    if 0<=ni<h and 0<=nj<w:
     t+=g[ni][nj]
   res[i][j]=t%10
 return res
"""
        return None

@handler_decorator
class HandlerNeighborMax(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Each cell becomes max of its neighbors including itself
        for i in range(h):
            for j in range(w):
                max_val = inp[i, j]
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w:
                        max_val = max(max_val, inp[ni, nj])
                test[i, j] = max_val
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   m=g[i][j]
   for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
    ni,nj=i+di,j+dj
    if 0<=ni<h and 0<=nj<w:
     m=max(m,g[ni][nj])
   res[i][j]=m
 return res
"""
        return None

@handler_decorator
class HandlerNeighborMin(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Each cell becomes min of its neighbors including itself
        for i in range(h):
            for j in range(w):
                min_val = inp[i, j]
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w:
                        min_val = min(min_val, inp[ni, nj])
                test[i, j] = min_val
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   m=g[i][j]
   for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
    ni,nj=i+di,j+dj
    if 0<=ni<h and 0<=nj<w:
     m=min(m,g[ni][nj])
   res[i][j]=m
 return res
"""
        return None

@handler_decorator
class HandlerDiagonalNeighborSum(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Sum diagonal neighbors only
        for i in range(h):
            for j in range(w):
                total = 0
                for di, dj in [(-1,-1), (-1,1), (1,-1), (1,1)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w:
                        total += inp[ni, nj]
                test[i, j] = total % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   t=0
   for di,dj in[(-1,-1),(-1,1),(1,-1),(1,1)]:
    ni,nj=i+di,j+dj
    if 0<=ni<h and 0<=nj<w:
     t+=g[ni][nj]
   res[i][j]=t%10
 return res
"""
        return None

@handler_decorator
class HandlerConditionalPropagation(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Try different propagation patterns
        for direction in ['down', 'up', 'right', 'left']:
            test = np.copy(inp)
            
            if direction == 'down':
                for j in range(w):
                    for i in range(1, h):
                        if test[i, j] == 0 and test[i-1, j] != 0:
                            test[i, j] = test[i-1, j]
            elif direction == 'up':
                for j in range(w):
                    for i in range(h-2, -1, -1):
                        if test[i, j] == 0 and test[i+1, j] != 0:
                            test[i, j] = test[i+1, j]
            elif direction == 'right':
                for i in range(h):
                    for j in range(1, w):
                        if test[i, j] == 0 and test[i, j-1] != 0:
                            test[i, j] = test[i, j-1]
            elif direction == 'left':
                for i in range(h):
                    for j in range(w-2, -1, -1):
                        if test[i, j] == 0 and test[i, j+1] != 0:
                            test[i, j] = test[i, j+1]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Try downward propagation
        test = np.copy(inp)
        for j in range(w):
            for i in range(1, h):
                if test[i, j] == 0 and test[i-1, j] != 0:
                    test[i, j] = test[i-1, j]
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for j in range(w):
  for i in range(1,h):
   if res[i][j]==0 and res[i-1][j]!=0:
    res[i][j]=res[i-1][j]
 return res
"""
        
        # Try rightward propagation
        test = np.copy(inp)
        for i in range(h):
            for j in range(1, w):
                if test[i, j] == 0 and test[i, j-1] != 0:
                    test[i, j] = test[i, j-1]
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(1,w):
   if res[i][j]==0 and res[i][j-1]!=0:
    res[i][j]=res[i][j-1]
 return res
"""
        return None

@handler_decorator
class HandlerNeighborCount(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Count non-zero neighbors
        for i in range(h):
            for j in range(w):
                count = 0
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] != 0:
                        count += 1
                test[i, j] = count
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   c=0
   for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
    ni,nj=i+di,j+dj
    if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
     c+=1
   res[i][j]=c
 return res
"""
        return None

@handler_decorator
class HandlerConditionalMirror(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Mirror cells based on neighbor patterns
        for i in range(h):
            for j in range(w):
                if inp[i, j] == 0:
                    # Check for mirror pattern
                    if j > 0 and j < w-1 and inp[i, j-1] == inp[i, j+1] and inp[i, j-1] != 0:
                        test[i, j] = inp[i, j-1]
                    elif i > 0 and i < h-1 and inp[i-1, j] == inp[i+1, j] and inp[i-1, j] != 0:
                        test[i, j] = inp[i-1, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if g[i][j]==0:
    if j>0 and j<w-1 and g[i][j-1]==g[i][j+1]and g[i][j-1]!=0:
     res[i][j]=g[i][j-1]
    elif i>0 and i<h-1 and g[i-1][j]==g[i+1][j]and g[i-1][j]!=0:
     res[i][j]=g[i-1][j]
 return res
"""
        return None

@handler_decorator
class HandlerEdgeDetection(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Detect edges where color changes
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    is_edge = False
                    current_color = inp[i, j]
                    
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w:
                            if inp[ni, nj] != current_color:
                                is_edge = True
                                break
                        else:
                            is_edge = True
                            break
                    
                    if is_edge:
                        test[i, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    e=0
    c=g[i][j]
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w:
      if g[ni][nj]!=c:e=1;break
     else:e=1;break
    if e:res[i][j]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerConwayLifeRules(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Apply Conway's Game of Life rules
        for i in range(h):
            for j in range(w):
                # Count live neighbors (8-connectivity)
                live_neighbors = 0
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        if di == 0 and dj == 0:
                            continue
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] != 0:
                            live_neighbors += 1
                
                # Apply rules
                if inp[i, j] != 0:  # Cell is alive
                    if live_neighbors in [2, 3]:
                        test[i, j] = inp[i, j]
                else:  # Cell is dead
                    if live_neighbors == 3:
                        test[i, j] = 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   n=0
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     if di==0 and dj==0:continue
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
      n+=1
   if g[i][j]!=0:
    if n in[2,3]:res[i][j]=g[i][j]
   else:
    if n==3:res[i][j]=1
 return res
"""
        return None

@handler_decorator
class HandlerNeighborMedian(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Each cell becomes median of neighbors
        for i in range(h):
            for j in range(w):
                neighbors = []
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w:
                            neighbors.append(inp[ni, nj])
                
                if neighbors:
                    neighbors.sort()
                    median_idx = len(neighbors) // 2
                    test[i, j] = neighbors[median_idx]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   n=[]
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w:
      n.append(g[ni][nj])
   n.sort()
   res[i][j]=n[len(n)//2]
 return res
"""
        return None

@handler_decorator
class HandlerNeighborProduct(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Each cell becomes product of neighbors mod 10
        for i in range(h):
            for j in range(w):
                product = 1
                has_neighbor = False
                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w:
                        product = (product * inp[ni, nj]) % 10
                        has_neighbor = True
                
                test[i, j] = product if has_neighbor else 0
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   p=1
   hn=0
   for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
    ni,nj=i+di,j+dj
    if 0<=ni<h and 0<=nj<w:
     p=(p*g[ni][nj])%10
     hn=1
   res[i][j]=p if hn else 0
 return res
"""
        return None

@handler_decorator
class HandlerThresholdFilter(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Try different threshold values
        for threshold in range(1, 10):
            test = np.zeros_like(inp)
            
            for i in range(h):
                for j in range(w):
                    # Count neighbors above threshold
                    count = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] >= threshold:
                            count += 1
                    
                    if count >= 2:
                        test[i, j] = inp[i, j]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for threshold in range(1, 10):
            test = np.zeros_like(inp)
            
            for i in range(h):
                for j in range(w):
                    count = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] >= threshold:
                            count += 1
                    
                    if count >= 2:
                        test[i, j] = inp[i, j]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   c=0
   for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
    ni,nj=i+di,j+dj
    if 0<=ni<h and 0<=nj<w and g[ni][nj]>={threshold}:
     c+=1
   if c>=2:res[i][j]=g[i][j]
 return res
"""
        return None

@handler_decorator
class HandlerColorSpread(TransformationHandler):
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Colors spread to adjacent empty cells
        changes = True
        while changes:
            changes = False
            new_test = np.copy(test)
            
            for i in range(h):
                for j in range(w):
                    if test[i, j] == 0:
                        # Check adjacent cells for colors
                        adjacent_colors = []
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if 0 <= ni < h and 0 <= nj < w and test[ni, nj] != 0:
                                adjacent_colors.append(test[ni, nj])
                        
                        # If only one unique adjacent color, spread it
                        if adjacent_colors and len(set(adjacent_colors)) == 1:
                            new_test[i, j] = adjacent_colors[0]
                            changes = True
            
            test = new_test
            
            # Prevent infinite loops
            if np.array_equal(test, out):
                return True
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        # Color spreading with iteration is complex to implement compactly
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# SORTING AND ARRANGEMENT OPERATIONS - ENHANCED VERSION
# --------------------------------------------------------------------------------

@handler_decorator
class HandlerSortBySize(TransformationHandler):
    """Sorts objects by their size (number of pixels)"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Manual connected component labeling
        h, w = inp.shape
        visited = np.zeros((h, w), dtype=bool)
        objects = []
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0 and not visited[i, j]:
                    # BFS to find component
                    component = []
                    color = inp[i, j]
                    stack = [(i, j)]
                    
                    while stack:
                        ci, cj = stack.pop()
                        if visited[ci, cj]:
                            continue
                        visited[ci, cj] = True
                        component.append((ci, cj))
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < h and 0 <= nj < w and 
                                inp[ni, nj] == color and not visited[ni, nj]):
                                stack.append((ni, nj))
                    
                    objects.append((len(component), color, component))
        
        if not objects:
            return False
        
        # Sort by size
        sorted_objects = sorted(objects, key=lambda x: x[0])
        
        # Try different arrangements
        test = np.zeros_like(inp)
        
        # Linear arrangement
        col = 0
        for size, color, component in sorted_objects:
            min_j = min(p[1] for p in component)
            max_j = max(p[1] for p in component)
            width = max_j - min_j + 1
            
            if col + width <= w:
                for ci, cj in component:
                    new_j = col + (cj - min_j)
                    if new_j < w:
                        test[ci, new_j] = color
                col += width + 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 vis=[[0]*w for _ in range(h)]
 objs=[]
 for i in range(h):
  for j in range(w):
   if g[i][j]and not vis[i][j]:
    comp=[]
    c=g[i][j]
    stack=[(i,j)]
    while stack:
     y,x=stack.pop()
     if vis[y][x]:continue
     vis[y][x]=1
     comp.append((y,x))
     for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
      ny,nx=y+dy,x+dx
      if 0<=ny<h and 0<=nx<w and g[ny][nx]==c and not vis[ny][nx]:
       stack.append((ny,nx))
    objs.append((len(comp),c,comp))
 objs.sort()
 r=[[0]*w for _ in range(h)]
 col=0
 for sz,c,comp in objs:
  minj=min(p[1]for p in comp)
  maxj=max(p[1]for p in comp)
  wd=maxj-minj+1
  if col+wd<=w:
   for y,x in comp:
    nj=col+(x-minj)
    if nj<w:r[y][nj]=c
   col+=wd+1
 return r
"""
        return None

@handler_decorator
class HandlerArrangeInGrid(TransformationHandler):
    """Arranges objects in a regular grid pattern"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Find objects in input
        h, w = inp.shape
        visited = np.zeros((h, w), dtype=bool)
        inp_objects = []
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0 and not visited[i, j]:
                    component = []
                    color = inp[i, j]
                    stack = [(i, j)]
                    
                    while stack:
                        ci, cj = stack.pop()
                        if visited[ci, cj]:
                            continue
                        visited[ci, cj] = True
                        component.append((ci, cj))
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < h and 0 <= nj < w and 
                                inp[ni, nj] == color and not visited[ni, nj]):
                                stack.append((ni, nj))
                    
                    inp_objects.append((color, component))
        
        # Check if output has same number of objects in grid
        out_h, out_w = out.shape
        visited_out = np.zeros((out_h, out_w), dtype=bool)
        out_objects = []
        
        for i in range(out_h):
            for j in range(out_w):
                if out[i, j] != 0 and not visited_out[i, j]:
                    component = []
                    color = out[i, j]
                    stack = [(i, j)]
                    
                    while stack:
                        ci, cj = stack.pop()
                        if visited_out[ci, cj]:
                            continue
                        visited_out[ci, cj] = True
                        component.append((ci, cj))
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < out_h and 0 <= nj < out_w and 
                                out[ni, nj] == color and not visited_out[ni, nj]):
                                stack.append((ni, nj))
                    
                    # Calculate center
                    center_i = sum(p[0] for p in component) // len(component)
                    center_j = sum(p[1] for p in component) // len(component)
                    out_objects.append((center_i, center_j))
        
        if len(inp_objects) != len(out_objects):
            return False
        
        # Check for regular grid spacing
        if len(out_objects) >= 4:
            out_objects.sort()
            
            # Check if forms a grid
            rows = {}
            for i, j in out_objects:
                if i not in rows:
                    rows[i] = []
                rows[i].append(j)
            
            # Check regular spacing
            row_counts = [len(cols) for cols in rows.values()]
            if len(set(row_counts)) == 1:  # Same number per row
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            # Calculate grid dimensions
            inp = np.array(task_data['train'][0]['input'])
            h, w = inp.shape
            
            # Count objects
            visited = np.zeros((h, w), dtype=bool)
            num_objects = 0
            
            for i in range(h):
                for j in range(w):
                    if inp[i, j] != 0 and not visited[i, j]:
                        num_objects += 1
                        stack = [(i, j)]
                        color = inp[i, j]
                        
                        while stack:
                            ci, cj = stack.pop()
                            if visited[ci, cj]:
                                continue
                            visited[ci, cj] = True
                            
                            for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                                ni, nj = ci + di, cj + dj
                                if (0 <= ni < h and 0 <= nj < w and 
                                    inp[ni, nj] == color and not visited[ni, nj]):
                                    stack.append((ni, nj))
            
            # Find grid dimensions
            grid_size = int(num_objects ** 0.5)
            if grid_size * grid_size == num_objects:
                return f"""def p(g):
 h,w=len(g),len(g[0])
 vis=[[0]*w for _ in range(h)]
 objs=[]
 for i in range(h):
  for j in range(w):
   if g[i][j]and not vis[i][j]:
    comp=[]
    c=g[i][j]
    stack=[(i,j)]
    while stack:
     y,x=stack.pop()
     if vis[y][x]:continue
     vis[y][x]=1
     comp.append((y,x))
     for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
      ny,nx=y+dy,x+dx
      if 0<=ny<h and 0<=nx<w and g[ny][nx]==c and not vis[ny][nx]:
       stack.append((ny,nx))
    objs.append((c,comp))
 r=[[0]*w for _ in range(h)]
 gs={grid_size}
 sp=max(3,w//gs)
 for idx,(c,comp)in enumerate(objs):
  ri,ci=idx//gs,idx%gs
  for y,x in comp:
   ny,nx=ri*sp+(y-min(p[0]for p in comp)),ci*sp+(x-min(p[1]for p in comp))
   if ny<h and nx<w:r[ny][nx]=c
 return r
"""
        return None

@handler_decorator
class HandlerSortRows(TransformationHandler):
    """Sorts elements within each row"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check ascending sort
        test = np.array([sorted(row) for row in inp])
        if np.array_equal(test, out):
            return True
        
        # Check descending sort
        test = np.array([sorted(row, reverse=True) for row in inp])
        if np.array_equal(test, out):
            return True
        
        # Check sort only non-zero
        test = []
        for row in inp:
            non_zero = sorted([x for x in row if x != 0])
            result = []
            nz_idx = 0
            for x in row:
                if x == 0:
                    result.append(0)
                else:
                    result.append(non_zero[nz_idx])
                    nz_idx += 1
            test.append(result)
        test = np.array(test)
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        test = np.array([sorted(row) for row in inp])
        if np.array_equal(test, out):
            return f"""def p(g):
 return[sorted(r)for r in g]
"""
        
        test = np.array([sorted(row, reverse=True) for row in inp])
        if np.array_equal(test, out):
            return f"""def p(g):
 return[sorted(r,reverse=True)for r in g]
"""
        
        # Sort only non-zero
        test = []
        for row in inp:
            non_zero = sorted([x for x in row if x != 0])
            result = []
            nz_idx = 0
            for x in row:
                if x == 0:
                    result.append(0)
                else:
                    result.append(non_zero[nz_idx])
                    nz_idx += 1
            test.append(result)
        test = np.array(test)
        if np.array_equal(test, out):
            return f"""def p(g):
 res=[]
 for r in g:
  nz=sorted([x for x in r if x])
  row=[]
  idx=0
  for x in r:
   if x:row.append(nz[idx]);idx+=1
   else:row.append(0)
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerSortColumns(TransformationHandler):
    """Sorts elements within each column"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Ascending sort
        test = np.array([sorted(col) for col in inp.T]).T
        if np.array_equal(test, out):
            return True
        
        # Descending sort
        test = np.array([sorted(col, reverse=True) for col in inp.T]).T
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        test = np.array([sorted(col) for col in inp.T]).T
        if np.array_equal(test, out):
            return f"""def p(g):
 t=list(zip(*g))
 s=[sorted(c)for c in t]
 return[list(r)for r in zip(*s)]
"""
        
        test = np.array([sorted(col, reverse=True) for col in inp.T]).T
        if np.array_equal(test, out):
            return f"""def p(g):
 t=list(zip(*g))
 s=[sorted(c,reverse=True)for c in t]
 return[list(r)for r in zip(*s)]
"""
        return None

@handler_decorator
class HandlerSortByColorFrequency(TransformationHandler):
    """Sorts or maps colors by their frequency"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Count frequencies
        from collections import Counter
        flat = inp.flatten()
        counts = Counter(flat)
        
        # Map by frequency rank
        sorted_items = sorted(counts.items(), key=lambda x: (-x[1], x[0]))
        rank_map = {color: idx for idx, (color, _) in enumerate(sorted_items)}
        
        test = np.vectorize(lambda x: rank_map[x])(inp)
        if np.array_equal(test, out):
            return True
        
        # Map to frequency count
        freq_map = {color: count % 10 for color, count in counts.items()}
        test = np.vectorize(lambda x: freq_map[x])(inp)
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        from collections import Counter
        flat = inp.flatten()
        counts = Counter(flat)
        
        sorted_items = sorted(counts.items(), key=lambda x: (-x[1], x[0]))
        rank_map = {color: idx for idx, (color, _) in enumerate(sorted_items)}
        
        test = np.vectorize(lambda x: rank_map[x])(inp)
        if np.array_equal(test, out):
            map_str = ','.join([f'{k}:{v}' for k, v in rank_map.items()])
            return f"""def p(g):
 m={{{map_str}}}
 return[[m[x]for x in r]for r in g]
"""
        
        freq_map = {color: count % 10 for color, count in counts.items()}
        test = np.vectorize(lambda x: freq_map[x])(inp)
        if np.array_equal(test, out):
            # Count manually in generated code
            return f"""def p(g):
 f={{}}
 for r in g:
  for x in r:f[x]=f.get(x,0)+1
 return[[f[x]%10for x in r]for r in g]
"""
        return None

@handler_decorator
class HandlerSortDiagonally(TransformationHandler):
    """Sorts elements along diagonals"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Sort main diagonal only
        if h == w:
            diag = [inp[i, i] for i in range(h)]
            sorted_diag = sorted(diag)
            for i in range(h):
                test[i, i] = sorted_diag[i]
            
            if np.array_equal(test, out):
                return True
        
        # Sort all diagonals
        test = np.copy(inp)
        for start in range(-(h-1), w):
            diag_indices = []
            diag_values = []
            
            for i in range(h):
                j = i + start
                if 0 <= j < w:
                    diag_indices.append((i, j))
                    diag_values.append(inp[i, j])
            
            if diag_values:
                sorted_values = sorted(diag_values)
                for idx, (i, j) in enumerate(diag_indices):
                    test[i, j] = sorted_values[idx]
        
        if np.array_equal(test, out):
            return True
        
        # Sort anti-diagonals
        test = np.copy(inp)
        for start in range(h + w - 1):
            diag_indices = []
            diag_values = []
            
            for i in range(h):
                j = start - i
                if 0 <= j < w:
                    diag_indices.append((i, j))
                    diag_values.append(inp[i, j])
            
            if diag_values:
                sorted_values = sorted(diag_values)
                for idx, (i, j) in enumerate(diag_indices):
                    test[i, j] = sorted_values[idx]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Main diagonal only
        if h == w:
            test = np.copy(inp)
            diag = [inp[i, i] for i in range(h)]
            sorted_diag = sorted(diag)
            for i in range(h):
                test[i, i] = sorted_diag[i]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h={h}
 r=[row[:]for row in g]
 d=[g[i][i]for i in range(h)]
 s=sorted(d)
 for i in range(h):r[i][i]=s[i]
 return r
"""
        
        # All diagonals
        test = np.copy(inp)
        for start in range(-(h-1), w):
            diag_indices = []
            diag_values = []
            
            for i in range(h):
                j = i + start
                if 0 <= j < w:
                    diag_indices.append((i, j))
                    diag_values.append(inp[i, j])
            
            if diag_values:
                sorted_values = sorted(diag_values)
                for idx, (i, j) in enumerate(diag_indices):
                    test[i, j] = sorted_values[idx]
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 r=[row[:]for row in g]
 for s in range(-{h-1},{w}):
  idx=[]
  vals=[]
  for i in range(h):
   j=i+s
   if 0<=j<w:
    idx.append((i,j))
    vals.append(g[i][j])
  sv=sorted(vals)
  for k,(i,j)in enumerate(idx):
   r[i][j]=sv[k]
 return r
"""
        return None

@handler_decorator
class HandlerMoveNonZeroToFront(TransformationHandler):
    """Moves non-zero elements to the front"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Row-wise movement
        test = []
        for row in inp:
            non_zero = [x for x in row if x != 0]
            zeros = [x for x in row if x == 0]
            test.append(non_zero + zeros)
        test = np.array(test)
        
        if np.array_equal(test, out):
            return True
        
        # Column-wise movement
        test = []
        for col in inp.T:
            non_zero = [x for x in col if x != 0]
            zeros = [x for x in col if x == 0]
            test.append(non_zero + zeros)
        test = np.array(test).T
        
        if np.array_equal(test, out):
            return True
        
        # Move to back instead
        test = []
        for row in inp:
            zeros = [x for x in row if x == 0]
            non_zero = [x for x in row if x != 0]
            test.append(zeros + non_zero)
        test = np.array(test)
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Row-wise to front
        test = []
        for row in inp:
            non_zero = [x for x in row if x != 0]
            zeros = [x for x in row if x == 0]
            test.append(non_zero + zeros)
        test = np.array(test)
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[x for x in r if x]+[x for x in r if not x]for r in g]
"""
        
        # Column-wise to front
        test = []
        for col in inp.T:
            non_zero = [x for x in col if x != 0]
            zeros = [x for x in col if x == 0]
            test.append(non_zero + zeros)
        test = np.array(test).T
        
        if np.array_equal(test, out):
            return f"""def p(g):
 t=list(zip(*g))
 s=[[x for x in c if x]+[x for x in c if not x]for c in t]
 return[list(r)for r in zip(*s)]
"""
        
        # Row-wise to back
        test = []
        for row in inp:
            zeros = [x for x in row if x == 0]
            non_zero = [x for x in row if x != 0]
            test.append(zeros + non_zero)
        test = np.array(test)
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[x for x in r if not x]+[x for x in r if x]for r in g]
"""
        return None

@handler_decorator
class HandlerGroupByColor(TransformationHandler):
    """Groups elements by color"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        flat_inp = inp.flatten()
        flat_out = out.flatten()
        
        # Check if same elements
        if sorted(flat_inp) != sorted(flat_out):
            return False
        
        # Check if grouped by color
        last_color = -1
        seen_colors = set()
        
        for color in flat_out:
            if color != last_color:
                if color in seen_colors:
                    return False
                seen_colors.add(color)
                last_color = color
        
        return True
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h, w = task_data['train'][0]['input'].shape
            return f"""def p(g):
 h,w={h},{w}
 f=[x for r in g for x in r]
 c=[]
 for x in range(10):c.extend([x]*f.count(x))
 return[c[i*w:(i+1)*w]for i in range(h)]
"""
        return None

@handler_decorator
class HandlerSortByBrightness(TransformationHandler):
    """Sorts elements by brightness (value)"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Row-wise sort
        test = np.array([sorted(row) for row in inp])
        if np.array_equal(test, out):
            return True
        
        test = np.array([sorted(row, reverse=True) for row in inp])
        if np.array_equal(test, out):
            return True
        
        # Global sort
        flat = sorted(inp.flatten())
        test = np.array(flat).reshape(inp.shape)
        if np.array_equal(test, out):
            return True
        
        flat = sorted(inp.flatten(), reverse=True)
        test = np.array(flat).reshape(inp.shape)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        test = np.array([sorted(row) for row in inp])
        if np.array_equal(test, out):
            return f"""def p(g):
 return[sorted(r)for r in g]
"""
        
        test = np.array([sorted(row, reverse=True) for row in inp])
        if np.array_equal(test, out):
            return f"""def p(g):
 return[sorted(r,reverse=True)for r in g]
"""
        
        h, w = inp.shape
        flat = sorted(inp.flatten())
        test = np.array(flat).reshape(inp.shape)
        if np.array_equal(test, out):
            return f"""def p(g):
 f=sorted([x for r in g for x in r])
 return[f[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        
        flat = sorted(inp.flatten(), reverse=True)
        test = np.array(flat).reshape(inp.shape)
        if np.array_equal(test, out):
            return f"""def p(g):
 f=sorted([x for r in g for x in r],reverse=True)
 return[f[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerSortByDistance(TransformationHandler):
    """Sorts elements by distance from center or corner"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Try different reference points
        reference_points = [
            (h//2, w//2),  # Center
            (0, 0),        # Top-left
            (0, w-1),      # Top-right
            (h-1, 0),      # Bottom-left
            (h-1, w-1)     # Bottom-right
        ]
        
        for ref_i, ref_j in reference_points:
            # Collect non-zero values with distances
            elements = []
            for i in range(h):
                for j in range(w):
                    if inp[i, j] != 0:
                        dist = abs(i - ref_i) + abs(j - ref_j)  # Manhattan distance
                        elements.append((dist, inp[i, j], i, j))
            
            if not elements:
                continue
            
            # Sort by distance
            elements.sort(key=lambda x: (x[0], x[1]))
            
            # Create distance-based positions
            positions = []
            for i in range(h):
                for j in range(w):
                    dist = abs(i - ref_i) + abs(j - ref_j)
                    positions.append((dist, i, j))
            
            positions.sort(key=lambda x: x[0])
            
            # Place elements
            test = np.zeros_like(inp)
            for idx, (_, val, _, _) in enumerate(elements):
                if idx < len(positions):
                    _, pi, pj = positions[idx]
                    test[pi, pj] = val
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Center distance
        ref_i, ref_j = h//2, w//2
        
        elements = []
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    dist = abs(i - ref_i) + abs(j - ref_j)
                    elements.append((dist, inp[i, j], i, j))
        
        elements.sort(key=lambda x: (x[0], x[1]))
        
        positions = []
        for i in range(h):
            for j in range(w):
                dist = abs(i - ref_i) + abs(j - ref_j)
                positions.append((dist, i, j))
        
        positions.sort(key=lambda x: x[0])
        
        test = np.zeros_like(inp)
        for idx, (_, val, _, _) in enumerate(elements):
            if idx < len(positions):
                _, pi, pj = positions[idx]
                test[pi, pj] = val
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 ri,rj={ref_i},{ref_j}
 els=[]
 for i in range(h):
  for j in range(w):
   if g[i][j]:
    d=abs(i-ri)+abs(j-rj)
    els.append((d,g[i][j],i,j))
 els.sort()
 pos=[]
 for i in range(h):
  for j in range(w):
   d=abs(i-ri)+abs(j-rj)
   pos.append((d,i,j))
 pos.sort()
 r=[[0]*w for _ in range(h)]
 for idx,(d,v,i,j)in enumerate(els):
  if idx<len(pos):
   _,pi,pj=pos[idx]
   r[pi][pj]=v
 return r
"""
        return None

@handler_decorator
class HandlerArrangeByPattern(TransformationHandler):
    """Arranges objects in specific patterns"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Extract objects
        h, w = inp.shape
        visited = np.zeros((h, w), dtype=bool)
        objects = []
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0 and not visited[i, j]:
                    color = inp[i, j]
                    component = []
                    stack = [(i, j)]
                    
                    while stack:
                        ci, cj = stack.pop()
                        if visited[ci, cj]:
                            continue
                        visited[ci, cj] = True
                        component.append((ci, cj))
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < h and 0 <= nj < w and 
                                inp[ni, nj] == color and not visited[ni, nj]):
                                stack.append((ni, nj))
                    
                    objects.append((color, component))
        
        if not objects:
            return False
        
        # Check diagonal arrangement
        test = np.zeros_like(out)
        for idx, (color, _) in enumerate(objects):
            if idx < min(h, w):
                test[idx, idx] = color
        
        if np.array_equal(test, out):
            return True
        
        # Check spiral arrangement
        test = np.zeros_like(out)
        positions = []
        top, bottom, left, right = 0, h-1, 0, w-1
        
        while top <= bottom and left <= right:
            for j in range(left, right+1):
                positions.append((top, j))
            top += 1
            
            for i in range(top, bottom+1):
                positions.append((i, right))
            right -= 1
            
            if top <= bottom:
                for j in range(right, left-1, -1):
                    positions.append((bottom, j))
                bottom -= 1
            
            if left <= right:
                for i in range(bottom, top-1, -1):
                    positions.append((i, left))
                left += 1
        
        for idx, (color, _) in enumerate(objects):
            if idx < len(positions):
                i, j = positions[idx]
                test[i, j] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Extract and check diagonal
        visited = np.zeros((h, w), dtype=bool)
        objects = []
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0 and not visited[i, j]:
                    color = inp[i, j]
                    stack = [(i, j)]
                    
                    while stack:
                        ci, cj = stack.pop()
                        if visited[ci, cj]:
                            continue
                        visited[ci, cj] = True
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < h and 0 <= nj < w and 
                                inp[ni, nj] == color and not visited[ni, nj]):
                                stack.append((ni, nj))
                    
                    objects.append(color)
        
        test = np.zeros_like(out)
        for idx, color in enumerate(objects):
            if idx < min(h, w):
                test[idx, idx] = color
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 vis=[[0]*w for _ in range(h)]
 objs=[]
 for i in range(h):
  for j in range(w):
   if g[i][j]and not vis[i][j]:
    c=g[i][j]
    stack=[(i,j)]
    while stack:
     y,x=stack.pop()
     if vis[y][x]:continue
     vis[y][x]=1
     for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
      ny,nx=y+dy,x+dx
      if 0<=ny<h and 0<=nx<w and g[ny][nx]==c and not vis[ny][nx]:
       stack.append((ny,nx))
    objs.append(c)
 r=[[0]*w for _ in range(h)]
 for idx,c in enumerate(objs):
  if idx<min(h,w):r[idx][idx]=c
 return r
"""
        return None

@handler_decorator
class HandlerSortMatrixSpiral(TransformationHandler):
    """Sorts matrix elements in spiral order"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Extract spiral
        spiral = []
        test = np.copy(inp)
        top, bottom, left, right = 0, h-1, 0, w-1
        
        while top <= bottom and left <= right:
            for j in range(left, right+1):
                spiral.append(inp[top, j])
            top += 1
            
            for i in range(top, bottom+1):
                spiral.append(inp[i, right])
            right -= 1
            
            if top <= bottom:
                for j in range(right, left-1, -1):
                    spiral.append(inp[bottom, j])
                bottom -= 1
            
            if left <= right:
                for i in range(bottom, top-1, -1):
                    spiral.append(inp[i, left])
                left += 1
        
        # Sort spiral
        sorted_spiral = sorted(spiral)
        
        # Place back
        test = np.zeros_like(inp)
        idx = 0
        top, bottom, left, right = 0, h-1, 0, w-1
        
        while top <= bottom and left <= right and idx < len(sorted_spiral):
            for j in range(left, right+1):
                if idx < len(sorted_spiral):
                    test[top, j] = sorted_spiral[idx]
                    idx += 1
            top += 1
            
            for i in range(top, bottom+1):
                if idx < len(sorted_spiral):
                    test[i, right] = sorted_spiral[idx]
                    idx += 1
            right -= 1
            
            if top <= bottom:
                for j in range(right, left-1, -1):
                    if idx < len(sorted_spiral):
                        test[bottom, j] = sorted_spiral[idx]
                        idx += 1
                bottom -= 1
            
            if left <= right:
                for i in range(bottom, top-1, -1):
                    if idx < len(sorted_spiral):
                        test[i, left] = sorted_spiral[idx]
                        idx += 1
                left += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h, w = task_data['train'][0]['input'].shape
            return f"""def p(g):
 h,w={h},{w}
 s=[]
 t,b,l,r=0,h-1,0,w-1
 while t<=b and l<=r:
  for j in range(l,r+1):s.append(g[t][j])
  t+=1
  for i in range(t,b+1):s.append(g[i][r])
  r-=1
  if t<=b:
   for j in range(r,l-1,-1):s.append(g[b][j])
   b-=1
  if l<=r:
   for i in range(b,t-1,-1):s.append(g[i][l])
   l+=1
 s.sort()
 res=[[0]*w for _ in range(h)]
 idx=0
 t,b,l,r=0,h-1,0,w-1
 while t<=b and l<=r and idx<len(s):
  for j in range(l,r+1):
   if idx<len(s):res[t][j]=s[idx];idx+=1
  t+=1
  for i in range(t,b+1):
   if idx<len(s):res[i][r]=s[idx];idx+=1
  r-=1
  if t<=b:
   for j in range(r,l-1,-1):
    if idx<len(s):res[b][j]=s[idx];idx+=1
   b-=1
  if l<=r:
   for i in range(b,t-1,-1):
    if idx<len(s):res[i][l]=s[idx];idx+=1
   l+=1
 return res
"""
        return None

@handler_decorator
class HandlerStableSort(TransformationHandler):
    """Performs stable sorting with priority"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check priority-based sorting
        for priority_color in range(10):
            test = []
            for row in inp:
                priority = [x for x in row if x == priority_color]
                others = sorted([x for x in row if x != priority_color])
                test.append(priority + others)
            
            test = np.array(test)
            if np.array_equal(test, out):
                return True
        
        # Check stable sort by preserving order of equals
        test = []
        for row in inp:
            # Create (value, index) pairs
            indexed = [(val, idx) for idx, val in enumerate(row)]
            # Sort by value, preserving index order for equals
            indexed.sort(key=lambda x: x[0])
            test.append([val for val, _ in indexed])
        
        test = np.array(test)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for priority_color in range(10):
            test = []
            for row in inp:
                priority = [x for x in row if x == priority_color]
                others = sorted([x for x in row if x != priority_color])
                test.append(priority + others)
            
            test = np.array(test)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[x for x in r if x=={priority_color}]+sorted([x for x in r if x!={priority_color}])for r in g]
"""
        return None

@handler_decorator
class HandlerLexicographicSort(TransformationHandler):
    """Sorts rows lexicographically"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Sort rows
        sorted_rows = sorted([list(row) for row in inp])
        test = np.array(sorted_rows)
        
        if np.array_equal(test, out):
            return True
        
        # Sort columns
        transposed = list(zip(*inp))
        sorted_cols = sorted(transposed)
        test = np.array(list(zip(*sorted_cols)))
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        sorted_rows = sorted([list(row) for row in inp])
        test = np.array(sorted_rows)
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return sorted(g)
"""
        
        transposed = list(zip(*inp))
        sorted_cols = sorted(transposed)
        test = np.array(list(zip(*sorted_cols)))
        
        if np.array_equal(test, out):
            return f"""def p(g):
 t=list(zip(*g))
 s=sorted(t)
 return[list(r)for r in zip(*s)]
"""
        return None

@handler_decorator
class HandlerRankTransform(TransformationHandler):
    """Replaces values with their rank"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Global rank
        unique_values = sorted(set(inp.flatten()))
        rank_map = {val: rank for rank, val in enumerate(unique_values)}
        
        test = np.vectorize(lambda x: rank_map[x])(inp)
        if np.array_equal(test, out):
            return True
        
        # Row-wise rank
        test = []
        for row in inp:
            unique_in_row = sorted(set(row))
            row_rank_map = {val: rank for rank, val in enumerate(unique_in_row)}
            test.append([row_rank_map[x] for x in row])
        
        test = np.array(test)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        unique_values = sorted(set(inp.flatten()))
        rank_map = {val: rank for rank, val in enumerate(unique_values)}
        
        test = np.vectorize(lambda x: rank_map[x])(inp)
        if np.array_equal(test, out):
            map_str = ','.join([f'{v}:{r}' for v, r in rank_map.items()])
            return f"""def p(g):
 m={{{map_str}}}
 return[[m[x]for x in r]for r in g]
"""
        
        # Row-wise rank
        test = []
        for row in inp:
            unique_in_row = sorted(set(row))
            row_rank_map = {val: rank for rank, val in enumerate(unique_in_row)}
            test.append([row_rank_map[x] for x in row])
        
        test = np.array(test)
        if np.array_equal(test, out):
            return f"""def p(g):
 res=[]
 for r in g:
  u=sorted(set(r))
  m={{v:i for i,v in enumerate(u)}}
  res.append([m[x]for x in r])
 return res
"""
        return None

@handler_decorator
class HandlerPartitionSort(TransformationHandler):
    """Sorts within partitions of the grid"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Quadrant sorting
        if h % 2 == 0 and w % 2 == 0:
            h2, w2 = h // 2, w // 2
            test = np.copy(inp)
            
            # Sort each quadrant
            q1 = inp[:h2, :w2].flatten()
            q2 = inp[:h2, w2:].flatten()
            q3 = inp[h2:, :w2].flatten()
            q4 = inp[h2:, w2:].flatten()
            
            test[:h2, :w2] = sorted(q1).reshape(h2, w2)
            test[:h2, w2:] = sorted(q2).reshape(h2, w2)
            test[h2:, :w2] = sorted(q3).reshape(h2, w2)
            test[h2:, w2:] = sorted(q4).reshape(h2, w2)
            
            if np.array_equal(test, out):
                return True
        
        # Block sorting (3x3 blocks)
        if h % 3 == 0 and w % 3 == 0:
            test = np.copy(inp)
            for i in range(0, h, 3):
                for j in range(0, w, 3):
                    block = inp[i:i+3, j:j+3].flatten()
                    test[i:i+3, j:j+3] = sorted(block).reshape(3, 3)
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if h % 2 == 0 and w % 2 == 0:
            h2, w2 = h // 2, w // 2
            test = np.copy(inp)
            
            q1 = inp[:h2, :w2].flatten()
            q2 = inp[:h2, w2:].flatten()
            q3 = inp[h2:, :w2].flatten()
            q4 = inp[h2:, w2:].flatten()
            
            test[:h2, :w2] = sorted(q1).reshape(h2, w2)
            test[:h2, w2:] = sorted(q2).reshape(h2, w2)
            test[h2:, :w2] = sorted(q3).reshape(h2, w2)
            test[h2:, w2:] = sorted(q4).reshape(h2, w2)
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w={h},{w}
 h2,w2={h2},{w2}
 r=[row[:]for row in g]
 q1,q2,q3,q4=[],[],[],[]
 for i in range(h2):
  for j in range(w2):q1.append(g[i][j])
  for j in range(w2,w):q2.append(g[i][j])
 for i in range(h2,h):
  for j in range(w2):q3.append(g[i][j])
  for j in range(w2,w):q4.append(g[i][j])
 q1,q2,q3,q4=sorted(q1),sorted(q2),sorted(q3),sorted(q4)
 idx=0
 for i in range(h2):
  for j in range(w2):r[i][j]=q1[idx];idx+=1
 idx=0
 for i in range(h2):
  for j in range(w2,w):r[i][j]=q2[idx];idx+=1
 idx=0
 for i in range(h2,h):
  for j in range(w2):r[i][j]=q3[idx];idx+=1
 idx=0
 for i in range(h2,h):
  for j in range(w2,w):r[i][j]=q4[idx];idx+=1
 return r
"""
        
        if h % 3 == 0 and w % 3 == 0:
            test = np.copy(inp)
            for i in range(0, h, 3):
                for j in range(0, w, 3):
                    block = inp[i:i+3, j:j+3].flatten()
                    test[i:i+3, j:j+3] = sorted(block).reshape(3, 3)
            
            if np.array_equal(test, out):
                return f"""def p(g):
 r=[row[:]for row in g]
 for i in range(0,{h},3):
  for j in range(0,{w},3):
   b=[]
   for y in range(3):
    for x in range(3):b.append(g[i+y][j+x])
   b.sort()
   idx=0
   for y in range(3):
    for x in range(3):r[i+y][j+x]=b[idx];idx+=1
 return r
"""
        return None

@handler_decorator
class HandlerBubbleSort(TransformationHandler):
    """Simulates bubble sort visualization"""
    def can_handle(self, task_data, analysis):
        # This would need multiple training examples showing sort steps
        return False
        
    def generate_solution(self, task_data, analysis):
        return None

@handler_decorator
class HandlerMergeSort(TransformationHandler):
    """Performs merge sort pattern"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if it's a simple sorted version
        flat_sorted = sorted(inp.flatten())
        test = np.array(flat_sorted).reshape(inp.shape)
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        flat_sorted = sorted(inp.flatten())
        test = np.array(flat_sorted).reshape(inp.shape)
        
        if np.array_equal(test, out):
            h, w = inp.shape
            return f"""def p(g):
 f=sorted([x for r in g for x in r])
 return[f[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerRadixSort(TransformationHandler):
    """Sorts by individual digits (radix sort style)"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # For single-digit numbers, this is just regular sort
        test = []
        for row in inp:
            test.append(sorted(row))
        
        test = np.array(test)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        test = []
        for row in inp:
            test.append(sorted(row))
        
        test = np.array(test)
        if np.array_equal(test, out):
            return f"""def p(g):
 return[sorted(r)for r in g]
"""
        return None

@handler_decorator
class HandlerTopologicalSort(TransformationHandler):
    """Sorts based on dependencies or connections"""
    def can_handle(self, task_data, analysis):
        # This would require detecting dependency relationships
        return False
        
    def generate_solution(self, task_data, analysis):
        return None

@handler_decorator
class HandlerZigzagSort(TransformationHandler):
    """Sorts in zigzag pattern"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Extract zigzag order
        zigzag = []
        for i in range(h):
            if i % 2 == 0:
                zigzag.extend(inp[i, :])
            else:
                zigzag.extend(inp[i, ::-1])
        
        # Sort
        sorted_zigzag = sorted(zigzag)
        
        # Place back
        test = np.zeros_like(inp)
        idx = 0
        for i in range(h):
            if i % 2 == 0:
                for j in range(w):
                    test[i, j] = sorted_zigzag[idx]
                    idx += 1
            else:
                for j in range(w-1, -1, -1):
                    test[i, j] = sorted_zigzag[idx]
                    idx += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h, w = task_data['train'][0]['input'].shape
            return f"""def p(g):
 h,w={h},{w}
 z=[]
 for i in range(h):
  if i%2==0:z.extend(g[i])
  else:z.extend(g[i][::-1])
 z.sort()
 r=[[0]*w for _ in range(h)]
 idx=0
 for i in range(h):
  if i%2==0:
   for j in range(w):r[i][j]=z[idx];idx+=1
  else:
   for j in range(w-1,-1,-1):r[i][j]=z[idx];idx+=1
 return r
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# TEMPLATE AND SHAPE OPERATIONS - ENHANCED VERSION
# --------------------------------------------------------------------------------

@handler_decorator
class HandlerDetectCross(TransformationHandler):
    """Detects cross patterns in various orientations and sizes"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Standard cross pattern
        for i in range(1, h-1):
            for j in range(1, w-1):
                if (inp[i,j] != 0 and 
                    inp[i-1,j] != 0 and inp[i+1,j] != 0 and
                    inp[i,j-1] != 0 and inp[i,j+1] != 0):
                    test[i,j] = 5
        
        if np.array_equal(test, out):
            return True
        
        # Larger cross patterns
        for size in range(2, min(h//2, w//2)):
            test = np.zeros_like(inp)
            for i in range(size, h-size):
                for j in range(size, w-size):
                    if inp[i,j] != 0:
                        is_cross = True
                        # Check vertical line
                        for di in range(-size, size+1):
                            if inp[i+di,j] == 0:
                                is_cross = False
                                break
                        # Check horizontal line
                        if is_cross:
                            for dj in range(-size, size+1):
                                if inp[i,j+dj] == 0:
                                    is_cross = False
                                    break
                        if is_cross:
                            test[i,j] = inp[i,j]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Standard cross
        test = np.zeros_like(inp)
        for i in range(1, h-1):
            for j in range(1, w-1):
                if (inp[i,j] != 0 and 
                    inp[i-1,j] != 0 and inp[i+1,j] != 0 and
                    inp[i,j-1] != 0 and inp[i,j+1] != 0):
                    test[i,j] = 5
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(1,h-1):
  for j in range(1,w-1):
   if g[i][j]and g[i-1][j]and g[i+1][j]and g[i][j-1]and g[i][j+1]:r[i][j]=5
 return r
"""
        
        # Larger crosses
        for size in range(2, min(h//2, w//2)):
            test = np.zeros_like(inp)
            for i in range(size, h-size):
                for j in range(size, w-size):
                    if inp[i,j] != 0:
                        is_cross = True
                        for di in range(-size, size+1):
                            if inp[i+di,j] == 0:
                                is_cross = False
                                break
                        if is_cross:
                            for dj in range(-size, size+1):
                                if inp[i,j+dj] == 0:
                                    is_cross = False
                                    break
                        if is_cross:
                            test[i,j] = inp[i,j]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 s={size}
 for i in range(s,h-s):
  for j in range(s,w-s):
   if g[i][j]:
    c=1
    for d in range(-s,s+1):
     if not g[i+d][j]:c=0;break
    if c:
     for d in range(-s,s+1):
      if not g[i][j+d]:c=0;break
    if c:r[i][j]=g[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerDetectRectangles(TransformationHandler):
    """Detects filled and hollow rectangles"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Find rectangles and mark corners
        visited = np.zeros((h, w), dtype=bool)
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0 and not visited[i, j]:
                    color = inp[i, j]
                    
                    # Find rectangle bounds
                    min_i, max_i = i, i
                    min_j, max_j = j, j
                    
                    # Expand right
                    while max_j + 1 < w and inp[i, max_j + 1] == color:
                        max_j += 1
                    
                    # Expand down
                    while max_i + 1 < h:
                        all_match = True
                        for jj in range(min_j, max_j + 1):
                            if inp[max_i + 1, jj] != color:
                                all_match = False
                                break
                        if all_match:
                            max_i += 1
                        else:
                            break
                    
                    # Verify rectangle
                    is_rect = True
                    for ii in range(min_i, max_i + 1):
                        for jj in range(min_j, max_j + 1):
                            if inp[ii, jj] != color:
                                is_rect = False
                    
                    if is_rect and (max_i - min_i + 1) >= 2 and (max_j - min_j + 1) >= 2:
                        # Mark corners
                        test[min_i, min_j] = color
                        test[min_i, max_j] = color
                        test[max_i, min_j] = color
                        test[max_i, max_j] = color
                        
                        # Mark as visited
                        for ii in range(min_i, max_i + 1):
                            for jj in range(min_j, max_j + 1):
                                visited[ii, jj] = True
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 v=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]and not v[i][j]:
    c=g[i][j]
    mi,ma,mj,mx=i,i,j,j
    while mx+1<w and g[i][mx+1]==c:mx+=1
    while ma+1<h and all(g[ma+1][k]==c for k in range(mj,mx+1)):ma+=1
    ok=all(g[y][x]==c for y in range(mi,ma+1)for x in range(mj,mx+1))
    if ok and ma>mi and mx>mj:
     r[mi][mj]=r[mi][mx]=r[ma][mj]=r[ma][mx]=c
    for y in range(mi,ma+1):
     for x in range(mj,mx+1):v[y][x]=1
 return r
"""
        return None

@handler_decorator
class HandlerDetectCorners(TransformationHandler):
    """Detects L-shaped corners in all orientations"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    is_corner = False
                    
                    # Check all four corner types
                    # Top-left corner
                    if (i+1 < h and j+1 < w and 
                        inp[i+1, j] != 0 and inp[i, j+1] != 0 and
                        (i-1 < 0 or inp[i-1, j] == 0) and
                        (j-1 < 0 or inp[i, j-1] == 0)):
                        is_corner = True
                    
                    # Top-right corner
                    elif (i+1 < h and j-1 >= 0 and
                          inp[i+1, j] != 0 and inp[i, j-1] != 0 and
                          (i-1 < 0 or inp[i-1, j] == 0) and
                          (j+1 >= w or inp[i, j+1] == 0)):
                        is_corner = True
                    
                    # Bottom-left corner
                    elif (i-1 >= 0 and j+1 < w and
                          inp[i-1, j] != 0 and inp[i, j+1] != 0 and
                          (i+1 >= h or inp[i+1, j] == 0) and
                          (j-1 < 0 or inp[i, j-1] == 0)):
                        is_corner = True
                    
                    # Bottom-right corner
                    elif (i-1 >= 0 and j-1 >= 0 and
                          inp[i-1, j] != 0 and inp[i, j-1] != 0 and
                          (i+1 >= h or inp[i+1, j] == 0) and
                          (j+1 >= w or inp[i, j+1] == 0)):
                        is_corner = True
                    
                    if is_corner:
                        test[i, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]:
    c=0
    if i+1<h and j+1<w and g[i+1][j]and g[i][j+1]and(i==0or not g[i-1][j])and(j==0or not g[i][j-1]):c=1
    elif i+1<h and j>0and g[i+1][j]and g[i][j-1]and(i==0or not g[i-1][j])and(j+1>=w or not g[i][j+1]):c=1
    elif i>0and j+1<w and g[i-1][j]and g[i][j+1]and(i+1>=h or not g[i+1][j])and(j==0or not g[i][j-1]):c=1
    elif i>0and j>0and g[i-1][j]and g[i][j-1]and(i+1>=h or not g[i+1][j])and(j+1>=w or not g[i][j+1]):c=1
    if c:r[i][j]=g[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerDetectTPattern(TransformationHandler):
    """Detects T-shaped patterns in all four orientations"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i,j] != 0 and i > 0 and i < h-1 and j > 0 and j < w-1:
                    # Upright T
                    if (inp[i-1,j] != 0 and inp[i,j-1] != 0 and 
                        inp[i,j+1] != 0 and inp[i+1,j] == 0):
                        test[i,j] = inp[i,j]
                    
                    # Inverted T
                    elif (inp[i+1,j] != 0 and inp[i,j-1] != 0 and 
                          inp[i,j+1] != 0 and inp[i-1,j] == 0):
                        test[i,j] = inp[i,j]
                    
                    # Left T
                    elif (inp[i-1,j] != 0 and inp[i+1,j] != 0 and 
                          inp[i,j-1] != 0 and inp[i,j+1] == 0):
                        test[i,j] = inp[i,j]
                    
                    # Right T
                    elif (inp[i-1,j] != 0 and inp[i+1,j] != 0 and 
                          inp[i,j+1] != 0 and inp[i,j-1] == 0):
                        test[i,j] = inp[i,j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(1,h-1):
  for j in range(1,w-1):
   if g[i][j]:
    if g[i-1][j]and g[i][j-1]and g[i][j+1]and not g[i+1][j]:r[i][j]=g[i][j]
    elif g[i+1][j]and g[i][j-1]and g[i][j+1]and not g[i-1][j]:r[i][j]=g[i][j]
    elif g[i-1][j]and g[i+1][j]and g[i][j-1]and not g[i][j+1]:r[i][j]=g[i][j]
    elif g[i-1][j]and g[i+1][j]and g[i][j+1]and not g[i][j-1]:r[i][j]=g[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerDetectLines(TransformationHandler):
    """Detects horizontal and vertical lines"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Horizontal lines
        for i in range(h):
            nonzero = [j for j in range(w) if inp[i, j] != 0]
            if len(nonzero) >= 3:
                colors = [inp[i, j] for j in nonzero]
                if len(set(colors)) == 1 and nonzero[-1] - nonzero[0] + 1 == len(nonzero):
                    for j in range(w):
                        test[i, j] = inp[i, j]
        
        # Vertical lines
        for j in range(w):
            nonzero = [i for i in range(h) if inp[i, j] != 0]
            if len(nonzero) >= 3:
                colors = [inp[i, j] for i in nonzero]
                if len(set(colors)) == 1 and nonzero[-1] - nonzero[0] + 1 == len(nonzero):
                    for i in range(h):
                        test[i, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  nz=[j for j in range(w)if g[i][j]]
  if len(nz)>=3and len(set(g[i][j]for j in nz))==1and nz[-1]-nz[0]+1==len(nz):
   for j in range(w):r[i][j]=g[i][j]
 for j in range(w):
  nz=[i for i in range(h)if g[i][j]]
  if len(nz)>=3and len(set(g[i][j]for i in nz))==1and nz[-1]-nz[0]+1==len(nz):
   for i in range(h):r[i][j]=g[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerDetectDiagonalLines(TransformationHandler):
    """Detects diagonal lines in both directions"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Main diagonals (top-left to bottom-right)
        for start_i in range(h):
            points = []
            i, j = start_i, 0
            while i < h and j < w:
                if inp[i, j] != 0:
                    points.append((i, j))
                i += 1
                j += 1
            if len(points) >= 3:
                for p in points:
                    test[p[0], p[1]] = inp[p[0], p[1]]
        
        for start_j in range(1, w):
            points = []
            i, j = 0, start_j
            while i < h and j < w:
                if inp[i, j] != 0:
                    points.append((i, j))
                i += 1
                j += 1
            if len(points) >= 3:
                for p in points:
                    test[p[0], p[1]] = inp[p[0], p[1]]
        
        # Anti-diagonals (top-right to bottom-left)
        for start_i in range(h):
            points = []
            i, j = start_i, w - 1
            while i < h and j >= 0:
                if inp[i, j] != 0:
                    points.append((i, j))
                i += 1
                j -= 1
            if len(points) >= 3:
                for p in points:
                    test[p[0], p[1]] = inp[p[0], p[1]]
        
        for start_j in range(w-2, -1, -1):
            points = []
            i, j = 0, start_j
            while i < h and j >= 0:
                if inp[i, j] != 0:
                    points.append((i, j))
                i += 1
                j -= 1
            if len(points) >= 3:
                for p in points:
                    test[p[0], p[1]] = inp[p[0], p[1]]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 # Main diagonals
 for si in range(h):
  pts=[]
  i,j=si,0
  while i<h and j<w:
   if g[i][j]:pts.append((i,j))
   i+=1;j+=1
  if len(pts)>=3:
   for y,x in pts:r[y][x]=g[y][x]
 for sj in range(1,w):
  pts=[]
  i,j=0,sj
  while i<h and j<w:
   if g[i][j]:pts.append((i,j))
   i+=1;j+=1
  if len(pts)>=3:
   for y,x in pts:r[y][x]=g[y][x]
 # Anti-diagonals
 for si in range(h):
  pts=[]
  i,j=si,w-1
  while i<h and j>=0:
   if g[i][j]:pts.append((i,j))
   i+=1;j-=1
  if len(pts)>=3:
   for y,x in pts:r[y][x]=g[y][x]
 return r
"""
        return None

@handler_decorator
class HandlerDetectTriangles(TransformationHandler):
    """Detects triangular patterns"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Right triangles in various orientations
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    color = inp[i, j]
                    
                    # Bottom-left right triangle
                    is_triangle = True
                    size = 1
                    while i + size < h and j + size < w:
                        for di in range(size + 1):
                            for dj in range(size + 1 - di):
                                if i + di < h and j + dj < w:
                                    if inp[i + di, j + dj] != color:
                                        is_triangle = False
                                        break
                            if not is_triangle:
                                break
                        if not is_triangle:
                            break
                        size += 1
                    
                    if size >= 3:
                        test[i, j] = color
                    
                    # Check other orientations similarly
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Simple triangle detection
        test = np.zeros_like(inp)
        for i in range(h-2):
            for j in range(w-2):
                if inp[i, j] != 0:
                    # Check for 3x3 triangle pattern
                    if (inp[i,j] == inp[i+1,j] == inp[i+1,j+1] == 
                        inp[i+2,j] == inp[i+2,j+1] == inp[i+2,j+2]):
                        test[i:i+3, j:j+3] = inp[i, j]
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h-2):
  for j in range(w-2):
   if g[i][j]and g[i][j]==g[i+1][j]==g[i+1][j+1]==g[i+2][j]==g[i+2][j+1]==g[i+2][j+2]:
    for y in range(i,i+3):
     for x in range(j,j+3):r[y][x]=g[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerDetectCircles(TransformationHandler):
    """Detects circular patterns"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Simple circle detection using distance
        for ci in range(1, h-1):
            for cj in range(1, w-1):
                if inp[ci, cj] != 0:
                    # Check for circular pattern
                    for radius in [1, 2, 3]:
                        circle_cells = []
                        # Check 8 directions at radius
                        for di, dj in [(0,1), (1,1), (1,0), (1,-1), 
                                       (0,-1), (-1,-1), (-1,0), (-1,1)]:
                            i, j = ci + radius*di, cj + radius*dj
                            if 0 <= i < h and 0 <= j < w and inp[i, j] != 0:
                                circle_cells.append((i, j))
                        
                        if len(circle_cells) >= 6:  # Most of circle present
                            test[ci, cj] = inp[ci, cj]
                            for i, j in circle_cells:
                                test[i, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for ci in range(1,h-1):
  for cj in range(1,w-1):
   if g[ci][cj]:
    for rad in[1,2,3]:
     cells=[]
     for di,dj in[(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]:
      i,j=ci+rad*di,cj+rad*dj
      if 0<=i<h and 0<=j<w and g[i][j]:cells.append((i,j))
     if len(cells)>=6:
      r[ci][cj]=g[ci][cj]
      for i,j in cells:r[i][j]=g[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerFillShape(TransformationHandler):
    """Fills hollow shapes"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Flood fill approach
        for color in range(1, 10):
            mask = (inp == color)
            if np.any(mask):
                # Find interior points and fill
                filled = np.copy(mask)
                
                # Simple fill: if surrounded by color, fill it
                for i in range(1, h-1):
                    for j in range(1, w-1):
                        if not mask[i, j]:
                            # Check if surrounded
                            surrounded = True
                            # Check if we can reach edge
                            visited = set()
                            stack = [(i, j)]
                            can_reach_edge = False
                            
                            while stack and not can_reach_edge:
                                ci, cj = stack.pop()
                                if (ci, cj) in visited:
                                    continue
                                visited.add((ci, cj))
                                
                                if ci == 0 or ci == h-1 or cj == 0 or cj == w-1:
                                    can_reach_edge = True
                                    break
                                
                                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                                    ni, nj = ci + di, cj + dj
                                    if 0 <= ni < h and 0 <= nj < w:
                                        if not mask[ni, nj] and (ni, nj) not in visited:
                                            stack.append((ni, nj))
                            
                            if not can_reach_edge:
                                filled[i, j] = True
                
                test[filled] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[row[:] for row in g]
 for c in range(1,10):
  # Fill enclosed areas of color c
  for i in range(1,h-1):
   for j in range(1,w-1):
    if r[i][j]==0:
     # Check if enclosed by color c
     enclosed=1
     stack=[(i,j)]
     vis=set()
     while stack and enclosed:
      y,x=stack.pop()
      if(y,x)in vis:continue
      vis.add((y,x))
      if y==0or y==h-1or x==0or x==w-1:enclosed=0;break
      for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
       ny,nx=y+dy,x+dx
       if 0<=ny<h and 0<=nx<w and r[ny][nx]!=c and(ny,nx)not in vis:
        stack.append((ny,nx))
     if enclosed:
      for y,x in vis:r[y][x]=c
 return r
"""
        return None

@handler_decorator
class HandlerOutlineShape(TransformationHandler):
    """Creates outline of filled shapes"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    is_edge = False
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if ni < 0 or ni >= h or nj < 0 or nj >= w or inp[ni, nj] == 0:
                            is_edge = True
                            break
                    
                    if is_edge:
                        test[i, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]:
    edge=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if ni<0or ni>=h or nj<0or nj>=w or not g[ni][nj]:edge=1;break
    if edge:r[i][j]=g[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerDetectSymmetricShapes(TransformationHandler):
    """Detects shapes with horizontal or vertical symmetry"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Find connected components manually
        visited = np.zeros((h, w), dtype=bool)
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0 and not visited[i, j]:
                    # BFS to find component
                    component = []
                    stack = [(i, j)]
                    color = inp[i, j]
                    
                    while stack:
                        ci, cj = stack.pop()
                        if visited[ci, cj]:
                            continue
                        visited[ci, cj] = True
                        component.append((ci, cj))
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < h and 0 <= nj < w and 
                                inp[ni, nj] == color and not visited[ni, nj]):
                                stack.append((ni, nj))
                    
                    # Check symmetry of component
                    if component:
                        min_i = min(p[0] for p in component)
                        max_i = max(p[0] for p in component)
                        min_j = min(p[1] for p in component)
                        max_j = max(p[1] for p in component)
                        
                        # Extract shape
                        shape_h = max_i - min_i + 1
                        shape_w = max_j - min_j + 1
                        shape = np.zeros((shape_h, shape_w))
                        
                        for ci, cj in component:
                            shape[ci - min_i, cj - min_j] = 1
                        
                        # Check symmetries
                        h_symmetric = np.array_equal(shape, np.flipud(shape))
                        v_symmetric = np.array_equal(shape, np.fliplr(shape))
                        
                        if h_symmetric or v_symmetric:
                            for ci, cj in component:
                                test[ci, cj] = inp[ci, cj]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 vis=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]and not vis[i][j]:
    comp=[]
    stack=[(i,j)]
    c=g[i][j]
    while stack:
     y,x=stack.pop()
     if vis[y][x]:continue
     vis[y][x]=1
     comp.append((y,x))
     for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
      ny,nx=y+dy,x+dx
      if 0<=ny<h and 0<=nx<w and g[ny][nx]==c and not vis[ny][nx]:
       stack.append((ny,nx))
    if comp:
     # Check if symmetric
     ys=[p[0]for p in comp]
     xs=[p[1]for p in comp]
     miy,may,mix,max=min(ys),max(ys),min(xs),max(xs)
     # Simple symmetry check
     sym=1
     for y,x in comp:
      # Check horizontal symmetry
      mirror_y=miy+may-y
      if(mirror_y,x)not in comp:sym=0
     if sym:
      for y,x in comp:r[y][x]=g[y][x]
 return r
"""
        return None

@handler_decorator
class HandlerCountShapes(TransformationHandler):
    """Counts connected components"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Count connected components
        h, w = inp.shape
        visited = np.zeros((h, w), dtype=bool)
        count = 0
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0 and not visited[i, j]:
                    # BFS
                    stack = [(i, j)]
                    while stack:
                        ci, cj = stack.pop()
                        if visited[ci, cj]:
                            continue
                        visited[ci, cj] = True
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < h and 0 <= nj < w and 
                                inp[ni, nj] != 0 and not visited[ni, nj]):
                                stack.append((ni, nj))
                    count += 1
        
        # Single value output
        if out.shape == (1, 1) and out[0, 0] == count:
            return True
        
        # Fill with count
        if inp.shape == out.shape and np.all(out == count):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if out.shape == (1, 1):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 vis=[[0]*w for _ in range(h)]
 cnt=0
 for i in range(h):
  for j in range(w):
   if g[i][j]and not vis[i][j]:
    stack=[(i,j)]
    while stack:
     y,x=stack.pop()
     if vis[y][x]:continue
     vis[y][x]=1
     for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
      ny,nx=y+dy,x+dx
      if 0<=ny<h and 0<=nx<w and g[ny][nx]and not vis[ny][nx]:
       stack.append((ny,nx))
    cnt+=1
 return[[cnt]]
"""
        
        if inp.shape == out.shape:
            return f"""def p(g):
 h,w=len(g),len(g[0])
 vis=[[0]*w for _ in range(h)]
 cnt=0
 for i in range(h):
  for j in range(w):
   if g[i][j]and not vis[i][j]:
    stack=[(i,j)]
    while stack:
     y,x=stack.pop()
     if vis[y][x]:continue
     vis[y][x]=1
     for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
      ny,nx=y+dy,x+dx
      if 0<=ny<h and 0<=nx<w and g[ny][nx]and not vis[ny][nx]:
       stack.append((ny,nx))
    cnt+=1
 return[[cnt]*w for _ in range(h)]
"""
        return None

@handler_decorator
class HandlerDetectConnectedComponents(TransformationHandler):
    """Labels each connected component with a different color"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        visited = np.zeros((h, w), dtype=bool)
        test = np.zeros_like(inp)
        comp_id = 1
        
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0 and not visited[i, j]:
                    # BFS to label component
                    stack = [(i, j)]
                    while stack:
                        ci, cj = stack.pop()
                        if visited[ci, cj]:
                            continue
                        visited[ci, cj] = True
                        test[ci, cj] = (comp_id % 9) + 1
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < h and 0 <= nj < w and 
                                inp[ni, nj] != 0 and not visited[ni, nj]):
                                stack.append((ni, nj))
                    comp_id += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 vis=[[0]*w for _ in range(h)]
 r=[[0]*w for _ in range(h)]
 cid=1
 for i in range(h):
  for j in range(w):
   if g[i][j]and not vis[i][j]:
    stack=[(i,j)]
    while stack:
     y,x=stack.pop()
     if vis[y][x]:continue
     vis[y][x]=1
     r[y][x]=(cid%9)+1
     for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
      ny,nx=y+dy,x+dx
      if 0<=ny<h and 0<=nx<w and g[ny][nx]and not vis[ny][nx]:
       stack.append((ny,nx))
    cid+=1
 return r
"""
        return None

@handler_decorator
class HandlerExtractLargestShape(TransformationHandler):
    """Extracts the largest connected component"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        h, w = inp.shape
        visited = np.zeros((h, w), dtype=bool)
        components = []
        
        # Find all components
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0 and not visited[i, j]:
                    component = []
                    color = inp[i, j]
                    stack = [(i, j)]
                    
                    while stack:
                        ci, cj = stack.pop()
                        if visited[ci, cj]:
                            continue
                        visited[ci, cj] = True
                        component.append((ci, cj))
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < h and 0 <= nj < w and 
                                inp[ni, nj] == color and not visited[ni, nj]):
                                stack.append((ni, nj))
                    
                    components.append((component, color))
        
        if components:
            # Find largest
            largest = max(components, key=lambda x: len(x[0]))
            test = np.zeros_like(inp)
            
            for ci, cj in largest[0]:
                test[ci, cj] = largest[1]
            
            return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 vis=[[0]*w for _ in range(h)]
 comps=[]
 for i in range(h):
  for j in range(w):
   if g[i][j]and not vis[i][j]:
    comp=[]
    c=g[i][j]
    stack=[(i,j)]
    while stack:
     y,x=stack.pop()
     if vis[y][x]:continue
     vis[y][x]=1
     comp.append((y,x))
     for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
      ny,nx=y+dy,x+dx
      if 0<=ny<h and 0<=nx<w and g[ny][nx]==c and not vis[ny][nx]:
       stack.append((ny,nx))
    comps.append((comp,c))
 r=[[0]*w for _ in range(h)]
 if comps:
  largest=max(comps,key=lambda x:len(x[0]))
  for y,x in largest[0]:r[y][x]=largest[1]
 return r
"""
        return None

@handler_decorator
class HandlerDetectRepeatingTemplate(TransformationHandler):
    """Detects repeating templates in the grid"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        h, w = inp.shape
        
        # Check for repeating templates
        for th in range(2, min(h//2 + 1, 5)):
            for tw in range(2, min(w//2 + 1, 5)):
                template = inp[:th, :tw]
                
                # Check if template repeats
                is_repeating = True
                for i in range(0, h, th):
                    for j in range(0, w, tw):
                        sub_h = min(th, h - i)
                        sub_w = min(tw, w - j)
                        
                        if not np.array_equal(inp[i:i+sub_h, j:j+sub_w], 
                                            template[:sub_h, :sub_w]):
                            is_repeating = False
                            break
                    if not is_repeating:
                        break
                
                if is_repeating:
                    # Mark first occurrence
                    test = np.zeros_like(inp)
                    test[:th, :tw] = template
                    return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        for th in range(2, min(h//2 + 1, 5)):
            for tw in range(2, min(w//2 + 1, 5)):
                # Check if this template size works
                is_repeating = True
                template = []
                for i in range(th):
                    template.append(inp[i, :tw].tolist())
                
                for i in range(0, h, th):
                    for j in range(0, w, tw):
                        for ti in range(min(th, h-i)):
                            for tj in range(min(tw, w-j)):
                                if inp[i+ti, j+tj] != template[ti][tj]:
                                    is_repeating = False
                
                if is_repeating:
                    test = np.zeros_like(inp)
                    test[:th, :tw] = template
                    if np.array_equal(test, out):
                        # Generate template string
                        template_str = "["
                        for row in template:
                            template_str += "[" + ",".join(map(str, row)) + "],"
                        template_str = template_str[:-1] + "]"
                        
                        return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 t={template_str}
 for i in range({th}):
  for j in range({tw}):r[i][j]=t[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerDetectHollowShapes(TransformationHandler):
    """Detects hollow rectangles and squares"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Find hollow rectangles
        for i in range(h-2):
            for j in range(w-2):
                if inp[i, j] != 0:
                    # Check if top-left of hollow rectangle
                    color = inp[i, j]
                    
                    # Find potential rectangle bounds
                    for h2 in range(i+2, h):
                        for w2 in range(j+2, w):
                            # Check if forms hollow rectangle
                            is_hollow = True
                            
                            # Check borders
                            for y in range(i, h2+1):
                                if inp[y, j] != color or inp[y, w2] != color:
                                    is_hollow = False
                                    break
                            
                            if is_hollow:
                                for x in range(j, w2+1):
                                    if inp[i, x] != color or inp[h2, x] != color:
                                        is_hollow = False
                                        break
                            
                            # Check interior is empty
                            if is_hollow:
                                for y in range(i+1, h2):
                                    for x in range(j+1, w2):
                                        if inp[y, x] != 0:
                                            is_hollow = False
                                            break
                                    if not is_hollow:
                                        break
                            
                            if is_hollow:
                                # Mark hollow rectangle
                                for y in range(i, h2+1):
                                    test[y, j] = color
                                    test[y, w2] = color
                                for x in range(j, w2+1):
                                    test[i, x] = color
                                    test[h2, x] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h-2):
  for j in range(w-2):
   if g[i][j]:
    c=g[i][j]
    for h2 in range(i+2,h):
     for w2 in range(j+2,w):
      ok=1
      # Check borders
      for y in range(i,h2+1):
       if g[y][j]!=c or g[y][w2]!=c:ok=0;break
      if ok:
       for x in range(j,w2+1):
        if g[i][x]!=c or g[h2][x]!=c:ok=0;break
      # Check interior empty
      if ok:
       for y in range(i+1,h2):
        for x in range(j+1,w2):
         if g[y][x]:ok=0;break
      if ok:
       for y in range(i,h2+1):r[y][j]=r[y][w2]=c
       for x in range(j,w2+1):r[i][x]=r[h2][x]=c
 return r
"""
        return None

@handler_decorator
class HandlerShapeTransformation(TransformationHandler):
    """Detects rotated, flipped, or scaled shapes"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check for 90-degree rotation
        if inp.shape[0] == out.shape[1] and inp.shape[1] == out.shape[0]:
            # Check clockwise rotation
            rotated = np.rot90(inp, -1)
            if np.array_equal(rotated, out):
                return True
            
            # Check counter-clockwise
            rotated = np.rot90(inp, 1)
            if np.array_equal(rotated, out):
                return True
        
        # Check for flips
        if inp.shape == out.shape:
            # Horizontal flip
            if np.array_equal(np.fliplr(inp), out):
                return True
            
            # Vertical flip
            if np.array_equal(np.flipud(inp), out):
                return True
            
            # 180-degree rotation
            if np.array_equal(np.rot90(inp, 2), out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # 90-degree clockwise rotation
        if inp.shape[0] == out.shape[1] and inp.shape[1] == out.shape[0]:
            rotated = np.rot90(inp, -1)
            if np.array_equal(rotated, out):
                h, w = inp.shape
                return f"""def p(g):
 h,w={h},{w}
 return[[g[h-1-j][i]for j in range(h)]for i in range(w)]
"""
            
            # Counter-clockwise
            rotated = np.rot90(inp, 1)
            if np.array_equal(rotated, out):
                h, w = inp.shape
                return f"""def p(g):
 h,w={h},{w}
 return[[g[j][w-1-i]for j in range(h)]for i in range(w)]
"""
        
        # Flips
        if inp.shape == out.shape:
            h, w = inp.shape
            
            # Horizontal flip
            if np.array_equal(np.fliplr(inp), out):
                return f"""def p(g):
 return[row[::-1]for row in g]
"""
            
            # Vertical flip
            if np.array_equal(np.flipud(inp), out):
                return f"""def p(g):
 return g[::-1]
"""
            
            # 180-degree rotation
            if np.array_equal(np.rot90(inp, 2), out):
                return f"""def p(g):
 h,w={h},{w}
 return[[g[h-1-i][w-1-j]for j in range(w)]for i in range(h)]
"""
        return None

@handler_decorator
class HandlerDetectShapeIntersections(TransformationHandler):
    """Detects intersection points between shapes"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Find cells with multiple adjacent colors
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    adjacent_colors = set()
                    
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni, nj] != 0:
                            adjacent_colors.add(inp[ni, nj])
                    
                    # Mark if multiple colors meet
                    if len(adjacent_colors) > 1 or inp[i, j] not in adjacent_colors:
                        test[i, j] = 5  # Mark intersection
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]:
    colors=set()
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]:
      colors.add(g[ni][nj])
    if len(colors)>1or g[i][j]not in colors:r[i][j]=5
 return r
"""
        return None

@handler_decorator
class HandlerDetectPatternCompletion(TransformationHandler):
    """Completes partial patterns"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if output adds cells to complete patterns
        diff = out.astype(int) - inp.astype(int)
        added = np.where(diff > 0)
        
        if len(added[0]) > 0:
            # Check if added cells complete rectangles
            h, w = inp.shape
            test = np.copy(inp)
            
            # Try to complete rectangles
            for i in range(h):
                for j in range(w):
                    if inp[i, j] != 0:
                        color = inp[i, j]
                        
                        # Look for partial rectangle patterns
                        # Check if three corners present
                        for di, dj in [(1, 1), (1, -1), (-1, 1), (-1, -1)]:
                            if (0 <= i+di < h and 0 <= j+dj < w and
                                0 <= i+di < h and 0 <= j < w and
                                0 <= i < h and 0 <= j+dj < w):
                                
                                if (inp[i, j] == color and
                                    inp[i+di, j] == color and
                                    inp[i, j+dj] == color and
                                    inp[i+di, j+dj] == 0):
                                    # Complete the rectangle
                                    test[i+di, j+dj] = color
            
            return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[row[:]for row in g]
 for i in range(h):
  for j in range(w):
   if g[i][j]:
    c=g[i][j]
    # Complete rectangles
    for di,dj in[(1,1),(1,-1),(-1,1),(-1,-1)]:
     if(0<=i+di<h and 0<=j+dj<w and
        g[i][j]==c and i+di<h and j+dj<w and
        0<=i+di<h and g[i+di][j]==c and
        0<=j+dj<w and g[i][j+dj]==c and
        not g[i+di][j+dj]):
      r[i+di][j+dj]=c
 return r
"""
        return None

@handler_decorator
class HandlerDetectLPattern(TransformationHandler):
    """Detects L-shaped patterns"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # L-patterns in all orientations
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    color = inp[i, j]
                    
                    # Check for L pattern (3x2 or 2x3)
                    # Vertical L
                    if (i+2 < h and j+1 < w and
                        inp[i+1, j] == color and inp[i+2, j] == color and
                        inp[i+2, j+1] == color):
                        test[i:i+3, j:j+2] = color
                    
                    # Horizontal L
                    elif (i+1 < h and j+2 < w and
                          inp[i, j+1] == color and inp[i, j+2] == color and
                          inp[i+1, j+2] == color):
                        test[i:i+2, j:j+3] = color
                    
                    # Other orientations...
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]:
    c=g[i][j]
    # Vertical L
    if(i+2<h and j+1<w and g[i+1][j]==c and g[i+2][j]==c and g[i+2][j+1]==c):
     for y in range(i,i+3):
      for x in range(j,j+2):r[y][x]=c
    # Horizontal L
    elif(i+1<h and j+2<w and g[i][j+1]==c and g[i][j+2]==c and g[i+1][j+2]==c):
     for y in range(i,i+2):
      for x in range(j,j+3):r[y][x]=c
 return r
"""
        return None

@handler_decorator
class HandlerExtractSmallestShape(TransformationHandler):
    """Extracts the smallest connected component"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        h, w = inp.shape
        visited = np.zeros((h, w), dtype=bool)
        components = []
        
        # Find all components
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0 and not visited[i, j]:
                    component = []
                    color = inp[i, j]
                    stack = [(i, j)]
                    
                    while stack:
                        ci, cj = stack.pop()
                        if visited[ci, cj]:
                            continue
                        visited[ci, cj] = True
                        component.append((ci, cj))
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < h and 0 <= nj < w and 
                                inp[ni, nj] == color and not visited[ni, nj]):
                                stack.append((ni, nj))
                    
                    if component:
                        components.append((component, color))
        
        if components and len(components) > 1:
            # Find smallest
            smallest = min(components, key=lambda x: len(x[0]))
            test = np.zeros_like(inp)
            
            for ci, cj in smallest[0]:
                test[ci, cj] = smallest[1]
            
            return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 vis=[[0]*w for _ in range(h)]
 comps=[]
 for i in range(h):
  for j in range(w):
   if g[i][j]and not vis[i][j]:
    comp=[]
    c=g[i][j]
    stack=[(i,j)]
    while stack:
     y,x=stack.pop()
     if vis[y][x]:continue
     vis[y][x]=1
     comp.append((y,x))
     for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
      ny,nx=y+dy,x+dx
      if 0<=ny<h and 0<=nx<w and g[ny][nx]==c and not vis[ny][nx]:
       stack.append((ny,nx))
    if comp:comps.append((comp,c))
 r=[[0]*w for _ in range(h)]
 if len(comps)>1:
  smallest=min(comps,key=lambda x:len(x[0]))
  for y,x in smallest[0]:r[y][x]=smallest[1]
 return r
"""
        return None

@handler_decorator
class HandlerDetectSquares(TransformationHandler):
    """Detects filled and hollow squares"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Find squares
        for i in range(h):
            for j in range(w):
                if inp[i, j] != 0:
                    color = inp[i, j]
                    
                    # Check different square sizes
                    for size in range(2, min(h-i, w-j)+1):
                        # Check if forms a square
                        is_square = True
                        
                        # Check filled square
                        for di in range(size):
                            for dj in range(size):
                                if inp[i+di, j+dj] != color:
                                    is_square = False
                                    break
                            if not is_square:
                                break
                        
                        if is_square:
                            # Mark square
                            for di in range(size):
                                for dj in range(size):
                                    test[i+di, j+dj] = color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return f"""def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]:
    c=g[i][j]
    for s in range(2,min(h-i,w-j)+1):
     ok=1
     for di in range(s):
      for dj in range(s):
       if g[i+di][j+dj]!=c:ok=0;break
      if not ok:break
     if ok:
      for di in range(s):
       for dj in range(s):r[i+di][j+dj]=c
 return r
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# SEQUENCE AND PATTERN GENERATION - ENHANCED VERSION
# --------------------------------------------------------------------------------

@handler_decorator
class HandlerGenerateSequence(TransformationHandler):
    """Generates simple incrementing sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Single row incrementing sequence
        if h == 1:
            if np.array_equal(out[0], np.arange(w) % 10):
                return True
            # Starting from different values
            for start in range(1, 10):
                if np.array_equal(out[0], np.arange(start, start + w) % 10):
                    return True
        
        # Column-wise sequences
        if w == 1:
            if np.array_equal(out[:, 0], np.arange(h) % 10):
                return True
        
        # Full grid sequential
        flat = out.flatten()
        if np.array_equal(flat, np.arange(len(flat)) % 10):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        if h == 1 and np.array_equal(out[0], np.arange(w) % 10):
            return f"""def p(g):
 return[[i%10for i in range({w})]]
"""
        
        if h == 1:
            for start in range(1, 10):
                if np.array_equal(out[0], np.arange(start, start + w) % 10):
                    return f"""def p(g):
 return[[(i+{start})%10for i in range({w})]]
"""
        
        if w == 1 and np.array_equal(out[:, 0], np.arange(h) % 10):
            return f"""def p(g):
 return[[i%10]for i in range({h})]
"""
        
        flat = out.flatten()
        if np.array_equal(flat, np.arange(len(flat)) % 10):
            return f"""def p(g):
 return[[(i*{w}+j)%10for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerFibonacciPattern(TransformationHandler):
    """Generates Fibonacci and generalized Fibonacci sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        if len(flat) >= 3:
            # Standard Fibonacci
            fib = [flat[0], flat[1]]
            for i in range(2, len(flat)):
                fib.append((fib[-1] + fib[-2]) % 10)
            
            if np.array_equal(flat, np.array(fib[:len(flat)])):
                return True
            
            # Tribonacci (sum of last 3)
            if len(flat) >= 4:
                trib = [flat[0], flat[1], flat[2]]
                for i in range(3, len(flat)):
                    trib.append((trib[-1] + trib[-2] + trib[-3]) % 10)
                
                if np.array_equal(flat, np.array(trib[:len(flat)])):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        if len(flat) >= 3:
            # Standard Fibonacci
            fib = [flat[0], flat[1]]
            for i in range(2, len(flat)):
                fib.append((fib[-1] + fib[-2]) % 10)
            
            if np.array_equal(flat, np.array(fib[:len(flat)])):
                a, b = flat[0], flat[1]
                return f"""def p(g):
 h,w={h},{w}
 f=[{a},{b}]
 for i in range(2,h*w):
  f.append((f[-1]+f[-2])%10)
 res=[]
 for i in range(h):
  res.append(f[i*w:(i+1)*w])
 return res
"""
            
            # Tribonacci
            if len(flat) >= 4:
                trib = [flat[0], flat[1], flat[2]]
                for i in range(3, len(flat)):
                    trib.append((trib[-1] + trib[-2] + trib[-3]) % 10)
                
                if np.array_equal(flat, np.array(trib[:len(flat)])):
                    a, b, c = flat[0], flat[1], flat[2]
                    return f"""def p(g):
 h,w={h},{w}
 f=[{a},{b},{c}]
 for i in range(3,h*w):
  f.append((f[-1]+f[-2]+f[-3])%10)
 res=[]
 for i in range(h):
  res.append(f[i*w:(i+1)*w])
 return res
"""
        return None

@handler_decorator
class HandlerPrimePattern(TransformationHandler):
    """Generates patterns based on prime numbers"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # First 30 primes for checking
        primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101,103,107,109,113]
        
        # Direct prime sequence mod 10
        if len(flat) <= len(primes):
            test = [p % 10 for p in primes[:len(flat)]]
            if np.array_equal(flat, test):
                return True
        
        # Prime indices (1 if prime, 0 if not)
        test = []
        for i in range(len(flat)):
            is_prime = i in primes
            test.append(1 if is_prime else 0)
        if np.array_equal(flat, test):
            return True
        
        # Sum of prime factors
        test = []
        for i in range(len(flat)):
            if i < 2:
                test.append(0)
            else:
                factor_sum = 0
                n = i
                for p in primes:
                    if p > n:
                        break
                    while n % p == 0:
                        factor_sum += p
                        n //= p
                test.append(factor_sum % 10)
        
        if np.array_equal(flat, test):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        # For small grids, hardcode prime sequence
        primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]
        
        if len(flat) <= 25:
            test = [p % 10 for p in primes[:len(flat)]]
            if np.array_equal(flat, test):
                prime_str = ','.join(str(p % 10) for p in primes[:len(flat)])
                return f"""def p(g):
 v=[{prime_str}]
 return[v[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        
        # Prime indicator pattern
        if len(flat) <= 30:
            test = []
            prime_set = set(primes)
            for i in range(len(flat)):
                test.append(1 if i in prime_set else 0)
            
            if np.array_equal(flat, test):
                # Generate compact prime checking
                return f"""def p(g):
 res=[]
 for i in range({h}):
  r=[]
  for j in range({w}):
   n=i*{w}+j
   v=1if n in[2,3,5,7,11,13,17,19,23,29]else 0
   r.append(v)
  res.append(r)
 return res
"""
        return None

@handler_decorator
class HandlerArithmeticSequence(TransformationHandler):
    """Generates arithmetic progressions with various differences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        if len(flat) < 2:
            return False
        
        # Check for arithmetic sequence
        diff = flat[1] - flat[0]
        for i in range(2, len(flat)):
            if flat[i] - flat[i-1] != diff:
                return False
        
        return True
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        start = flat[0]
        diff = flat[1] - flat[0] if len(flat) > 1 else 0
        
        # Optimize for special cases
        if diff == 0:
            return f"""def p(g):
 return[[{start}]*{w}for _ in range({h})]
"""
        elif diff == 1 and start == 0:
            return f"""def p(g):
 return[[(i*{w}+j)%10for j in range({w})]for i in range({h})]
"""
        else:
            return f"""def p(g):
 h,w={h},{w}
 res=[]
 v={start}
 for i in range(h):
  row=[]
  for j in range(w):
   row.append(v%10)
   v+={diff}
  res.append(row)
 return res
"""

@handler_decorator
class HandlerGeometricSequence(TransformationHandler):
    """Generates geometric progressions"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        if len(flat) < 3:
            return False
        
        # Check for multiplicative pattern mod 10
        for ratio in range(2, 10):
            test = [flat[0]]
            for i in range(1, len(flat)):
                test.append((test[-1] * ratio) % 10)
            
            if np.array_equal(flat, test):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        for ratio in range(2, 10):
            test = [flat[0]]
            for i in range(1, len(flat)):
                test.append((test[-1] * ratio) % 10)
            
            if np.array_equal(flat, test):
                return f"""def p(g):
 h,w={h},{w}
 res=[]
 v={flat[0]}
 for i in range(h):
  row=[]
  for j in range(w):
   row.append(v)
   v=(v*{ratio})%10
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerTriangularNumbers(TransformationHandler):
    """Generates triangular number sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Triangular numbers: T(n) = n(n+1)/2
        triangular = [n*(n+1)//2 % 10 for n in range(len(flat))]
        if np.array_equal(flat, triangular):
            return True
        
        # Starting from 1
        triangular = [(n+1)*(n+2)//2 % 10 for n in range(len(flat))]
        if np.array_equal(flat, triangular):
            return True
        
        # With offset
        for offset in range(1, 10):
            triangular = [(n*(n+1)//2 + offset) % 10 for n in range(len(flat))]
            if np.array_equal(flat, triangular):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        triangular = [n*(n+1)//2 % 10 for n in range(len(flat))]
        if np.array_equal(flat, triangular):
            return f"""def p(g):
 return[[(n*(n+1)//2)%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        
        triangular = [(n+1)*(n+2)//2 % 10 for n in range(len(flat))]
        if np.array_equal(flat, triangular):
            return f"""def p(g):
 return[[((n+1)*(n+2)//2)%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        
        for offset in range(1, 10):
            triangular = [(n*(n+1)//2 + offset) % 10 for n in range(len(flat))]
            if np.array_equal(flat, triangular):
                return f"""def p(g):
 return[[((n*(n+1)//2)+{offset})%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerSquareNumbers(TransformationHandler):
    """Generates square number sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Square numbers
        squares = [n*n % 10 for n in range(len(flat))]
        if np.array_equal(flat, squares):
            return True
        
        # Starting from 1
        squares = [(n+1)*(n+1) % 10 for n in range(len(flat))]
        if np.array_equal(flat, squares):
            return True
        
        # Centered square numbers: n^2 + (n-1)^2
        centered = [n*n + max(0, (n-1)*(n-1)) for n in range(len(flat))]
        centered = [c % 10 for c in centered]
        if np.array_equal(flat, centered):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        squares = [n*n % 10 for n in range(len(flat))]
        if np.array_equal(flat, squares):
            return f"""def p(g):
 return[[(n*n)%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        
        squares = [(n+1)*(n+1) % 10 for n in range(len(flat))]
        if np.array_equal(flat, squares):
            return f"""def p(g):
 return[[((n+1)**2)%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        
        centered = [n*n + max(0, (n-1)*(n-1)) for n in range(len(flat))]
        centered = [c % 10 for c in centered]
        if np.array_equal(flat, centered):
            return f"""def p(g):
 return[[(n*n+max(0,(n-1)**2))%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerCubicNumbers(TransformationHandler):
    """Generates cubic and higher power sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Cubic numbers
        cubes = [n**3 % 10 for n in range(len(flat))]
        if np.array_equal(flat, cubes):
            return True
        
        # Fourth powers
        fourths = [n**4 % 10 for n in range(len(flat))]
        if np.array_equal(flat, fourths):
            return True
        
        # Fifth powers
        fifths = [n**5 % 10 for n in range(len(flat))]
        if np.array_equal(flat, fifths):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        cubes = [n**3 % 10 for n in range(len(flat))]
        if np.array_equal(flat, cubes):
            return f"""def p(g):
 return[[(n**3)%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        
        fourths = [n**4 % 10 for n in range(len(flat))]
        if np.array_equal(flat, fourths):
            return f"""def p(g):
 return[[(n**4)%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        
        fifths = [n**5 % 10 for n in range(len(flat))]
        if np.array_equal(flat, fifths):
            return f"""def p(g):
 return[[(n**5)%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerPentagonalNumbers(TransformationHandler):
    """Generates pentagonal number sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Pentagonal numbers: P(n) = n(3n-1)/2
        pentagonal = [n*(3*n-1)//2 % 10 for n in range(len(flat))]
        if np.array_equal(flat, pentagonal):
            return True
        
        # Generalized pentagonal (includes negative n)
        gen_pent = []
        for i in range(len(flat)):
            n = (i + 1) // 2
            if i % 2 == 0:
                gen_pent.append((n * (3*n - 1) // 2) % 10)
            else:
                gen_pent.append((n * (3*n + 1) // 2) % 10)
        
        if np.array_equal(flat, gen_pent):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        pentagonal = [n*(3*n-1)//2 % 10 for n in range(len(flat))]
        if np.array_equal(flat, pentagonal):
            return f"""def p(g):
 return[[(n*(3*n-1)//2)%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        
        # Generalized pentagonal
        gen_pent = []
        for i in range(len(flat)):
            n = (i + 1) // 2
            if i % 2 == 0:
                gen_pent.append((n * (3*n - 1) // 2) % 10)
            else:
                gen_pent.append((n * (3*n + 1) // 2) % 10)
        
        if np.array_equal(flat, gen_pent):
            return f"""def p(g):
 res=[]
 idx=0
 for i in range({h}):
  row=[]
  for j in range({w}):
   n=(idx+1)//2
   v=(n*(3*n-1)//2)%10if idx%2==0else(n*(3*n+1)//2)%10
   row.append(v)
   idx+=1
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerHexagonalNumbers(TransformationHandler):
    """Generates hexagonal number sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Hexagonal numbers: H(n) = 2n^2 - n
        hexagonal = [(2*n*n - n) % 10 for n in range(len(flat))]
        if np.array_equal(flat, hexagonal):
            return True
        
        # Centered hexagonal: 3n(n-1) + 1
        centered_hex = [3*n*(n-1) + 1 for n in range(len(flat))]
        centered_hex = [h % 10 for h in centered_hex]
        if np.array_equal(flat, centered_hex):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        hexagonal = [(2*n*n - n) % 10 for n in range(len(flat))]
        if np.array_equal(flat, hexagonal):
            return f"""def p(g):
 return[[(2*n*n-n)%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        
        centered_hex = [(3*n*(n-1) + 1) % 10 for n in range(len(flat))]
        if np.array_equal(flat, centered_hex):
            return f"""def p(g):
 return[[(3*n*(n-1)+1)%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerPowerSequence(TransformationHandler):
    """Generates sequences based on powers of fixed bases"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Powers of 2, 3, etc.
        for base in range(2, 10):
            powers = [pow(base, i, 10) for i in range(len(flat))]
            if np.array_equal(flat, powers):
                return True
        
        # Alternating sign powers
        for base in range(2, 10):
            alt_powers = [pow(-base, i, 10) if i % 2 else pow(base, i, 10) for i in range(len(flat))]
            if np.array_equal(flat, alt_powers):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        for base in range(2, 10):
            powers = [pow(base, i, 10) for i in range(len(flat))]
            if np.array_equal(flat, powers):
                return f"""def p(g):
 return[[pow({base},i*{w}+j,10)for j in range({w})]for i in range({h})]
"""
        
        # For alternating powers, use explicit calculation
        for base in range(2, 10):
            alt_powers = []
            val = 1
            for i in range(len(flat)):
                alt_powers.append(val % 10)
                val = (val * base) % 10 if i % 2 == 0 else (-val * base) % 10
            
            if np.array_equal(flat, alt_powers):
                return f"""def p(g):
 res=[]
 v=1
 for i in range({h}):
  row=[]
  for j in range({w}):
   row.append(v%10)
   idx=i*{w}+j
   v=(v*{base})%10if idx%2==0else(-v*{base})%10
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerFactorialSequence(TransformationHandler):
    """Generates factorial sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Factorial mod 10 has a short cycle
        fact_cycle = [1, 1, 2, 6, 4, 0, 0, 0, 0, 0]
        
        # Direct factorial sequence
        factorials = []
        for n in range(len(flat)):
            if n < 10:
                factorials.append(fact_cycle[n])
            else:
                factorials.append(0)
        
        if np.array_equal(flat, factorials):
            return True
        
        # Subfactorial (derangements)
        subfact = [1, 0, 1, 2, 9, 4, 5, 4, 9, 6]  # !n mod 10
        if len(flat) <= 10 and np.array_equal(flat, subfact[:len(flat)]):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        fact_cycle = [1, 1, 2, 6, 4, 0, 0, 0, 0, 0]
        factorials = []
        for n in range(len(flat)):
            if n < 10:
                factorials.append(fact_cycle[n])
            else:
                factorials.append(0)
        
        if np.array_equal(flat, factorials):
            return f"""def p(g):
 f=[1,1,2,6,4,0,0,0,0,0]
 return[[f[min(i*{w}+j,9)]for j in range({w})]for i in range({h})]
"""
        
        subfact = [1, 0, 1, 2, 9, 4, 5, 4, 9, 6]
        if len(flat) <= 10 and np.array_equal(flat, subfact[:len(flat)]):
            sub_str = ','.join(map(str, subfact[:len(flat)]))
            return f"""def p(g):
 s=[{sub_str}]
 return[s[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerDigitalRoot(TransformationHandler):
    """Generates sequences based on digital roots and digit operations"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Digital root sequence
        digital_roots = []
        for i in range(len(flat)):
            dr = i % 9
            if dr == 0 and i > 0:
                dr = 9
            digital_roots.append(dr % 10)
        
        if np.array_equal(flat, digital_roots):
            return True
        
        # Digit product
        digit_products = []
        for i in range(len(flat)):
            prod = 1
            for d in str(i):
                prod *= int(d)
            digit_products.append(prod % 10)
        
        if np.array_equal(flat, digit_products):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        # Digital root
        digital_roots = []
        for i in range(len(flat)):
            dr = i % 9
            if dr == 0 and i > 0:
                dr = 9
            digital_roots.append(dr % 10)
        
        if np.array_equal(flat, digital_roots):
            return f"""def p(g):
 return[[9if(i*{w}+j)%9==0and i*{w}+j>0else(i*{w}+j)%9for j in range({w})]for i in range({h})]
"""
        
        # Digit product - for small grids
        if len(flat) <= 20:
            digit_products = []
            for i in range(len(flat)):
                prod = 1
                for d in str(i):
                    prod *= int(d)
                digit_products.append(prod % 10)
            
            if np.array_equal(flat, digit_products):
                prod_str = ','.join(map(str, digit_products))
                return f"""def p(g):
 d=[{prod_str}]
 return[d[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerCollatzSequence(TransformationHandler):
    """Generates Collatz sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        if len(flat) < 3:
            return False
        
        # Check for Collatz sequence starting from various values
        for start in range(1, 20):
            collatz = [start]
            n = start
            while len(collatz) < len(flat):
                if n % 2 == 0:
                    n = n // 2
                else:
                    n = 3 * n + 1
                collatz.append(n % 10)
            
            if np.array_equal(flat, [c % 10 for c in collatz[:len(flat)]]):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        # Find which starting value works
        for start in range(1, 20):
            collatz = [start]
            n = start
            while len(collatz) < len(flat):
                if n % 2 == 0:
                    n = n // 2
                else:
                    n = 3 * n + 1
                collatz.append(n % 10)
            
            if np.array_equal(flat, [c % 10 for c in collatz[:len(flat)]]):
                # Pre-compute the sequence for code golf
                seq_str = ','.join(str(c % 10) for c in collatz[:len(flat)])
                return f"""def p(g):
 c=[{seq_str}]
 return[c[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerBellNumbers(TransformationHandler):
    """Generates Bell number sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Bell numbers (number of partitions of a set)
        # First few: 1, 1, 2, 5, 15, 52, 203...
        bell_mod10 = [1, 1, 2, 5, 5, 2, 3, 7, 7, 0]  # B(n) mod 10 for n=0..9
        
        if len(flat) <= 10 and np.array_equal(flat, bell_mod10[:len(flat)]):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        bell_mod10 = [1, 1, 2, 5, 5, 2, 3, 7, 7, 0]
        
        if len(flat) <= 10 and np.array_equal(flat, bell_mod10[:len(flat)]):
            bell_str = ','.join(map(str, bell_mod10[:len(flat)]))
            return f"""def p(g):
 b=[{bell_str}]
 return[b[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerPartitionNumbers(TransformationHandler):
    """Generates partition function sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Partition function p(n) - number of ways to partition n
        # First few: 1, 1, 2, 3, 5, 7, 11, 15, 22, 30...
        partition_mod10 = [1, 1, 2, 3, 5, 7, 1, 5, 2, 0]  # p(n) mod 10
        
        if len(flat) <= 10 and np.array_equal(flat, partition_mod10[:len(flat)]):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        partition_mod10 = [1, 1, 2, 3, 5, 7, 1, 5, 2, 0]
        
        if len(flat) <= 10 and np.array_equal(flat, partition_mod10[:len(flat)]):
            part_str = ','.join(map(str, partition_mod10[:len(flat)]))
            return f"""def p(g):
 p=[{part_str}]
 return[p[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerThueMoreSequence(TransformationHandler):
    """Generates Thue-Morse sequence"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Thue-Morse: count 1s in binary representation mod 2
        thue_morse = []
        for i in range(len(flat)):
            thue_morse.append(bin(i).count('1') % 2)
        
        if np.array_equal(flat, thue_morse):
            return True
        
        # Inverted Thue-Morse
        inv_thue = [1 - t for t in thue_morse]
        if np.array_equal(flat, inv_thue):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        thue_morse = [bin(i).count('1') % 2 for i in range(len(flat))]
        
        if np.array_equal(flat, thue_morse):
            return f"""def p(g):
 return[[bin(i*{w}+j).count('1')%2for j in range({w})]for i in range({h})]
"""
        
        inv_thue = [1 - t for t in thue_morse]
        if np.array_equal(flat, inv_thue):
            return f"""def p(g):
 return[[1-bin(i*{w}+j).count('1')%2for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerAlternatingSequence(TransformationHandler):
    """Generates alternating patterns"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        if len(flat) < 2:
            return False
        
        # Simple alternating between two values
        if len(set(flat)) == 2:
            vals = sorted(list(set(flat)))
            alternating = [vals[i%2] for i in range(len(flat))]
            if np.array_equal(flat, alternating):
                return True
            
            # Starting with second value
            alternating = [vals[(i+1)%2] for i in range(len(flat))]
            if np.array_equal(flat, alternating):
                return True
        
        # Alternating signs
        if len(flat) >= 3:
            alt_signs = [flat[0]]
            sign = 1
            for i in range(1, len(flat)):
                sign *= -1
                alt_signs.append((alt_signs[0] + sign * i) % 10)
            
            if np.array_equal(flat, alt_signs):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        if len(set(flat)) == 2:
            vals = sorted(list(set(flat)))
            alternating = [vals[i%2] for i in range(len(flat))]
            
            if np.array_equal(flat, alternating):
                return f"""def p(g):
 v=[{vals[0]},{vals[1]}]
 return[[v[(i*{w}+j)%2]for j in range({w})]for i in range({h})]
"""
            
            alternating = [vals[(i+1)%2] for i in range(len(flat))]
            if np.array_equal(flat, alternating):
                return f"""def p(g):
 v=[{vals[0]},{vals[1]}]
 return[[v[(i*{w}+j+1)%2]for j in range({w})]for i in range({h})]
"""
        
        # Alternating signs
        if len(flat) >= 3:
            start = flat[0]
            return f"""def p(g):
 return[[({start}+(-1)**(i*{w}+j)*(i*{w}+j))%10for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerRepeatingPattern(TransformationHandler):
    """Generates repeating patterns of various lengths"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Check for repeating patterns of length 2 to 10
        for pattern_len in range(2, min(11, len(flat)//2 + 1)):
            pattern = flat[:pattern_len]
            repeating = list(pattern) * (len(flat) // pattern_len + 1)
            if np.array_equal(flat, repeating[:len(flat)]):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        for pattern_len in range(2, min(11, len(flat)//2 + 1)):
            pattern = flat[:pattern_len]
            repeating = list(pattern) * (len(flat) // pattern_len + 1)
            if np.array_equal(flat, repeating[:len(flat)]):
                pattern_str = ','.join(map(str, pattern))
                return f"""def p(g):
 pat=[{pattern_str}]
 return[[pat[(i*{w}+j)%{pattern_len}]for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerPalindromicSequence(TransformationHandler):
    """Generates palindromic patterns"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        
        # Row-wise palindromes
        if all(list(row) == list(row)[::-1] for row in out):
            return True
        
        # Column-wise palindromes
        if all(list(out[:, j]) == list(out[:, j])[::-1] for j in range(out.shape[1])):
            return True
        
        # Full grid palindrome
        flat = out.flatten()
        if list(flat) == list(flat)[::-1]:
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Row-wise palindromes with same pattern
        if all(list(row) == list(row)[::-1] for row in out):
            pattern = out[0]
            if all(np.array_equal(row, pattern) for row in out):
                pat_str = ','.join(map(str, pattern))
                return f"""def p(g):
 return[[{pat_str}]for _ in range({h})]
"""
            
            # Different palindromes per row
            if h <= 5:
                rows = []
                for row in out:
                    rows.append('[' + ','.join(map(str, row)) + ']')
                rows_str = ','.join(rows)
                return f"""def p(g):
 return[{rows_str}]
"""
        
        # Full grid palindrome
        flat = out.flatten()
        if list(flat) == list(flat)[::-1] and len(flat) <= 30:
            if w % 2 == 1:  # Odd width, can create palindrome row by row
                half = w // 2
                return f"""def p(g):
 res=[]
 v=0
 for i in range({h}):
  row=[]
  for j in range({half}):
   row.append(v%10)
   v+=1
  row.append(v%10)
  v+=1
  for j in range({half}):
   row.append(row[{half}-1-j])
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerRunLengthSequence(TransformationHandler):
    """Generates sequences based on run-length encoding patterns"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Look-and-say sequence elements
        # 1, 11, 21, 1211, 111221, 312211...
        # Taking digit sums mod 10: 1, 2, 3, 5, 6, 8...
        look_say_sums = [1, 2, 3, 5, 6, 8, 3, 7, 8, 8]
        
        if len(flat) <= 10 and np.array_equal(flat, look_say_sums[:len(flat)]):
            return True
        
        # Van Eck sequence
        van_eck = [0, 0, 1, 0, 2, 0, 2, 2, 1, 6]
        if len(flat) <= 10 and np.array_equal(flat, van_eck[:len(flat)]):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        look_say_sums = [1, 2, 3, 5, 6, 8, 3, 7, 8, 8]
        if len(flat) <= 10 and np.array_equal(flat, look_say_sums[:len(flat)]):
            seq_str = ','.join(map(str, look_say_sums[:len(flat)]))
            return f"""def p(g):
 s=[{seq_str}]
 return[s[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        
        van_eck = [0, 0, 1, 0, 2, 0, 2, 2, 1, 6]
        if len(flat) <= 10 and np.array_equal(flat, van_eck[:len(flat)]):
            seq_str = ','.join(map(str, van_eck[:len(flat)]))
            return f"""def p(g):
 v=[{seq_str}]
 return[v[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerBitCountSequence(TransformationHandler):
    """Generates sequences based on bit manipulation"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Population count (number of 1s in binary)
        popcount = [bin(i).count('1') % 10 for i in range(len(flat))]
        if np.array_equal(flat, popcount):
            return True
        
        # Binary weight (position of rightmost 1)
        binary_weight = []
        for i in range(len(flat)):
            if i == 0:
                binary_weight.append(0)
            else:
                weight = (i & -i).bit_length() - 1
                binary_weight.append(weight % 10)
        
        if np.array_equal(flat, binary_weight):
            return True
        
        # Gray code
        gray_code = [i ^ (i >> 1) for i in range(len(flat))]
        gray_mod = [g % 10 for g in gray_code]
        if np.array_equal(flat, gray_mod):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        popcount = [bin(i).count('1') % 10 for i in range(len(flat))]
        if np.array_equal(flat, popcount):
            return f"""def p(g):
 return[[bin(i*{w}+j).count('1')%10for j in range({w})]for i in range({h})]
"""
        
        # Binary weight
        binary_weight = []
        for i in range(len(flat)):
            if i == 0:
                binary_weight.append(0)
            else:
                weight = (i & -i).bit_length() - 1
                binary_weight.append(weight % 10)
        
        if np.array_equal(flat, binary_weight):
            return f"""def p(g):
 res=[]
 for i in range({h}):
  row=[]
  for j in range({w}):
   n=i*{w}+j
   row.append(0if n==0else((n&-n).bit_length()-1)%10)
  res.append(row)
 return res
"""
        
        # Gray code
        gray_mod = [(i ^ (i >> 1)) % 10 for i in range(len(flat))]
        if np.array_equal(flat, gray_mod):
            return f"""def p(g):
 return[[(n^(n>>1))%10for n in range(i*{w},(i+1)*{w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerDivisorSequence(TransformationHandler):
    """Generates sequences based on divisor functions"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Number of divisors
        divisor_count = []
        for i in range(len(flat)):
            if i == 0:
                divisor_count.append(0)
            else:
                count = sum(1 for d in range(1, i+1) if i % d == 0)
                divisor_count.append(count % 10)
        
        if np.array_equal(flat, divisor_count):
            return True
        
        # Sum of divisors
        divisor_sum = []
        for i in range(len(flat)):
            if i == 0:
                divisor_sum.append(0)
            else:
                s = sum(d for d in range(1, i+1) if i % d == 0)
                divisor_sum.append(s % 10)
        
        if np.array_equal(flat, divisor_sum):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        # For small grids, hardcode the sequences
        if len(flat) <= 20:
            divisor_count = []
            for i in range(len(flat)):
                if i == 0:
                    divisor_count.append(0)
                else:
                    count = sum(1 for d in range(1, i+1) if i % d == 0)
                    divisor_count.append(count % 10)
            
            if np.array_equal(flat, divisor_count):
                div_str = ','.join(map(str, divisor_count))
                return f"""def p(g):
 d=[{div_str}]
 return[d[i*{w}:(i+1)*{w}]for i in range({h})]
"""
            
            divisor_sum = []
            for i in range(len(flat)):
                if i == 0:
                    divisor_sum.append(0)
                else:
                    s = sum(d for d in range(1, i+1) if i % d == 0)
                    divisor_sum.append(s % 10)
            
            if np.array_equal(flat, divisor_sum):
                sum_str = ','.join(map(str, divisor_sum))
                return f"""def p(g):
 s=[{sum_str}]
 return[s[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerModularCycle(TransformationHandler):
    """Generates modular arithmetic cycles"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Simple modular cycles
        for mod in range(2, 10):
            cycle = [i % mod for i in range(len(flat))]
            if np.array_equal(flat, cycle):
                return True
            
            # Multiplicative cycles
            for mult in range(2, 10):
                cycle = [(i * mult) % mod for i in range(len(flat))]
                if np.array_equal(flat, cycle):
                    return True
            
            # Power cycles
            cycle = [pow(2, i, mod) for i in range(len(flat))]
            if np.array_equal(flat, cycle):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        for mod in range(2, 10):
            cycle = [i % mod for i in range(len(flat))]
            if np.array_equal(flat, cycle):
                return f"""def p(g):
 return[[(i*{w}+j)%{mod}for j in range({w})]for i in range({h})]
"""
            
            for mult in range(2, 10):
                cycle = [(i * mult) % mod for i in range(len(flat))]
                if np.array_equal(flat, cycle):
                    return f"""def p(g):
 return[[((i*{w}+j)*{mult})%{mod}for j in range({w})]for i in range({h})]
"""
            
            cycle = [pow(2, i, mod) for i in range(len(flat))]
            if np.array_equal(flat, cycle):
                return f"""def p(g):
 return[[pow(2,i*{w}+j,{mod})for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerRecurrenceRelation(TransformationHandler):
    """Generates linear recurrence sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        if len(flat) < 4:
            return False
        
        # Second-order linear recurrence: a(n) = c1*a(n-1) + c2*a(n-2)
        for c1 in range(-3, 4):
            for c2 in range(-3, 4):
                if c1 == 0 and c2 == 0:
                    continue
                
                test = [flat[0], flat[1]]
                for i in range(2, len(flat)):
                    test.append((c1 * test[-1] + c2 * test[-2]) % 10)
                
                if np.array_equal(flat, test[:len(flat)]):
                    return True
        
        # Third-order recurrence
        if len(flat) >= 5:
            for c1 in range(-2, 3):
                for c2 in range(-2, 3):
                    for c3 in range(-2, 3):
                        if c1 == 0 and c2 == 0 and c3 == 0:
                            continue
                        
                        test = [flat[0], flat[1], flat[2]]
                        for i in range(3, len(flat)):
                            test.append((c1 * test[-1] + c2 * test[-2] + c3 * test[-3]) % 10)
                        
                        if np.array_equal(flat, test[:len(flat)]):
                            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        # Second-order recurrence
        for c1 in range(-3, 4):
            for c2 in range(-3, 4):
                if c1 == 0 and c2 == 0:
                    continue
                
                test = [flat[0], flat[1]]
                for i in range(2, len(flat)):
                    test.append((c1 * test[-1] + c2 * test[-2]) % 10)
                
                if np.array_equal(flat, test[:len(flat)]):
                    return f"""def p(g):
 h,w={h},{w}
 s=[{flat[0]},{flat[1]}]
 for i in range(2,h*w):
  s.append(({c1}*s[-1]+{c2}*s[-2])%10)
 res=[]
 for i in range(h):
  res.append(s[i*w:(i+1)*w])
 return res
"""
        
        # Third-order recurrence
        if len(flat) >= 5:
            for c1 in range(-2, 3):
                for c2 in range(-2, 3):
                    for c3 in range(-2, 3):
                        if c1 == 0 and c2 == 0 and c3 == 0:
                            continue
                        
                        test = [flat[0], flat[1], flat[2]]
                        for i in range(3, len(flat)):
                            test.append((c1 * test[-1] + c2 * test[-2] + c3 * test[-3]) % 10)
                        
                        if np.array_equal(flat, test[:len(flat)]):
                            return f"""def p(g):
 h,w={h},{w}
 s=[{flat[0]},{flat[1]},{flat[2]}]
 for i in range(3,h*w):
  s.append(({c1}*s[-1]+{c2}*s[-2]+{c3}*s[-3])%10)
 res=[]
 for i in range(h):
  res.append(s[i*w:(i+1)*w])
 return res
"""
        return None

@handler_decorator
class HandlerDigitSum(TransformationHandler):
    """Generates sequences based on digit sums"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Standard digit sum
        digit_sums = []
        for i in range(len(flat)):
            s = sum(int(d) for d in str(i))
            digit_sums.append(s % 10)
        
        if np.array_equal(flat, digit_sums):
            return True
        
        # Iterated digit sum until single digit
        iterated_sums = []
        for i in range(len(flat)):
            n = i
            while n >= 10:
                n = sum(int(d) for d in str(n))
            iterated_sums.append(n)
        
        if np.array_equal(flat, iterated_sums):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        # Standard digit sum
        digit_sums = []
        for i in range(len(flat)):
            s = sum(int(d) for d in str(i))
            digit_sums.append(s % 10)
        
        if np.array_equal(flat, digit_sums):
            return f"""def p(g):
 return[[sum(int(d)for d in str(i*{w}+j))%10for j in range({w})]for i in range({h})]
"""
        
        # Iterated digit sum
        iterated_sums = []
        for i in range(len(flat)):
            n = i
            while n >= 10:
                n = sum(int(d) for d in str(n))
            iterated_sums.append(n)
        
        if np.array_equal(flat, iterated_sums):
            # Digital root formula: 1 + (n-1) % 9
            return f"""def p(g):
 return[[1+((i*{w}+j-1)%9)if i*{w}+j>0else 0for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerWaveSequence(TransformationHandler):
    """Generates wave-like oscillating sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        if len(flat) < 3:
            return False
        
        # Triangular wave patterns
        for amplitude in range(1, 5):
            for period in range(2, 10):
                wave = []
                value = flat[0] if len(flat) > 0 else 0
                direction = 1
                
                for i in range(len(flat)):
                    wave.append(value % 10)
                    if (i + 1) % period == 0:
                        direction *= -1
                    if i < len(flat) - 1:
                        value += direction * amplitude
                
                if np.array_equal(flat, wave):
                    return True
        
        # Sawtooth wave
        for step in range(1, 5):
            for period in range(2, 10):
                sawtooth = []
                for i in range(len(flat)):
                    sawtooth.append(((i % period) * step) % 10)
                
                if np.array_equal(flat, sawtooth):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        # Triangular wave
        for amplitude in range(1, 5):
            for period in range(2, 10):
                wave = []
                value = flat[0] if len(flat) > 0 else 0
                direction = 1
                
                for i in range(len(flat)):
                    wave.append(value % 10)
                    if (i + 1) % period == 0:
                        direction *= -1
                    if i < len(flat) - 1:
                        value += direction * amplitude
                
                if np.array_equal(flat, wave):
                    return f"""def p(g):
 res=[]
 v={flat[0]}
 d=1
 for i in range({h}):
  row=[]
  for j in range({w}):
   row.append(v%10)
   if(i*{w}+j+1)%{period}==0:d*=-1
   if i*{w}+j<{h*w-1}:v+=d*{amplitude}
  res.append(row)
 return res
"""
        
        # Sawtooth wave
        for step in range(1, 5):
            for period in range(2, 10):
                sawtooth = [((i % period) * step) % 10 for i in range(len(flat))]
                
                if np.array_equal(flat, sawtooth):
                    return f"""def p(g):
 return[[(((i*{w}+j)%{period})*{step})%10for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerInterleavedSequence(TransformationHandler):
    """Generates interleaved sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        if len(flat) < 4:
            return False
        
        # Two interleaved arithmetic sequences
        even_seq = flat[0::2]
        odd_seq = flat[1::2]
        
        if len(even_seq) >= 2 and len(odd_seq) >= 2:
            # Check if both are arithmetic
            even_diff = even_seq[1] - even_seq[0] if len(even_seq) > 1 else 0
            odd_diff = odd_seq[1] - odd_seq[0] if len(odd_seq) > 1 else 0
            
            even_arith = all(even_seq[i] - even_seq[i-1] == even_diff for i in range(1, len(even_seq)))
            odd_arith = all(odd_seq[i] - odd_seq[i-1] == odd_diff for i in range(1, len(odd_seq)))
            
            if even_arith and odd_arith:
                return True
            
            # Check if both are geometric
            if even_seq[0] != 0 and odd_seq[0] != 0:
                even_geometric = True
                odd_geometric = True
                
                for base in range(2, 10):
                    even_test = [even_seq[0] * pow(base, i, 10) for i in range(len(even_seq))]
                    odd_test = [odd_seq[0] * pow(base, i, 10) for i in range(len(odd_seq))]
                    
                    if np.array_equal(even_seq, [e % 10 for e in even_test]) and \
                       np.array_equal(odd_seq, [o % 10 for o in odd_test]):
                        return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        even_seq = flat[0::2]
        odd_seq = flat[1::2]
        
        if len(even_seq) >= 2 and len(odd_seq) >= 2:
            even_diff = even_seq[1] - even_seq[0] if len(even_seq) > 1 else 0
            odd_diff = odd_seq[1] - odd_seq[0] if len(odd_seq) > 1 else 0
            
            even_arith = all(even_seq[i] - even_seq[i-1] == even_diff for i in range(1, len(even_seq)))
            odd_arith = all(odd_seq[i] - odd_seq[i-1] == odd_diff for i in range(1, len(odd_seq)))
            
            if even_arith and odd_arith:
                e0, o0 = even_seq[0], odd_seq[0]
                return f"""def p(g):
 res=[]
 e,o={e0},{o0}
 for i in range({h}):
  row=[]
  for j in range({w}):
   if(i*{w}+j)%2==0:
    row.append(e%10)
    e+={even_diff}
   else:
    row.append(o%10)
    o+={odd_diff}
  res.append(row)
 return res
"""
        return None

@handler_decorator
class HandlerLucasSequence(TransformationHandler):
    """Generates Lucas and generalized Lucas sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        if len(flat) < 3:
            return False
        
        # Standard Lucas numbers
        lucas = [2, 1]
        for i in range(2, len(flat)):
            lucas.append((lucas[-1] + lucas[-2]) % 10)
        
        if np.array_equal(flat, lucas[:len(flat)]):
            return True
        
        # Generalized Lucas sequences
        for a in range(10):
            for b in range(10):
                if a == 0 and b == 0:
                    continue
                test = [a, b]
                for i in range(2, len(flat)):
                    test.append((test[-1] + test[-2]) % 10)
                
                if np.array_equal(flat, test[:len(flat)]):
                    return True
        
        # Lucas-like with different recurrence
        for a in range(10):
            for b in range(10):
                test = [a, b]
                for i in range(2, len(flat)):
                    test.append((2 * test[-1] + test[-2]) % 10)
                
                if np.array_equal(flat, test[:len(flat)]):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        # Standard Lucas
        lucas = [2, 1]
        for i in range(2, len(flat)):
            lucas.append((lucas[-1] + lucas[-2]) % 10)
        
        if np.array_equal(flat, lucas[:len(flat)]):
            return f"""def p(g):
 h,w={h},{w}
 s=[2,1]
 for i in range(2,h*w):
  s.append((s[-1]+s[-2])%10)
 res=[]
 for i in range(h):
  res.append(s[i*w:(i+1)*w])
 return res
"""
        
        # Generalized Lucas
        for a in range(10):
            for b in range(10):
                if a == 0 and b == 0:
                    continue
                test = [a, b]
                for i in range(2, len(flat)):
                    test.append((test[-1] + test[-2]) % 10)
                
                if np.array_equal(flat, test[:len(flat)]):
                    return f"""def p(g):
 h,w={h},{w}
 s=[{a},{b}]
 for i in range(2,h*w):
  s.append((s[-1]+s[-2])%10)
 res=[]
 for i in range(h):
  res.append(s[i*w:(i+1)*w])
 return res
"""
        
        # Modified Lucas recurrence
        for a in range(10):
            for b in range(10):
                test = [a, b]
                for i in range(2, len(flat)):
                    test.append((2 * test[-1] + test[-2]) % 10)
                
                if np.array_equal(flat, test[:len(flat)]):
                    return f"""def p(g):
 h,w={h},{w}
 s=[{a},{b}]
 for i in range(2,h*w):
  s.append((2*s[-1]+s[-2])%10)
 res=[]
 for i in range(h):
  res.append(s[i*w:(i+1)*w])
 return res
"""
        return None

@handler_decorator
class HandlerCatalanNumbers(TransformationHandler):
    """Generates Catalan number sequences"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        flat = out.flatten()
        
        # Catalan numbers mod 10
        catalan_mod10 = [1, 1, 2, 5, 4, 2, 2, 9, 6, 2, 6, 2, 5, 2, 8, 7, 8, 6, 4, 4]
        
        if len(flat) <= 20 and np.array_equal(flat, catalan_mod10[:len(flat)]):
            return True
        
        # Shifted Catalan
        for shift in range(1, 10):
            shifted = [(c + shift) % 10 for c in catalan_mod10[:len(flat)]]
            if np.array_equal(flat, shifted):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        flat = out.flatten()
        
        catalan_mod10 = [1, 1, 2, 5, 4, 2, 2, 9, 6, 2, 6, 2, 5, 2, 8, 7, 8, 6, 4, 4]
        
        if len(flat) <= 20 and np.array_equal(flat, catalan_mod10[:len(flat)]):
            cat_str = ','.join(map(str, catalan_mod10[:len(flat)]))
            return f"""def p(g):
 c=[{cat_str}]
 return[c[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        
        # Shifted Catalan
        for shift in range(1, 10):
            shifted = [(c + shift) % 10 for c in catalan_mod10[:len(flat)]]
            if np.array_equal(flat, shifted):
                cat_str = ','.join(map(str, shifted))
                return f"""def p(g):
 c=[{cat_str}]
 return[c[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# REGION PARTITIONING - ENHANCED VERSION WITH COMPLETE IMPLEMENTATIONS
# --------------------------------------------------------------------------------

@handler_decorator
class HandlerPartitionGrid(TransformationHandler):
    """Partitions grid into colored quadrants"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Check various quadrant partitioning schemes
        if h % 2 == 0 and w % 2 == 0:
            h2, w2 = h // 2, w // 2
            
            # Standard quadrant coloring
            test = np.zeros_like(inp)
            test[:h2, :w2] = 1
            test[:h2, w2:] = 2
            test[h2:, :w2] = 3
            test[h2:, w2:] = 4
            
            if np.array_equal(test, out):
                return True
            
            # Alternative quadrant patterns
            quadrant_patterns = [
                (1, 2, 2, 3),
                (1, 1, 2, 2),
                (1, 2, 3, 1),
                (1, 2, 4, 3),
                (2, 1, 1, 2),
                (1, 3, 2, 4)
            ]
            
            for colors in quadrant_patterns:
                test = np.zeros_like(inp)
                test[:h2, :w2] = colors[0]
                test[:h2, w2:] = colors[1]
                test[h2:, :w2] = colors[2]
                test[h2:, w2:] = colors[3]
                
                if np.array_equal(test, out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        if h % 2 == 0 and w % 2 == 0:
            h2, w2 = h // 2, w // 2
            
            # Standard quadrant pattern
            test = np.zeros_like(inp)
            test[:h2, :w2] = 1
            test[:h2, w2:] = 2
            test[h2:, :w2] = 3
            test[h2:, w2:] = 4
            
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[1+2*(i>={h2})+(j>={w2})-(j>={w2}and i<{h2})for j in range({w})]for i in range({h})]
"""
            
            # Check alternative patterns
            quadrant_patterns = [
                (1, 2, 2, 3),
                (1, 1, 2, 2),
                (1, 2, 3, 1),
                (1, 2, 4, 3),
                (2, 1, 1, 2),
                (1, 3, 2, 4)
            ]
            
            for colors in quadrant_patterns:
                test = np.zeros_like(inp)
                test[:h2, :w2] = colors[0]
                test[:h2, w2:] = colors[1]
                test[h2:, :w2] = colors[2]
                test[h2:, w2:] = colors[3]
                
                if np.array_equal(test, out):
                    # Generate compact solution based on pattern
                    c0, c1, c2, c3 = colors
                    return f"""def p(g):
 return[[{c0}if i<{h2}and j<{w2}else{c1}if i<{h2}else{c2}if j<{w2}else{c3}for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerSectorPartition(TransformationHandler):
    """Partitions grid into angular sectors from center or corners"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Four sectors from center
        ch, cw = h // 2, w // 2
        test = np.zeros((h, w), dtype=int)
        
        for i in range(h):
            for j in range(w):
                if i < ch and j < cw:
                    test[i,j] = 1
                elif i < ch and j >= cw:
                    test[i,j] = 2
                elif i >= ch and j >= cw:
                    test[i,j] = 3
                else:
                    test[i,j] = 4
        
        if np.array_equal(test, out):
            return True
        
        # Eight sectors (with diagonals)
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                dy, dx = i - ch, j - cw
                if dy >= 0 and dx >= 0:
                    test[i,j] = 1 if dy >= dx else 2
                elif dy >= 0 and dx < 0:
                    test[i,j] = 3 if dy >= -dx else 4
                elif dy < 0 and dx < 0:
                    test[i,j] = 5 if -dy >= -dx else 6
                else:
                    test[i,j] = 7 if -dy >= dx else 8
        
        if np.array_equal(test % 10, out):
            return True
        
        # Six sectors (hexagonal-like)
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                dy, dx = i - ch, j - cw
                if abs(dy) < abs(dx) / 2:
                    test[i,j] = 1 if dx > 0 else 4
                elif dy > 0:
                    test[i,j] = 2 if dx > 0 else 3
                else:
                    test[i,j] = 5 if dx < 0 else 6
        
        if np.array_equal(test % 10, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        ch, cw = h // 2, w // 2
        
        # Four sectors
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                if i < ch and j < cw:
                    test[i,j] = 1
                elif i < ch and j >= cw:
                    test[i,j] = 2
                elif i >= ch and j >= cw:
                    test[i,j] = 3
                else:
                    test[i,j] = 4
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[1+(i>={ch})*2+(j>={cw})-(j>={cw}and i<{ch})for j in range({w})]for i in range({h})]
"""
        
        # Eight sectors
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                dy, dx = i - ch, j - cw
                if dy >= 0 and dx >= 0:
                    test[i,j] = 1 if dy >= dx else 2
                elif dy >= 0 and dx < 0:
                    test[i,j] = 3 if dy >= -dx else 4
                elif dy < 0 and dx < 0:
                    test[i,j] = 5 if -dy >= -dx else 6
                else:
                    test[i,j] = 7 if -dy >= dx else 8
        
        if np.array_equal(test % 10, out):
            return f"""def p(g):
 ch,cw={ch},{cw}
 res=[]
 for i in range({h}):
  r=[]
  for j in range({w}):
   y,x=i-ch,j-cw
   if y>=0and x>=0:v=1if y>=x else 2
   elif y>=0and x<0:v=3if y>=-x else 4
   elif y<0and x<0:v=5if-y>=-x else 6
   else:v=7if-y>=x else 8
   r.append(v%10)
  res.append(r)
 return res
"""
        
        # Six sectors
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                dy, dx = i - ch, j - cw
                if abs(dy) < abs(dx) / 2:
                    test[i,j] = 1 if dx > 0 else 4
                elif dy > 0:
                    test[i,j] = 2 if dx > 0 else 3
                else:
                    test[i,j] = 5 if dx < 0 else 6
        
        if np.array_equal(test % 10, out):
            return f"""def p(g):
 ch,cw={ch},{cw}
 res=[]
 for i in range({h}):
  r=[]
  for j in range({w}):
   y,x=i-ch,j-cw
   if abs(y)<abs(x)/2:v=1if x>0else 4
   elif y>0:v=2if x>0else 3
   else:v=5if x<0else 6
   r.append(v%10)
  res.append(r)
 return res
"""
        return None

@handler_decorator
class HandlerVoronoiPartition(TransformationHandler):
    """Creates Voronoi-like partitions based on distance to fixed points"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Simple 2-point Voronoi (vertical split)
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                d1 = abs(j - w//3)
                d2 = abs(j - 2*w//3)
                test[i,j] = 1 if d1 <= d2 else 2
        
        if np.array_equal(test, out):
            return True
        
        # Simple 2-point Voronoi (horizontal split)
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                d1 = abs(i - h//3)
                d2 = abs(i - 2*h//3)
                test[i,j] = 1 if d1 <= d2 else 2
        
        if np.array_equal(test, out):
            return True
        
        # Corner-based Voronoi
        test = np.zeros((h, w), dtype=int)
        corners = [(0,0), (0,w-1), (h-1,0), (h-1,w-1)]
        
        for i in range(h):
            for j in range(w):
                distances = [abs(i-ci) + abs(j-cj) for ci, cj in corners]
                test[i,j] = np.argmin(distances) + 1
        
        if np.array_equal(test, out):
            return True
        
        # Center + corners Voronoi
        test = np.zeros((h, w), dtype=int)
        points = [(h//2, w//2), (0,0), (0,w-1), (h-1,0), (h-1,w-1)]
        
        for i in range(h):
            for j in range(w):
                distances = [abs(i-pi) + abs(j-pj) for pi, pj in points]
                test[i,j] = np.argmin(distances) + 1
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Vertical split Voronoi
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                d1 = abs(j - w//3)
                d2 = abs(j - 2*w//3)
                test[i,j] = 1 if d1 <= d2 else 2
        
        if np.array_equal(test, out):
            w1, w2 = w//3, 2*w//3
            return f"""def p(g):
 return[[1if abs(j-{w1})<=abs(j-{w2})else 2for j in range({w})]for _ in range({h})]
"""
        
        # Horizontal split Voronoi
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                d1 = abs(i - h//3)
                d2 = abs(i - 2*h//3)
                test[i,j] = 1 if d1 <= d2 else 2
        
        if np.array_equal(test, out):
            h1, h2 = h//3, 2*h//3
            return f"""def p(g):
 return[[1if abs(i-{h1})<=abs(i-{h2})else 2for j in range({w})]for i in range({h})]
"""
        
        # Corner-based Voronoi
        test = np.zeros((h, w), dtype=int)
        corners = [(0,0), (0,w-1), (h-1,0), (h-1,w-1)]
        
        for i in range(h):
            for j in range(w):
                distances = [abs(i-ci) + abs(j-cj) for ci, cj in corners]
                test[i,j] = np.argmin(distances) + 1
        
        if np.array_equal(test, out):
            return f"""def p(g):
 h,w={h},{w}
 res=[]
 for i in range(h):
  r=[]
  for j in range(w):
   d=[i+j,i+w-1-j,h-1-i+j,h-1-i+w-1-j]
   r.append(d.index(min(d))+1)
  res.append(r)
 return res
"""
        
        # Center + corners Voronoi
        test = np.zeros((h, w), dtype=int)
        points = [(h//2, w//2), (0,0), (0,w-1), (h-1,0), (h-1,w-1)]
        
        for i in range(h):
            for j in range(w):
                distances = [abs(i-pi) + abs(j-pj) for pi, pj in points]
                test[i,j] = np.argmin(distances) + 1
        
        if np.array_equal(test, out):
            ch, cw = h//2, w//2
            return f"""def p(g):
 h,w={h},{w}
 ch,cw={ch},{cw}
 res=[]
 for i in range(h):
  r=[]
  for j in range(w):
   d=[abs(i-ch)+abs(j-cw),i+j,i+w-1-j,h-1-i+j,h-1-i+w-1-j]
   r.append(d.index(min(d))+1)
  res.append(r)
 return res
"""
        return None

@handler_decorator
class HandlerHexagonalPartition(TransformationHandler):
    """Creates hexagonal-like tiling patterns"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Simplified hexagonal pattern
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                test[i,j] = ((i//3) + (j//2) + (i//3)*(j//2)) % 3
        
        if np.array_equal(test, out):
            return True
        
        # Alternative hex pattern
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                test[i,j] = (i + 2*(j//3) + (i%2)*(j%3)) % 4
        
        if np.array_equal(test, out):
            return True
        
        # Offset hex grid
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                offset = (i % 2) * 2
                test[i,j] = ((j + offset) // 4) % 3
        
        if np.array_equal(test, out):
            return True
        
        # Honeycomb approximation
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                hex_i = i // 3
                hex_j = j // 2 + (hex_i % 2)
                test[i,j] = (hex_i + hex_j) % 6
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Simplified hexagonal pattern
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                test[i,j] = ((i//3) + (j//2) + (i//3)*(j//2)) % 3
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[((i//3)+(j//2)+(i//3)*(j//2))%3for j in range({w})]for i in range({h})]
"""
        
        # Alternative hex pattern
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                test[i,j] = (i + 2*(j//3) + (i%2)*(j%3)) % 4
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[(i+2*(j//3)+(i%2)*(j%3))%4for j in range({w})]for i in range({h})]
"""
        
        # Offset hex grid
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                offset = (i % 2) * 2
                test[i,j] = ((j + offset) // 4) % 3
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[((j+(i%2)*2)//4)%3for j in range({w})]for i in range({h})]
"""
        
        # Honeycomb approximation
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                hex_i = i // 3
                hex_j = j // 2 + (hex_i % 2)
                test[i,j] = (hex_i + hex_j) % 6
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[(i//3+j//2+(i//3)%2)%6for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerFractalPartition(TransformationHandler):
    """Creates fractal-like recursive partitions"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Sierpinski-like pattern using bitwise AND
        if h == w and h in [2, 4, 8, 16]:
            test = np.zeros((h, w), dtype=int)
            for i in range(h):
                for j in range(w):
                    test[i,j] = 1 if (i & j) == 0 else 0
            
            if np.array_equal(test, out):
                return True
        
        # Pascal's triangle mod 2 (Sierpinski)
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(min(i+1, w)):
                # Check if C(i,j) is odd
                test[i,j] = 1 if (i & j) == j else 0
        
        if np.array_equal(test, out):
            return True
        
        # Cantor set approximation
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                # Check if in middle third at any scale
                val = 1
                size = max(h, w)
                while size >= 3:
                    if (i % size) >= size//3 and (i % size) < 2*size//3:
                        val = 0
                        break
                    if (j % size) >= size//3 and (j % size) < 2*size//3:
                        val = 0
                        break
                    size //= 3
                test[i,j] = val
        
        if np.array_equal(test, out):
            return True
        
        # Hilbert curve approximation
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                # Simple Hilbert-like pattern
                level = 0
                ii, jj = i, j
                while ii > 0 or jj > 0:
                    level += (ii % 2) + (jj % 2)
                    ii //= 2
                    jj //= 2
                test[i,j] = level % 10
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Sierpinski using bitwise AND
        if h == w and h in [2, 4, 8, 16]:
            test = np.zeros((h, w), dtype=int)
            for i in range(h):
                for j in range(w):
                    test[i,j] = 1 if (i & j) == 0 else 0
            
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[1if(i&j)==0else 0for j in range({w})]for i in range({h})]
"""
        
        # Pascal's triangle mod 2
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(min(i+1, w)):
                test[i,j] = 1 if (i & j) == j else 0
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[1if j<=i and(i&j)==j else 0for j in range({w})]for i in range({h})]
"""
        
        # Cantor set approximation (simplified)
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                # Simplified version checking specific scales
                val = 1
                for k in [3, 9, 27]:
                    if k > max(h, w):
                        break
                    if (i % k) >= k//3 and (i % k) < 2*k//3:
                        val = 0
                        break
                    if (j % k) >= k//3 and (j % k) < 2*k//3:
                        val = 0
                        break
                test[i,j] = val
        
        if np.array_equal(test, out):
            checks = []
            for k in [3, 9, 27]:
                if k <= max(h, w):
                    checks.append(k)
            
            if len(checks) == 1:
                k = checks[0]
                return f"""def p(g):
 return[[0if({k}//3<=i%{k}<2*{k}//3)or({k}//3<=j%{k}<2*{k}//3)else 1for j in range({w})]for i in range({h})]
"""
            elif len(checks) == 2:
                k1, k2 = checks[0], checks[1]
                return f"""def p(g):
 return[[0if any({k}//3<=i%{k}<2*{k}//3or{k}//3<=j%{k}<2*{k}//3for k in[{k1},{k2}])else 1for j in range({w})]for i in range({h})]
"""
        
        # Hilbert curve approximation
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                level = 0
                ii, jj = i, j
                while ii > 0 or jj > 0:
                    level += (ii % 2) + (jj % 2)
                    ii //= 2
                    jj //= 2
                test[i,j] = level % 10
        
        if np.array_equal(test, out):
            return f"""def p(g):
 res=[]
 for i in range({h}):
  r=[]
  for j in range({w}):
   l,a,b=0,i,j
   while a>0or b>0:
    l+=(a%2)+(b%2)
    a//=2
    b//=2
   r.append(l%10)
  res.append(r)
 return res
"""
        return None

@handler_decorator
class HandlerPolarPartition(TransformationHandler):
    """Creates patterns based on polar coordinates"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        ch, cw = h // 2, w // 2
        
        # Simplified angular sectors without atan2
        # Using quadrant and comparison logic
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                dy, dx = i - ch, j - cw
                
                # 8 sectors using slope comparisons
                if dy == 0 and dx == 0:
                    test[i,j] = 0
                elif dx >= 0 and dy >= 0:
                    # First quadrant
                    if dy < dx * 0.414:  # tan(22.5°) ≈ 0.414
                        test[i,j] = 0
                    elif dy < dx * 2.414:  # tan(67.5°) ≈ 2.414
                        test[i,j] = 1
                    else:
                        test[i,j] = 2
                elif dx < 0 and dy >= 0:
                    # Second quadrant
                    if dy < -dx * 0.414:
                        test[i,j] = 4
                    elif dy < -dx * 2.414:
                        test[i,j] = 3
                    else:
                        test[i,j] = 2
                elif dx < 0 and dy < 0:
                    # Third quadrant
                    if -dy < -dx * 0.414:
                        test[i,j] = 4
                    elif -dy < -dx * 2.414:
                        test[i,j] = 5
                    else:
                        test[i,j] = 6
                else:
                    # Fourth quadrant
                    if -dy < dx * 0.414:
                        test[i,j] = 0
                    elif -dy < dx * 2.414:
                        test[i,j] = 7
                    else:
                        test[i,j] = 6
        
        if np.array_equal(test, out):
            return True
        
        # Radial rings
        test = np.zeros((h, w), dtype=int)
        max_dist = max(h, w)
        for i in range(h):
            for j in range(w):
                dy, dx = i - ch, j - cw
                dist_sq = dy*dy + dx*dx
                # Approximate rings without sqrt
                test[i,j] = min(9, dist_sq // (max_dist // 3))
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        ch, cw = h // 2, w // 2
        
        # Simplified 8 sectors
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                dy, dx = i - ch, j - cw
                
                if dy == 0 and dx == 0:
                    test[i,j] = 0
                elif dx >= 0 and dy >= 0:
                    if dy * 10 < dx * 4:
                        test[i,j] = 0
                    elif dy * 10 < dx * 24:
                        test[i,j] = 1
                    else:
                        test[i,j] = 2
                elif dx < 0 and dy >= 0:
                    if dy * 10 < -dx * 4:
                        test[i,j] = 4
                    elif dy * 10 < -dx * 24:
                        test[i,j] = 3
                    else:
                        test[i,j] = 2
                elif dx < 0 and dy < 0:
                    if -dy * 10 < -dx * 4:
                        test[i,j] = 4
                    elif -dy * 10 < -dx * 24:
                        test[i,j] = 5
                    else:
                        test[i,j] = 6
                else:
                    if -dy * 10 < dx * 4:
                        test[i,j] = 0
                    elif -dy * 10 < dx * 24:
                        test[i,j] = 7
                    else:
                        test[i,j] = 6
        
        if np.array_equal(test, out):
            return f"""def p(g):
 ch,cw={ch},{cw}
 res=[]
 for i in range({h}):
  r=[]
  for j in range({w}):
   y,x=i-ch,j-cw
   if y==0and x==0:v=0
   elif x>=0and y>=0:v=0if y*10<x*4else 1if y*10<x*24else 2
   elif x<0and y>=0:v=4if y*10<-x*4else 3if y*10<-x*24else 2
   elif x<0and y<0:v=4if-y*10<-x*4else 5if-y*10<-x*24else 6
   else:v=0if-y*10<x*4else 7if-y*10<x*24else 6
   r.append(v)
  res.append(r)
 return res
"""
        
        # Radial rings
        test = np.zeros((h, w), dtype=int)
        max_dist = max(h, w)
        for i in range(h):
            for j in range(w):
                dy, dx = i - ch, j - cw
                dist_sq = dy*dy + dx*dx
                test[i,j] = min(9, dist_sq // (max_dist // 3))
        
        if np.array_equal(test, out):
            divisor = max_dist // 3
            return f"""def p(g):
 ch,cw={ch},{cw}
 return[[min(9,((i-ch)**2+(j-cw)**2)//{divisor})for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerMosaicPattern(TransformationHandler):
    """Creates mosaic-like patterns with irregular tiles"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Brick pattern
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            offset = (i // 2) % 2
            for j in range(w):
                test[i,j] = ((j + offset * 2) // 4) % 3
        
        if np.array_equal(test, out):
            return True
        
        # Irregular checkerboard
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                bi = i // (2 + (i % 3))
                bj = j // (2 + (j % 3))
                test[i,j] = (bi + bj) % 4
        
        if np.array_equal(test, out):
            return True
        
        # Staggered tiles
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                offset = (i // 3) % 3
                test[i,j] = ((j + offset) // 3) % 5
        
        if np.array_equal(test, out):
            return True
        
        # Diamond mosaic
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                # Create diamond-shaped regions
                test[i,j] = ((i//2 + j//2) + (i//2) * (j//2)) % 6
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Brick pattern
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            offset = (i // 2) % 2
            for j in range(w):
                test[i,j] = ((j + offset * 2) // 4) % 3
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[((j+((i//2)%2)*2)//4)%3for j in range({w})]for i in range({h})]
"""
        
        # Irregular checkerboard
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                bi = i // (2 + (i % 3))
                bj = j // (2 + (j % 3))
                test[i,j] = (bi + bj) % 4
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[(i//(2+i%3)+j//(2+j%3))%4for j in range({w})]for i in range({h})]
"""
        
        # Staggered tiles
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                offset = (i // 3) % 3
                test[i,j] = ((j + offset) // 3) % 5
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[((j+(i//3)%3)//3)%5for j in range({w})]for i in range({h})]
"""
        
        # Diamond mosaic
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                test[i,j] = ((i//2 + j//2) + (i//2) * (j//2)) % 6
        
        if np.array_equal(test, out):
            return f"""def p(g):
 return[[(i//2+j//2+(i//2)*(j//2))%6for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerTreemapPartition(TransformationHandler):
    """Creates treemap-like rectangular subdivisions"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Simple binary space partitioning
        # Level 1: split horizontally
        test = np.zeros((h, w), dtype=int)
        if h >= 2:
            test[:h//2, :] = 0
            test[h//2:, :] = 1
            
            # Level 2: split each half vertically
            if w >= 2:
                test[:h//2, :w//2] = 0
                test[:h//2, w//2:] = 1
                test[h//2:, :w//2] = 2
                test[h//2:, w//2:] = 3
            
            if np.array_equal(test, out):
                return True
        
        # Alternative: split vertically first
        test = np.zeros((h, w), dtype=int)
        if w >= 2:
            test[:, :w//2] = 0
            test[:, w//2:] = 1
            
            # Level 2: split each half horizontally
            if h >= 2:
                test[:h//2, :w//2] = 0
                test[h//2:, :w//2] = 1
                test[:h//2, w//2:] = 2
                test[h//2:, w//2:] = 3
            
            if np.array_equal(test, out):
                return True
        
        # Golden ratio subdivision
        test = np.zeros((h, w), dtype=int)
        if h >= 3 and w >= 3:
            # Approximate golden ratio as 5:3
            h1 = (h * 5) // 8
            w1 = (w * 5) // 8
            
            test[:h1, :w1] = 0
            test[:h1, w1:] = 1
            test[h1:, :w1] = 2
            test[h1:, w1:] = 3
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Simple binary space partitioning - horizontal first
        test = np.zeros((h, w), dtype=int)
        if h >= 2 and w >= 2:
            h2, w2 = h//2, w//2
            test[:h2, :w2] = 0
            test[:h2, w2:] = 1
            test[h2:, :w2] = 2
            test[h2:, w2:] = 3
            
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[(i//{h2})*2+(j//{w2})for j in range({w})]for i in range({h})]
"""
        
        # Vertical first
        test = np.zeros((h, w), dtype=int)
        if h >= 2 and w >= 2:
            h2, w2 = h//2, w//2
            test[:h2, :w2] = 0
            test[h2:, :w2] = 1
            test[:h2, w2:] = 2
            test[h2:, w2:] = 3
            
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[(j//{w2})*2+(i//{h2})for j in range({w})]for i in range({h})]
"""
        
        # Golden ratio subdivision
        if h >= 3 and w >= 3:
            h1 = (h * 5) // 8
            w1 = (w * 5) // 8
            
            test = np.zeros((h, w), dtype=int)
            test[:h1, :w1] = 0
            test[:h1, w1:] = 1
            test[h1:, :w1] = 2
            test[h1:, w1:] = 3
            
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[0if i<{h1}and j<{w1}else 1if i<{h1}else 2if j<{w1}else 3for j in range({w})]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerIrregularGrid(TransformationHandler):
    """Creates grids with irregular cell sizes"""
    def can_handle(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Grid with increasing cell sizes
        test = np.zeros((h, w), dtype=int)
        boundaries_h = [0]
        boundaries_w = [0]
        
        # Create boundaries with increasing spacing
        size = 1
        while boundaries_h[-1] < h:
            boundaries_h.append(min(boundaries_h[-1] + size, h))
            size += 1
        
        size = 1
        while boundaries_w[-1] < w:
            boundaries_w.append(min(boundaries_w[-1] + size, w))
            size += 1
        
        # Color cells
        color = 0
        for i in range(len(boundaries_h) - 1):
            for j in range(len(boundaries_w) - 1):
                test[boundaries_h[i]:boundaries_h[i+1], 
                     boundaries_w[j]:boundaries_w[j+1]] = color % 10
                color += 1
        
        if np.array_equal(test, out):
            return True
        
        # Logarithmic grid
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                # Use log-like boundaries
                cell_i = 0
                cell_j = 0
                val = 1
                while val <= i:
                    cell_i += 1
                    val *= 2
                val = 1
                while val <= j:
                    cell_j += 1
                    val *= 2
                test[i,j] = (cell_i + cell_j) % 10
        
        if np.array_equal(test, out):
            return True
        
        # Simple irregular: different sized stripes
        test = np.zeros((h, w), dtype=int)
        boundaries = [0, 2, 5, 8, 12, 16, 20, 25, 30]
        
        for i in range(h):
            for j in range(w):
                cell_i = 0
                cell_j = 0
                for k in range(len(boundaries)-1):
                    if boundaries[k] <= i < boundaries[k+1]:
                        cell_i = k
                    if boundaries[k] <= j < boundaries[k+1]:
                        cell_j = k
                test[i,j] = (cell_i + cell_j) % 10
        
        if np.array_equal(test, out):
            return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        out = np.array(task_data['train'][0]['output'])
        h, w = out.shape
        
        # Grid with increasing cell sizes
        test = np.zeros((h, w), dtype=int)
        boundaries_h = [0]
        boundaries_w = [0]
        
        size = 1
        while boundaries_h[-1] < h:
            boundaries_h.append(min(boundaries_h[-1] + size, h))
            size += 1
        
        size = 1
        while boundaries_w[-1] < w:
            boundaries_w.append(min(boundaries_w[-1] + size, w))
            size += 1
        
        color = 0
        for i in range(len(boundaries_h) - 1):
            for j in range(len(boundaries_w) - 1):
                test[boundaries_h[i]:boundaries_h[i+1], 
                     boundaries_w[j]:boundaries_w[j+1]] = color % 10
                color += 1
        
        if np.array_equal(test, out):
            # Generate compact solution
            # For triangular numbers: position n is at sum(1..n) = n(n+1)/2
            return f"""def p(g):
 res=[]
 for i in range({h}):
  r=[]
  for j in range({w}):
   # Find which cell i and j belong to
   ci=cj=0
   s=1
   t=0
   while t+s<=i:
    t+=s
    s+=1
    ci+=1
   s=1
   t=0
   while t+s<=j:
    t+=s
    s+=1
    cj+=1
   r.append((ci*{len(boundaries_w)-1}+cj)%10)
  res.append(r)
 return res
"""
        
        # Logarithmic grid
        test = np.zeros((h, w), dtype=int)
        for i in range(h):
            for j in range(w):
                cell_i = 0
                cell_j = 0
                val = 1
                while val <= i:
                    cell_i += 1
                    val *= 2
                val = 1
                while val <= j:
                    cell_j += 1
                    val *= 2
                test[i,j] = (cell_i + cell_j) % 10
        
        if np.array_equal(test, out):
            return f"""def p(g):
 res=[]
 for i in range({h}):
  r=[]
  for j in range({w}):
   ci=cj=0
   v=1
   while v<=i:
    ci+=1
    v*=2
   v=1
   while v<=j:
    cj+=1
    v*=2
   r.append((ci+cj)%10)
  res.append(r)
 return res
"""
        
        # Simple irregular with fixed boundaries
        test = np.zeros((h, w), dtype=int)
        boundaries = [0, 2, 5, 8, 12, 16, 20, 25, 30]
        
        # Find which boundaries are actually used
        used_boundaries = [b for b in boundaries if b < max(h, w)]
        used_boundaries.append(max(h, w))
        
        valid = True
        for i in range(h):
            for j in range(w):
                cell_i = cell_j = 0
                for k in range(len(used_boundaries)-1):
                    if used_boundaries[k] <= i < used_boundaries[k+1]:
                        cell_i = k
                    if used_boundaries[k] <= j < used_boundaries[k+1]:
                        cell_j = k
                test[i,j] = (cell_i + cell_j) % 10
        
        if np.array_equal(test, out):
            bounds_str = ','.join(map(str, used_boundaries[:-1]))
            return f"""def p(g):
 b=[{bounds_str}]
 res=[]
 for i in range({h}):
  r=[]
  for j in range({w}):
   ci=cj=0
   for k in range(len(b)-1):
    if b[k]<=i<b[k+1]:ci=k
    if b[k]<=j<b[k+1]:cj=k
   r.append((ci+cj)%10)
  res.append(r)
 return res
"""
        return None

# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# ITERATIVE AND RECURSIVE TRANSFORMATION HANDLERS
# --------------------------------------------------------------------------------

@handler_decorator
class HandlerIterativeFloodFill(TransformationHandler):
    """Fills regions iteratively by expanding from seed points"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if output is result of iterative expansion
        current = np.copy(inp)
        for _ in range(max(inp.shape)):
            next_state = np.copy(current)
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    if current[i, j] == 0:
                        # Check neighbors for non-zero values
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if (0 <= ni < current.shape[0] and 
                                0 <= nj < current.shape[1] and 
                                current[ni, nj] != 0):
                                next_state[i, j] = current[ni, nj]
                                break
            
            if np.array_equal(next_state, out):
                return True
            if np.array_equal(next_state, current):
                break
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(99):
  n=[row[:]for row in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    if r[i][j]==0:
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if 0<=ni<len(r)and 0<=nj<len(r[0])and r[ni][nj]:
       n[i][j]=r[ni][nj];break
  if n==r:break
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerCellularAutomaton(TransformationHandler):
    """Applies cellular automaton rules iteratively"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Try various CA rules
        current = np.copy(inp)
        for _ in range(20):
            next_state = np.zeros_like(current)
            
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    # Count neighbors
                    neighbors = 0
                    neighbor_vals = []
                    
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            ni, nj = i + di, j + dj
                            if (0 <= ni < current.shape[0] and 
                                0 <= nj < current.shape[1]):
                                if current[ni, nj] != 0:
                                    neighbors += 1
                                    neighbor_vals.append(current[ni, nj])
                    
                    # Apply rules
                    if current[i, j] == 0 and neighbors == 3:
                        # Birth rule
                        next_state[i, j] = max(set(neighbor_vals), key=neighbor_vals.count) if neighbor_vals else 0
                    elif current[i, j] != 0 and neighbors in [2, 3]:
                        # Survival rule
                        next_state[i, j] = current[i, j]
                    else:
                        # Death rule
                        next_state[i, j] = 0
            
            if np.array_equal(next_state, out):
                return True
            if np.array_equal(next_state, current):
                break
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(20):
  n=[[0]*len(r[0])for _ in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    c=0;v=[]
    for di in[-1,0,1]:
     for dj in[-1,0,1]:
      if di==0and dj==0:continue
      ni,nj=i+di,j+dj
      if 0<=ni<len(r)and 0<=nj<len(r[0])and r[ni][nj]:
       c+=1;v.append(r[ni][nj])
    if r[i][j]==0and c==3:n[i][j]=max(v,key=v.count)
    elif r[i][j]and c in[2,3]:n[i][j]=r[i][j]
  if n==r:break
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerIterativeGrowth(TransformationHandler):
    """Grows patterns by one pixel per iteration"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Grow non-zero regions
        current = np.copy(inp)
        for iteration in range(10):
            next_state = np.copy(current)
            
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    if current[i, j] != 0:
                        # Expand to orthogonal neighbors
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if (0 <= ni < current.shape[0] and 
                                0 <= nj < current.shape[1] and 
                                current[ni, nj] == 0):
                                next_state[ni, nj] = current[i, j]
            
            if np.array_equal(next_state, out):
                return True
            if np.array_equal(next_state, current):
                break
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(10):
  n=[row[:]for row in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    if r[i][j]:
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if 0<=ni<len(r)and 0<=nj<len(r[0])and r[ni][nj]==0:
       n[ni][nj]=r[i][j]
  if n==r:break
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerWavePropagation(TransformationHandler):
    """Propagates values in waves from source points"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Wave propagation with distance-based values
        current = np.copy(inp)
        wave_num = 1
        
        for _ in range(max(inp.shape)):
            next_state = np.copy(current)
            any_change = False
            
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    if current[i, j] == 0:
                        # Check if adjacent to wave front
                        adjacent_to_wave = False
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if (0 <= ni < current.shape[0] and 
                                0 <= nj < current.shape[1] and 
                                current[ni, nj] == wave_num):
                                adjacent_to_wave = True
                                break
                        
                        if adjacent_to_wave:
                            next_state[i, j] = (wave_num + 1) % 10
                            any_change = True
            
            if np.array_equal(next_state, out):
                return True
            if not any_change:
                wave_num = (wave_num + 1) % 10
                if wave_num == 0:
                    wave_num = 1
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 w=1
 for _ in range(50):
  n=[row[:]for row in r]
  ch=0
  for i in range(len(r)):
   for j in range(len(r[0])):
    if r[i][j]==0:
     adj=0
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if 0<=ni<len(r)and 0<=nj<len(r[0])and r[ni][nj]==w:
       adj=1;break
     if adj:n[i][j]=(w+1)%10;ch=1
  if not ch:w=(w+1)%10
  if w==0:w=1
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerRecursiveSubdivision(TransformationHandler):
    """Recursively subdivides regions"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check for recursive subdivision patterns
        h, w = inp.shape
        
        # Try binary subdivision
        def subdivide(grid, depth):
            if depth == 0:
                return grid
            
            new_grid = np.copy(grid)
            h, w = grid.shape
            
            if h >= 2 and w >= 2:
                # Find regions to subdivide
                for i in range(0, h, 2):
                    for j in range(0, w, 2):
                        # Check if 2x2 block is uniform non-zero
                        if (i+1 < h and j+1 < w and
                            grid[i,j] != 0 and 
                            grid[i,j] == grid[i+1,j] == grid[i,j+1] == grid[i+1,j+1]):
                            # Subdivide with pattern
                            new_grid[i,j] = grid[i,j]
                            new_grid[i+1,j] = (grid[i,j] + 1) % 10
                            new_grid[i,j+1] = (grid[i,j] + 2) % 10
                            new_grid[i+1,j+1] = (grid[i,j] + 3) % 10
            
            return subdivide(new_grid, depth - 1)
        
        for depth in range(1, 5):
            result = subdivide(inp, depth)
            if np.array_equal(result, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(5):
  n=[row[:]for row in r]
  for i in range(0,len(r),2):
   for j in range(0,len(r[0]),2):
    if(i+1<len(r)and j+1<len(r[0])and r[i][j]and
       r[i][j]==r[i+1][j]==r[i][j+1]==r[i+1][j+1]):
     n[i][j]=r[i][j]
     n[i+1][j]=(r[i][j]+1)%10
     n[i][j+1]=(r[i][j]+2)%10
     n[i+1][j+1]=(r[i][j]+3)%10
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerIterativeRotation(TransformationHandler):
    """Rotates patterns incrementally"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check for incremental rotation
        current = np.copy(inp)
        
        for rotation_count in range(1, 5):
            # Rotate by 90 degrees
            current = np.rot90(current, -1)
            
            if np.array_equal(current, out):
                return True
        
        # Try partial rotations (rotating only non-zero elements)
        h, w = inp.shape
        if h == w:
            for step in range(1, h):
                rotated = np.copy(inp)
                
                # Rotate elements around center
                ch, cw = h // 2, w // 2
                for i in range(h):
                    for j in range(w):
                        if inp[i, j] != 0:
                            # Calculate rotated position
                            di, dj = i - ch, j - cw
                            # Rotate by step positions
                            angle = step * 2 * np.pi / (h * w)
                            new_di = int(di * np.cos(angle) - dj * np.sin(angle) + 0.5)
                            new_dj = int(di * np.sin(angle) + dj * np.cos(angle) + 0.5)
                            new_i, new_j = ch + new_di, cw + new_dj
                            
                            if 0 <= new_i < h and 0 <= new_j < w:
                                rotated[new_i, new_j] = inp[i, j]
                
                if np.array_equal(rotated, out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        current = np.copy(inp)
        for rotation_count in range(1, 5):
            current = np.rot90(current, -1)
            if np.array_equal(current, out):
                return f"""def p(g):
 r=g
 for _ in range({rotation_count}):
  r=[list(row)for row in zip(*r[::-1])]
 return r
"""
        return None

@handler_decorator
class HandlerIterativeSymmetry(TransformationHandler):
    """Builds symmetric patterns iteratively"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Try building horizontal symmetry
        current = np.copy(inp)
        for _ in range(w // 2):
            next_state = np.copy(current)
            
            # Copy from left to right
            for i in range(h):
                for j in range(w // 2):
                    if current[i, j] != 0 and current[i, w-1-j] == 0:
                        next_state[i, w-1-j] = current[i, j]
            
            if np.array_equal(next_state, out):
                return True
            current = next_state
        
        # Try building vertical symmetry
        current = np.copy(inp)
        for _ in range(h // 2):
            next_state = np.copy(current)
            
            # Copy from top to bottom
            for i in range(h // 2):
                for j in range(w):
                    if current[i, j] != 0 and current[h-1-i, j] == 0:
                        next_state[h-1-i, j] = current[i, j]
            
            if np.array_equal(next_state, out):
                return True
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Horizontal symmetry
        current = np.copy(inp)
        for _ in range(w // 2):
            next_state = np.copy(current)
            for i in range(h):
                for j in range(w // 2):
                    if current[i, j] != 0 and current[i, w-1-j] == 0:
                        next_state[i, w-1-j] = current[i, j]
            if np.array_equal(next_state, out):
                return f"""def p(g):
 r=[row[:]for row in g]
 h,w={h},{w}
 for _ in range({w//2}):
  for i in range(h):
   for j in range(w//2):
    if r[i][j]and not r[i][w-1-j]:
     r[i][w-1-j]=r[i][j]
 return r
"""
            current = next_state
        
        # Vertical symmetry
        current = np.copy(inp)
        for _ in range(h // 2):
            next_state = np.copy(current)
            for i in range(h // 2):
                for j in range(w):
                    if current[i, j] != 0 and current[h-1-i, j] == 0:
                        next_state[h-1-i, j] = current[i, j]
            if np.array_equal(next_state, out):
                return f"""def p(g):
 r=[row[:]for row in g]
 h,w={h},{w}
 for _ in range({h//2}):
  for i in range(h//2):
   for j in range(w):
    if r[i][j]and not r[h-1-i][j]:
     r[h-1-i][j]=r[i][j]
 return r
"""
            current = next_state
        
        return None

@handler_decorator
class HandlerIterativeErosion(TransformationHandler):
    """Erodes shapes by removing edge pixels iteratively"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        current = np.copy(inp)
        
        for _ in range(10):
            next_state = np.copy(current)
            
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    if current[i, j] != 0:
                        # Check if edge pixel
                        is_edge = False
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if (ni < 0 or ni >= current.shape[0] or 
                                nj < 0 or nj >= current.shape[1] or 
                                current[ni, nj] == 0):
                                is_edge = True
                                break
                        
                        if is_edge:
                            next_state[i, j] = 0
            
            if np.array_equal(next_state, out):
                return True
            if np.array_equal(next_state, current):
                break
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(10):
  n=[row[:]for row in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    if r[i][j]:
     edge=0
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if ni<0or ni>=len(r)or nj<0or nj>=len(r[0])or r[ni][nj]==0:
       edge=1;break
     if edge:n[i][j]=0
  if n==r:break
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerRecursiveFractal(TransformationHandler):
    """Builds fractal patterns through recursive application"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check for self-similar patterns
        h, w = inp.shape
        
        # Try Sierpinski-like pattern
        current = np.copy(inp)
        
        for iteration in range(5):
            next_state = np.zeros_like(current)
            
            # Apply fractal rule
            for i in range(h):
                for j in range(w):
                    if current[i, j] != 0:
                        # Place copies at specific positions
                        next_state[i, j] = current[i, j]
                        
                        # Try to place scaled copies
                        if i*2 < h and j*2 < w:
                            next_state[i*2, j*2] = current[i, j]
                        if i*2 < h and j*2+1 < w:
                            next_state[i*2, j*2+1] = current[i, j]
                        if i*2+1 < h and j*2 < w:
                            next_state[i*2+1, j*2] = current[i, j]
            
            if np.array_equal(next_state, out):
                return True
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 h,w=len(r),len(r[0])
 for _ in range(5):
  n=[[0]*w for _ in range(h)]
  for i in range(h):
   for j in range(w):
    if r[i][j]:
     n[i][j]=r[i][j]
     if i*2<h and j*2<w:n[i*2][j*2]=r[i][j]
     if i*2<h and j*2+1<w:n[i*2][j*2+1]=r[i][j]
     if i*2+1<h and j*2<w:n[i*2+1][j*2]=r[i][j]
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerIterativeSmoothing(TransformationHandler):
    """Smooths patterns by averaging neighboring values"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        current = np.copy(inp)
        
        for _ in range(10):
            next_state = np.zeros_like(current)
            
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    # Collect neighbor values
                    values = []
                    
                    for di, dj in [(0,0), (0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < current.shape[0] and 
                            0 <= nj < current.shape[1]):
                            values.append(current[ni, nj])
                    
                    # Use most common non-zero value
                    non_zero = [v for v in values if v != 0]
                    if non_zero:
                        from collections import Counter
                        counts = Counter(non_zero)
                        next_state[i, j] = counts.most_common(1)[0][0]
                    else:
                        next_state[i, j] = 0
            
            if np.array_equal(next_state, out):
                return True
            if np.array_equal(next_state, current):
                break
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(10):
  n=[[0]*len(r[0])for _ in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    v=[]
    for di,dj in[(0,0),(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<len(r)and 0<=nj<len(r[0]):
      v.append(r[ni][nj])
    nz=[x for x in v if x]
    if nz:n[i][j]=max(nz,key=nz.count)
  if n==r:break
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerRecursiveZoom(TransformationHandler):
    """Recursively zooms in or out on patterns"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check for zoom out (downscaling)
        if out.shape[0] * 2 == inp.shape[0] and out.shape[1] * 2 == inp.shape[1]:
            # 2x2 block averaging
            test = np.zeros_like(out)
            for i in range(out.shape[0]):
                for j in range(out.shape[1]):
                    # Get most common value in 2x2 block
                    block = []
                    for di in range(2):
                        for dj in range(2):
                            if i*2+di < inp.shape[0] and j*2+dj < inp.shape[1]:
                                block.append(inp[i*2+di, j*2+dj])
                    
                    non_zero = [v for v in block if v != 0]
                    if non_zero:
                        from collections import Counter
                        test[i, j] = Counter(non_zero).most_common(1)[0][0]
            
            return np.array_equal(test, out)
        
        # Check for zoom in (upscaling)
        if inp.shape[0] * 2 == out.shape[0] and inp.shape[1] * 2 == out.shape[1]:
            test = np.zeros_like(out)
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    # Replicate each pixel to 2x2 block
                    for di in range(2):
                        for dj in range(2):
                            if i*2+di < out.shape[0] and j*2+dj < out.shape[1]:
                                test[i*2+di, j*2+dj] = inp[i, j]
            
            return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Zoom out
        if out.shape[0] * 2 == inp.shape[0] and out.shape[1] * 2 == inp.shape[1]:
            return """def p(g):
 h,w=len(g)//2,len(g[0])//2
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   b=[]
   for di in range(2):
    for dj in range(2):
     b.append(g[i*2+di][j*2+dj])
   nz=[v for v in b if v]
   if nz:r[i][j]=max(nz,key=nz.count)
 return r
"""
        
        # Zoom in
        if inp.shape[0] * 2 == out.shape[0] and inp.shape[1] * 2 == out.shape[1]:
            return """def p(g):
 h,w=len(g)*2,len(g[0])*2
 r=[[0]*w for _ in range(h)]
 for i in range(len(g)):
  for j in range(len(g[0])):
   for di in range(2):
    for dj in range(2):
     r[i*2+di][j*2+dj]=g[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerIterativePathfinding(TransformationHandler):
    """Finds paths between points iteratively"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Find start and end points
        start_points = []
        end_points = []
        
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] == 1:
                    start_points.append((i, j))
                elif inp[i, j] == 2:
                    end_points.append((i, j))
        
        if not start_points or not end_points:
            return False
        
        # Try to connect points
        current = np.copy(inp)
        
        for _ in range(inp.shape[0] + inp.shape[1]):
            next_state = np.copy(current)
            
            # Expand from existing path
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    if current[i, j] in [1, 3]:  # Start or path
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if (0 <= ni < current.shape[0] and 
                                0 <= nj < current.shape[1]):
                                if current[ni, nj] == 0:
                                    next_state[ni, nj] = 3  # Path marker
                                elif current[ni, nj] == 2:
                                    # Reached end point
                                    return True
            
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(99):
  n=[row[:]for row in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    if r[i][j]in[1,3]:
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if 0<=ni<len(r)and 0<=nj<len(r[0]):
       if r[ni][nj]==0:n[ni][nj]=3
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerIterativeBlending(TransformationHandler):
    """Blends adjacent colors iteratively"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        current = np.copy(inp)
        
        for _ in range(10):
            next_state = np.copy(current)
            
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    if current[i, j] == 0:
                        # Get adjacent colors
                        colors = []
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if (0 <= ni < current.shape[0] and 
                                0 <= nj < current.shape[1] and 
                                current[ni, nj] != 0):
                                colors.append(current[ni, nj])
                        
                        # Blend colors
                        if len(colors) >= 2:
                            # Use average or create new color
                            blend = (sum(colors) // len(colors)) % 10
                            if blend == 0:
                                blend = 1
                            next_state[i, j] = blend
            
            if np.array_equal(next_state, out):
                return True
            if np.array_equal(next_state, current):
                break
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(10):
  n=[row[:]for row in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    if r[i][j]==0:
     c=[]
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if 0<=ni<len(r)and 0<=nj<len(r[0])and r[ni][nj]:
       c.append(r[ni][nj])
     if len(c)>=2:
      b=(sum(c)//len(c))%10
      if b==0:b=1
      n[i][j]=b
  if n==r:break
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerRecursivePattern(TransformationHandler):
    """Applies patterns recursively at different scales"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Try recursive pattern application
        def apply_pattern(grid, pattern_size):
            if pattern_size < 2:
                return grid
            
            new_grid = np.copy(grid)
            h, w = grid.shape
            
            for i in range(0, h - pattern_size + 1, pattern_size):
                for j in range(0, w - pattern_size + 1, pattern_size):
                    # Check if region has pattern
                    region = grid[i:i+pattern_size, j:j+pattern_size]
                    
                    if np.any(region != 0):
                        # Apply transformation to region
                        center_val = region[pattern_size//2, pattern_size//2]
                        if center_val != 0:
                            # Create pattern around center
                            for di in range(pattern_size):
                                for dj in range(pattern_size):
                                    if region[di, dj] == 0:
                                        dist = abs(di - pattern_size//2) + abs(dj - pattern_size//2)
                                        new_grid[i+di, j+dj] = (center_val + dist) % 10
            
            return apply_pattern(new_grid, pattern_size // 2)
        
        for initial_size in [8, 4, 2]:
            result = apply_pattern(inp, initial_size)
            if np.array_equal(result, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 ps=4
 while ps>=2:
  n=[row[:]for row in r]
  for i in range(0,len(r)-ps+1,ps):
   for j in range(0,len(r[0])-ps+1,ps):
    cv=r[i+ps//2][j+ps//2]
    if cv:
     for di in range(ps):
      for dj in range(ps):
       if r[i+di][j+dj]==0:
        d=abs(di-ps//2)+abs(dj-ps//2)
        n[i+di][j+dj]=(cv+d)%10
  r=n
  ps//=2
 return r
"""
        return None

@handler_decorator
class HandlerIterativeConnectivity(TransformationHandler):
    """Connects components iteratively"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        current = np.copy(inp)
        
        # Find components
        components = []
        visited = np.zeros_like(inp, dtype=bool)
        
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0 and not visited[i, j]:
                    # BFS to find component
                    component = []
                    stack = [(i, j)]
                    color = inp[i, j]
                    
                    while stack:
                        ci, cj = stack.pop()
                        if visited[ci, cj]:
                            continue
                        visited[ci, cj] = True
                        component.append((ci, cj))
                        
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = ci + di, cj + dj
                            if (0 <= ni < inp.shape[0] and 
                                0 <= nj < inp.shape[1] and 
                                inp[ni, nj] == color and 
                                not visited[ni, nj]):
                                stack.append((ni, nj))
                    
                    components.append((color, component))
        
        # Try to connect components
        for _ in range(max(inp.shape)):
            next_state = np.copy(current)
            
            # Grow from each component
            for color, positions in components:
                for i, j in positions:
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < current.shape[0] and 
                            0 <= nj < current.shape[1] and 
                            current[ni, nj] == 0):
                            next_state[ni, nj] = color
            
            if np.array_equal(next_state, out):
                return True
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(20):
  n=[row[:]for row in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    if r[i][j]:
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if 0<=ni<len(r)and 0<=nj<len(r[0])and r[ni][nj]==0:
       n[ni][nj]=r[i][j]
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerIterativeRule(TransformationHandler):
    """Applies simple transformation rules iteratively"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Try simple replacement rules
        current = np.copy(inp)
        
        for _ in range(10):
            next_state = np.copy(current)
            
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    # Rule: If surrounded by same color, change to that color
                    if current[i, j] == 0:
                        surrounding = []
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if (0 <= ni < current.shape[0] and 
                                0 <= nj < current.shape[1]):
                                surrounding.append(current[ni, nj])
                        
                        non_zero = [s for s in surrounding if s != 0]
                        if non_zero and all(v == non_zero[0] for v in non_zero):
                            next_state[i, j] = non_zero[0]
                    
                    # Rule: Increment color value
                    elif current[i, j] != 0:
                        # Check if should increment
                        neighbors_higher = False
                        for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                            ni, nj = i + di, j + dj
                            if (0 <= ni < current.shape[0] and 
                                0 <= nj < current.shape[1] and 
                                current[ni, nj] > current[i, j]):
                                neighbors_higher = True
                                break
                        
                        if neighbors_higher:
                            next_state[i, j] = (current[i, j] + 1) % 10
            
            if np.array_equal(next_state, out):
                return True
            if np.array_equal(next_state, current):
                break
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(10):
  n=[row[:]for row in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    if r[i][j]==0:
     s=[]
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if 0<=ni<len(r)and 0<=nj<len(r[0]):
       s.append(r[ni][nj])
     nz=[v for v in s if v]
     if nz and all(v==nz[0]for v in nz):
      n[i][j]=nz[0]
    elif r[i][j]:
     h=0
     for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
      ni,nj=i+di,j+dj
      if 0<=ni<len(r)and 0<=nj<len(r[0])and r[ni][nj]>r[i][j]:
       h=1;break
     if h:n[i][j]=(r[i][j]+1)%10
  if n==r:break
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerConvergence(TransformationHandler):
    """Runs until convergence to stable state"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        current = np.copy(inp)
        
        # Generic convergence loop
        for iteration in range(50):
            next_state = np.copy(current)
            
            # Apply various local rules
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    # Count neighbors by value
                    neighbor_counts = {}
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            ni, nj = i + di, j + dj
                            if (0 <= ni < current.shape[0] and 
                                0 <= nj < current.shape[1]):
                                val = current[ni, nj]
                                neighbor_counts[val] = neighbor_counts.get(val, 0) + 1
                    
                    # Apply majority rule
                    if neighbor_counts:
                        max_count = max(neighbor_counts.values())
                        candidates = [v for v, c in neighbor_counts.items() if c == max_count and v != 0]
                        
                        if candidates and max_count >= 3:
                            next_state[i, j] = min(candidates)  # Deterministic choice
            
            if np.array_equal(next_state, out):
                return True
            if np.array_equal(next_state, current):
                # Converged but not to target
                break
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(50):
  n=[row[:]for row in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    nc={}
    for di in[-1,0,1]:
     for dj in[-1,0,1]:
      if di==0and dj==0:continue
      ni,nj=i+di,j+dj
      if 0<=ni<len(r)and 0<=nj<len(r[0]):
       v=r[ni][nj]
       nc[v]=nc.get(v,0)+1
    if nc:
     mx=max(nc.values())
     cd=[v for v,c in nc.items()if c==mx and v]
     if cd and mx>=3:n[i][j]=min(cd)
  if n==r:break
  r=n
 return r
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# PATTERN MATCHING AND CONSTRAINT-BASED HANDLERS
# --------------------------------------------------------------------------------

@handler_decorator
class HandlerPatternReplace(TransformationHandler):
    """Finds and replaces specific patterns in the grid"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Look for 2x2 pattern replacements
        for pattern_h in [2, 3]:
            for pattern_w in [2, 3]:
                if inp.shape[0] < pattern_h or inp.shape[1] < pattern_w:
                    continue
                
                # Find all unique patterns in input
                patterns = {}
                for i in range(inp.shape[0] - pattern_h + 1):
                    for j in range(inp.shape[1] - pattern_w + 1):
                        pattern = tuple(inp[i:i+pattern_h, j:j+pattern_w].flatten())
                        if pattern not in patterns:
                            patterns[pattern] = []
                        patterns[pattern].append((i, j))
                
                # Check if output differs only in these patterns
                test = np.copy(inp)
                for pattern, positions in patterns.items():
                    # See what this pattern becomes in output
                    if positions:
                        i, j = positions[0]
                        out_pattern = out[i:i+pattern_h, j:j+pattern_w]
                        
                        # Apply to all occurrences
                        for pi, pj in positions:
                            test[pi:pi+pattern_h, pj:pj+pattern_w] = out_pattern
                
                if np.array_equal(test, out):
                    return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        for pattern_h in [2, 3]:
            for pattern_w in [2, 3]:
                if inp.shape[0] < pattern_h or inp.shape[1] < pattern_w:
                    continue
                
                # Find pattern mapping
                pattern_map = {}
                for i in range(inp.shape[0] - pattern_h + 1):
                    for j in range(inp.shape[1] - pattern_w + 1):
                        inp_pattern = tuple(inp[i:i+pattern_h, j:j+pattern_w].flatten())
                        out_pattern = tuple(out[i:i+pattern_h, j:j+pattern_w].flatten())
                        
                        if inp_pattern != out_pattern:
                            pattern_map[inp_pattern] = out_pattern
                
                if pattern_map and len(pattern_map) <= 3:
                    # Generate compact replacement code
                    return f"""def p(g):
 r=[row[:]for row in g]
 h,w={pattern_h},{pattern_w}
 m={{{','.join(f'{k}:{v}' for k,v in pattern_map.items())}}}
 for i in range(len(r)-h+1):
  for j in range(len(r[0])-w+1):
   pat=tuple(r[i+y][j+x]for y in range(h)for x in range(w))
   if pat in m:
    new=m[pat]
    for y in range(h):
     for x in range(w):
      r[i+y][j+x]=new[y*w+x]
 return r
"""
        return None

@handler_decorator
class HandlerBoundaryTrace(TransformationHandler):
    """Traces boundaries of shapes"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Detect boundary pixels
        test = np.zeros_like(inp)
        
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    # Check if on boundary
                    is_boundary = False
                    
                    # Check 4-connectivity
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (ni < 0 or ni >= inp.shape[0] or 
                            nj < 0 or nj >= inp.shape[1] or 
                            inp[ni, nj] == 0):
                            is_boundary = True
                            break
                    
                    if is_boundary:
                        test[i, j] = inp[i, j]
        
        if np.array_equal(test, out):
            return True
        
        # Try 8-connectivity boundary
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    is_boundary = False
                    
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            ni, nj = i + di, j + dj
                            if (ni < 0 or ni >= inp.shape[0] or 
                                nj < 0 or nj >= inp.shape[1] or 
                                inp[ni, nj] == 0):
                                is_boundary = True
                                break
                        if is_boundary:
                            break
                    
                    if is_boundary:
                        test[i, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # 4-connectivity
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    is_boundary = False
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (ni < 0 or ni >= inp.shape[0] or 
                            nj < 0 or nj >= inp.shape[1] or 
                            inp[ni, nj] == 0):
                            is_boundary = True
                            break
                    if is_boundary:
                        test[i, j] = inp[i, j]
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]:
    b=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if ni<0or ni>=h or nj<0or nj>=w or not g[ni][nj]:
      b=1;break
    if b:r[i][j]=g[i][j]
 return r
"""
        
        # 8-connectivity
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    is_boundary = False
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            ni, nj = i + di, j + dj
                            if (ni < 0 or ni >= inp.shape[0] or 
                                nj < 0 or nj >= inp.shape[1] or 
                                inp[ni, nj] == 0):
                                is_boundary = True
                                break
                        if is_boundary:
                            break
                    if is_boundary:
                        test[i, j] = inp[i, j]
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]:
    b=0
    for di in[-1,0,1]:
     for dj in[-1,0,1]:
      if di==0and dj==0:continue
      ni,nj=i+di,j+dj
      if ni<0or ni>=h or nj<0or nj>=w or not g[ni][nj]:
       b=1;break
     if b:break
    if b:r[i][j]=g[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerMaskApplication(TransformationHandler):
    """Applies masks based on patterns or conditions"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if output is masked version of input
        # Mask by even/odd positions
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if (i + j) % 2 == 0:
                    test[i, j] = 0
        
        if np.array_equal(test, out):
            return True
        
        # Mask by checkerboard
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if (i + j) % 2 == 1:
                    test[i, j] = 0
        
        if np.array_equal(test, out):
            return True
        
        # Mask by distance from center
        ch, cw = inp.shape[0] // 2, inp.shape[1] // 2
        for radius in range(1, max(inp.shape)):
            test = np.copy(inp)
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    dist = abs(i - ch) + abs(j - cw)
                    if dist > radius:
                        test[i, j] = 0
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Even position mask
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if (i + j) % 2 == 0:
                    test[i, j] = 0
        
        if np.array_equal(test, out):
            return """def p(g):
 return[[g[i][j]if(i+j)%2else 0for j in range(len(g[0]))]for i in range(len(g))]
"""
        
        # Odd position mask
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if (i + j) % 2 == 1:
                    test[i, j] = 0
        
        if np.array_equal(test, out):
            return """def p(g):
 return[[g[i][j]if(i+j)%2==0else 0for j in range(len(g[0]))]for i in range(len(g))]
"""
        
        # Distance mask
        ch, cw = inp.shape[0] // 2, inp.shape[1] // 2
        for radius in range(1, max(inp.shape)):
            test = np.copy(inp)
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    dist = abs(i - ch) + abs(j - cw)
                    if dist > radius:
                        test[i, j] = 0
            
            if np.array_equal(test, out):
                return f"""def p(g):
 ch,cw={ch},{cw}
 return[[g[i][j]if abs(i-ch)+abs(j-cw)<={radius}else 0for j in range(len(g[0]))]for i in range(len(g))]
"""
        return None

@handler_decorator
class HandlerConstraintSatisfaction(TransformationHandler):
    """Fills grid based on constraint satisfaction"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check if output satisfies local constraints
        # Constraint: no two adjacent cells have same non-zero value
        test = np.copy(inp)
        
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if test[i, j] == 0:
                    # Find valid value
                    forbidden = set()
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1] and 
                            test[ni, nj] != 0):
                            forbidden.add(test[ni, nj])
                    
                    # Assign smallest valid value
                    for val in range(1, 10):
                        if val not in forbidden:
                            test[i, j] = val
                            break
        
        if np.array_equal(test, out):
            return True
        
        # Constraint: sum of neighbors must equal cell value
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] == 0:
                    neighbor_sum = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1]):
                            neighbor_sum += inp[ni, nj]
                    test[i, j] = neighbor_sum % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # No adjacent same value constraint
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if test[i, j] == 0:
                    forbidden = set()
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1] and 
                            test[ni, nj] != 0):
                            forbidden.add(test[ni, nj])
                    
                    for val in range(1, 10):
                        if val not in forbidden:
                            test[i, j] = val
                            break
        
        if np.array_equal(test, out):
            return """def p(g):
 r=[row[:]for row in g]
 for i in range(len(r)):
  for j in range(len(r[0])):
   if r[i][j]==0:
    f=set()
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<len(r)and 0<=nj<len(r[0])and r[ni][nj]:
      f.add(r[ni][nj])
    for v in range(1,10):
     if v not in f:
      r[i][j]=v;break
 return r
"""
        
        # Sum constraint
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] == 0:
                    neighbor_sum = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1]):
                            neighbor_sum += inp[ni, nj]
                    test[i, j] = neighbor_sum % 10
        
        if np.array_equal(test, out):
            return """def p(g):
 r=[row[:]for row in g]
 for i in range(len(r)):
  for j in range(len(r[0])):
   if g[i][j]==0:
    s=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<len(r)and 0<=nj<len(r[0]):
      s+=g[ni][nj]
    r[i][j]=s%10
 return r
"""
        return None

@handler_decorator
class HandlerSymmetryComplete(TransformationHandler):
    """Completes partial symmetric patterns"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Horizontal symmetry completion
        test = np.copy(inp)
        for i in range(h):
            for j in range(w // 2):
                if test[i, j] != 0 and test[i, w-1-j] == 0:
                    test[i, w-1-j] = test[i, j]
                elif test[i, j] == 0 and test[i, w-1-j] != 0:
                    test[i, j] = test[i, w-1-j]
        
        if np.array_equal(test, out):
            return True
        
        # Vertical symmetry completion
        test = np.copy(inp)
        for i in range(h // 2):
            for j in range(w):
                if test[i, j] != 0 and test[h-1-i, j] == 0:
                    test[h-1-i, j] = test[i, j]
                elif test[i, j] == 0 and test[h-1-i, j] != 0:
                    test[i, j] = test[h-1-i, j]
        
        if np.array_equal(test, out):
            return True
        
        # Diagonal symmetry
        if h == w:
            test = np.copy(inp)
            for i in range(h):
                for j in range(w):
                    if test[i, j] != 0 and test[j, i] == 0:
                        test[j, i] = test[i, j]
                    elif test[i, j] == 0 and test[j, i] != 0:
                        test[i, j] = test[j, i]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Horizontal symmetry
        test = np.copy(inp)
        for i in range(h):
            for j in range(w // 2):
                if test[i, j] != 0 and test[i, w-1-j] == 0:
                    test[i, w-1-j] = test[i, j]
                elif test[i, j] == 0 and test[i, w-1-j] != 0:
                    test[i, j] = test[i, w-1-j]
        
        if np.array_equal(test, out):
            return f"""def p(g):
 r=[row[:]for row in g]
 h,w={h},{w}
 for i in range(h):
  for j in range(w//2):
   if r[i][j]and not r[i][w-1-j]:r[i][w-1-j]=r[i][j]
   elif not r[i][j]and r[i][w-1-j]:r[i][j]=r[i][w-1-j]
 return r
"""
        
        # Vertical symmetry
        test = np.copy(inp)
        for i in range(h // 2):
            for j in range(w):
                if test[i, j] != 0 and test[h-1-i, j] == 0:
                    test[h-1-i, j] = test[i, j]
                elif test[i, j] == 0 and test[h-1-i, j] != 0:
                    test[i, j] = test[h-1-i, j]
        
        if np.array_equal(test, out):
            return f"""def p(g):
 r=[row[:]for row in g]
 h,w={h},{w}
 for i in range(h//2):
  for j in range(w):
   if r[i][j]and not r[h-1-i][j]:r[h-1-i][j]=r[i][j]
   elif not r[i][j]and r[h-1-i][j]:r[i][j]=r[h-1-i][j]
 return r
"""
        
        # Diagonal symmetry
        if h == w:
            test = np.copy(inp)
            for i in range(h):
                for j in range(w):
                    if test[i, j] != 0 and test[j, i] == 0:
                        test[j, i] = test[i, j]
                    elif test[i, j] == 0 and test[j, i] != 0:
                        test[i, j] = test[j, i]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 r=[row[:]for row in g]
 n={h}
 for i in range(n):
  for j in range(n):
   if r[i][j]and not r[j][i]:r[j][i]=r[i][j]
   elif not r[i][j]and r[j][i]:r[i][j]=r[j][i]
 return r
"""
        return None

@handler_decorator
class HandlerLayerExtraction(TransformationHandler):
    """Extracts specific layers or color planes"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Extract single color layer
        for color in range(1, 10):
            test = np.zeros_like(inp)
            test[inp == color] = color
            
            if np.array_equal(test, out):
                return True
        
        # Extract multiple colors
        for color1 in range(1, 10):
            for color2 in range(color1 + 1, 10):
                test = np.zeros_like(inp)
                test[inp == color1] = color1
                test[inp == color2] = color2
                
                if np.array_equal(test, out):
                    return True
        
        # Extract based on value threshold
        for threshold in range(1, 10):
            test = np.zeros_like(inp)
            test[inp >= threshold] = inp[inp >= threshold]
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Single color
        for color in range(1, 10):
            test = np.zeros_like(inp)
            test[inp == color] = color
            
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[v if v=={color}else 0for v in r]for r in g]
"""
        
        # Two colors
        for color1 in range(1, 10):
            for color2 in range(color1 + 1, 10):
                test = np.zeros_like(inp)
                test[inp == color1] = color1
                test[inp == color2] = color2
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[[v if v in[{color1},{color2}]else 0for v in r]for r in g]
"""
        
        # Threshold
        for threshold in range(1, 10):
            test = np.zeros_like(inp)
            test[inp >= threshold] = inp[inp >= threshold]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[v if v>={threshold}else 0for v in r]for r in g]
"""
        return None

@handler_decorator
class HandlerCompression(TransformationHandler):
    """Compresses patterns using run-length or other encoding"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check for downsampling (taking every nth pixel)
        for step in [2, 3, 4]:
            if (out.shape[0] == (inp.shape[0] + step - 1) // step and 
                out.shape[1] == (inp.shape[1] + step - 1) // step):
                
                test = inp[::step, ::step]
                if np.array_equal(test, out):
                    return True
        
        # Check for removing empty rows/columns
        non_empty_rows = [i for i in range(inp.shape[0]) if np.any(inp[i, :] != 0)]
        non_empty_cols = [j for j in range(inp.shape[1]) if np.any(inp[:, j] != 0)]
        
        if non_empty_rows and non_empty_cols:
            compressed = inp[np.ix_(non_empty_rows, non_empty_cols)]
            if compressed.shape == out.shape and np.array_equal(compressed, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Downsampling
        for step in [2, 3, 4]:
            if (out.shape[0] == (inp.shape[0] + step - 1) // step and 
                out.shape[1] == (inp.shape[1] + step - 1) // step):
                
                test = inp[::step, ::step]
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[r[::{step}]for r in g[::{step}]]
"""
        
        # Remove empty rows/columns
        non_empty_rows = [i for i in range(inp.shape[0]) if np.any(inp[i, :] != 0)]
        non_empty_cols = [j for j in range(inp.shape[1]) if np.any(inp[:, j] != 0)]
        
        if non_empty_rows and non_empty_cols:
            compressed = inp[np.ix_(non_empty_rows, non_empty_cols)]
            if compressed.shape == out.shape and np.array_equal(compressed, out):
                return """def p(g):
 nr=[i for i,r in enumerate(g)if any(r)]
 nc=[j for j in range(len(g[0]))if any(g[i][j]for i in range(len(g)))]
 return[[g[i][j]for j in nc]for i in nr]
"""
        return None

@handler_decorator
class HandlerDistanceTransform(TransformationHandler):
    """Transforms based on distance to nearest feature"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Find non-zero positions
        feature_positions = []
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    feature_positions.append((i, j))
        
        if not feature_positions:
            return False
        
        # Calculate distance transform
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    test[i, j] = inp[i, j]
                else:
                    # Find minimum distance to feature
                    min_dist = float('inf')
                    for fi, fj in feature_positions:
                        dist = abs(i - fi) + abs(j - fj)
                        min_dist = min(min_dist, dist)
                    
                    test[i, j] = min_dist % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 f=[(i,j)for i in range(len(g))for j in range(len(g[0]))if g[i][j]]
 r=[[0]*len(g[0])for _ in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]:r[i][j]=g[i][j]
   else:r[i][j]=min(abs(i-fi)+abs(j-fj)for fi,fj in f)%10
 return r
"""
        return None

@handler_decorator
class HandlerConvolution(TransformationHandler):
    """Applies convolution-like operations"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # 3x3 average filter
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                sum_val = 0
                count = 0
                
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1]):
                            sum_val += inp[ni, nj]
                            count += 1
                
                test[i, j] = (sum_val // count) if count > 0 else 0
        
        if np.array_equal(test, out):
            return True
        
        # Edge detection kernel
        test = np.zeros_like(inp)
        for i in range(1, inp.shape[0] - 1):
            for j in range(1, inp.shape[1] - 1):
                # Sobel-like operator
                val = (4 * inp[i, j] - 
                       inp[i-1, j] - inp[i+1, j] - 
                       inp[i, j-1] - inp[i, j+1])
                test[i, j] = abs(val) % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Average filter
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                sum_val = 0
                count = 0
                
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1]):
                            sum_val += inp[ni, nj]
                            count += 1
                
                test[i, j] = (sum_val // count) if count > 0 else 0
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   s=c=0
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w:
      s+=g[ni][nj];c+=1
   r[i][j]=s//c if c else 0
 return r
"""
        
        # Edge detection
        test = np.zeros_like(inp)
        for i in range(1, inp.shape[0] - 1):
            for j in range(1, inp.shape[1] - 1):
                val = (4 * inp[i, j] - 
                       inp[i-1, j] - inp[i+1, j] - 
                       inp[i, j-1] - inp[i, j+1])
                test[i, j] = abs(val) % 10
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 r=[[0]*w for _ in range(h)]
 for i in range(1,h-1):
  for j in range(1,w-1):
   v=4*g[i][j]-g[i-1][j]-g[i+1][j]-g[i][j-1]-g[i][j+1]
   r[i][j]=abs(v)%10
 return r
"""
        return None

@handler_decorator
class HandlerMorphological(TransformationHandler):
    """Applies morphological operations like dilation and erosion"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Dilation
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1] and 
                            inp[ni, nj] == 0):
                            test[ni, nj] = inp[i, j]
        
        if np.array_equal(test, out):
            return True
        
        # Erosion
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    # Check if all neighbors are non-zero
                    all_neighbors = True
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1]):
                            if inp[ni, nj] == 0:
                                all_neighbors = False
                                break
                        else:
                            all_neighbors = False
                            break
                    
                    if all_neighbors:
                        test[i, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Dilation
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1] and 
                            inp[ni, nj] == 0):
                            test[ni, nj] = inp[i, j]
        
        if np.array_equal(test, out):
            return """def p(g):
 r=[row[:]for row in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]:
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<len(g)and 0<=nj<len(g[0])and g[ni][nj]==0:
      r[ni][nj]=g[i][j]
 return r
"""
        
        # Erosion
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    all_neighbors = True
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1]):
                            if inp[ni, nj] == 0:
                                all_neighbors = False
                                break
                        else:
                            all_neighbors = False
                            break
                    
                    if all_neighbors:
                        test[i, j] = inp[i, j]
        
        if np.array_equal(test, out):
            return """def p(g):
 r=[[0]*len(g[0])for _ in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]:
    a=1
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<len(g)and 0<=nj<len(g[0]):
      if g[ni][nj]==0:a=0;break
     else:a=0;break
    if a:r[i][j]=g[i][j]
 return r
"""
        return None

@handler_decorator
class HandlerPathConnectivity(TransformationHandler):
    """Creates or verifies path connectivity between points"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Find special points (value > 5)
        special_points = []
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] > 5:
                    special_points.append((i, j, inp[i, j]))
        
        if len(special_points) < 2:
            return False
        
        # Connect points with straight lines
        test = np.copy(inp)
        for idx in range(len(special_points) - 1):
            i1, j1, _ = special_points[idx]
            i2, j2, _ = special_points[idx + 1]
            
            # Draw line between points
            if abs(i2 - i1) > abs(j2 - j1):
                # More vertical
                for i in range(min(i1, i2), max(i1, i2) + 1):
                    j = j1 + (j2 - j1) * (i - i1) // (i2 - i1) if i2 != i1 else j1
                    if test[i, j] == 0:
                        test[i, j] = 3
            else:
                # More horizontal
                for j in range(min(j1, j2), max(j1, j2) + 1):
                    i = i1 + (i2 - i1) * (j - j1) // (j2 - j1) if j2 != j1 else i1
                    if test[i, j] == 0:
                        test[i, j] = 3
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 pts=[(i,j)for i in range(len(g))for j in range(len(g[0]))if g[i][j]>5]
 for k in range(len(pts)-1):
  i1,j1=pts[k]
  i2,j2=pts[k+1]
  if abs(i2-i1)>abs(j2-j1):
   for i in range(min(i1,i2),max(i1,i2)+1):
    j=j1+(j2-j1)*(i-i1)//(i2-i1)if i2!=i1else j1
    if r[i][j]==0:r[i][j]=3
  else:
   for j in range(min(j1,j2),max(j1,j2)+1):
    i=i1+(i2-i1)*(j-j1)//(j2-j1)if j2!=j1else i1
    if r[i][j]==0:r[i][j]=3
 return r
"""
        return None

@handler_decorator
class HandlerOverlay(TransformationHandler):
    """Overlays multiple patterns or layers"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Check if output combines odd and even position values differently
        if inp.shape == out.shape:
            test = np.zeros_like(inp)
            
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    if (i + j) % 2 == 0:
                        test[i, j] = inp[i, j]
                    else:
                        # Transform value for odd positions
                        test[i, j] = (inp[i, j] * 2) % 10 if inp[i, j] != 0 else 0
            
            if np.array_equal(test, out):
                return True
            
            # Try XOR-like operation
            test = np.zeros_like(inp)
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    # Combine with neighbors using XOR-like operation
                    val = inp[i, j]
                    for di, dj in [(0,1), (1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1]):
                            val = (val + inp[ni, nj]) % 10
                    test[i, j] = val
            
            if np.array_equal(test, out):
                return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape == out.shape:
            test = np.zeros_like(inp)
            
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    if (i + j) % 2 == 0:
                        test[i, j] = inp[i, j]
                    else:
                        test[i, j] = (inp[i, j] * 2) % 10 if inp[i, j] != 0 else 0
            
            if np.array_equal(test, out):
                return """def p(g):
 return[[g[i][j]if(i+j)%2==0else(g[i][j]*2)%10if g[i][j]else 0for j in range(len(g[0]))]for i in range(len(g))]
"""
            
            # XOR-like
            test = np.zeros_like(inp)
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    val = inp[i, j]
                    for di, dj in [(0,1), (1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1]):
                            val = (val + inp[ni, nj]) % 10
                    test[i, j] = val
            
            if np.array_equal(test, out):
                return """def p(g):
 r=[[0]*len(g[0])for _ in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   v=g[i][j]
   for di,dj in[(0,1),(1,0)]:
    ni,nj=i+di,j+dj
    if 0<=ni<len(g)and 0<=nj<len(g[0]):
     v=(v+g[ni][nj])%10
   r[i][j]=v
 return r
"""
        return None

@handler_decorator
class HandlerFrequencyDomain(TransformationHandler):
    """Applies frequency-domain-like transformations"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Check for periodic patterns
        h, w = inp.shape
        
        # Sine-like pattern
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                # Create wave pattern
                wave_val = int(5 * (1 + np.sin(2 * np.pi * j / w)))
                test[i, j] = (inp[i, j] + wave_val) % 10 if inp[i, j] != 0 else wave_val
        
        if np.array_equal(test, out):
            return True
        
        # Frequency filtering (keep only certain periodic components)
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                # Simple high-pass filter effect
                if j > 0:
                    diff = abs(inp[i, j] - inp[i, j-1])
                    test[i, j] = diff % 10
                else:
                    test[i, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # High-pass filter
        test = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                if j > 0:
                    diff = abs(inp[i, j] - inp[i, j-1])
                    test[i, j] = diff % 10
                else:
                    test[i, j] = inp[i, j]
        
        if np.array_equal(test, out):
            return """def p(g):
 return[[abs(g[i][j]-g[i][j-1])%10if j>0else g[i][j]for j in range(len(g[0]))]for i in range(len(g))]
"""
        return None

@handler_decorator
class HandlerNeighborhoodRule(TransformationHandler):
    """Applies rules based on neighborhood configurations"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Rule based on neighbor configuration
        test = np.zeros_like(inp)
        
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                # Get 3x3 neighborhood
                neighbors = []
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1]):
                            neighbors.append(inp[ni, nj])
                        else:
                            neighbors.append(0)
                
                # Apply rule based on pattern
                center = neighbors[4]  # Center cell
                
                # Rule: if center is 0 and has exactly 3 non-zero neighbors, 
                # take the most common value
                if center == 0:
                    non_zero = [n for n in neighbors if n != 0]
                    if len(non_zero) == 3:
                        from collections import Counter
                        test[i, j] = Counter(non_zero).most_common(1)[0][0]
                    else:
                        test[i, j] = 0
                else:
                    # Rule: increment if surrounded by lower values
                    lower_count = sum(1 for n in neighbors if 0 < n < center)
                    if lower_count >= 4:
                        test[i, j] = (center + 1) % 10
                    else:
                        test[i, j] = center
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[[0]*len(g[0])for _ in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   nb=[]
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     ni,nj=i+di,j+dj
     if 0<=ni<len(g)and 0<=nj<len(g[0]):
      nb.append(g[ni][nj])
     else:nb.append(0)
   c=nb[4]
   if c==0:
    nz=[n for n in nb if n]
    if len(nz)==3:
     r[i][j]=max(nz,key=nz.count)
   else:
    lc=sum(1for n in nb if 0<n<c)
    r[i][j]=(c+1)%10if lc>=4else c
 return r
"""
        return None

@handler_decorator
class HandlerScanline(TransformationHandler):
    """Processes grid using scanline algorithms"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Horizontal scanline fill
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            inside = False
            last_color = 0
            
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    if not inside:
                        inside = True
                        last_color = inp[i, j]
                    else:
                        # Found second boundary
                        if inp[i, j] == last_color:
                            # Fill between boundaries
                            for k in range(j):
                                if test[i, k] == 0:
                                    test[i, k] = last_color
                            inside = False
                elif inside and test[i, j] == 0:
                    test[i, j] = last_color
        
        if np.array_equal(test, out):
            return True
        
        # Vertical scanline
        test = np.copy(inp)
        for j in range(inp.shape[1]):
            inside = False
            last_color = 0
            
            for i in range(inp.shape[0]):
                if inp[i, j] != 0:
                    if not inside:
                        inside = True
                        last_color = inp[i, j]
                    else:
                        # Found second boundary
                        if inp[i, j] == last_color:
                            # Fill between boundaries
                            for k in range(i):
                                if test[k, j] == 0:
                                    test[k, j] = last_color
                            inside = False
                elif inside and test[i, j] == 0:
                    test[i, j] = last_color
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Horizontal scanline
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            inside = False
            last_color = 0
            
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    if not inside:
                        inside = True
                        last_color = inp[i, j]
                    else:
                        if inp[i, j] == last_color:
                            for k in range(j):
                                if test[i, k] == 0:
                                    test[i, k] = last_color
                            inside = False
                elif inside and test[i, j] == 0:
                    test[i, j] = last_color
        
        if np.array_equal(test, out):
            return """def p(g):
 r=[row[:]for row in g]
 for i in range(len(g)):
  ins=0;lc=0
  for j in range(len(g[0])):
   if g[i][j]:
    if not ins:ins=1;lc=g[i][j]
    elif g[i][j]==lc:
     for k in range(j):
      if r[i][k]==0:r[i][k]=lc
     ins=0
   elif ins and r[i][j]==0:r[i][j]=lc
 return r
"""
        return None

@handler_decorator
class HandlerGradientFill(TransformationHandler):
    """Creates gradient fills between points"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Linear gradient
        test = np.zeros_like(inp)
        
        # Horizontal gradient
        for i in range(inp.shape[0]):
            start_val = None
            start_j = None
            
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    if start_val is None:
                        start_val = inp[i, j]
                        start_j = j
                        test[i, j] = start_val
                    else:
                        # Create gradient between points
                        end_val = inp[i, j]
                        end_j = j
                        
                        for k in range(start_j + 1, end_j):
                            progress = (k - start_j) / (end_j - start_j)
                            test[i, k] = int(start_val + progress * (end_val - start_val)) % 10
                        
                        test[i, j] = end_val
                        start_val = end_val
                        start_j = end_j
        
        if np.array_equal(test, out):
            return True
        
        # Radial gradient
        center_points = [(i, j) for i in range(inp.shape[0]) 
                         for j in range(inp.shape[1]) if inp[i, j] != 0]
        
        if center_points:
            test = np.zeros_like(inp)
            ci, cj = center_points[0]
            center_val = inp[ci, cj]
            
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    dist = abs(i - ci) + abs(j - cj)
                    test[i, j] = (center_val - dist) % 10 if center_val - dist >= 0 else 0
            
            return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Radial gradient
        center_points = [(i, j) for i in range(inp.shape[0]) 
                         for j in range(inp.shape[1]) if inp[i, j] != 0]
        
        if center_points:
            test = np.zeros_like(inp)
            ci, cj = center_points[0]
            center_val = inp[ci, cj]
            
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    dist = abs(i - ci) + abs(j - cj)
                    test[i, j] = (center_val - dist) % 10 if center_val - dist >= 0 else 0
            
            if np.array_equal(test, out):
                return f"""def p(g):
 pts=[(i,j)for i in range(len(g))for j in range(len(g[0]))if g[i][j]]
 if pts:
  ci,cj=pts[0]
  cv=g[ci][cj]
  return[[(cv-abs(i-ci)-abs(j-cj))%10if cv-abs(i-ci)-abs(j-cj)>=0else 0for j in range(len(g[0]))]for i in range(len(g))]
 return g
"""
        return None

@handler_decorator
class HandlerSetOperations(TransformationHandler):
    """Performs set operations on regions"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Union of two colors
        for color1 in range(1, 10):
            for color2 in range(color1 + 1, 10):
                test = np.zeros_like(inp)
                test[(inp == color1) | (inp == color2)] = 1
                
                if np.array_equal(test, out):
                    return True
        
        # Intersection (where multiple colors overlap)
        # This would need a different input format
        
        # Difference (color1 minus color2 regions)
        for color1 in range(1, 10):
            for color2 in range(1, 10):
                if color1 != color2:
                    test = np.copy(inp)
                    # Remove color1 where adjacent to color2
                    for i in range(inp.shape[0]):
                        for j in range(inp.shape[1]):
                            if inp[i, j] == color1:
                                adjacent_to_color2 = False
                                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                                    ni, nj = i + di, j + dj
                                    if (0 <= ni < inp.shape[0] and 
                                        0 <= nj < inp.shape[1] and 
                                        inp[ni, nj] == color2):
                                        adjacent_to_color2 = True
                                        break
                                
                                if adjacent_to_color2:
                                    test[i, j] = 0
                    
                    if np.array_equal(test, out):
                        return True
        
        return False
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Union
        for color1 in range(1, 10):
            for color2 in range(color1 + 1, 10):
                test = np.zeros_like(inp)
                test[(inp == color1) | (inp == color2)] = 1
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 return[[1if v in[{color1},{color2}]else 0for v in r]for r in g]
"""
        
        # Difference
        for color1 in range(1, 10):
            for color2 in range(1, 10):
                if color1 != color2:
                    test = np.copy(inp)
                    for i in range(inp.shape[0]):
                        for j in range(inp.shape[1]):
                            if inp[i, j] == color1:
                                adjacent_to_color2 = False
                                for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                                    ni, nj = i + di, j + dj
                                    if (0 <= ni < inp.shape[0] and 
                                        0 <= nj < inp.shape[1] and 
                                        inp[ni, nj] == color2):
                                        adjacent_to_color2 = True
                                        break
                                
                                if adjacent_to_color2:
                                    test[i, j] = 0
                    
                    if np.array_equal(test, out):
                        return f"""def p(g):
 r=[row[:]for row in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]=={color1}:
    adj=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<len(g)and 0<=nj<len(g[0])and g[ni][nj]=={color2}:
      adj=1;break
    if adj:r[i][j]=0
 return r
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
# --------------------------------------------------------------------------------
# CREATIVE ITERATIVE/RECURSIVE ATOMIC TRANSFORMATIONS WITH 1D CONVERSIONS
# --------------------------------------------------------------------------------

@handler_decorator
class HandlerRaster1DTransform(TransformationHandler):
    """Converts to 1D, applies transformation, converts back"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Row-major order with various transformations
        flat = inp.flatten()
        
        # Try sliding window average
        test_flat = np.zeros_like(flat)
        for i in range(len(flat)):
            window = []
            for j in range(max(0, i-1), min(len(flat), i+2)):
                if flat[j] != 0:
                    window.append(flat[j])
            test_flat[i] = sum(window) // len(window) if window else 0
        
        test = test_flat.reshape(h, w)
        if np.array_equal(test, out):
            return True
        
        # Try cumulative sum mod 10
        test_flat = np.zeros_like(flat)
        cumsum = 0
        for i in range(len(flat)):
            cumsum = (cumsum + flat[i]) % 10
            test_flat[i] = cumsum
        
        test = test_flat.reshape(h, w)
        if np.array_equal(test, out):
            return True
        
        # Try difference encoding
        test_flat = np.zeros_like(flat)
        test_flat[0] = flat[0]
        for i in range(1, len(flat)):
            test_flat[i] = (flat[i] - flat[i-1]) % 10
        
        test = test_flat.reshape(h, w)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        h, w = inp.shape
        
        # Sliding window
        flat = inp.flatten()
        test_flat = np.zeros_like(flat)
        for i in range(len(flat)):
            window = []
            for j in range(max(0, i-1), min(len(flat), i+2)):
                if flat[j] != 0:
                    window.append(flat[j])
            test_flat[i] = sum(window) // len(window) if window else 0
        
        test = test_flat.reshape(h, w)
        if np.array_equal(test, out):
            return f"""def p(g):
 f=[x for r in g for x in r]
 n=[]
 for i in range(len(f)):
  w=[f[j]for j in range(max(0,i-1),min(len(f),i+2))if f[j]]
  n.append(sum(w)//len(w)if w else 0)
 return[n[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        
        # Cumulative sum
        flat = inp.flatten()
        test_flat = np.zeros_like(flat)
        cumsum = 0
        for i in range(len(flat)):
            cumsum = (cumsum + flat[i]) % 10
            test_flat[i] = cumsum
        
        test = test_flat.reshape(h, w)
        if np.array_equal(test, out):
            return f"""def p(g):
 f=[x for r in g for x in r]
 s=0;n=[]
 for x in f:s=(s+x)%10;n.append(s)
 return[n[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        
        # Difference encoding
        flat = inp.flatten()
        test_flat = np.zeros_like(flat)
        test_flat[0] = flat[0]
        for i in range(1, len(flat)):
            test_flat[i] = (flat[i] - flat[i-1]) % 10
        
        test = test_flat.reshape(h, w)
        if np.array_equal(test, out):
            return f"""def p(g):
 f=[x for r in g for x in r]
 n=[f[0]]+[(f[i]-f[i-1])%10for i in range(1,len(f))]
 return[n[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerSpiralTransform(TransformationHandler):
    """Transforms using spiral ordering"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Extract spiral
        spiral = []
        top, bottom, left, right = 0, h-1, 0, w-1
        
        while top <= bottom and left <= right:
            for j in range(left, right+1):
                spiral.append(inp[top, j])
            top += 1
            
            for i in range(top, bottom+1):
                spiral.append(inp[i, right])
            right -= 1
            
            if top <= bottom:
                for j in range(right, left-1, -1):
                    spiral.append(inp[bottom, j])
                bottom -= 1
            
            if left <= right:
                for i in range(bottom, top-1, -1):
                    spiral.append(inp[i, left])
                left += 1
        
        # Apply transformations
        # Shift by position
        transformed = [(val + idx) % 10 if val != 0 else 0 for idx, val in enumerate(spiral)]
        
        # Place back in spiral
        test = np.zeros_like(inp)
        idx = 0
        top, bottom, left, right = 0, h-1, 0, w-1
        
        while top <= bottom and left <= right and idx < len(transformed):
            for j in range(left, right+1):
                if idx < len(transformed):
                    test[top, j] = transformed[idx]
                    idx += 1
            top += 1
            
            for i in range(top, bottom+1):
                if idx < len(transformed):
                    test[i, right] = transformed[idx]
                    idx += 1
            right -= 1
            
            if top <= bottom:
                for j in range(right, left-1, -1):
                    if idx < len(transformed):
                        test[bottom, j] = transformed[idx]
                        idx += 1
                bottom -= 1
            
            if left <= right:
                for i in range(bottom, top-1, -1):
                    if idx < len(transformed):
                        test[i, left] = transformed[idx]
                        idx += 1
                left += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h, w = task_data['train'][0]['input'].shape
            return f"""def p(g):
 s=[];t,b,l,r=0,{h-1},0,{w-1}
 while t<=b and l<=r:
  s.extend(g[t][l:r+1]);t+=1
  for i in range(t,b+1):s.append(g[i][r])
  r-=1
  if t<=b:s.extend(g[b][r:l-1:-1]);b-=1
  if l<=r:
   for i in range(b,t-1,-1):s.append(g[i][l])
   l+=1
 tr=[(v+i)%10if v else 0for i,v in enumerate(s)]
 res=[[0]*{w}for _ in range({h})]
 idx=0;t,b,l,r=0,{h-1},0,{w-1}
 while t<=b and l<=r and idx<len(tr):
  for j in range(l,r+1):
   if idx<len(tr):res[t][j]=tr[idx];idx+=1
  t+=1
  for i in range(t,b+1):
   if idx<len(tr):res[i][r]=tr[idx];idx+=1
  r-=1
  if t<=b:
   for j in range(r,l-1,-1):
    if idx<len(tr):res[b][j]=tr[idx];idx+=1
   b-=1
  if l<=r:
   for i in range(b,t-1,-1):
    if idx<len(tr):res[i][l]=tr[idx];idx+=1
   l+=1
 return res
"""
        return None

@handler_decorator
class HandlerZigzagTransform(TransformationHandler):
    """Transforms using zigzag ordering"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Extract zigzag order
        zigzag = []
        for i in range(h):
            if i % 2 == 0:
                zigzag.extend(inp[i, :])
            else:
                zigzag.extend(inp[i, ::-1])
        
        # Apply transformations
        # Run-length encoding effect
        transformed = []
        i = 0
        while i < len(zigzag):
            val = zigzag[i]
            count = 1
            while i + count < len(zigzag) and zigzag[i + count] == val:
                count += 1
            
            # Encode as value, count pairs
            for _ in range(count):
                transformed.append((val + count - 1) % 10 if val != 0 else 0)
            i += count
        
        # Place back in zigzag order
        test = np.zeros_like(inp)
        idx = 0
        for i in range(h):
            if i % 2 == 0:
                for j in range(w):
                    if idx < len(transformed):
                        test[i, j] = transformed[idx]
                        idx += 1
            else:
                for j in range(w-1, -1, -1):
                    if idx < len(transformed):
                        test[i, j] = transformed[idx]
                        idx += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h, w = task_data['train'][0]['input'].shape
            return f"""def p(g):
 z=[]
 for i in range({h}):
  if i%2==0:z.extend(g[i])
  else:z.extend(g[i][::-1])
 t=[];i=0
 while i<len(z):
  v=z[i];c=1
  while i+c<len(z)and z[i+c]==v:c+=1
  for _ in range(c):t.append((v+c-1)%10if v else 0)
  i+=c
 r=[[0]*{w}for _ in range({h})];idx=0
 for i in range({h}):
  if i%2==0:
   for j in range({w}):
    if idx<len(t):r[i][j]=t[idx];idx+=1
  else:
   for j in range({w}-1,-1,-1):
    if idx<len(t):r[i][j]=t[idx];idx+=1
 return r
"""
        return None

@handler_decorator
class HandlerPythonBuiltins(TransformationHandler):
    """Uses Python built-in functions creatively"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Using zip to transpose and process
        transposed = list(zip(*inp))
        
        # Process each column with map
        processed = []
        for col in transposed:
            # Apply filter to remove zeros, then redistribute
            non_zero = list(filter(lambda x: x != 0, col))
            if non_zero:
                # Cycle through non-zero values
                new_col = []
                for i in range(len(col)):
                    if col[i] != 0:
                        new_col.append(non_zero[i % len(non_zero)])
                    else:
                        new_col.append(0)
                processed.append(new_col)
            else:
                processed.append(col)
        
        test = np.array(list(zip(*processed)))
        if np.array_equal(test, out):
            return True
        
        # Using enumerate and list comprehension
        flat = [x for row in inp for x in row]
        transformed = [x if i % 2 == 0 else (x * 2) % 10 for i, x in enumerate(flat)]
        test = np.array(transformed).reshape(inp.shape)
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Enumerate transformation
        flat = [x for row in inp for x in row]
        transformed = [x if i % 2 == 0 else (x * 2) % 10 for i, x in enumerate(flat)]
        test = np.array(transformed).reshape(inp.shape)
        
        if np.array_equal(test, out):
            h, w = inp.shape
            return f"""def p(g):
 f=[x for r in g for x in r]
 t=[x if i%2==0else(x*2)%10for i,x in enumerate(f)]
 return[t[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerIterativeKernel(TransformationHandler):
    """Applies tiny kernels iteratively"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        current = np.copy(inp)
        
        # 2x2 kernel that rotates values
        for iteration in range(10):
            next_state = np.copy(current)
            
            for i in range(0, current.shape[0] - 1, 2):
                for j in range(0, current.shape[1] - 1, 2):
                    # Rotate 2x2 block clockwise
                    if (current[i, j] != 0 or current[i, j+1] != 0 or 
                        current[i+1, j] != 0 or current[i+1, j+1] != 0):
                        temp = current[i, j]
                        next_state[i, j] = current[i+1, j]
                        next_state[i+1, j] = current[i+1, j+1]
                        next_state[i+1, j+1] = current[i, j+1]
                        next_state[i, j+1] = temp
            
            if np.array_equal(next_state, out):
                return True
            if np.array_equal(next_state, current):
                break
            current = next_state
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(10):
  n=[row[:]for row in r]
  for i in range(0,len(r)-1,2):
   for j in range(0,len(r[0])-1,2):
    if r[i][j]or r[i][j+1]or r[i+1][j]or r[i+1][j+1]:
     t=r[i][j]
     n[i][j]=r[i+1][j]
     n[i+1][j]=r[i+1][j+1]
     n[i+1][j+1]=r[i][j+1]
     n[i][j+1]=t
  if n==r:break
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerHilbertCurve(TransformationHandler):
    """Transforms using Hilbert curve ordering"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Simple pseudo-Hilbert ordering for small grids
        if h == 4 and w == 4:
            # Hardcoded 4x4 Hilbert curve order
            hilbert_order = [
                (0,0), (1,0), (1,1), (0,1),
                (0,2), (0,3), (1,3), (1,2),
                (2,2), (2,3), (3,3), (3,2),
                (3,1), (2,1), (2,0), (3,0)
            ]
            
            # Extract values in Hilbert order
            hilbert_vals = [inp[i, j] for i, j in hilbert_order]
            
            # Transform - increment by position
            transformed = [(v + idx) % 10 if v != 0 else 0 for idx, v in enumerate(hilbert_vals)]
            
            # Place back
            test = np.zeros_like(inp)
            for idx, (i, j) in enumerate(hilbert_order):
                test[i, j] = transformed[idx]
            
            return np.array_equal(test, out)
        
        return False
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 ho=[(0,0),(1,0),(1,1),(0,1),(0,2),(0,3),(1,3),(1,2),
     (2,2),(2,3),(3,3),(3,2),(3,1),(2,1),(2,0),(3,0)]
 hv=[g[i][j]for i,j in ho]
 tr=[(v+idx)%10if v else 0for idx,v in enumerate(hv)]
 r=[[0]*4for _ in range(4)]
 for idx,(i,j)in enumerate(ho):r[i][j]=tr[idx]
 return r
"""
        return None

@handler_decorator
class HandlerColumnMajorTransform(TransformationHandler):
    """Transforms using column-major ordering"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Column-major flatten
        col_major = []
        for j in range(w):
            for i in range(h):
                col_major.append(inp[i, j])
        
        # Apply transformation - XOR with position
        transformed = [(v ^ (idx % 8)) % 10 if v != 0 else 0 for idx, v in enumerate(col_major)]
        
        # Reshape back in column-major order
        test = np.zeros_like(inp)
        idx = 0
        for j in range(w):
            for i in range(h):
                test[i, j] = transformed[idx]
                idx += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h, w = task_data['train'][0]['input'].shape
            return f"""def p(g):
 cm=[]
 for j in range({w}):
  for i in range({h}):cm.append(g[i][j])
 tr=[(v^(idx%8))%10if v else 0for idx,v in enumerate(cm)]
 r=[[0]*{w}for _ in range({h})];idx=0
 for j in range({w}):
  for i in range({h}):r[i][j]=tr[idx];idx+=1
 return r
"""
        return None

@handler_decorator
class HandlerDiagonalOrder(TransformationHandler):
    """Transforms using diagonal ordering"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        h, w = inp.shape
        
        # Extract diagonals
        diagonals = []
        for d in range(h + w - 1):
            diagonal = []
            for i in range(h):
                j = d - i
                if 0 <= j < w:
                    diagonal.append(inp[i, j])
            diagonals.extend(diagonal)
        
        # Transform - reverse each group of same values
        transformed = []
        i = 0
        while i < len(diagonals):
            val = diagonals[i]
            group = [val]
            i += 1
            while i < len(diagonals) and diagonals[i] == val:
                group.append(diagonals[i])
                i += 1
            transformed.extend(reversed(group))
        
        # Place back along diagonals
        test = np.zeros_like(inp)
        idx = 0
        for d in range(h + w - 1):
            for i in range(h):
                j = d - i
                if 0 <= j < w and idx < len(transformed):
                    test[i, j] = transformed[idx]
                    idx += 1
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h, w = task_data['train'][0]['input'].shape
            return f"""def p(g):
 d=[]
 for s in range({h+w-1}):
  for i in range({h}):
   j=s-i
   if 0<=j<{w}:d.append(g[i][j])
 t=[];i=0
 while i<len(d):
  v=d[i];gr=[v];i+=1
  while i<len(d)and d[i]==v:gr.append(d[i]);i+=1
  t.extend(gr[::-1])
 r=[[0]*{w}for _ in range({h})];idx=0
 for s in range({h+w-1}):
  for i in range({h}):
   j=s-i
   if 0<=j<{w}and idx<len(t):r[i][j]=t[idx];idx+=1
 return r
"""
        return None

@handler_decorator
class HandlerBitManipulation(TransformationHandler):
    """Uses bit manipulation for transformations"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Bit rotation
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    # Rotate bits left by 1
                    val = inp[i, j]
                    test[i, j] = ((val << 1) | (val >> 3)) & 0xF
                    if test[i, j] >= 10:
                        test[i, j] = test[i, j] % 10
        
        if np.array_equal(test, out):
            return True
        
        # Bit reversal
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    # Reverse bits (for 4-bit numbers)
                    val = inp[i, j]
                    reversed_val = 0
                    for _ in range(4):
                        reversed_val = (reversed_val << 1) | (val & 1)
                        val >>= 1
                    test[i, j] = reversed_val % 10
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        # Bit rotation
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    val = inp[i, j]
                    test[i, j] = ((val << 1) | (val >> 3)) & 0xF
                    if test[i, j] >= 10:
                        test[i, j] = test[i, j] % 10
        
        if np.array_equal(test, out):
            return """def p(g):
 return[[((v<<1)|(v>>3))&0xF%10if v else 0for v in r]for r in g]
"""
        return None

@handler_decorator
class HandlerFunctionalComposition(TransformationHandler):
    """Composes multiple small functions"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Function composition: increment, then modulo by neighbor count
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0:
                    # Count non-zero neighbors
                    count = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if (0 <= ni < inp.shape[0] and 
                            0 <= nj < inp.shape[1] and 
                            inp[ni, nj] != 0):
                            count += 1
                    
                    # Apply composition
                    val = inp[i, j]
                    val = (val + 1) % 10  # First function
                    if count > 0:
                        val = val % (count + 1)  # Second function
                    test[i, j] = val
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[[0]*len(g[0])for _ in g]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]:
    c=sum(1for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]if 0<=i+di<len(g)and 0<=j+dj<len(g[0])and g[i+di][j+dj])
    v=(g[i][j]+1)%10
    if c:v=v%(c+1)
    r[i][j]=v
 return r
"""
        return None

@handler_decorator
class HandlerPrimeTransform(TransformationHandler):
    """Uses prime numbers for transformations"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Small primes
        primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
        
        # Transform by mapping to nth prime
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i, j] != 0 and inp[i, j] < len(primes):
                    test[i, j] = primes[inp[i, j]] % 10
                else:
                    test[i, j] = inp[i, j]
        
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 pr=[2,3,5,7,11,13,17,19,23,29]
 return[[pr[v]%10if v and v<len(pr)else v for v in r]for r in g]
"""
        return None

@handler_decorator
class HandlerFibonacciTransform(TransformationHandler):
    """Uses Fibonacci sequence for transformations"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Generate Fibonacci numbers mod 10
        fib = [0, 1]
        for _ in range(20):
            fib.append((fib[-1] + fib[-2]) % 10)
        
        # Transform non-zero values to Fibonacci sequence
        flat = [x for row in inp for x in row]
        fib_idx = 0
        transformed = []
        
        for val in flat:
            if val != 0:
                transformed.append(fib[fib_idx % len(fib)])
                fib_idx += 1
            else:
                transformed.append(0)
        
        test = np.array(transformed).reshape(inp.shape)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h, w = task_data['train'][0]['input'].shape
            return f"""def p(g):
 fib=[0,1]
 for _ in range(20):fib.append((fib[-1]+fib[-2])%10)
 f=[x for r in g for x in r];fi=0;t=[]
 for v in f:
  if v:t.append(fib[fi%len(fib)]);fi+=1
  else:t.append(0)
 return[t[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

@handler_decorator
class HandlerRecursivePartition(TransformationHandler):
    """Recursively partitions and transforms grid"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        def recursive_transform(grid, depth=0):
            if depth >= 3 or grid.shape[0] < 2 or grid.shape[1] < 2:
                return grid
            
            h, w = grid.shape
            result = np.copy(grid)
            
            # Transform each quadrant differently
            h2, w2 = h // 2, w // 2
            
            # Top-left: increment
            result[:h2, :w2] = (grid[:h2, :w2] + 1) % 10
            
            # Top-right: decrement
            result[:h2, w2:] = (grid[:h2, w2:] - 1) % 10
            
            # Bottom-left: double
            result[h2:, :w2] = (grid[h2:, :w2] * 2) % 10
            
            # Bottom-right: recurse
            result[h2:, w2:] = recursive_transform(grid[h2:, w2:], depth + 1)
            
            return result
        
        test = recursive_transform(inp)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 def rt(gr,d=0):
  if d>=3or len(gr)<2or len(gr[0])<2:return gr
  h,w=len(gr),len(gr[0]);r=[row[:]for row in gr]
  h2,w2=h//2,w//2
  for i in range(h2):
   for j in range(w2):r[i][j]=(gr[i][j]+1)%10
  for i in range(h2):
   for j in range(w2,w):r[i][j]=(gr[i][j]-1)%10
  for i in range(h2,h):
   for j in range(w2):r[i][j]=(gr[i][j]*2)%10
  br=rt([gr[i][w2:]for i in range(h2,h)],d+1)
  for i in range(len(br)):
   for j in range(len(br[0])):r[h2+i][w2+j]=br[i][j]
  return r
 return rt(g)
"""
        return None

@handler_decorator
class HandlerIterativeBlur(TransformationHandler):
    """Applies iterative blurring with small kernels"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        current = np.copy(inp)
        
        for _ in range(3):
            next_state = np.zeros_like(current)
            
            for i in range(current.shape[0]):
                for j in range(current.shape[1]):
                    # Simple 2x1 kernel
                    if j < current.shape[1] - 1:
                        next_state[i, j] = (current[i, j] + current[i, j+1]) // 2
                    else:
                        next_state[i, j] = current[i, j]
            
            current = next_state
        
        return np.array_equal(current, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            return """def p(g):
 r=[row[:]for row in g]
 for _ in range(3):
  n=[[0]*len(r[0])for _ in r]
  for i in range(len(r)):
   for j in range(len(r[0])):
    if j<len(r[0])-1:n[i][j]=(r[i][j]+r[i][j+1])//2
    else:n[i][j]=r[i][j]
  r=n
 return r
"""
        return None

@handler_decorator
class HandlerChainedMicroOps(TransformationHandler):
    """Chains multiple micro-operations"""
    def can_handle(self, task_data, analysis):
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.shape != out.shape:
            return False
        
        # Chain: flatten -> filter odds -> double -> mod 10 -> reshape
        flat = [x for row in inp for x in row]
        
        # Keep track of positions
        result = []
        for i, val in enumerate(flat):
            if val % 2 == 1:  # Odd values
                result.append((val * 2) % 10)
            else:
                result.append(val)
        
        test = np.array(result).reshape(inp.shape)
        return np.array_equal(test, out)
        
    def generate_solution(self, task_data, analysis):
        if self.can_handle(task_data, analysis):
            h, w = task_data['train'][0]['input'].shape
            return f"""def p(g):
 f=[x for r in g for x in r]
 r=[(v*2)%10if v%2else v for v in f]
 return[r[i*{w}:(i+1)*{w}]for i in range({h})]
"""
        return None

# ================================================================================
# ================================================================================

In [ ]:
class SolutionCombiner:
    """Enhanced solution combination strategies"""
    
    @staticmethod
    def combine_sequential(handlers: List[Tuple[TransformationHandler, str]], 
                          task_data: dict) -> Optional[str]:
        """Apply handlers in sequence with optimization"""
        if len(handlers) < 2:
            return None
        
        # Extract minimal function bodies
        functions = []
        for handler, solution in handlers[:3]:  # Limit to 3 for size
            lines = solution.strip().split('\n')
            # Extract core logic
            core_lines = []
            for line in lines[1:-1]:  # Skip def and return
                if line.strip() and not line.strip().startswith('#'):
                    core_lines.append(line[1:] if line.startswith(' ') else line)
            
            if core_lines:
                functions.append('\n'.join(core_lines))
        
        # Build compact sequential combination
        combined = "def p(g):\n"
        
        # Inline the transformations
        combined += " r=g\n"
        for i, func_body in enumerate(functions):
            # Compact representation
            if len(func_body) < 100:  # Short enough to inline
                combined += " " + func_body.replace('\n', '\n ').replace('g', 'r') + "\n"
            else:
                # Create mini function
                combined += f" r=(lambda x:{func_body.replace('g', 'x').replace(chr(10), ';')})(r)\n"
        
        combined += " return r"
        
        return SolutionOptimizer.optimize(combined)
    
    @staticmethod
    def combine_iterative(handlers: List[Tuple[TransformationHandler, str]], 
                         task_data: dict, iterations: int = 5) -> Optional[str]:
        """Combine handlers for iterative application"""
        if not handlers:
            return None
        
        handler, solution = handlers[0]
        
        # Extract core transformation
        lines = solution.strip().split('\n')
        core = []
        for line in lines[1:-1]:
            if line.strip():
                core.append(line[1:] if line.startswith(' ') else line)
        
        # Build iterative version
        combined = "def p(g):\n"
        combined += " r=[row[:] for row in g]\n"
        combined += f" for _ in range({iterations}):\n"
        combined += "  n=[row[:] for row in r]\n"
        
        # Add core transformation
        for line in core:
            combined += "  " + line.replace('res', 'n').replace('g', 'r') + "\n"
        
        combined += "  if n==r:break\n"
        combined += "  r=n\n"
        combined += " return r"
        
        return SolutionOptimizer.optimize(combined)
    
    @staticmethod
    def combine_1d_transform(handler: TransformationHandler, solution: str,
                           ordering: str = "row_major") -> Optional[str]:
        """Convert handler to work on 1D representation"""
        
        orderings = {
            "row_major": {
                "flatten": "f=[x for r in g for x in r]",
                "reshape": "return[f[i*w:(i+1)*w]for i in range(h)]"
            },
            "col_major": {
                "flatten": "f=[];w=len(g[0])\nfor j in range(w):\n for i in range(len(g)):f.append(g[i][j])",
                "reshape": "r=[[0]*w for _ in range(h)];idx=0\nfor j in range(w):\n for i in range(h):r[i][j]=f[idx];idx+=1\nreturn r"
            },
            "spiral": {
                "flatten": "f=[];t,b,l,r=0,h-1,0,w-1\nwhile t<=b and l<=r:\n f.extend(g[t][l:r+1]);t+=1\n for i in range(t,b+1):f.append(g[i][r])\n r-=1\n if t<=b:f.extend(g[b][r:l-1:-1]);b-=1\n if l<=r:\n  for i in range(b,t-1,-1):f.append(g[i][l])\n  l+=1",
                "reshape": "r=[[0]*w for _ in range(h)];idx=0;t,b,l,r=0,h-1,0,w-1\nwhile t<=b and l<=r and idx<len(f):\n for j in range(l,r+1):\n  if idx<len(f):r[t][j]=f[idx];idx+=1\n t+=1\n for i in range(t,b+1):\n  if idx<len(f):r[i][r]=f[idx];idx+=1\n r-=1\n if t<=b:\n  for j in range(r,l-1,-1):\n   if idx<len(f):r[b][j]=f[idx];idx+=1\n  b-=1\n if l<=r:\n  for i in range(b,t-1,-1):\n   if idx<len(f):r[i][l]=f[idx];idx+=1\n  l+=1\nreturn r"
            }
        }
        
        if ordering not in orderings:
            ordering = "row_major"
        
        # Extract 2D transformation logic
        lines = solution.strip().split('\n')
        transform_lines = []
        for line in lines[1:-1]:
            if 'for i' in line or 'for j' in line or '[i][j]' in line:
                # This is 2D specific, skip
                continue
            transform_lines.append(line.strip())
        
        # Build 1D version
        combined = "def p(g):\n"
        combined += " h,w=len(g),len(g[0])\n"
        combined += " " + orderings[ordering]["flatten"].replace('\n', '\n ') + "\n"
        
        # Add 1D transformation
        combined += " # Transform in 1D\n"
        combined += " nf=[(x+i)%10 if x else 0 for i,x in enumerate(f)]\n"  # Example transform
        
        combined += " " + orderings[ordering]["reshape"].replace('\n', '\n ')
        
        return SolutionOptimizer.optimize(combined)
    
    @staticmethod
    def combine_conditional_region(handlers: List[Tuple[TransformationHandler, str]], 
                                 task_data: dict) -> Optional[str]:
        """Apply different handlers to different regions"""
        if len(handlers) < 2:
            return None
        
        # Detect regions in input
        inp = np.array(task_data['train'][0]['input'])
        h, w = inp.shape
        
        combined = "def p(g):\n"
        combined += " h,w=len(g),len(g[0])\n"
        combined += " r=[row[:]for row in g]\n"
        
        # Apply first handler to non-zero regions
        handler1_core = SolutionCombiner._extract_core(handlers[0][1])
        combined += " # Transform non-zero regions\n"
        combined += " for i in range(h):\n"
        combined += "  for j in range(w):\n"
        combined += "   if g[i][j]:\n"
        combined += "    " + handler1_core.replace('\n', '\n    ') + "\n"
        
        # Apply second handler to zero regions
        if len(handlers) > 1:
            handler2_core = SolutionCombiner._extract_core(handlers[1][1])
            combined += " # Transform zero regions\n"
            combined += " for i in range(h):\n"
            combined += "  for j in range(w):\n"
            combined += "   if not g[i][j]:\n"
            combined += "    " + handler2_core.replace('\n', '\n    ') + "\n"
        
        combined += " return r"
        
        return SolutionOptimizer.optimize(combined)
    
    @staticmethod
    def _extract_core(solution: str) -> str:
        """Extract core transformation logic"""
        lines = solution.strip().split('\n')
        core_lines = []
        
        for line in lines:
            line = line.strip()
            if line and not line.startswith(('def', 'return', '#')):
                # Look for transformation operations
                if any(op in line for op in ['=', '+=', '-=', '*=', '//=', '%=']):
                    core_lines.append(line)
        
        return '\n'.join(core_lines[:3])  # Limit to 3 lines


class SolutionOptimizer:
    """Advanced optimizer for aggressive code size reduction"""
    
    @staticmethod
    def optimize(solution: str) -> str:
        lines = solution.strip().split('\n')
        
        # Phase 1: Aggressive single-character variables
        var_map = {
            'result': 'r', 'row': 'r', 'col': 'c', 'temp': 't', 'value': 'v',
            'grid': 'g', 'mask': 'm', 'test': 't', 'output': 'o', 'input': 'i',
            'height': 'h', 'width': 'w', 'index': 'i', 'jndex': 'j', 'count': 'c',
            'visited': 'v', 'stack': 's', 'queue': 'q', 'current': 'c', 'next': 'n',
            'neighbor': 'n', 'component': 'p', 'objects': 'o', 'pattern': 'p'
        }
        
        # Build regex patterns for whole-word replacement
        import re
        for old, new in var_map.items():
            pattern = r'\b' + old + r'\b'
            solution = re.sub(pattern, new, solution)
        
        # Phase 2: Compact operations
        replacements = {
            'return ': 'return',
            'range(0,': 'range(',
            'range(len(': 'range(len(',
            'for i in range(len(g))': 'for i,r in enumerate(g)',
            'for j in range(len(g[0]))': 'for j,v in enumerate(r)',
            'if x != 0': 'if x',
            'if x == 0': 'if not x',
            '== True': '',
            '== False': ' not',
            'append(': 'append(',
            ') )': '))',
            '( (': '((',
            '    ': ' ',  # Single space for indentation
            '\n\n': '\n',
            '; ': ';',
            ', ': ','
        }
        
        for old, new in replacements.items():
            solution = solution.replace(old, new)
        
        # Phase 3: Inline simple operations
        lines = solution.split('\n')
        optimized = []
        skip_indices = set()
        
        for i, line in enumerate(lines):
            if i in skip_indices:
                continue
                
            # Inline single-use variables
            if '=' in line and i + 1 < len(lines):
                stripped = line.strip()
                if stripped.count('=') == 1:
                    var_name = stripped.split('=')[0].strip()
                    var_value = stripped.split('=')[1].strip()
                    
                    # Check if variable is used only once in next line
                    next_line = lines[i + 1]
                    if var_name in next_line and next_line.count(var_name) == 1:
                        # Inline it
                        new_next = next_line.replace(var_name, f'({var_value})')
                        optimized.append(new_next)
                        skip_indices.add(i + 1)
                        continue
            
            optimized.append(line)
        
        # Phase 4: Compress list comprehensions
        solution = '\n'.join(optimized)
        
        # Nested comprehension optimization
        solution = re.sub(
            r'\[\[(.+?) for (.+?) in (.+?)\] for (.+?) in (.+?)\]',
            r'[[(\1)for \2 in(\3)]for \4 in(\5)]',
            solution
        )
        
        # Phase 5: Remove unnecessary whitespace
        lines = solution.split('\n')
        final = []
        
        for line in lines:
            # Preserve indentation but minimize spaces
            indent = len(line) - len(line.lstrip())
            content = line.strip()
            
            # Minimize spaces around operators
            content = re.sub(r'\s*([=+\-*/%<>!]+)\s*', r'\1', content)
            content = re.sub(r'\s*,\s*', ',', content)
            content = re.sub(r'\s*:\s*', ':', content)
            
            # Special cases where spaces are needed
            content = re.sub(r'(\w)(if|for|in|and|or|not)(\w)', r'\1 \2 \3', content)
            content = re.sub(r'(return)(\w)', r'\1 \2', content)
            content = re.sub(r'(def|class)(\w)', r'\1 \2', content)
            
            if content:
                final.append(' ' * indent + content)
        
        # Phase 6: Use semicolons for multiple statements
        result = []
        for line in final:
            if line.strip() and not line.strip().startswith(('def', 'class', 'if', 'for', 'while', 'return')):
                # Check if we can combine with previous line
                if result and not result[-1].strip().endswith(':'):
                    if len(result[-1] + ';' + line.strip()) < 80:  # Reasonable line length
                        result[-1] = result[-1] + ';' + line.strip()
                        continue
            result.append(line)
        
        return '\n'.join(result)


import numpy as np
import json
import os
import time
import zipfile
from typing import List, Dict, Optional, Tuple, Set, Any, Callable
from collections import defaultdict, Counter
from abc import ABC, abstractmethod
import re
from itertools import combinations

# ================================================================================
# ROBUST BASE HANDLER WITH COMPREHENSIVE ERROR HANDLING
# ================================================================================

class TransformationHandler(ABC):
    """Base class with comprehensive error handling and shape validation"""
    
    def __init__(self):
        self.name = self.__class__.__name__
    
    @abstractmethod
    def can_handle(self, task_data: dict, analysis: dict) -> bool:
        """Determine if this handler can process the given task"""
        pass
    
    @abstractmethod
    def generate_solution(self, task_data: dict, analysis: dict) -> Optional[str]:
        """Generate solution code for the task"""
        pass
    
    def safe_can_handle(self, task_data: dict, analysis: dict) -> bool:
        """Wrapper with comprehensive error handling"""
        try:
            # Validate task data structure first
            if not self._validate_task_data(task_data):
                return False
            
            # Ensure analysis is valid
            if not isinstance(analysis, dict):
                return False
            
            # Call the actual can_handle with error protection
            return self.can_handle(task_data, analysis)
            
        except Exception:
            return False
    
    def safe_generate_solution(self, task_data: dict, analysis: dict) -> Optional[str]:
        """Wrapper with error handling for solution generation"""
        try:
            # Validate before generation
            if not self._validate_task_data(task_data):
                return None
            
            solution = self.generate_solution(task_data, analysis)
            
            # Validate generated solution
            if solution and isinstance(solution, str) and 'def p(g):' in solution:
                return solution
            
            return None
            
        except Exception:
            return None
    
    def _validate_task_data(self, task_data: dict) -> bool:
        """Comprehensive task data validation"""
        try:
            if not isinstance(task_data, dict):
                return False
            
            if 'train' not in task_data or not isinstance(task_data['train'], list):
                return False
            
            if len(task_data['train']) == 0:
                return False
            
            for example in task_data['train']:
                if not isinstance(example, dict):
                    return False
                
                if 'input' not in example or 'output' not in example:
                    return False
                
                # Validate input structure
                if not isinstance(example['input'], list):
                    return False
                
                if len(example['input']) == 0:
                    return False
                
                # Check each row
                for row in example['input']:
                    if not isinstance(row, list) or len(row) == 0:
                        return False
                
                # Validate output structure similarly
                if not isinstance(example['output'], list):
                    return False
                
                if len(example['output']) == 0:
                    return False
                
                for row in example['output']:
                    if not isinstance(row, list) or len(row) == 0:
                        return False
            
            return True
            
        except Exception:
            return False
    
    def verify_solution(self, solution: str, task_data: dict) -> bool:
        """Enhanced solution verification with robust error handling"""
        try:
            # Create safe execution environment
            safe_globals = {
                '__builtins__': {
                    'range': range, 'len': len, 'min': min, 'max': max,
                    'sum': sum, 'abs': abs, 'enumerate': enumerate, 'zip': zip,
                    'list': list, 'set': set, 'sorted': sorted, 'reversed': reversed,
                    'filter': filter, 'map': map, 'any': any, 'all': all,
                    'int': int, 'str': str, 'tuple': tuple, 'dict': dict,
                }
            }
            
            local_namespace = {}
            exec(solution, safe_globals, local_namespace)
            
            if 'p' not in local_namespace:
                return False
            
            p = local_namespace['p']
            
            # Test on all training examples
            for example in task_data['train']:
                try:
                    # Create deep copy to prevent modification
                    input_grid = [row[:] for row in example['input']]
                    expected = example['output']
                    
                    # Execute function
                    actual = p(input_grid)
                    
                    # Validate output format
                    if not isinstance(actual, list):
                        return False
                    
                    # Compare results
                    if not self._grids_equal(actual, expected):
                        return False
                        
                except Exception:
                    return False
            
            return True
            
        except Exception:
            return False
    
    def _grids_equal(self, grid1: Any, grid2: Any) -> bool:
        """Safely compare two grids"""
        try:
            # First check if both are lists
            if not (isinstance(grid1, list) and isinstance(grid2, list)):
                return False
            
            # Check dimensions
            if len(grid1) != len(grid2):
                return False
            
            # Check each row
            for i in range(len(grid1)):
                if not (isinstance(grid1[i], list) and isinstance(grid2[i], list)):
                    return False
                
                if len(grid1[i]) != len(grid2[i]):
                    return False
                
                # Compare elements
                for j in range(len(grid1[i])):
                    if grid1[i][j] != grid2[i][j]:
                        return False
            
            return True
            
        except Exception:
            return False


# ================================================================================
# SAFE TASK ANALYZER WITH COMPREHENSIVE ERROR HANDLING
# ================================================================================

class SafeTaskAnalyzer:
    """Task analyzer that handles all edge cases gracefully"""
    
    def analyze(self, task_data: dict) -> dict:
        """Analyze task with complete error protection"""
        analysis = {
            'shape_changes': set(),
            'color_changes': {},
            'transform_types': set(),
            'has_patterns': False,
            'has_objects': False,
            'is_iterative': False,
            'benefits_from_1d': False,
            'has_recursive_structure': False,
            'symmetry_type': 'none',
            'is_local_transform': False,
            'complexity_score': 0,
            'is_valid': True,
            'error_messages': []
        }
        
        try:
            # Validate task structure
            if not self._validate_task_structure(task_data):
                analysis['is_valid'] = False
                analysis['error_messages'].append("Invalid task structure")
                return analysis
            
            # Safely analyze each example
            for idx, example in enumerate(task_data['train']):
                try:
                    # Convert to numpy arrays safely
                    inp_array = self._safe_to_numpy(example['input'])
                    out_array = self._safe_to_numpy(example['output'])
                    
                    if inp_array is None or out_array is None:
                        analysis['error_messages'].append(f"Example {idx}: Invalid array format")
                        continue
                    
                    # Shape analysis
                    analysis['shape_changes'].add((inp_array.shape, out_array.shape))
                    
                    # Safe color analysis
                    self._analyze_colors_safe(inp_array, out_array, analysis)
                    
                    # Pattern detection
                    if self._has_repeating_pattern_safe(inp_array):
                        analysis['has_patterns'] = True
                    
                    # Object detection
                    if self._has_distinct_objects_safe(inp_array):
                        analysis['has_objects'] = True
                    
                    # Advanced pattern detection (only on first example)
                    if idx == 0:
                        self._analyze_advanced_patterns_safe(inp_array, out_array, analysis)
                    
                except Exception as e:
                    analysis['error_messages'].append(f"Example {idx}: Analysis failed")
            
            # Calculate complexity
            analysis['complexity_score'] = self._calculate_complexity_safe(analysis)
            
        except Exception:
            analysis['is_valid'] = False
            analysis['error_messages'].append("Critical analysis error")
        
        return analysis
    
    def _validate_task_structure(self, task_data: dict) -> bool:
        """Validate basic task structure"""
        try:
            if not isinstance(task_data, dict):
                return False
            
            if 'train' not in task_data or not isinstance(task_data['train'], list):
                return False
            
            if len(task_data['train']) == 0:
                return False
            
            for example in task_data['train']:
                if not isinstance(example, dict):
                    return False
                
                if 'input' not in example or 'output' not in example:
                    return False
                
                # Check that input and output are valid grids
                for grid in [example['input'], example['output']]:
                    if not isinstance(grid, list) or len(grid) == 0:
                        return False
                    
                    width = len(grid[0]) if isinstance(grid[0], list) else 0
                    if width == 0:
                        return False
                    
                    # Check all rows have same width
                    for row in grid:
                        if not isinstance(row, list) or len(row) != width:
                            return False
            
            return True
            
        except Exception:
            return False
    
    def _safe_to_numpy(self, grid: Any) -> Optional[np.ndarray]:
        """Safely convert grid to numpy array"""
        try:
            if not isinstance(grid, list):
                return None
            
            # Validate rectangular shape
            if len(grid) == 0:
                return None
            
            width = len(grid[0]) if isinstance(grid[0], list) else 0
            if width == 0:
                return None
            
            # Check all rows have same width
            for row in grid:
                if not isinstance(row, list) or len(row) != width:
                    return None
            
            # Convert to numpy
            arr = np.array(grid, dtype=int)
            
            # Validate 2D
            if arr.ndim != 2:
                return None
            
            return arr
            
        except Exception:
            return None
    
    def _analyze_colors_safe(self, inp: np.ndarray, out: np.ndarray, analysis: dict) -> None:
        """Safely analyze color transformations"""
        try:
            # Get unique colors
            inp_colors = set(np.unique(inp).tolist())
            out_colors = set(np.unique(out).tolist())
            
            # Initialize color changes
            for color in inp_colors:
                if color not in analysis['color_changes']:
                    analysis['color_changes'][color] = set()
            
            # Only analyze if shapes match
            if inp.shape == out.shape:
                for color in inp_colors:
                    # Find where this color appears in input
                    mask = (inp == color)
                    # See what colors appear in those positions in output
                    transformed_colors = set(out[mask].tolist())
                    analysis['color_changes'][color].update(transformed_colors)
                    
        except Exception:
            pass
    
    def _has_repeating_pattern_safe(self, grid: np.ndarray) -> bool:
        """Safely detect repeating patterns"""
        try:
            h, w = grid.shape
            
            # Check for small repeating patterns
            for pattern_h in [2, 3]:
                for pattern_w in [2, 3]:
                    if h >= pattern_h * 2 and w >= pattern_w * 2:
                        # Check if grid can be tiled by a pattern
                        pattern = grid[:pattern_h, :pattern_w]
                        
                        # Look for this pattern elsewhere
                        for i in range(0, h - pattern_h + 1, pattern_h):
                            for j in range(0, w - pattern_w + 1, pattern_w):
                                if (i > 0 or j > 0):
                                    test_region = grid[i:i+pattern_h, j:j+pattern_w]
                                    if np.array_equal(pattern, test_region):
                                        return True
            
            return False
            
        except Exception:
            return False
    
    def _has_distinct_objects_safe(self, grid: np.ndarray) -> bool:
        """Safely detect distinct objects"""
        try:
            # Count non-zero unique values
            unique_vals = np.unique(grid)
            non_zero_vals = [v for v in unique_vals if v != 0]
            
            # Multiple non-zero values suggest objects
            return len(non_zero_vals) >= 2
            
        except Exception:
            return False
    
    def _analyze_advanced_patterns_safe(self, inp: np.ndarray, out: np.ndarray, 
                                       analysis: dict) -> None:
        """Safely analyze advanced patterns"""
        try:
            # Check if shapes are compatible
            if inp.shape == out.shape:
                # Iterative pattern detection
                diff_count = np.sum(inp != out)
                total_cells = inp.size
                
                if 0 < diff_count < total_cells * 0.5:
                    analysis['is_iterative'] = True
                
                # Check for symmetry
                analysis['symmetry_type'] = self._detect_symmetry_safe(inp, out)
                
                # Check if transformation is local
                analysis['is_local_transform'] = self._check_local_transform_safe(inp, out)
            
            # Check if would benefit from 1D transformation
            if inp.size > 0 and out.size > 0:
                analysis['benefits_from_1d'] = self._check_1d_benefit_safe(inp, out)
            
        except Exception:
            pass
    
    def _detect_symmetry_safe(self, inp: np.ndarray, out: np.ndarray) -> str:
        """Safely detect symmetry"""
        try:
            if inp.shape != out.shape:
                return "none"
            
            # Check various symmetries
            if np.array_equal(out, np.fliplr(inp)):
                return "horizontal_flip"
            
            if np.array_equal(out, np.flipud(inp)):
                return "vertical_flip"
            
            # Check rotations
            for k in range(1, 4):
                if np.array_equal(out, np.rot90(inp, k)):
                    return f"rotation_{k*90}"
            
            return "none"
            
        except Exception:
            return "none"
    
    def _check_local_transform_safe(self, inp: np.ndarray, out: np.ndarray) -> bool:
        """Safely check if transformation is local"""
        try:
            if inp.shape != out.shape:
                return False
            
            h, w = inp.shape
            
            # Sample a few positions
            sample_positions = [
                (h//4, w//4),
                (h//2, w//2),
                (3*h//4, 3*w//4)
            ]
            
            local_count = 0
            
            for i, j in sample_positions:
                if 1 <= i < h-1 and 1 <= j < w-1:
                    # Get 3x3 neighborhood
                    neighborhood = inp[max(0,i-1):min(h,i+2), max(0,j-1):min(w,j+2)]
                    
                    # Check if output depends on neighborhood
                    if neighborhood.size > 0:
                        n_sum = np.sum(neighborhood)
                        n_max = np.max(neighborhood)
                        
                        if out[i, j] in [n_sum % 10, n_max]:
                            local_count += 1
            
            return local_count >= 2
            
        except Exception:
            return False
    
    def _check_1d_benefit_safe(self, inp: np.ndarray, out: np.ndarray) -> bool:
        """Safely check if 1D transformation would help"""
        try:
            if inp.shape != out.shape:
                return False
            
            # Flatten arrays
            flat_in = inp.flatten()
            flat_out = out.flatten()
            
            # Check for position-dependent transformations
            position_dependent = 0
            
            for i in range(min(len(flat_in), 10)):  # Check first 10 elements
                if flat_in[i] != 0 and flat_out[i] != flat_in[i]:
                    # Check if output depends on position
                    if flat_out[i] == (i % 10) or flat_out[i] == ((flat_in[i] + i) % 10):
                        position_dependent += 1
            
            return position_dependent >= 3
            
        except Exception:
            return False
    
    def _calculate_complexity_safe(self, analysis: dict) -> int:
        """Safely calculate complexity score"""
        try:
            score = 0
            
            # Shape complexity
            if len(analysis.get('shape_changes', set())) > 1:
                score += 20
            
            # Color complexity
            color_count = len(analysis.get('color_changes', {}))
            score += min(color_count * 5, 30)
            
            # Pattern complexity
            if analysis.get('has_patterns', False):
                score += 15
            
            if analysis.get('has_objects', False):
                score += 10
            
            if analysis.get('is_iterative', False):
                score += 10
            
            # Error penalty
            error_count = len(analysis.get('error_messages', []))
            score += min(error_count * 5, 20)
            
            return min(score, 100)
            
        except Exception:
            return 50  # Default medium complexity


# ================================================================================
# REGISTRY WITH ERROR HANDLING
# ================================================================================

class HandlerRegistry:
    """Registry for transformation handlers with error handling"""
    
    def __init__(self):
        self.handlers = []
        self.performance_stats = defaultdict(lambda: {
            'calls': 0,
            'successes': 0,
            'time': 0
        })
    
    def register(self, handler: TransformationHandler) -> None:
        """Register a handler with validation"""
        if isinstance(handler, TransformationHandler):
            self.handlers.append(handler)
    
    def get_all_handlers(self) -> List[TransformationHandler]:
        """Get all registered handlers"""
        return self.handlers.copy()


# ================================================================================
# ENHANCED SOLUTION PROCESSING (Keep existing implementation)
# ================================================================================

# [Include the SolutionOptimizer, SolutionCombiner classes from the original code]
# [These are already well-implemented and don't need changes]

# ================================================================================
# ROBUST TASK PROCESSOR
# ================================================================================

class RobustTaskProcessor:
    """Task processor with comprehensive error handling"""
    
    def __init__(self, registry: HandlerRegistry):
        self.registry = registry
        self.analyzer = SafeTaskAnalyzer()
        self.optimizer = SolutionOptimizer()
        self.combiner = SolutionCombiner()
        self.cache = {}
        self.error_log = []
        self.handler_performance = defaultdict(lambda: {
            'attempts': 0,
            'successes': 0,
            'avg_time': 0,
            'complexity_solved': []
        })
    
    def process_task(self, task_data: dict) -> str:
        """Process task with complete error protection"""
        try:
            # Validate task data first
            if not self._validate_task_data(task_data):
                return self._safe_fallback()
            
            # Check cache
            cache_key = self._get_cache_key(task_data)
            if cache_key and cache_key in self.cache:
                return self.cache[cache_key]
            
            # Analyze task
            analysis = self.analyzer.analyze(task_data)
            
            # Check if analysis succeeded
            if not analysis.get('is_valid', False):
                self.error_log.append({
                    'task': 'unknown',
                    'phase': 'analysis',
                    'errors': analysis.get('error_messages', [])
                })
                return self._safe_fallback()
            
            # Get handlers
            handlers = self.registry.get_all_handlers()
            
            if not handlers:
                return self._safe_fallback()
            
            # Smart handler selection based on analysis
            handlers = self._prioritize_handlers(handlers, analysis)
            
            # Try each handler with safety wrapper
            for handler in handlers[:30]:  # Limit iterations
                try:
                    handler_name = handler.name
                    self.handler_performance[handler_name]['attempts'] += 1
                    
                    start_time = time.time()
                    
                    # Use safe methods
                    if handler.safe_can_handle(task_data, analysis):
                        solution = handler.safe_generate_solution(task_data, analysis)
                        
                        if solution and handler.verify_solution(solution, task_data):
                            # Success!
                            elapsed = time.time() - start_time
                            perf = self.handler_performance[handler_name]
                            perf['successes'] += 1
                            perf['avg_time'] = (perf['avg_time'] * (perf['successes'] - 1) + elapsed) / perf['successes']
                            perf['complexity_solved'].append(analysis.get('complexity_score', 50))
                            
                            optimized = self.optimizer.optimize(solution)
                            
                            if cache_key:
                                self.cache[cache_key] = optimized
                            
                            return optimized
                            
                except Exception as e:
                    # Log but continue
                    self.error_log.append({
                        'handler': handler.name,
                        'phase': 'execution',
                        'error': str(e)[:100]
                    })
                    continue
            
            # Try combinations if single handlers failed
            return self._try_combinations(task_data, analysis, handlers)
            
        except Exception as e:
            # Ultimate fallback
            self.error_log.append({
                'phase': 'critical',
                'error': str(e)[:100]
            })
            return self._safe_fallback()
    
    def _validate_task_data(self, task_data: dict) -> bool:
        """Validate task data structure"""
        try:
            if not isinstance(task_data, dict):
                return False
            
            if 'train' not in task_data:
                return False
            
            if not isinstance(task_data['train'], list) or len(task_data['train']) == 0:
                return False
            
            # Validate each example
            for example in task_data['train']:
                if not isinstance(example, dict):
                    return False
                
                if 'input' not in example or 'output' not in example:
                    return False
                
                # Check grid structure
                for grid in [example['input'], example['output']]:
                    if not isinstance(grid, list) or len(grid) == 0:
                        return False
                    
                    # Check rectangular shape
                    width = len(grid[0]) if isinstance(grid[0], list) else -1
                    if width <= 0:
                        return False
                    
                    for row in grid:
                        if not isinstance(row, list) or len(row) != width:
                            return False
            
            return True
            
        except Exception:
            return False
    
    def _prioritize_handlers(self, handlers: List[TransformationHandler], 
                           analysis: dict) -> List[TransformationHandler]:
        """Prioritize handlers based on analysis"""
        try:
            scored_handlers = []
            
            for handler in handlers:
                score = 0
                handler_name = handler.name.lower()
                
                # Historical performance
                perf = self.handler_performance.get(handler.name, {})
                if perf.get('attempts', 0) > 0:
                    success_rate = perf.get('successes', 0) / perf['attempts']
                    score += success_rate * 50
                
                # Match handler type to detected patterns
                if analysis.get('is_iterative') and 'iterative' in handler_name:
                    score += 30
                
                if analysis.get('has_patterns') and 'pattern' in handler_name:
                    score += 25
                
                if analysis.get('symmetry_type') != 'none' and any(
                    term in handler_name for term in ['symmetry', 'flip', 'rotate']):
                    score += 25
                
                if analysis.get('is_local_transform') and any(
                    term in handler_name for term in ['kernel', 'local', 'neighbor']):
                    score += 20
                
                # Simple tasks prefer simple handlers
                complexity = analysis.get('complexity_score', 50)
                if complexity < 30 and any(
                    term in handler_name for term in ['simple', 'basic', 'identity']):
                    score += 20
                
                scored_handlers.append((score, handler))
            
            # Sort by score
            scored_handlers.sort(key=lambda x: x[0], reverse=True)
            
            return [handler for _, handler in scored_handlers]
            
        except Exception:
            # Return original order if prioritization fails
            return handlers
    
    def _try_combinations(self, task_data: dict, analysis: dict, 
                         handlers: List[TransformationHandler]) -> str:
        """Try handler combinations as fallback"""
        try:
            # Find handlers that produce valid solutions
            valid_handlers = []
            
            for handler in handlers[:10]:
                try:
                    if handler.safe_can_handle(task_data, analysis):
                        solution = handler.safe_generate_solution(task_data, analysis)
                        if solution:
                            valid_handlers.append((handler, solution))
                except Exception:
                    continue
            
            # Try sequential combination
            if len(valid_handlers) >= 2:
                try:
                    combined = self.combiner.combine_sequential(
                        valid_handlers[:2], task_data)
                    
                    if combined and self._quick_verify(combined, task_data):
                        return self.optimizer.optimize(combined)
                except Exception:
                    pass
            
            # No successful combination
            return self._safe_fallback()
            
        except Exception:
            return self._safe_fallback()
    
    def _quick_verify(self, solution: str, task_data: dict) -> bool:
        """Quick verification on first example only"""
        try:
            safe_globals = {
                '__builtins__': {
                    'range': range, 'len': len, 'min': min, 'max': max,
                    'sum': sum, 'abs': abs, 'enumerate': enumerate, 'zip': zip,
                    'list': list, 'set': set, 'sorted': sorted, 'reversed': reversed,
                    'filter': filter, 'map': map, 'any': any, 'all': all,
                    'int': int, 'str': str, 'tuple': tuple,
                }
            }
            
            local_namespace = {}
            exec(solution, safe_globals, local_namespace)
            
            if 'p' not in local_namespace:
                return False
            
            p = local_namespace['p']
            example = task_data['train'][0]
            
            # Create deep copy
            input_copy = [row[:] for row in example['input']]
            
            try:
                result = p(input_copy)
                return result == example['output']
            except Exception:
                return False
                
        except Exception:
            return False
    
    def _get_cache_key(self, task_data: dict) -> Optional[str]:
        """Generate cache key for task"""
        try:
            # Use first example for cache key
            example = task_data['train'][0]
            inp = example['input']
            out = example['output']
            
            # Create string representation
            inp_str = str(inp)
            out_str = str(out)
            
            # Combine with shapes
            inp_shape = (len(inp), len(inp[0]) if inp else 0)
            out_shape = (len(out), len(out[0]) if out else 0)
            
            key = f"{inp_shape}_{out_shape}_{hash(inp_str)}_{hash(out_str)}"
            
            return key
            
        except Exception:
            return None
    
    def _safe_fallback(self) -> str:
        """Return a safe fallback solution"""
        return "def p(g):\n return g"


# ================================================================================
# COMPETITION RUNNER
# ================================================================================

class RobustCompetitionRunner:
    """Competition runner with comprehensive error handling"""
    
    def __init__(self, input_dir: str, output_dir: str, registry: HandlerRegistry):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.registry = registry
        self.processor = RobustTaskProcessor(registry)
        self.solution_stats = defaultdict(int)
    
    def run(self):
        """Run competition with robust error handling"""
        os.makedirs(self.output_dir, exist_ok=True)
        
        solutions = {}
        solved_count = 0
        total_count = 0
        start_time = time.time()
        
        print(f"{'='*60}")
        print("ARC-AGI Code Golf Competition - Robust Runner")
        print(f"{'='*60}\n")
        print(f"Registered handlers: {len(self.registry.get_all_handlers())}\n")
        
        for task_num in range(1, 401):
            task_id = f"{task_num:03d}"
            task_file = os.path.join(self.input_dir, f"task{task_id}.json")
            
            try:
                # Load task
                with open(task_file) as f:
                    task_data = json.load(f)
                
                task_start = time.time()
                
                # Process task
                solution = self.processor.process_task(task_data)
                
                task_time = time.time() - task_start
                
                solutions[task_id] = solution
                
                # Verify solution
                if self._verify_solution(solution, task_data):
                    print(f"✅ Task {task_id} solved ({task_time:.2f}s) - {len(solution)} chars")
                    solved_count += 1
                    self.solution_stats['solved'] += 1
                else:
                    print(f"❌ Task {task_id} fallback ({task_time:.2f}s)")
                    self.solution_stats['fallback'] += 1
                
                total_count += 1
                
                # Progress update
                if task_num % 50 == 0:
                    elapsed = time.time() - start_time
                    rate = solved_count / total_count * 100 if total_count > 0 else 0
                    print(f"\nProgress: {task_num}/400 | Success rate: {rate:.1f}% | Time: {elapsed:.1f}s\n")
                    
            except Exception as e:
                print(f"❌ Task {task_id} - error: {str(e)[:50]}")
                solutions[task_id] = "def p(g):\n return g"
                total_count += 1
                self.solution_stats['error'] += 1
        
        # Write solutions
        self._write_solutions(solutions)
        
        # Print statistics
        total_time = time.time() - start_time
        self._print_statistics(solved_count, total_count, total_time, solutions)
    
    def _verify_solution(self, solution: str, task_data: dict) -> bool:
        """Verify solution on all training examples"""
        try:
            safe_globals = {
                '__builtins__': {
                    'range': range, 'len': len, 'min': min, 'max': max,
                    'sum': sum, 'abs': abs, 'enumerate': enumerate, 'zip': zip,
                    'list': list, 'set': set, 'sorted': sorted, 'reversed': reversed,
                    'filter': filter, 'map': map, 'any': any, 'all': all,
                    'int': int, 'str': str, 'tuple': tuple,
                }
            }
            
            local_namespace = {}
            exec(solution, safe_globals, local_namespace)
            
            if 'p' not in local_namespace:
                return False
            
            p = local_namespace['p']
            
            # Test all examples
            for example in task_data['train']:
                try:
                    input_copy = [row[:] for row in example['input']]
                    result = p(input_copy)
                    
                    if result != example['output']:
                        return False
                        
                except Exception:
                    return False
            
            return True
            
        except Exception:
            return False
    
    def _write_solutions(self, solutions: dict) -> None:
        """Write solutions to files"""
        total_chars = 0
        
        for task_id, code in solutions.items():
            output_file = os.path.join(self.output_dir, f"task{task_id}.py")
            with open(output_file, "w") as f:
                f.write(code)
            total_chars += len(code)
        
        # Create submission zip
        with zipfile.ZipFile("submission.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
            for task_id in solutions:
                file_path = os.path.join(self.output_dir, f"task{task_id}.py")
                zipf.write(file_path, f"task{task_id}.py")
    
    def _print_statistics(self, solved: int, total: int, time: float, solutions: dict) -> None:
        """Print final statistics"""
        total_chars = sum(len(code) for code in solutions.values())
        
        print(f"\n{'='*60}")
        print("FINAL RESULTS")
        print(f"{'='*60}")
        print(f"✅ Solved: {solved}/{total} ({solved/total*100:.1f}% if total > 0 else 0)")
        print(f"⏱️  Total time: {time:.1f}s ({time/total:.2f}s per task if total > 0 else 0)")
        print(f"📏 Total chars: {total_chars:,} ({total_chars//total if total > 0 else 0} avg)")
        print(f"📦 Submission: submission.zip")
        
        print(f"\nSolution Statistics:")
        for stat_type, count in sorted(self.solution_stats.items()):
            print(f"  {stat_type}: {count}")
        
        # Print handler performance
        print(f"\nTop Performing Handlers:")
        top_handlers = sorted(
            [(name, stats) for name, stats in self.processor.handler_performance.items()
             if stats['successes'] > 0],
            key=lambda x: x[1]['successes'],
            reverse=True
        )[:10]
        
        for name, stats in top_handlers:
            rate = stats['successes'] / stats['attempts'] * 100 if stats['attempts'] > 0 else 0
            avg_complexity = sum(stats['complexity_solved']) / len(stats['complexity_solved']) if stats['complexity_solved'] else 0
            print(f"  {name}: {stats['successes']} solved ({rate:.1f}%), "
                  f"{stats['avg_time']:.3f}s avg, complexity: {avg_complexity:.1f}")
        
        # Print error summary if any
        if self.processor.error_log:
            print(f"\nError Summary (showing first 5):")
            for error in self.processor.error_log[:5]:
                print(f"  {error.get('phase', 'unknown')}: {error.get('error', 'unknown')[:80]}")


# ================================================================================
# ENTRY POINT
# ================================================================================

if __name__ == "__main__":
    # Initialize registry
    registry = HandlerRegistry()
    
    # Import handlers safely
    import inspect
    import sys
    
    current_module = sys.modules[__name__]
    
    # Find and register all handler classes
    handler_count = 0
    for name, obj in inspect.getmembers(current_module):
        if (inspect.isclass(obj) and 
            issubclass(obj, TransformationHandler) and 
            obj != TransformationHandler):
            try:
                if not inspect.isabstract(obj):
                    handler_instance = obj()
                    registry.register(handler_instance)
                    handler_count += 1
            except Exception as e:
                print(f"Warning: Could not instantiate {name}: {str(e)[:50]}")
    
    print(f"Successfully registered {handler_count} handlers")
    
    # Create and run the robust competition runner
    runner = RobustCompetitionRunner(
        input_dir="/kaggle/input/google-code-golf-2025",
        output_dir="submission",
        registry=registry
    )
    
    # Run the competition
    runner.run()